In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_dir = '../data/PoliInfo2-EntityLinking-FormalRun-v20200703/AnswerSheet'
os.listdir(data_dir)

['PoliInfo2-EntityLinking-JA-Formal-Test.tsv',
 'PoliInfo2-EntityLinking-JA-Formal-Training.tsv']

In [3]:
df_train = pd.read_table(os.path.join(data_dir,'PoliInfo2-EntityLinking-JA-Formal-Training.tsv'))


In [4]:
df_train["exists_in_wiki"] = (df_train["wikipediaタイトル"].isnull()) + (df_train["wikipediaタイトル"]=="NIL")

/home/naraki/.pyenv/versions/3.6.5/lib/python3.6/site-packages/pandas/core/computation/expressions.py:178: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [5]:
df_train['形態素'] = df_train['形態素'].replace('　',' ')


In [6]:
split_train_text = df_train['形態素'].values[1:]


In [12]:
from janome.tokenizer import Tokenizer

t = Tokenizer()

t.tokenize('走れ')[0].part_of_speech[:2]

'動詞'

In [17]:
# select_conditions = ['動詞', '形容詞', '名詞']
select_conditions = ['名詞']
train_word_list = []
tmp_sentence = []
for w in split_train_text:
    if w == "○":
#         train_sentence_list.append(' '.join(tmp_sentence))
        train_word_list.append(tmp_sentence)
        tmp_sentence = []
    elif w != " " and w != "─":
        if t.tokenize(str(w))[0].part_of_speech[:2] in select_conditions:
            tmp_sentence.append(str(w))

In [45]:
train_sentence_list = []
tmp_sentence = []
for w in split_train_text:
    if w == "○":
#         train_sentence_list.append(' '.join(tmp_sentence))
        train_sentence_list.append(tmp_sentence)
        tmp_sentence = []
    elif w != " " and w != "─":
        tmp_sentence.append(str(w))

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
model = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b', use_idf=True)
model.fit(train_sentence_list)
# print(model.vocabulary_)


TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b')

In [44]:
tfidfs = model.fit_transform(train_sentence_list)
terms = model.get_feature_names()
tfidfs.toarray()[0].sum()


9.696798390150544

In [15]:
from itertools import combinations, dropwhile
from collections import Counter, OrderedDict

def bform2pair(bform_2l, min_cnt=5):
    pair_all = []
    
    for bform_l in bform_2l:
        pair_l = list(combinations(set(bform_l), 2))
        
        for i,pair in enumerate(pair_l):
            pair_l[i] = tuple(sorted(pair))
            
        pair_all += pair_l

    pair_count = Counter(pair_all)

    for key, count in dropwhile(lambda key_count: key_count[1] >= min_cnt, pair_count.most_common()):
        del pair_count[key]

    return pair_count

def pair2jaccard(pair_count, bform_2l, edge_th=0.4):
    # jaccard係数を計算

    # 単語ごとの出現章数
    word_count = Counter()
    for bform_l in bform_2l:
        word_count += Counter(set(bform_l))

    # 単語ペアごとのjaccard係数を計算
    jaccard_coef = []
    for pair, cnt in pair_count.items():
        jaccard_coef.append(cnt / (word_count[pair[0]] + word_count[pair[1]] - cnt))

    # jaccard係数がedge_th未満の単語ペアを除外
    jaccard_dict = OrderedDict()
    for (pair, cnt), coef in zip(pair_count.items(), jaccard_coef):
        if coef >= edge_th:
            jaccard_dict[pair] = coef
            print(pair, cnt, coef, word_count[pair[0]], word_count[pair[1]], sep='\t')

    return jaccard_dict


In [19]:
pair_count = bform2pair(train_word_list, min_cnt=5)
jaccard_dict = pair2jaccard(pair_count, train_word_list, edge_th=0.4)


('中川', '真')	8	0.5333333333333333	10	13
('義偉', '菅')	9	0.6	9	15
('清水', '貴之')	33	0.868421052631579	35	36
('神本', '美恵子')	5	1.0	5	5
('わか子', '矢田')	8	1.0	8	8
('堀江', '裕')	6	1.0	6	6
('奨', '難波')	32	0.9696969696969697	32	33
('二十', '平成')	61	0.5083333333333333	116	65
('大門', '実紀史')	21	0.875	24	21
('官房', '長官')	16	0.5333333333333333	24	22
('三', '年')	88	0.4	140	168
('和田', '政宗')	8	0.8888888888888888	9	8
('参考', '政府')	120	0.48	155	215
('人', '参考')	141	0.5444015444015444	245	155
('智子', '田村')	6	0.5454545454545454	9	8
('博之', '細田')	11	0.7333333333333333	11	15
('人', '政府')	143	0.45110410094637227	245	215
('国務', '大臣')	67	0.44370860927152317	67	151
('康人', '瓦林')	9	1.0	9	9
('異動', '選任')	7	1.0	7	7
('辞任', '選任')	7	1.0	7	7
('補欠', '選任')	7	1.0	7	7
('異動', '辞任')	7	1.0	7	7
('異動', '補欠')	7	1.0	7	7
('議員', '院')	128	0.6597938144329897	154	168
('衆議', '院')	152	0.9047619047619048	152	168
('仁君', '小沢')	25	0.8064516129032258	26	30
('里見', '隆治')	15	0.8823529411764706	17	15
('稔', '西村')	23	0.6216216216216216	25	35
('康', '稔')	23	0.76666666666

In [57]:
sorted(c.items(), key=lambda x:x[1])

[(('とおり', '人'), 30),
 (('君', '発議'), 30),
 (('の', '太郎'), 30),
 (('の', '発議'), 30),
 (('会', '八'), 30),
 (('会', '推進'), 30),
 (('会', '会議'), 30),
 (('は', '官'), 30),
 (('地方', '年'), 30),
 (('政府', '金'), 30),
 (('に', '発議'), 30),
 (('日', '者'), 30),
 (('。', '小沢'), 30),
 (('に', '要求'), 30),
 (('の', '要求'), 30),
 (('の', '区域'), 30),
 (('を', '貴之'), 30),
 (('二', '及び'), 30),
 (('ただいま', 'まし'), 30),
 (('し', '清水'), 30),
 (('を', '清水'), 30),
 (('二', '報告'), 30),
 (('報告', '委員'), 30),
 (('と', '官'), 30),
 (('いたし', '施設'), 30),
 (('いたし', '審議'), 30),
 (('ため', 'とおり'), 30),
 (('委員', '施設'), 30),
 (('案', '長'), 30),
 (('六', '案'), 30),
 (('求め', '者'), 30),
 (('か', '外'), 30),
 (('と', '決定'), 30),
 (('、', '決定'), 30),
 (('。', '決定'), 30),
 (('ます', '決定'), 30),
 (('行い', '）'), 30),
 (('議題', '）'), 30),
 (('行い', '（'), 30),
 (('議題', '（'), 30),
 (('委員', '質疑'), 30),
 (('あるいは', 'せ'), 30),
 (('ぐらい', 'それ'), 30),
 (('たい', '関し'), 30),
 (('さ', '一般'), 30),
 (('れ', '一般'), 30),
 (('から', '最初'), 30),
 (('ます', '最初'), 30),
 (('も', '最初'), 30),
 (('か'

In [ ]:
jaccard_dict

In [7]:
train_text = ''.join([w if w is str else str(w) for w in split_train_text])


In [8]:
train_text.count('法')

1205

In [11]:
df_train['IOB2'].value_counts()

I    1478
B     265
Name: IOB2, dtype: int64

In [12]:
df_train['形態素'].value_counts()['法']

357

In [13]:
df_train['形態素'].value_counts()['法案']

217

In [14]:
df_train['形態素'].value_counts()['法律']

196

In [15]:
df_train['形態素'].value_counts()['法務']

73

In [16]:
train_mentions = df_train[~df_train['メンション'].isnull()]['メンション'].values


In [17]:
train_mention_names = list(df_train[df_train['IOB2']=='B']['メンション'].value_counts().keys())
print('train mention name: ', len(train_mention_names), '種類')
df_train[df_train['IOB2']=='B']['メンション'].value_counts()

train mention name:  101 種類


円滑化法                           18
働き方改革を推進するための関係法律の整備に関する法律案    14
ＩＲ法案                           11
特定複合観光施設区域の整備の推進に関する法律案        10
経営承継円滑化法                        9
                               ..
犯罪収益移転防止法                       1
ＩＲ推進法                           1
地球温暖化対策推進法                      1
過労死等防止対策推進法                     1
食品衛生法                           1
Name: メンション, Length: 101, dtype: int64

In [18]:
train_entity_names = list(df_train[df_train['IOB2']=='B']['wikipediaタイトル'].value_counts().keys())
print('train mention name: ', len(train_entity_names), '種類')
df_train[df_train['IOB2']=='B']['wikipediaタイトル'].value_counts()

train mention name:  46 種類


NIL                                        67
特定複合観光施設区域の整備の推進に関する法律                     60
中小企業者等に対する金融の円滑化を図るための臨時措置に関する法律           26
働き方改革関連法                                   19
中小企業における経営の承継の円滑化に関する法律                    11
平和安全法制                                      9
政治資金規正法                                     6
国等による環境物品等の調達の推進等に関する法律                     5
労働基準法                                       5
風俗営業等の規制及び業務の適正化等に関する法律                     5
財政法                                         4
貸金業法                                        4
工場三法                                        3
総合保養地域整備法                                   3
労働契約法                                       2
中小企業信用保険法                                   2
特定秘密の保護に関する法律                               2
犯罪による収益の移転防止に関する法律                          2
関税暫定措置法                                     2
所得税法                                        2
少年法                                         1
大規模小売店舗立地法                        

In [19]:
train_mention_names

['円滑化法',
 '働き方改革を推進するための関係法律の整備に関する法律案',
 'ＩＲ法案',
 '特定複合観光施設区域の整備の推進に関する法律案',
 '経営承継円滑化法',
 'ＩＲ推進法案',
 '推進法案',
 'カジノ法案',
 '政治資金規正法',
 '一般職の職員の給与に関する法律等の一部を改正する法律案',
 '労働安全衛生法の一部を改正する法律案',
 '環太平洋パートナーシップ協定の締結に伴う関係法律の整備に関する法律の一部を改正する法律案',
 'グリーン購入法',
 '金融円滑化法',
 '戦争法',
 '特別職の職員の給与に関する法律の一部を改正する法律案',
 '推進法',
 'クリーンウッド法',
 '財政法',
 '中小企業金融円滑化法',
 '貸金業法',
 '労働基準法',
 'ＴＰＰ整備法',
 'ＩＲ法',
 'リゾート法',
 '働き方改革関連法案',
 '風営法',
 '防衛省の職員の給与等に関する法律の一部を改正する法律案',
 '中小企業における経営の承継の円滑化に関する法律等の一部を改正する法律案',
 '国会議員の秘書の給与等に関する法律の一部を改正する法律案',
 '小規模企業振興基本法',
 '裁判官の報酬等に関する法律の一部を改正する法律案',
 '過労死促進法',
 '特定秘密保護法',
 '基本法',
 '中小企業信用保険法',
 '所得税法',
 '検察官の俸給等に関する法律の一部を改正する法律案',
 '平和安全法制',
 '関税暫定措置法',
 '機構法',
 '地方交付税法の一部を改正する法律案',
 '土壌汚染対策法',
 '環太平洋パートナーシップ協定の締結に伴う関係法律の整備に関する法律',
 'パワハラ規制法案',
 '承継法',
 '中小企業基本法',
 '一般職の職員の給与に関する法律',
 '合法伐採木材等の流通及び利用の促進に関する法律',
 '土壌汚染防止法',
 '犯収法',
 '立地法',
 '公務員給与法',
 '労働者派遣法',
 'ＩＲの推進法',
 '改正労働契約法',
 '容器包装リサイクル法',
 '風適法',
 'ＩＲ、カジノ解禁法案',
 '風俗適正化法',
 '労基法',
 '競馬法',
 '競輪法',
 '事業承継円滑化法',
 '工場三法',
 'カ

In [20]:
train_text

'平成二十八年十二月十三日（火曜日）   午前十時開会    ─────────────   委員の異動 十二月十二日    辞任         補欠選任     青山 繁晴君     佐藤  啓君     進藤金日子君     山東 昭子君     神本美恵子君     森本 真治君     田村 智子君     大門実紀史君     浅田  均君     清水 貴之君 十二月十三日    辞任         補欠選任     小野田紀美君     こやり隆史君     森本 真治君     神本美恵子君     大門実紀史君     田村 智子君    ─────────────  出席者は左のとおり。    委員長         難波 奨二君    理 事                上月 良祐君                高野光二郎君                相原久美子君                西田 実仁君    委 員                足立 敏之君                有村 治子君                江島  潔君                岡田  広君                こやり隆史君                佐藤  啓君                山東 昭子君                豊田 俊郎君                和田 政宗君                神本美恵子君                森本 真治君                矢田わか子君                里見 隆治君                田村 智子君                大門実紀史君                清水 貴之君                山本 太郎君   衆議院議員       発議者      細田 博之君       発議者      岩屋  毅君       発議者      西村 康稔君       発議者      小沢 鋭仁君       発議者      松浪 健太君   国務大臣       国務大臣       （内閣官房長官） 菅  義偉君   事務局側       常任委員会専門       員      

In [99]:
df_train[df_train["IOB2"]=='B']["exists_in_wiki"].sum()/len(df_train[df_train["IOB2"]=='B'])

0.2528301886792453

In [21]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [22]:
printmd("**bold text**")

**bold text**

In [23]:
word_tags = df_train['IOB2'].values[1:]
split_train_text = df_train['形態素'].values[1:]


In [75]:
train_md_words = []
for idx, word in enumerate(split_train_text):
    tag = df_train['IOB2'][idx+1]
    if tag=="B" or tag=="I":
        train_md_words.append("**"+str(word)+"**")
        if idx+2 < len(df_train) and df_train['IOB2'][idx+2]!="I":
            train_md_words.append("( `" + str(df_train['wikipediaタイトル'][idx+1])+"` )")
    else:
        if word=="○":
            word = "<br/> ○"
        train_md_words.append(str(word))


In [34]:
train_md_words = ["**"+str(word)+"**" if (tag=="B" or tag=="I") else str(word) for tag, word in zip(word_tags, split_train_text)]


In [76]:
printmd(' '.join(train_md_words[:100000]))

平成 二十 八 年 十 二 月 十 三 日 （ 火曜 日 ）       午前 十 時 開会         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       委員 の 異動   十 二 月 十 二 日         辞任                   補欠 選任           青山   繁晴 君           佐藤     啓 君           進藤 金 日子 君           山東   昭子 君           神本 美恵子 君           森本   真治 君           田村   智子 君           大門 実紀史 君           浅田     均 君           清水   貴之 君   十 二 月 十 三 日         辞任                   補欠 選任           小野田 紀美 君           こ やり 隆史 君           森本   真治 君           神本 美恵子 君           大門 実紀史 君           田村   智子 君         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─     出席 者 は 左 の とおり 。         委員 長                   難波   奨 二 君         理   事                                 上月   良祐 君                                 高野 光二郎 君                                 相原 久美子 君                                 西田   実仁 君         委   員                                 足立   敏之 君                                 有村   治子 君                                 江島     潔 君                                 岡田     広 君                                 こ やり 隆史 君                                 佐藤     啓 君                                 山東   昭子 君                                 豊田   俊郎 君                                 和田   政宗 君                                 神本 美恵子 君                                 森本   真治 君                                 矢田 わか子 君                                 里見   隆治 君                                 田村   智子 君                                 大門 実紀史 君                                 清水   貴之 君                                 山本   太郎 君       衆議 院 議員               発議 者             細田   博之 君               発議 者             岩屋     毅 君               発議 者             西村   康 稔 君               発議 者             小沢   鋭 仁君               発議 者             松浪   健太 君       国務 大臣               国務 大臣               （ 内閣 官房 長官 ）   菅     義偉 君       事務 局側               常任 委員 会 専門               員                 藤田   昌三 君       政府 参考 人               内閣 官房 内閣 審               議 官               中川     真 君               内閣 府 地方 創生               推進 事務 局 審議               官                 藤原     豊 君               警察 庁 生活 安全               局 長               種谷   良二 君               法務 大臣 官房 審               議 官               加藤   俊治 君               文部 科学 大臣 官               房 審議 官           神山     修 君               厚生 労働 省 社会               ・ 援護 局 障害 保               健 福祉 部 長         堀江     裕 君               観光 庁 審議 官       瓦林   康人 君         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─     本日 の 会議 に 付し た 案件 <br/> ○ 政府 参考 人 の 出席 要求 に 関する 件 <br/> ○ **特定** **複合** **観光** **施設** **区域** **の** **整備** **の** **推進** **に** **関する** **法** **律** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) （ 衆議 院 提出 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 委員 長 （ 難波 奨 二 君 ）   ただいま から 内閣 委員 会 を 開会 いたし ます 。   委員 の 異動 に つい て 御 報告 いたし ます 。   本日 まで に 、 神本 美恵子 さん 、 浅田 均 君 、 青山 繁晴 君 、 進藤 金 日子 君 、 田村 智子 さん 及び 小野田 紀美 さん が 委員 を 辞任 さ れ 、 その 補欠 と し て 森本 真治 君 、 清水 貴之 君 、 佐藤 啓 君 、 山東 昭子 さん 、 大門 実紀史 君 及び こ やり 隆史 君 が 選任 さ れ まし た 。         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 委員 長 （ 難波 奨 二 君 ）   政府 参考 人 の 出席 要求 に 関する 件 に つい て お 諮り いたし ます 。   **特定** **複合** **観光** **施設** **区域** **の** **整備** **の** **推進** **に** **関する** **法律** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) の 審査 の ため 、 本日 の 委員 会 に 、 理事 会 協議 の とおり 、 政府 参考 人 と し て 内閣 官房 内閣 審議 官 中川 真 君 外 六 名 の 出席 を 求め 、 その 説明 を 聴取 する こと に 御 異議 ござい ませ ん か 。       〔 「 異議 なし 」 と 呼ぶ 者 あり 〕 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   御 異議 ない と 認め 、 さよう 決定 いたし ます 。         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 委員 長 （ 難波 奨 二 君 ）   **特定** **複合** **観光** **施設** **区域** **の** **整備** **の** **推進** **に** **関する** **法律** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) を 議題 と し 、 質疑 を 行い ます 。   質疑 の ある 方 は 順次 御 発言 願い ます 。 <br/> ○ 上月 良祐 君   自由 民主 党 の 上月 良祐 で ござい ます 。   これ まで かなり 入念 な 議論 が なさ れ て き た と 思い ます が 、 まだ 何 点 か 確認 し たい こと が あり ます の で 、 あるいは 再 確認 を し て おき たい こと も あり ます の で 、 まず 自民 党 の 中 で 私 が 最初 に 質問 を さ せ て いただき たい と いう ふう に 思い ます 。   最初 に 、 ＩＲ の 基本 的 な 考え 方 に 関し まし て な ん です が 、 国際 会議 が かなり 日本 が 劣勢 に なっ て いる ん で は ない か と いう お 話 を 聞き ます 。 今 どんな 現状 に なっ て いる の か 、 それ から 、 どれ ぐらい の 規模 の 会議 場 と いう の が 国際 的 に 見れ ば 一般 的 と いう の か 、 これ ぐらい ない と 駄目 だ よ ね と いう の に 対し て 日本 が どんな 現状 な の か 、 そこ を 教え て いただき たい と 思い ます 。 <br/> ○ 政府 参考 人 （ 瓦林 康人 君 ）   お 答え 申し上げ ます 。   我が 国 の 国際 会議 の 開催 件数 で ござい ます 。 非 営利 の 国際 団体 で ござい ます 国際 会議 協会 の 統計 に より ます と 、 我が 国 の 国際 会議 開催 件数 は 、 世界 全体 で 見 ます と 二 〇 一 三 年 以降 七 位 、 アジア 太平 洋 地域 で は 二 〇 一 二 年 以降 一 位 を 堅持 し て おり ます 。 ただし 、 アジア 大洋 州 の 競合 国 も 近年 開催 件数 を 伸ばし て おり ます 。 我が 国 の ほか 、 中国 、 韓国 、 シンガポール 、 豪州 、 これ ら 主要 五 か国 の 国際 会議 開催 件数 の 中 で の 我が 国 が 占める シェア 、 これ は 一 九 九 一 年 時点 で 五 一 ％ でし た が 、 以後 低下 を 続け て 、 二 〇 一 五 年 に は 二 六 ％ に 低下 し て おり ます 。   国際 会議 場 に つき まし て で ござい ます 。 これ に つき まし て は 、 他 の アジア 諸国 の 主要 施設 を 見 ます と 、 シンガポール で 収容 人数 一 万 人 を 超える 会議 場 が 二 か所 整備 さ れ て いる ほか 、 香港 で は 八千 人 規模 、 また 韓国 で は 七千 人 規模 の 大 規模 な 会議 場 が 整備 さ れ て おり ます 。 我が 国 の 国際 会議 場 に つき まし て は 、 収容 人数 約 五千 人 の 会議 場 、 これ が 二 か所 ござい ます が 、 これ が 最大 規模 と なっ て ござい ます 。   以上 で ござい ます 。 <br/> ○ 上月 良祐 君   ありがとう ござい ます 。   僕 も 資料 を ちょっと もらっ たり し まし て 、 より 詳しく 聞か せ て いただき まし た けれど も 、 かなり 劣勢 に なっ て き て いる 傾向 が 顕著 に なっ て いる ん だ と 思い ます 。 そう いう 意味 で は 、 会議 場 自体 も 、 ハード も 足り ない し 、 日本 で 会議 を やっ て くれ と いう ＰＲ も 足り ない の か も しれ ませ ん 。 いろんな こと が ある ん だ と 思い ます が 、 やはり 会議 に 来 た 人 の 会議 以外 の 、 まあ いろんな 余暇 活動 と いう ん でしょう か 、 そう いう の も 含め て 全体 を やっぱり 整え て いか なきゃ いけ ない と いう の は 今 の 状況 を 見 たら もう 明らか だ と いう ふう に 思い ます 。 これ 、 急激 に シェア も 落ち て い ます けれど も 、 これ から この 傾向 が 更に 続い て いけ ば かなり 危機 的 な 状況 に なっ て いく と 思い ます の で 、 こう いっ た こと は よく 頭 に 置い て 対応 し て いか なきゃ いけ ない と 思っ て おり ます 。   ただ 、 やっぱり 会議 も 、 ただ 大きな 会議 場 が あれ ば いい と いう の で は なく て 、 やはり その 会議 で やる よう な 学術 的 な こと に つい て の 日本 が 先進 的 な ところ で ある か どう か 、 研究 が 進ん で いる か どう か 、 やっぱり 日本 で やっ た 方 が いい よ ね と いう よう な コンテンツ が 重要 だ と 思い ます し 、 やはり 会議 の 開催 だけ じゃ なく て 、 観光 で 来る 方 と いう の で あれ ば 、 日本 の 文化 あるいは スポーツ も 含め た 観光 コンテンツ の 充実 と いう の も 大変 重要 だ と 思い ます 。   ＩＲ の 中 に カジノ が ある の と 一緒 で 、 日本 観光 全体 の 中 に ＩＲ が ある ん だ と 思い ます の で 、 そう いう 意味 で は ＩＲ と ほか の ところ の 連携 と いう の が もう 極めて 重要 だ と 思っ て おり まし て 、 その 点 に つい て 、 改めて もう 一 回 提案 者 の 御 意見 を お 伺い し たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 細田 博之 君 ）   上月 議員 が 御 質問 さ れ た とおり で ござい ます 。 我が 国 は 、 有名 な 観光 地 で ゴールデン ルート など と 言わ れ て いる 京都 と 奈良 だけ が 観光 地 で は ない 、 我々 の 出身 地 の 四十 七 都 道 府 県 に どこ も いい ところ が あり 、 歴史 が あり 、 すばらしい 景観 も ある わけ で ござい ます 。   したがっ て 、 もちろん ＩＲ と いう の は 、 今 の 国際 会議 場 に 造っ て おら れ た 、 ビッグ サイト と か メッセ と か パシフィコ と か いろんな 名称 が あり ます けど 、 それ 、 ただ 来 て 、 ただ 開催 し て 、 ただ 帰る ん じゃ ほとんど 楽しく ない よう な こと が あっ て 魅力 が ない 、 ここ を 充実 さ せ なきゃ なら ない と いう こと は 第 一 の 目的 で ござい ます が 、 やはり 第 二 は 、 ある ところ へ 来 た と 、 この ＩＲ に 、 そして 日本 の どこ か 観光 し たい と いう とき に 、 観光 の いろいろ な 都 道 府 県 や 会社 の 人 たち が そこ に 常 に い て 、 今 は ちょうど 梅 が 咲い て いる から 水戸 の 偕楽 園 に 行く の は 物すごく いい ルート です よ と いっ て そこ に 案内 し たり 、 今 は 桜 が どこ が いい から どこ へ 行っ たら どう です か 、 あるいは 古い もの は ここ に いい もの が あり ます よ と 、 そう いう ふう な 全 日本 の 観光 の 言わば メッカ と し て まず 設置 する と 、 そう いう 目的 も 当然 入ら なけれ ば なら ない 。   シンガポール と か 小さい ところ は そこ へ 行け ば もう ほか へ 行き よう が あり ませ ん から 、 日本 は どこ で も そこ を 拠点 と し て いい ところ が たくさん ある わけ で ござい ます から 、 そう いっ た 観光 の 拠点 に する と いう こと は いい 考え 方 で あり 、 我々 も そう いう こと を 実現 し て いき たい と 思っ て おり ます 。 <br/> ○ 上月 良祐 君   私 も 、 最近 は 行け て い ませ ん が 、 海外 旅行 は 大変 好き だっ た の で 夫婦 で よく 行っ て おり ます 。 帰っ て くる と 、 何 と いう ん でしょう か 、 ちょっと ほっと し た と いう ん でしょう か 、 ちょっと 疲れ も 出 たり 、 最後 成田 空港 など へ 帰っ て くる と 、 また 来週 すぐ 行き たい と まで は ちょっと 旅行 で は 、 ビジネス は 違う でしょう けど 、 思え ない ぐらい 、 まあ 緊張 も し て いる ん でしょう 、 疲れ も 出る と いう の は あり ます が 、 実 は ラスベガス だけ は 来週 また 行き たい と 思っ た ん です ね 。 それ は 嫁 さん も 一緒 でし た 。   それ は 、 別 に カジノ を し たかっ た わけ じゃ ない ん です よ 、 これ は 。 やっぱり 町 が 本当 に 魅力 的 だっ た ん です ね 。 大変 すばらしい ショー が あっ たり 、 Ｋ — 1 の 予選 なんか も やっ て い まし た です ね 、 私 も あそこ で 見 て 感動 し まし た けど 、 いろんな テーマ パーク が あっ たり 、 ショッピング も 大変 充実 し て い まし た 。 すごい 楽しかっ た です 。 何 より 一番 楽しかっ た の は やっぱり レストラン だっ た ね と いう 話 を 昨日 の 晩 も 嫁 さん と し まし た ん です 。 やっぱり 町 が 大変 魅力 的 だっ た と いう の が ある 。 それ が 重要 だ と 思う ん です ね 。   大門 先生 だっ た か 、 シンガポール は ちっちゃな 国 だ から ＩＲ が 必要 だ けど 、 日本 は ほか に も 魅力 ある じゃ ない か と 。 私 は 、 で も 、 その 魅力 こそ が 魅力 な ん で あっ て 、 そこ と の 連携 を どう 図る か と いう こと なし に 日本 型 の ＩＲ と いう の は 絶対 に 成果 が 出 ない と 思っ て おり ます の で 、 くれぐれ も そこ は 、 観光 庁 の 人 も 来 られ て い ます から 、 くれぐれ も よろしく お 願い し たい と 思い ます 。   それ で 、 私 、 実 は この 質問 し た の は 、 ちょっと 心配 し て いる ん です よ 。 と いう の は 、 日本 は 大抵 どこ の 地域 も 、 ほか の 地域 見 て い まし て 、 まね する こと 多い ん です ね 。 それ で 、 どこ か が 最初 、 先頭 バッター で 一生 懸命 やっ て うまく いっ た 、 トップ バッター で うまく いっ た と 、 そう する と 次々 また 出 て くる 可能 性 が あっ て 、 これ は アトランティック シティー の 例 を 見 て も 、 余り たくさん あっ て いい もの で も ない と 思う ん です よ 、 お 互い 共倒れ し ちゃう 可能 性 が ある から 。   日本 の 中 で おのずと やっぱり 数 に 制限 が ある ん だ と 思う ん です よ 。 日本 に 百 個 も 二百 個 も あっ て いい わけ が ない の で 、 やっぱり 厳密 に 数 の 制限 は 加える べき だ と いう ふう に 思っ て おり まし て 、 これ 衆 段階 で の 附帯 で も 言わ れ て おり ます けれど も 、 とにかく 厳格 に 少数 に 限っ て もらい たい 。 次々 造っ て いっ て 共倒れ に なる よう な こと を 、 ちょっと 前 に 走っ た ところ が うまく いっ た から 、 ここ も まね し たい 、 ここ も まね し たい と いう こと に なら ない よう な 厳格 な 運営 を し て もらい たい と 思っ て いる ん です が 、 この 数 の 制限 も 含め て どんな ふう に お 考え か を お 尋ね し ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   大事 な 御 視点 だ と 思い ます 。 この 法案 で 、 目的 に 、 観光 振興 、 地域 の 経済 振興 、 それ から 財政 に も 資する と 、 財政 改善 に も 寄与 する と いう こと で あり ます の で 、 どこ へ かしこ に も 造る と いう こと で は なく て 、 ある 程度 一定 の 規模 以上 の もの で あっ て 、 まさに 日本 経済 全体 に 資する よう な 、 そう いっ た こと が 期待 できる もの 、 その 中 で は 地域 の 特性 を 生かし た もの と いう こと に なっ て くる と 思い ます 。   一方 で 、 依存 症 対策 など 、 そう いっ た 負 の 側面 に も 配慮 し なけれ ば いけ ませ ん の で 、 そう いっ た こと を 全体 的 に 考慮 し ます と 、 私 は 十 も 二十 も 日本 全国 に 造る と いう こと で は ない と 思い ます し 、 温泉 旅館 の 横 に ちょこっと 何 か カジノ を 一部 置く よう な 、 そんな こと も 想定 を し て い ない わけ で あり ます 。 私 ども 、 最初 の 段階 で は せいぜい 二 か所 、 三 か所 、 こう いっ た ところ で 、 これ は もちろん 地方 で 議会 の 同意 を 得 て 手 を 挙げ て くる と いう 方式 を 考え て おり ます が 、 そう いっ た 中 で 限定 的 に 認定 を し て 、 それ で その 様子 も 見 ながら 、 効果 で ある と か 成果 で ある と か あるいは 課題 、 こう いっ た もの も 検証 し ながら 、 段階 的 に 増やし て いく の が 適切 だ と いう ふう に 考え て おり ます 。   あわせ て 、 総数 に つい て も 、 そんな 十 も 二十 も 想定 は し て おり ませ ん の で 、 衆議 院 内閣 委員 会 に おけ る 附帯 決議 に おき まし て も 、 その 数 に つい て は 「 厳格 に 少数 に 限る こと と し 、 区域 認定 数 の 上限 を 法定 する こと 。 」 と いう ふう に さ れ て おり ます の で 、 こう いっ た 点 も 含め て 、 政府 が 一 年 以内 を 目途 に 提出 を する 実施 法 案 検討 の 際 に は 適切 に 判断 を する と いう ふう に 考え て おり ます 。 <br/> ○ 上月 良祐 君   ありがとう ござい まし た 。   ある 種 の **リゾート** **法** ( `総合保養地域整備法` ) の 反省 と いう ん でしょう か 、 そう いっ た こと も 踏まえ まし て 、 せっかく 造る ん だっ たら 、 しっかり その 辺 の 制限 も 掛け て 、 いい もの を ちゃんと 造っ て いただき たい と 思っ て おり ます 。 そして 、 その 上 で 連携 も しっかり 図っ て いただき たい と 思い ます の で 、 よろしく お 願い し ます 。   法務 省 の 方 、 来 て いらっしゃる と 思い ます 。 ちょっと 順番 が 違い ます が 、 法務 省 が 十 二 月 七 日 に 出さ れ た ペーパー が あり まし て 、 前回 の 審議 の 中 で 大門 先生 から も 質疑 が あっ た ん です ね 。 で 、 書き 方 が やや 分かり にくい の で 御 質問 が 。 答弁 を さ れ て いる 、 私 は ちゃんと 答弁 さ れ て いる と 思い ます けれど も 、 昨日 の 参考 人 で も 御 意見 が 分かれ て い た ん です よ 、 これ を どう 読む か で 。   改めて そこ を ちょっと 確認 さ せ て もらい たい ん です が 、 この 十 二 月 七 日 ペーパー の 中 で 、 目的 の 公益 性 （ 収益 の 使途 を 公益 性 の ある もの に 限る こと を 含む 。 ） と いう の と 、 運営 主体 等 の 性格 （ 官 又 は それ に 準ずる 団体 に 限る など ） の ところ の その 意味 を 一 例 だ と 、 昨日 の 参考 人 の 先生 は お 一人 、 渡邉 先生 か な 、 書い て あっ たり し た ん です が 、 もう 一 回 ちょっと 確認 さ せ て ください 。 <br/> ○ 政府 参考 人 （ 加藤 俊治 君 ）   お 答え を 申し上げ ます 。   まず 、 提出 し た ペーパー の 書き 方 に つい て 、 分かり にくい 点 が あっ た 点 に つい て は お わび を 申し上げ ます 。   改めて 御 説明 申し上げ ます と 、 御 指摘 の 資料 は 、 刑法 が 賭博 を 犯罪 と し て いる 趣旨 や 、 特別 法 に おい て 賭博 に 当たる 行為 を 許容 する 場合 に 刑法 と の 整合 性 を 害する こと と なら ない よう 考慮 さ れる べき 事項 等 に つい て 、 刑法 を 所管 する 法務 省 の 立場 から お 示し を し た もの で ござい ます 。   その 資料 に も 記載 し て あり ます と おり 、 既存 の 公営 競技 等 に 係る 特別 法 の 立案 に 当たっ て は 、 基本 法 で ある 刑法 が 賭博 を 犯罪 と 規定 し て いる 趣旨 を 没却 し ない よう な 制度 上 の 配慮 が さ れ て いる もの と 認識 し て おり 、 法務 省 と いたし まし て も 、 例えば 目的 の 公益 性 で あり ます と か 運営 主体 等 の 性格 、 それ 以下 記載 し て おり ます 諸 要素 等 に 着目 し て これ まで 意見 を 述べ て き た ところ で ござい ます 。 すなわち 、 これ ら の 諸 要素 は 、 刑法 が 賭博 を 犯罪 と 規定 し た 趣旨 と 整合 し て いる もの で ある か どう か を 判断 する 上 で の 考慮 要素 で ある と 考え て おり ます 。   その 上 で 、 その 考慮 要素 に 付さ れ た 括弧 書き に 記載 し て ある 内容 で ござい ます が 、 これ は 各 考慮 要素 に 関し 、 既存 の 公営 競技 等 に おい て 実際 に どの よう な 制度 や 規制 と し て 反映 、 具体 化 さ れ て いる か と いう 観点 から 例 を 挙げ た もの で ござい ます 。 例えば 、 目的 の 公益 性 の 次 に 括弧 書き で 「 （ 収益 の 使途 を 公益 性 の ある もの に 限る こと も 含む 。 ） 」 と あり ます の は 、 目的 の 公益 性 を 担保 する 制度 の 一 例 と し て 収益 の 使途 を 公共 性 の ある もの に 限る こと を 挙げ た もの で ござい ます 。 また 、 運営 主体 等 の 性格 の 次 に 括弧 書き で 「 （ 官 又 は それ に 準じる 団体 に 限る など ） 」 と あり ます の も 、 既存 の 公営 競技 等 に おい て 現に 運営 主体 等 が 官 又 は それ に 準じる 団体 に 限ら れ て いる こと を 踏まえ て 、 運営 主体 等 に 関する 制度 の 例 と し て 官 又 は それ に 準ずる 団体 に 限る こと を 挙げ た もの で ござい ます 。   ただいま 申し上げ た よう な 意味 で 、 御 指摘 の 資料 の 括弧 書き は 各 考慮 要素 の 例示 で ござい まし て 、 括弧 書き に ある もの しか 許容 さ れ ない と いう 意味 で 記載 し て いる もの で は ござい ませ ん 。 <br/> ○ 上月 良祐 君   改めて よく 分かり まし た 。   大変 重要 な 点 だ と 思う ん です けど 、 もう ちょっと 分かり やすく 紙 に 書か ない と 、 そう いう ふう に 誤解 さ れる 可能 性 ある と 思う ん です よ 。 これ は 、 法律 の 文言 読ん で いる 人 だ と 、 ああ 、 なるほど 、 そう いう ふう に 読む ん だ な と いう の も 何 と なく 分かる ん です けど 、 やっぱり これ 、 法務 省 が 出し たら 、 読む の は 一般 の 人 も 読ん だり する し 、 もう 少し 、 何 と いう ん です か 、 誤解 さ れ ない よう な 書き 方 を し ない と いけ ない と 思い ます の で 、 くれぐれ も そこ は 気 を 付け て いただき たい と 思い ます 。   その こと は 、 内容 は 分かり まし た の で 、 法務 省 の 方 は もう これ で 結構 です から 、 御 退席 いただい て 結構 です 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   加藤 審議 官 は 御 退席 いただい て 結構 で ござい ます 。 <br/> ○ 上月 良祐 君   済み ませ ん 、 それ で は 続き まし て 、 依存 症 対策 に つい て お 聞き し たい と 思い ます 。   これ まで かなり 時間 を 積み重ね て 議論 を する 中 で 、 依存 症 に つい て は 、 この カジノ だけ で は なく て 既存 の ギャンブル や 遊技 に 関し て も 含め て 総合 的 な 対策 を 取っ て いか なけれ ば いけ ない と いう こと が よく 分かっ て き た と いう ふう に 思い ます 。 せっかく 内閣 委員 会 で かみ合っ た と いい ます か 、 しっかり 充実 し た 審議 を し て いる の で 、 この こと は 絶対 に 施策 に 生かし て いっ て いただき たい と 思っ て いる ん です ね 。   それ で 、 山本 委員 が 御 質問 さ れ た 中 の ビッグ イシュー 基金 の 本 も 、 カジノ 化 し て いる 日本 も 、 私 も 改めて 読ま せ て もらい まし た けど 、 大変 示唆 に 富む 本 だ と 思い ます 。 こう いっ た もの が 、 審議 の 中 で 出 て き た もの は しっかり 生かし て これ から の 対策 に 是非 つなげ て いっ て ほしい と 思う ん です が 、 まず 厚労 省 の 方 に お 聞き し ます が 、 五百 三十 六 万 人 と いう 数字 、 これ は 取りあえず 今 出し て いる もの で あっ て 、 今 いろいろ 調査 を さ れ て いらっしゃる と いう ふう に 聞い て おり ます けれど も 、 今 の 調査 の 状況 と いう ん でしょう か 、 対策 に つい て 今 いろいろ 研究 と か も さ れ て いらっしゃる よう です が 、 その 状況 を ちょっと 教え て ください 。 <br/> ○ 政府 参考 人 （ 堀江 裕 君 ）   お 答え 申し上げ ます 。   厚生 労働 省 に おい て 平成 二十 五 年度 に 行い まし た 調査 の 結果 、 依存 症 が 疑わ れる 方 が 成人 の 四 ・ 八 ％ と 推計 いたし まし た と いう こと だっ た わけ で ござい ます けれど も 、 これ 、 御 本人 に アンケート と いい ます か 、 御 自分 で 記入 し て いただく よう な 形式 の もの で あっ た 、 それ だけ を 集計 し た 形 に なっ て い ます と いう こと 、 それ から 、 あるいは 遊技 が いろいろ 混じっ て い て 、 そう し た もの で ある と 、 それ から サンプル 数 は 四千 人 程度 で あっ た と いう こと で あっ た わけ で ござい ます の で 、 その 辺 を 踏まえ まし て 、 平成 二十 八 年度 、 現在 で ござい ます けれど も 、 サンプル 数 は 四千 余り と いたし ます と し て 、 遊技 から ほか の 公営 競技 等 の もの が しっかり 分かれ て 把握 できる よう に 、 そして また 、 調査 員 なり 医師 なり が 御 回答 いただく 方 に 出向い て よく 面接 を し ながら 丁寧 に お 聞き する と いう よう な 形 で の 調査 を 今 進め て いる ところ で ござい ます 。   それ から 、 同じ 調査 の 違う 枠組み の 中 で な ん です けれど も 、 再三 衆議 院 の 方 で も 議論 あっ た わけ で ござい ます が 、 内観 療法 です と か 認知 行動 療法 です と か あるいは 集団 療法 です と いっ た よう な 治療 法 が あり まし て 、 それぞれ の 患者 さん に は それぞれ なり に 治っ て いただける よう に 聞い て いる わけ で ござい ます けれど も 、 例えば ほか の 病気 、 がん で あれ ば 放射 線 治療 です と か 切除 を する と か あるいは 薬 を 飲ん で いただく と 、 どう いう とき に は どう いう もの が 一番 適当 で ある か と いう よう な 意味 合い で の 療法 が まだ きちっと 確立 し て い ない と いう こと で ござい ます の で 、 そう し た もの に 向け まし て の 、 今 あり ます 先ほど の 療法 の それぞれ の 評価 と いう こと を 今 の 研究 の 中 で 進め させ て いただい て いる と 、 こう いう よう な こと で ござい ます 。 <br/> ○ 上月 良祐 君   しっかり 対策 は 、 まだ どう いう ふう な 症状 に は どう いう ふう な 治療 を すれ ば 結果 どう か と いう の が 確定 さ れ て い ない わけ です ね 。 これ は 果てし が ない の か も しれ ない けれど も 、 今 それ が 始まっ た と いう こと で 、 そこ も しっかり やっ て もらい たい と 思い ます し 、 人数 は 五百 三十 六 万 人 より も 増え よう が 減ろう が 、 ちゃんと 数字 は 出し て ほしい と 思う ん です よ 。 で ない と 、 これ から 対策 を し て 増え た か 減っ た か 評価 でき なく なり ます から 、 ベース が 分から ない と 。 この ベースライン を きちっと まず 固め て 、 それ も 今 年度 末 に 何 と か ある 程度 出し て こ られる の か どう か 分かり ませ ん が 、 それ が また 、 若干 こう いう ふう に 精査 に やっ たら 、 少し 動い たら 、 その こと も 含め て きちんと 出し て いただき たい 。 そして 、 その 説明 を し て いただき たい と 思い ます の で 、 そこ は よろしく お 願い し ます 。   あと 、 文科 省 で も 取組 が 今 進ん で いる と 聞い て おり まし て 、 そこ の ところ は ちょっと 宣伝 し て ください 、 ちゃんと やっ て いる と いう こと を 。 お 願い し ます 。 <br/> ○ 政府 参考 人 （ 神山 修 君 ）   お 答え いたし ます 。   子供 たち が 成長 し 大人 に なっ た 際 に 、 ギャンブル や 遊技 など に 依存 せ ず に 自律 的 かつ 健康 的 に 生き て いく こと は 大変 重要 で ある と 認識 し て ござい ます 。 この ため に 、 まず 学校 教育 に おき まし て 、 学習 指導 要領 など に 基づき まし て 、 保健 体育 、 道徳 、 家庭 科 と いっ た 教科 に おき まし て 、 児童 生徒 の 発達 の 段階 に 応じ まし て 、 欲求 や ストレス に 対する 自分 なり の 対処 法 や 望ましい 生活 習慣 など を 身 に 付け られる よう に 指導 し て いる ところ で ござい ます 。   また 、 児童 生徒 だけ で は なく 、 青少年 を 取り巻く 大人 に 対し まし て も 依存 症 の 危険 性 など を 周知 する こと が 大変 重要 で ある と 、 この よう な 認識 に 立ち まし て 、 平成 二十 八 年度 、 今 年度 から で ござい ます が 、 文部 科学 省 に おき まし て は 、 青少年 や その 保護 者 など を 対象 と し て 地域 に おけ る 依存 症 予防 対策 の 開催 、 こう いっ た よう な もの を 支援 いたし ます 依存 症 予防 教育 推進 事業 を 行っ て いる ところ で ござい ます 。 <br/> ○ 上月 良祐 君   聞き ます と 、 子供 たち だけ じゃ なく て その 親 御 さん も 含め て 対策 を やっ て 、 地域 に 近い ところ で やっ て いる と いう こと な ん です 。 これ も 大変 重要 な 取組 で 、 来年 以降 も また 続い て いく ん だ と 思い ます けれど も 、 そう いう ふう な 取組 を しっかり 、 始まっ て だんだん 深まっ て いく ん だ と 思い ます から 、 これ は やっ て もらい たい と 思い ます 。   それ で 、 提案 者 の 方 に これ で お 聞き し たい と 思う ん です が 、 これ 実態 把握 、 まず ちゃんと やら ない と 効果 が 検証 でき ませ ん から 、 それ は 役所 の 方 で きちんと やっ て いただける と いう こと な ん で 、 あと 、 これ 公営 ギャンブル と いう の は 、 江島 先生 から も お 話 あっ た 、 ばらばら な ん です ね 。 それぞれ の 役所 に ばらばら で やら せる と 、 例えば 〇 ・ 三 人 ずつ やる みたい な こと に なっ ちゃっ て 中途 半端 に なる の で 、 まとめ て きちっと し た 体制 や 仕組み で やる べき だ と 。 これ は やら れる と いう こと で 西村 先生 から お 話 、 提案 者 から 御 答弁 が あっ た ん です が 。   あと 、 公営 ギャンブル は お 互い が 競争 で も ある ん です ね 。 だ から 、 お 互い 頑張っ ちゃう から 、 より 売上げ も 考える と 少し ずつ 頑張ら ない と いけ ない 、 お 互い 競い合う と いう こと に なっ て 、 そう いう 競い合う 仕組み が ギャンブル 依存 症 を 招く よう な こと も ある か も しれ ない ん です ね 。 だ から 、 振興 と 規制 と いう ん でしょう か 、 振興 と 対応 する ところ を ある 意味 分ける こと に も 意味 が あり まし て 、 そう いう 意味 で は 、 各省 は その 辺 の ところ を きちっと 見 て 規制 と 振興 を やっ て いる 。 そこ は 分かれ ない に し て も 、 ギャンブル 依存 症 は そこ から 離れ た ところ で 、 各省 、 ある 意味 指導 し ながら ちゃんと 対応 する よう な 牽制 機能 を 持っ た 仕組み と いう ん でしょう か 、 総合 かつ そう いう ふう な 機能 を 持っ た 仕組み を しっかり 取る べき だ と 思っ て おり ます 。   それ から 、 今回 の カジノ は 、 ある 意味 で 振興 する の は 観光 庁 と か 内閣 官房 と か 内閣 府 と か で やら れる ん でしょう けど 、 カジノ 管理 委員 会 と いう こと で きちっと 規制 を する ところ が 、 牽制 機能 を 持つ ところ が できる わけ です から 、 そう いう 意味 で も これ まで と は やっぱり 形態 が 違う 。 私 も 公営 ギャンブル の 関係 やっ て い まし た けれど も 、 今 まで と は 違っ て 牽制 機能 が きちっと 働き やすい よう な 仕組み に なっ て いる と 思う の で 、 そう いう 形 で 、 しっかり 管理 する ところ は 管理 し て いただい て 、 振興 する ところ は 振興 し て いただい て 、 牽制 機能 を 果たし ながら やっ て いっ て いただき たい と 思う ん です 。   特に この 依存 症 に 対し て は これ まで ずっと 議論 し て き て 、 カジノ の 出発 を 待た ず に 、 この 法案 が 仮 に 成立 する に し て も 、 それ を 、 カジノ が 実際 に できる とき から と か と いう ん じゃ なく て 、 これ 一刻 も 早く 、 今 もう ずっと やり 始め て いる 取組 を 更に 進化 さ せ て いっ て もらい たい と 思う ん です けれど も 、 この 辺 の ところ の 御 対応 に つい て 是非 お 聞き し たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   大変 大事 な 御 指摘 を いただい た と 思っ て おり ます 。 今回 の この 法 案 の 審議 に おい て も この ギャンブル 依存 症 対策 と いう の は 大きな 論点 と なっ て おり ます し 、 私 ども も これ は しっかり 取り組ま なきゃ いけ ない な と いう こと を 改めて 痛感 を し て いる ところ で ござい ます が 。   政府 の 取組 、 いろいろ ござい まし た けれど も 、 まだまだ 不 十分 だ と いう ふう に 我々 も 認識 を いたし て おり ます の で 、 ギャンブル 等 依存 症 患者 へ の 対策 です ね 、 これ を 抜本 的 に 政府 に おい て は 強化 を し て もらい たい と 思っ て おり ます し 、 この 実態 把握 の ため の 体制 の 整備 、 そして その 原因 を しっかり と 把握 、 分析 を する こと が 大事 だ と いう ふう に 思っ て おり ます 。 あわせ て 、 患者 の 相談 体制 や 臨床 医療 体制 の 強化 、 これ も 必要 だ と いう ふう に 認識 を いたし て おり ます 。 あわせ て 、 文科 省 から あり まし た けれど も 、 教育 上 の 取組 、 これ も 是非 強化 を し て もらい たい と いう ふう に 思っ て おり ます 。   そして 、 御 指摘 の 、 カジノ に とどまら ず 、 他 の 公営 ギャンブル あるいは 遊技 等 に 起因 する 依存 症 も 含め て 、 この ギャンブル 等 依存 症 対策 に 関する 国 の 取組 を 抜本 的 に 強化 を す べき だ と いう ふう に 考え て おり ます 。 総合 的 、 包括 的 に 対処 する ため の 御 指摘 の あっ た 新た な 仕組み 、 体制 、 こう し た もの を 設け て もらい 、 関係 省庁 が 十分 連携 し て 取組 を 構築 し 、 強化 を す べき だ と いう ふう に 認識 を いたし て おり ます 。   あわせ て 、 来 年度 の 予算 要求 も いたし て いる よう で あり ます けれど も 、 是非 十分 な 予算 も 確保 し て もらい 、 御 指摘 の よう に 、 一 年 以内 を 目途 に 提出 さ れる 実施 法案 を 待た ず に 、 これ は 直ちに 政府 に おい て は 取組 を 強化 を し て もらい たい と 思い ます し 、 私 ども 議連 と し て も 、 これ も 直ちに 取組 を 強化 を し て いき たい と いう ふう に 考え て いる ところ で ござい ます 。 <br/> ○ 上月 良祐 君   大変 前向き な 答弁 を ありがとう ござい まし た 。 これ まで の 審議 が 本当 に 今 の 御 答弁 で すごく 意味 が ある と 思い ます 。 予算 の こと も これ から 聞こう と 思っ たら ちゃんと 言っ て くださっ た ん で 、 今年 は もう 予算 査定 進ん で いる と いう こと は ある か も しれ ませ ん けれど も 、 しっかり 予算 に つい て も 確保 し て やっ て いただき たい と 思い ます の で 、 お 願い いたし ます 。   あと 、 マネ ロン に つい て ちょっと お 聞き し ます 。   マネ ロン に つい て は 、 昨日 の 参考 人 で も 、 法律 で かなり きちっと やれる ん だ と いう こと を お 聞き し まし た 。 むしろ 依存 症 の 方 が 難しい と いう こと も 聞い た ん で 先 に 聞い た ん です が 、 念 の ため これ も 確認 さ せ て もらい たい ん です が 、 やっぱり カジノ の 事業 主体 の 廉潔 性 の 確保 の 措置 、 これ は 衆 の 附帯 に も あり まし た 、 厳格 な 入場 規制 、 これ も 重要 です 。   しかし 、 それ に 加え て さらに 、 やっぱり マネ ロン と いう の は 事業 者 が 何 ら か の 形 で 、 あるいは そこ で 働い て いる 人 な の か も しれ ませ ん が 、 関わら ない と 、 勝た せ て お 金 を あげる と いう こと は でき ませ ん から 、 やっぱり そう いう 意味 で は 事業 者 の 管理 と いう の は 運営 開始 後 、 そこ 大変 重要 だ と 思っ て おり まし て 、 監視 カメラ の 設置 で ある と か 従業 員 の 教育 と か 、 そう いっ た もの は 絶対 に 欠かせ ない こと だ と 思っ て おり ます の で 、 その こと は しっかり やっ て いただき たい 。 もちろん 、 これ は もう ＦＡ ＴＦ 対応 は もちろん の こと です よ 、 それ に 加え て しっかり やっ て いただき たい と 思っ て おり ます 。   この こと は 決意 だけ ちょっと 聞か せ て いただき たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   先生 おっしゃる よう に 、 マネ ロン 対策 と いう の は 非常 に 重要 な 課題 だ と 私 ども も 考え て おり ます 。   現行 の カジノ は 、 施設 内 に 数千 台 の 監視 カメラ が ござい まし て 、 全て の プレー を 監視 し 記録 を する と いう 仕組み に なっ て おり ます の で 、 事業 者 に おい て も そう いう 努力 を しっかり し て いただく と とも に 、 先生 御 案内 の ＦＡ ＴＦ の 勧告 に 従っ て 国内 法 を しっかり 整備 し て 対応 する こと に なる と 思い ます 。 現在 の **犯** **収** **法** ( `犯罪による収益の移転防止に関する法律` ) 、 **犯罪** **収益** **移転** **防止** **法** ( `犯罪による収益の移転防止に関する法律` ) の 改正 等 など の 措置 を しっかり 政府 に とっ て もらっ て 、 マネ ロン 対策 も 遺漏 なき よう に しっかり やっ て いく 、 その 前提 で この 構想 を お 願い し て いる ところ で ござい ます 。 <br/> ○ 上月 良祐 君   ありがとう ござい まし た 。 答弁 を 聞く と ちょっと 安心 を いたし ます 。   あと 、 ジャンケット の 対策 も 、 今 ランド オペレーター と いう の が 旅行 業 界 で いろいろ 問題 に なっ て おり まし て 、 今 国交 省 の 方 で 調査 など も やっ て おり ます けれど も 、 いわゆる ジャンケット と いう もの に 対する 対策 も 慎重 に 取扱い を やっ て いただき たい と 思っ て おり ます し 、 あと 、 マネ ロン 対策 を 徹底 する ため に 、 疑わしい 取引 が あっ た よう な とき に は 、 その ため に 新た な 税制 を つくっ て くれ と は 言い ませ ん が 、 既存 税制 は しっかり 適用 し て 、 厳格 な 適用 を し て いただき たい と 思っ て おり ます の で 、 その こと に つい て も 受け止め て いただき たい と 思い ます が 、 どう でしょう か 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   ジャンケット に つい て は 、 衆議 院 の 審議 の 段階 で も 、 また 参議 院 の この 当 委員 会 で の 審議 に おい て も 御 指摘 を いただき まし た 。   ジャンケット は 、 確か に 集客 上 の メリット は ある もの の 、 この 制度 が ゲーム へ の 過剰 な のめり込み 等 多く の マイナス の 側面 も 持っ て いる と 我々 認識 し て おり ます の で 、 ジャンケット の 扱い に つい て は 政府 に おい て 極めて 慎重 に 検討 を さ れる べき と 考え て おり ます 。   また 、 先生 御 指摘 の 税制 上 の 措置 に つき まし て も 、 税務 調査 等 の 措置 が 政府 に おい て しっかり 行わ れ 、 適切 に 判断 さ れる もの と 考え て おり ます 。 <br/> ○ 上月 良祐 君   税 の こと は 、 別 に 誰 で も 彼 で も と は 言い ませ ん 。 怪しい 取引 が あっ た よう な とき に は しっかり 、 まあ これ も ほか と 同じ だ と 思い ます 、 そう いっ た とき に は しっかり やる ん でしょう から 、 しっかり やっ て ほしい と いう こと で あり ます 。   最後 に 、 もう 一 点 だけ ちょっと 御 要望 し て おき ます 。   これ は 江島 先生 から お 話 が あっ た いわゆる 均てん 化 の 話 です 。   行政 需要 が 、 これ 、 まだ 余り みんな ちゃん と 考え られ て い ない ん だ と 思う ん です が 、 僕 は 地方 自治 の 現場 に 長く い た の で よく 分かる ん です けど 、 どこ か で か あっ た 場合 に 、 その 横 の 市 町 村 は まだ いい ん です よ 、 まだ 目配り できる から 。 県域 越え た 隣 の 県 に ある 空港 の 周り と か 、 県域 を 越え た 隣 の 県 に ある クルーズ 船 が 着く 例えば 港 の ところ と か 、 そこ の 間 の 動線 と か 、 そう いう ところ に 需要 も 出 て くる と 思う ん です 。   これ 、 実 は 岩屋 先生 が 極めて 的確 な 御 答弁 を さ れ て いらっしゃい ます 、 前回 の 審議 の とき に 。 な の で 、 その こと を 踏まえ て 、 周辺 自治 体 など に 対し て その こと を しっかり 踏まえ た 対応 、 納付 金 関係 に つい て は 検討 を 政府 の 方 で これ は し て いただけれ ば いい と 思い ます の で 、 その こと を しっかり お 願い を し たい と いう ふう に 思い ます 。 これ は 前 御 答弁 いただい て い ます の で 、 御 答弁 求め ませ ん の で 。   以上 、 そう いう ふう に 私 から いろいろ 申し上げ まし た 。 是非 、 魅力 の ある ＩＲ が できる よう に 、 しっかり 管理 と 、 しっかり 振興 等 を やっ て いただける よう に お 願い を 申し上げ まし て 、 私 から の 質問 を 終わら せ て いただき ます 。   ありがとう ござい まし た 。 <br/> ○ 高野 光二郎 君   自由 民主 党 の 高野 光二郎 で ござい ます 。   それ で は 、 早速 質問 を さ せ て いただき ます 。   ＩＲ に は 、 ただ の リゾート 施設 で は 実現 が でき ない 大きな 経済 社会 的 波及 効果 が ある と 考え ます 。 昨日 の 慎重 派 の 鳥畑 参考 人 の 資料 で も 明らか に なっ た よう に 、 二 〇 〇 九 年 から 二 〇 一 六 年 、 マカオ の カジノ の 収益 の 推移 は 年間 を 通じ て 一定 の 収益 を 上げ て おり 、 シーズン に よっ て 収益 に 著しい 増減 は 認め られ ませ ん 。 例えば 、 ホテル 部門 の 閑散 期 、 あるいは 国際 会議 が 開か れ ない 平時 に おけ る 不 利益 が ある 場合 で も 、 カジノ 部門 の 安定 し た 収益 に よっ て 不 採算 部門 も 稼働 が でき 、 雇用 者 の 継続 し た 就労 場所 が 確保 が 実現 を でき て おり ます 。   カジノ を エンジン と し て 機能 さ せ 、 この ＩＲ 導入 に よっ て かなう 国際 会議 の 誘致 、 民間 企業 、 学術 界 など 幅広い 情報 の ハブ と し て 機能 し 得る 展示 会 の 開催 など に よっ て 、 日本 の 産官 学 の 国際 競争 力 の 底上げ が ＩＲ 導入 に よっ て 可能 に なる と 考え て おり ます 。 まさに 成長 戦略 の 要 だ と 考え て おり ます 。   そこ で 、 確認 を し たい こと が あり ます 。 カジノ の 収益 は そもそも 一体 何 で つくら れ て いる の か 。 カジノ が 負け た 者 から ふんだくる システム で ある か の よう な 発言 が 党首 討論 に おい て も 見受け られ まし た が 、 カジノ の 収入 源 で ある 控除 率 に つい て の 理解 が 不足 し て いる ため だ と 私 は 考え て い ます 。   カジノ の 収益 が 一体 何 で つくら れ て いる の か 、 収益 構造 に つい て お 伺い を いたし ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   例えば 、 競馬 と か の 公営 競技 、 富くじ に 該当 する もの は 、 掛金 から 一定 の 割合 を 施行 者 が 控除 し て 、 控除 後 の 残額 を お 客 様 で 配分 する と いう こと に なっ て いる わけ で ござい ます 。 大体 、 公営 競技 の 控除 率 は 、 先生 御 案内 の とおり 、 二 五 ％ ぐらい に なっ て いる わけ で ござい ます 。   他方 、 御 質問 の あっ た カジノ と いう ゲーミング に おき まし て は 、 ゲーム の 種類 に 応じ て 決め られる 勝敗 の 数学 的 な 確率 です ね 、 それ に よっ て 施行 者 の 収益 が 決まっ て くる と 。 例えば 、 テーブル ゲーム など に おい て は 大体 九十 数 ％ が お 客 さん に 戻る けれど も 、 その 数 ％ が 施行 者 側 に 残る と いう 数学 的 な 確率 が あり ます 。 これ 、 数多く 繰り返し て いく こと に よっ て その 理論 値 の 中 に ずっと 収まっ て いく わけ です ね 、 大数 の 法則 で 。 そう やっ て その 施行 者 側 に 利益 が 残る と 、 そう いう 収益 構造 だ と いう ふう に 申し上げ て いい と 思い ます 。 <br/> ○ 高野 光二郎 君   つまり 、 例えば 、 私 、 カジノ やっ た こと ない ん です けど 、 ルーレット で いう と 、 三十 八 の ボックス が あっ て 、 それ に 玉 を 投げ て 、 一 枚 だけ に あげ て 、 一 枚 賭け て 一 個 当たっ たら 、 一 枚 だっ たら 三十 八 枚 に ふだん なら なる ん だ けど 、 それ が 三十 六 枚 しか なら ない と いう こと で ござい ます よ ね 。 これ ら も 踏まえ て 、 入場 料 なんか も あり ます の で 、 事業 者 は そう いっ た ところ で 利益 を 増し て いる と いう こと で ござい ます 。 だ から 、 決して 負け た 人 だけ から 取る と いう こと で は ない と いう こと を 国民 の 皆 さん に しっかり と 理解 を し て いただき たい と 思い ます 。   続き まし て 、 特定 複合 観光 施設 を 設置 する メリット です が 、 具体 的 に は 、 莫大 な 関連 施設 の 建設 、 建築 、 設備 需要 や 、 カジノ や ホテル の アミューズメント ホテル 施設 など の 雇用 が 創出 さ れ ます し 、 観光 客 増加 に よる 経済 波及 効果 、 カジノ の 収益 に よる 主に 自治 体 の 税収 増 も 寄与 さ れ ます 。   昨年 、 横浜 市 は 独自 の 試算 を し て 、 カジノ を 含む ＩＲ 全体 の 経済 効果 は 年間 四千 百 億 円 と し 、 税収 を 増加 六十 一 億 円 と 想定 を し て おり ます 。 また 、 横浜 商工 会議 所 の 試算 に よれ ば 、 五千 六百 億 円 の 経済 波及 効果 が ある と 試算 を さ れ て おり ます 。   しかし 、 一方 で 、 社会 的 な 負 の 影響 も 懸念 を さ れ て おり ます 。 とりわけ マネー ロンダリング 対策 に つい て は 、 規制 と 徹底 し た 管理 に より 対策 強化 は 可能 だ と 考え ます 。 しかし ながら 、 依存 症 対策 に つい て は 、 カジノ が 導入 さ れ て い ない 現状 で も その 対策 は 不 十分 だ と 言わ ざる を 得 ない 、 認め ざる を 得 ない と 考え て い ます 。   発議 者 は 、 先日 の 江島 委員 の 質問 に 対し て 、 カジノ 収益 に よる 国 へ の 納付 金 の 使途 に つい て 、 依存 症 対策 の 実施 へ の 十分 な 配慮 と いっ た 点 で 使わ なけれ ば なら ない と し て 、 実施 法 に よる 依存 症 対策 強化 を 求め て い ます 。 これ も 大変 必要 な こと だ と 思い ます 。 今回 の 法律 は プログラム 法 で ござい ます の で 、 実施 法 で しっかり と こう いっ た こと を 位置 付け を し なけれ ば いけ ない と いう ふう に 思っ て おり ます 。   ここ で 、 質問 を さ せ て いただき ます 。   今度 は 、 認定 を する 地方 公共 団体 の 区域 は 、 当初 は 全国 で 二、三 か所 を 想定 し 、 モデル 的 に 検証 し 、 慎重 に 進め て いく と いう こと で ござい ます 。 フランス で は 百 年 以上 前 の 一 九 〇 七 年 に カジノ 法 が 成立 を し 、 温泉 観光 地 に 限っ て 認定 さ れ た そう です 。 後 に 、 カジノ 施設 許可 を 人口 五十 万 人 以上 の 観光 都市 と いう ルール を 作り まし た 。 誘致 に 当たり 、 地方 公共 団体 の 日本 の 人口 規模 や 国際 空港 の 有無 、 距離 や 所要 時間 、 あるいは 大型 クルーズ 船 の 就航 可能 な 港湾 と の 距離 など 、 様々 な 項目 が 考え られ ます 。   発議 者 は 、 区域 の 認定 基準 と し て 具体 的 に どの よう な こと を 考え られ て いる の か 、 お 伺い を いたし ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   お 答え を 申し上げ ます 。   具体 的 な この まさに ＩＲ 施設 、 ＩＲ 区域 、 施設 区域 の 認定 基準 に つい て は 、 法案 の 施行 後 一 年 以内 を 目途 に 政府 が 策定 を し ます 実施 法案 の 中 に 規定 を さ れる と いう こと に なり ます けれど も 、 私 ども と し て は 、 まさに 御 指摘 の あっ た 観光 客 の 増加 や 多様 な サービス 提供 に よる 雇用 の 拡大 など 大きな 経済 効果 が 見込ま れる と いう こと が まず 第 一 で あり ます の で 、 一定 規模 以上 の もの と いう こと に なっ て いく か と 思い ます 。   それ から 、 御 指摘 の あっ た 地域 の 魅力 の 向上 あるいは 地方 創生 、 町づくり 、 こう いっ た もの に も 貢献 する こと 、 あるいは それぞれ の 地域 独自 の 文化 発信 、 個性 を 生かし た 発信 、 クール ジャパン の 推進 、 こう いっ た もの に も 寄与 する と いう こと が 基本 的 な 認定 基準 と し て 私 ども 想定 を し て いる ところ で ござい ます 。   それ から 、 もう 御 案内 の とおり 、 地元 住民 の 理解 を 得る と いう こと も 大事 な 点 で あり ます の で 、 地方 自治 体 が 認定 の 申請 を する に 当たっ て は 地方 公共 団体 の 議会 の 同意 を 要件 と する と いう こと も 考え られる と いう ふう に 思い ます 。   さらに 、 御 指摘 の あり まし た 、 温泉 を 始め と し て 様々 な 地域 の 観光 資源 の 活用 の 状況 あるいは 地方 公共 団体 の 人口 、 空港 や 港湾 の 立地 状況 、 こう いっ た もの に つい て も 、 これ は 当然 、 ＩＲ 施設 区域 の 認定 に 当たっ て は 重要 な 判断 材料 に なる と いう ふう に 思っ て おり ます し 、 地域 が 手 を 挙げ て も 、 民間 事業 者 が 投資 を し なけれ ば これ は 成り立ち ませ ん の で 、 民間 事業 者 が 投資 を 判断 する に 当たっ て も 、 こう し た 人口 の 状況 あるいは 地域 の 観光 資源 、 空港 、 港湾 の 状況 、 こう いっ た もの に つい て も 、 投資 の 判断 に 当たっ て も 大事 な 判断 材料 に なる と いう ふう に 思っ て おり ます 。 <br/> ○ 高野 光二郎 君   ＩＲ 施設 は 民設 民営 です ね 。 ＩＲ 施設 は 様々 な 施設 が 複合 的 に 存在 いたし ます 。 三十 四 年 前 に 設立 を し た 東京 ディズニー ランド の 初期 投資 は 約 一千 八百 億 円 と 言わ れ て い ます が 、 年間 入場 者 数 が 三千 万 人 を 超え た 今 、 新 アトラクション の 導入 を 連続 さ せ 、 年間 五百 億 円 の 投資 を 十 年間 、 五千 億 円 の 投資 を 続け 、 顧客 確保 対策 を する と 発表 いたし て おり ます 。   ＩＲ の 設置 や 運営 に 参入 が できる 事業 者 は 莫大 な 初期 投資 や ランニング コスト が 掛かる と 思い ます が 、 どの よう な 業界 の 参入 が 想定 を さ れる の か 、 お 伺い を いたし ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   お 答え いたし ます 。   先生 御 指摘 の よう に 、 ＩＲ と いう の は あくまで も 統合 型 の 施設 で ござい まし て 、 その 中 に 認め られる カジノ の 面積 も 三 ％ 程度 の もの を 私 ども 想定 し て いる わけ で ござい ます 。 したがい まし て 、 初期 投資 は かなり 大きな もの に なる と 想定 を し て おり ます 。 例えば 、 シンガポール で は 二 か所 の ＩＲ を 造っ て おり ます が 、 それぞれ 五千 億 円 、 四千 七百 億 円 と いう 投資 規模 に なっ て おり ます が 、 今 何 も 事業 者 も 地域 も 決まっ て い ない 段階 で 初期 投資 が どの ぐらい に なる の か と いう の は 明確 に お 答え でき ない わけ です が 、 そう いっ た 例 を 参照 いたし まし て も 、 それ ら の 規模 を 我が 国 で 行う 場合 は 上回る と いう こと を 想定 を いたし て おり ます 。   また 、 現 時点 に おい て どの よう な 事業 者 が 参入 する と 思っ て いる か と いう こと で ござい ます が 、 私 ども と し て は 、 ＩＲ と いう 統合 型 の 施設 を 国際 競争 力 を 持っ た 魅力 的 な もの に し て いただく ため に は 国 の 内外 の 英知 を 結集 し て いただく こと が 望ましい の で は ない か な と いう ふう に 思っ て おり ます の で 、 外国 企業 だけ で は なく て 、 国内 の 事業 者 と の 場合 に よっ て は ジョイント ベンチャー みたい な 形 に なる こと が 望ましい の で は ない か と 。 そこ に は 様々 な 業態 が 入っ て くる ん だ と 思い ます 。 カジノ 部分 以外 は 我が 国 の 企業 の ノウハウ を 十分 に 生かせる 施設 に なっ て いく と 思い ます の で 、 内外 の 英知 が そこ に 結集 さ れ て いく と いう こと を 想定 を いたし て おり ます 。 <br/> ○ 高野 光二郎 君   岩屋 先生 の 先ほど の 答弁 に 少し 関わり が 出 て くる ん です が 、 日本 に は カジノ の 文化 は 正直 根付い て い ない と 思い ます の で 、 カジノ 施設 を 設置 、 運営 する 日本 企業 の ノウハウ は 海外 企業 に 水 を 空け られ て いる と 私 は 推察 を いたし て おり ます 。 しかし ながら 、 日本 企業 の コンプライアンス 遵守 の 精神 又 は 日本 の 伝統 文化 に 通じる 日本 企業 の 参入 が 、 私 は 国内 企業 の 育成 と 成長 に 資する と 考え て おり ます 。   しかし 、 カジノ が 、 いざ 設置 を し て 、 蓋 を 開け て みれ ば 全て 外国 企業 だっ た と いう こと は やはり 思わしく ない と 思っ て おり ます が 、 その 辺 に つい て の お 考え を お 伺い し ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   外国 企業 が カジノ の 運営 事業 者 と なり 得る か どう か と いう こと に つい て は 、 例えば ＷＴＯ で の 規定 など 国際 的 な サービス 協定 等 の 状況 を 踏まえ ながら 、 今後 、 実施 法 の 立案 過程 に おい て 検討 さ れる こと に なる と 考え て おり ます 。   その 上 で 、 実施 法 に おい て 、 仮 に 外国 企業 が カジノ 運営 事業 者 に なる 、 あるいは その ＩＲ の 事業 に 参画 する と いう こと に なっ た 場合 で あっ て も 、 観光 や 地域 経済 の 振興 、 財政 の 改善 に 資する と いう 目的 を 達成 できる と いう こと で あれ ば 設置 者 に なり 得る こと は 当然 で ござい ます が 、 当然 、 我が 国 の 法令 に 全て 従っ て 納付 金 あるいは 税 の 納付 等 の 義務 は 全部 果たして いただく と いう こと に なる わけ で ござい ます 。 <br/> ○ 高野 光二郎 君   それ で は 最後 に 、 タイム スケジュール に つい て やはり 気 に なり ます 、 お 伺い を いたし ます 。   衆議 院 の 解散 に より 廃案 と なっ た 同法 案 は 、 今 から 二 年 前 、 平成 二十 六 年 六 月 十 八 日 の 衆議 院 内閣 委員 会 にて 同法 案 が 議論 さ れ て い た よう で ござい ます が 、 ＩＲ は いつ オープン が できる か が 私 は 重要 だ と 考え て おり ます 。 今回 の 発議 者 の 皆 様 が 主導 さ れ て こ られ まし た 、 超 党派 議連 の 皆 様 が まさに もう 十 三 年 も の 間 を 掛け て ＩＲ の 有効 性 に つい て 検証 さ れ て き た と 心 から 敬意 を 表し ます 。   その 間 に 、 二 〇 二 〇 年 の 東京 オリンピック ・ パラリンピック の 誘致 が 成功 いたし まし た 。 世界 一 の イベント を 誘致 し た こと も あり 、 当然 外国 人 観光 客 誘致 に も 多大 な 貢献 が 期待 さ れる と 思っ て おり ます 。   二 年 前 の 衆議 院 の 内閣 委員 会 の 別 の 発議 者 、 皆 さん で は ない 別 の 発議 者 の 答弁 で 、 実施 法 成立 に 一 年 、 施設 の 開発 に 三 年 掛かっ た と し て も 、 六 年間 の 時間 軸 が あれ ば 二 〇 二 〇 年 に 間に合う と 思っ て いる と の 答弁 を さ れ て おり ます が 、 それ から 今 遅れ て 二 年 の スタート 、 さらに 、 今回 の 審議 で も 改めて マネー ロンダリング 対策 や 依存 症 対策 の 必要 性 が 明確 に 浮かび上がり まし た 。 実施 法 で は 万全 を 期さ なけれ ば なり ませ ん 。   そこ で 、 既に 開催 が 決まっ て い ます 東京 オリンピック ・ パラリンピック 、 さらに は 二 〇 一 九 年 ラグビー ワールド カップ 、 二 〇 二 一 年 ワールド マスターズ ゲームズ 、 さらに 大阪 府 が 誘致 を 目指し 国 も 支援 する 方針 の 二 〇二五年大阪万博誘致 に 向け て 、 ＩＲ の 設置 、 オープン 時期 を どの よう に 戦略 的 に 結び付け て いく の か 、 お 伺い を いたし ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   お 答え を 申し上げ ます 。   御 指摘 の とおり 、 仮 に 本 法案 が 成立 し た と し て も 、 そこ から 一 年 以内 を 目途 に 政府 の 実施 法 案 、 具体 的 な カジノ の 規制 法 案 、 申請 の 手続 に つい て の 法案 、 そう いっ た もの が 出 て き ます の で 、 そこ まで まず 一 年 掛かり ます し 、 そこ から 、 地方 公共 団体 が 手 を 挙げ て 、 審査 を し 、 認め られ 、 そして 事業 者 の 認可 も あり 許可 も あり 、 そして 、 そこ から 建設 が また 始まる と いう こと です の で 、 具体 的 に 考え て いき ます と 、 仮 に この 法案 が でき て 、 そして 実施 法案 が 成立 し て も 、 そこ から 数 年 は 掛かる もの と いう ふう に 思い ます の で 、 そう し ます と 、 当初 想定 を し て おり まし た 二 〇 二 〇 年 の 東京 オリンピック ・ パラリンピック に は 、 現実 的 に は なかなか 間に合わ せる の は 難しい と いう ところ だ と いう ふう に 思い ます 。   むしろ 、 建設 は ひょっと し たら その 前後 から 始まっ て いる か も しれ ませ ん の で 、 そう いう 意味 で 、 御 指摘 の あっ た いろんな 国際 的 な イベント 、 国内 で 誘致 を し て いる いろんな イベント も 念頭 に 置き ながら 、 この 二 〇 二 〇 年 前後 の 切れ目 ない 経済 対策 、 そして 観光 振興 と いう こと を 念頭 に 、 これ から そう いっ た こと も 配慮 し ながら ＩＲ の 整備 が なさ れ て いく と いう こと を 期待 を し たい と いう ふう に 思っ て おり ます 。 <br/> ○ 高野 光二郎 君   丁寧 な 御 答弁 ありがとう ござい まし た 。   以上 で 終わり ます 。 <br/> ○ 和田 政宗 君   自由 民主 党 の 和田 政宗 です 。   私 は ギャンブル 依存 症 に つい て 聞い て まいり ます 。   ギャンブル 依存 症 を 完全 に 防ぐ に は どう する か 。 それ は 、 公営 競技 、 パチンコ 、 パチスロ を 含め て ギャンブル を 全て 日本 から なくし て しまう こと です 。 しかし 、 これ は なかなか 現実 的 で は あり ませ ん 。 また 、 ギャンブル を やり たい と 思う 人 が 社会 に 存在 する 中 で 、 全く 禁止 を し て しまえ ば 、 ギャンブル は 裏 に 潜り込ん で しまう こと が 考え られる わけ です 。 言わば 必要 悪 に 近い 存在 。 そう し た 中 で 、 公営 競技 や ｔｏ ｔｏ など で 違法 性 阻却 を 認め 、 その 収益 を 社会 に 貢献 さ せ て いこう と いう ふう に なっ て いる わけ です 。   これ は 、 ギャンブル を 求める 声 が 先 か 、 ギャンブル を つくっ た の が 先 か と いう 論 に なっ て いく と いう ふう に も 思い ます けれど も 、 ギャンブル を 全く なくす の は 現 時点 に おい て 現実 的 で は ない わけ です 。 だ から こそ 、 私 は 、 ギャンブル に おい て は 賭博 性 や 射幸 性 を 低く 抑える と とも に 、 ギャンブル 依存 症 対策 を 根本 的 に 構築 を し て いか なく て は なら ない と いう ふう に 考え て おり ます 。   カジノ が 日本 に おい て 将来 できる 、 これ は 、 これ まで の 日本 に おけ る 既存 の ギャンブル と 違い ます の で 、 抑止 策 や 予防 策 を 取っ て いけ ば 依存 症 が 大きく 増大 する と いう こと は ない と いう ふう に 思い ます けれど も 、 やはり ギャンブル に つい て は 人 それぞれ と いう もの が あり ます の で 、 ゼロ と いう わけ に は いか ない わけ で ござい ます 。   ＩＲ の 必要 性 を 考え た 場合 に 、 昨日 の 参考 人 質疑 で も 、 カジノ が 運営 開始 に なる 際 、 ギャンブル 依存 を 防止 する 組織 や 体制 を つくる 必要 性 が 述べ られ て おり ます 。 先ほど 上月 理事 から も 同様 の 質問 が ござい まし た けれど も 、 運営 開始 に 先行 し て 体制 や 組織 を つくる こと 、 こう いっ た こと に つい て 発議 者 は どの よう に お 考え か 、 お 聞か せ ください 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   もう 本当 に 和田 委員 に おか れ て も 、 御 自身 の 経験 も 含め て 前回 も 質疑 が ござい まし た 。   まさに 、 ギャンブル 依存 症 対策 と いう の は 私 ども も 最 重要 課題 だ と 認識 を いたし て おり ます の で 、 今回 の 審議 を 通じ て 様々 な 御 指摘 を いただき まし た こと を 私 ども 議連 と し て も 直ちに 取り組ん で いき たい と 思い ます し 、 政府 に おい て も これ は 予算 要求 も し て おり ます し 、 今 既に 実施 し て おり ます けれど も 、 これ は まだ 不 十分 だ と 言わ ざる を 得 ませ ん の で 、 体制 の 強化 を 急い で やっ て いただき たい と いう ふう に 思っ て いる ところ で ござい ます 。   実施 法案 が 一 年 以内 を 目途 に 出 て まいり ます けれど も 、 もちろん その 実施 法案 の 出す に 当たっ て の 検討 過程 で も 、 政府 に おい て は 、 本部 に おい て は 推進 会議 、 有識 者 を 集め て 会議 を 開く こと に も なっ て おり ます の で 、 専門 家 の 知見 、 知恵 を 結集 し 、 また シンガポール など 海外 で の 事例 も 踏まえ て 、 しっかり と ギャンブル 依存 症 対策 、 取り組ん で いただき たい と 思っ て い ます が 、 これ は もう 直ちに 取り組む べき 課題 だ と いう ふう に 認識 を いたし て おり ます 。 <br/> ○ 和田 政宗 君   もう この ギャンブル 依存 症 対策 に つい て は 、 既存 の ギャンブル も 含め て しっかり やっ て いか なく て は なら ない と いう の は 、 これ は もう 与野党 共通 の 考え 方 で ある と いう ふう に 思っ て おり ます 。   そして 、 国民 の 声 に も しっかり と 向き合っ て いか なく て は なら ない と いう ふう に 思っ て おり ます が 、 そう し た 声 の 中 で 、 カジノ の 合法 化 、 これ は 多重 債務 者 を 増やす おそれ が ある と 主張 する 方 も い ます けれど も 、 発議 者 は その 声 に どう いう ふう に 向き合っ て いく でしょう か 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   これ まで 、 多重 債務 者 対策 と いたし まし て 累次 の **貸金** **業** **法** ( `貸金業法` ) 改正 等 の 対策 が 実施 を さ れ て き て おり ます 。 例えば **貸金** **業** **法** ( `貸金業法` ) に おき まし て は 、 個人 消費 者 に は 、 年収 等 を 確認 の 上 、 貸与 額 の 上限 が 定め られる 等 の 措置 が とら れ て いる わけ で ござい ます 。 今後 、 カジノ を 含む ＩＲ が 推進 さ れる こと に なっ た 場合 に おき まし て も 、 これ まで の 多重 債務 者 対策 は 有効 に 機能 する もの と いう ふう に 考え て おり ます 。   また 、 カジノ が 一定 の 金融 機能 を 持つ こと も 事実 で あろう か と 思い ます 。 したがい まし て 、 マネ ロン 対策 の 対象 施設 に も なっ て いく わけ で ござい ます 。 したがい まし て 、 実施 法 に おき まし て は 、 カジノ の 金融 業務 の 在り 方 と とも に 、 それ に 関連 する 対策 、 例えば 事業 者 に よる 顧客 管理 、 与信 管理 の 徹底 など 適切 に 検討 さ れる こと が 必要 で ある と いう ふう に 考え て おり ます 。 <br/> ○ 和田 政宗 君   それ で は 、 私 の 経験 、 見聞き し た こと から 一 つ 質問 を さ せ て いただき ます けれど も 、 私 が 昔 競馬 を やっ て い た 時代 に 、 府中 の 東京 競馬 場 、 ここ に 行き まし て 、 十 二 レース 終わっ て 府中 本町 の 駅 まで 帰る わけ です けれど も 、 帰る 途中 に パチンコ 店 や パチスロ 店 が あり まし て 、 競馬 の 負け 分 を 取り返そう と いう 人 たち が そう いっ た ところ に 入っ て いく わけ で あり ます 。 私 の 場合 は そう いっ た こと も なく 、 すぱっ と 帰っ て おり まし た けれど も 、 こう いっ た 実情 が ござい ます 。   カジノ の 特性 、 立地 条件 など は 、 何 と いい ます か 、 その ギャンブル と し て の 性質 も 含め て 、 そう いっ た 競馬 場 と は また 違う もの で ある と は 認識 を し て おり ます が 、 ＩＲ の 近隣 地域 に ほか の 公営 競技 の 設置 で あり ます と か パチンコ や パチスロ 店 の 出店 を 規制 する こと 、 これ も ギャンブル 依存 の 抑止 に つながる と いう ふう に 考え ます が 、 発議 者 は どの よう に お 考え に なり ます でしょう か 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   先生 の 本当 に 貴重 な 経験 を 踏まえ て 御 質問 を いただき まし た が 、 ギャンブル 依存 症 対策 に つき まし て は 、 先生 おっしゃる とおり 、 カジノ だけ で は なく て 、 他 の ギャンブル 等 に 起因 する 依存 症 を 含め て 包括 的 に 取組 を 考え て いか なく て は いけ ない と 思っ て おり ます 。   私 ども 、 ＩＲ と いう の は 非常 に 高 規格 な 、 グレード の 高い 総合 エンターテインメント 施設 と いう もの を 想定 し て おり ます の で 、 先生 御 指摘 あっ た よう に 、 ＩＲ の 近隣 地域 に 他 の 公営 ギャンブル や パチンコ 、 パチスロ 店 の 出店 が 行わ れる など と いう こと は 望ましい こと で は ない と いう ふう に 考え て おり まし て 、 そう いう こと も 含め て 実施 法案 の 立案 過程 に おい て 十分 検討 が なさ れる べき と いう ふう に 思っ て おり ます 。 <br/> ○ 和田 政宗 君   それ で は 次 に 、 運営 に つい て 聞い て いき たい と いう ふう に 思っ て おり ます 。   これ は 昨日 の 参考 人 質疑 で も 参考 人 の 方々 に お 聞き を いたし まし た けれど も 、 公営 競技 に おき まし て は 、 賭け の 対象 と なる の は 、 競輪 、 ボート 、 オート レース など 、 厳格 に 管理 さ れ た プロ プレーヤー で あり ます 。   一方 、 カジノ に おけ る ディーラー は 、 賭け の 勝負 の 相手 方 と なっ たり 、 ルーレット に 玉 を 入れる など 重要 な 役割 を 果たす わけ です けれど も 、 民間 の 方々 の 育成 と なる と いう ふう に 見 られ て いる わけ で あり ます 。 民間 の 育成 で 足り 得る の か 、 また 公的 な 資格 を 付与 し たり 管理 を 行う の か 、 発議 者 の 見解 を 伺い ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   本 法案 に おき まし て は 、 カジノ 施設 関係 者 は カジノ 管理 委員 会 が 行う 規則 に 従わ なけれ ば なら ない 旨 規定 を し て いる わけ で ござい ます が 、 この カジノ の 施設 関係 者 に は 、 カジノ の 設置 、 運営 を 行う 事業 者 のみ なら ず 、 機器 の 納入 業 者 、 あるいは 先生 今 おっしゃっ た カジノ で サービス 提供 を 行う ディーラー など 他 の 従業 員 も 含ま れる こと に なる わけ で ござい ます 。 したがい まし て 、 そこ に は 最高 位 の 廉潔 性 を 求め なけれ ば なり ませ ん 。 厳格 な 参入 規制 、 適格 性 の 審査 と 行為 規制 、 さらに は 監督 が 必要 だ と いう ふう に 考え て おり ます 。 分かり やすく 言え ば 、 従業 員 も 適格 性 審査 の 上 、 ライセンス の よう な もの を 得 て い なけれ ば なら ない と いう 仕組み に す べき だ と 思っ て おり ます 。   ディーラー と いう の は 主要 な その 中 の プレーヤー で ござい ます の で 、 この ディーラー の 規制 に つい て の 具体 的 な 内容 に つい て は 、 実施 法案 の 立案 過程 に おい て 、 資格 の 付与 の 当否 など も 含め て 、 今 申し上げ た よう な 考え 方 に 基づい て 十分 に 検討 さ れる もの に なる と 考え て おり ます 。 <br/> ○ 和田 政宗 君   次 に 、 カジノ の 運営 業 者 に つい て お 聞き を し たい と いう ふう に 思い ます 。   運営 業 者 、 これ は 、 安定 し た 収益 を 出し 続ける 、 法律 に 守ら れ た 言わば 特権 と も 言える 企業 が できる 可能 性 も ある と いう ふう に 考え て おり ます 。 一旦 カジノ の 運営 業 者 と し て 決定 し 参入 し た 企業 が 永続 的 に 運営 を 行う こと が できる と する の か 、 それ と も 、 投資 の 回収 と いう もの が 一定 年限 が あり ます と 見込める わけ で ござい ます の で 、 一定 年限 で 交代 さ せる こと を する の か 、 こう いっ た 可能 性 も 含め て 、 発議 者 の 見解 を 伺い ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   私 ども も 先生 と 同じ よう に 考え て おり まし て 、 この ライセンス と いう もの は 当然 有効 期限 、 期間 と いう もの が あっ て しかる べき だろう と いう ふう に 思っ て いる ところ で ござい ます 。   具体 的 に 例えば 何 年 に する の か 、 どう する の か と いう こと に つい て は これ から 実施 法 で 定め られる こと に なる わけ で あり ます が 、 仮 に 有効 期限 を 設ける と し た 場合 は 、 政府 あるいは 管理 委員 会 が 示し た 基準 を 施行 に おい て しっかり 果たし て き た か と いう こと など も 含め て 審査 の 基準 に なる 、 また そう いう 仕組み を つくる こと が 必要 で は ない か と 思っ て おり ます 。 <br/> ○ 和田 政宗 君   それ で は 、 最後 の 質問 に なり ます が 、 これ も 昨日 参考 人 質疑 で 参考 人 の 方 に お 聞き を いたし まし た 。 カジノ 運営 企業 が 仮 に 外国 企業 で あっ た 場合 に 、 収益 が 海外 に 流出 する こと に なり ます が 、 外国 企業 が カジノ 運営 業 者 と なり 得る の か どう か 、 また 、 外国 企業 が カジノ 運営 業 者 と なっ た 場合 、 その 収益 に つい て どれ だけ 納付 金 を 課す か 、 これ に つい て 発議 者 の 見解 を 伺い ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   先ほど も 申し上げ まし た よう に 、 私 ども 提出 者 と し て は 、 この ＩＲ の 建設 、 運営 に 際し て は 国内外 の 英知 が 結集 さ れる こと が 望ましい と 考え て おり ます けれど も 、 仮 に 外国 企業 が カジノ 運営 事業 者 と なる 、 あるいは 参画 を する と いう ふう に なっ た 場合 は 、 先ほど お 答え し た とおり 、 まず 国際 的 な サービス 協定 等 の 状況 を 踏まえ ながら 実施 法 に おい て 対応 方 が 定め られ て いく と いう こと に なろう か と 思い ます 。   仮 に 外国 企業 が 参画 を し た と し た 場合 で あっ て も 、 当然 の こと で ござい ます が 、 我が 国 の 法令 に のっとっ て 納付 金 あるいは 納税 等 の 義務 を しっかり 果たして いただく と いう こと に なる わけ で ござい ます の で 、 一方 的 に 外国 企業 が 利益 を 外 に 持ち出す など と いう こと に は なら ない と 考え て おり ます 。 <br/> ○ 和田 政宗 君   時間 が 参り まし た の で 終わり ます けれど も 、 この プログラム 法 に おい て の 論点 、 かなり 私 は 整理 を さ れ て き て いる と いう ふう に 思っ て おり ます 。 この 後 も 質疑 続き ます けれど も 、 しっかり と 議論 を し て 、 もし 仮 に 今回 の 法案 が 通る と いう こと に なり ますれ ば 、 この 後 実施 法 と いう こと も あり ます 。 一 年 を 掛け て 政府 が しっかり と やっ て いく と いう こと も あり ます 。 引き続き 、 ギャンブル 依存 症 を 含め て しっかり と し た 議論 を し て いか なく て は なら ない と いう ふう に 思っ て おり ます 。   以上 で 終わり ます 。 <br/> ○ 森本 真治 君   民進 党 ・ 新緑 風 会 の 森本 真治 で ござい ます 。 質問 の 機会 を いただき まし て ありがとう ござい ます 。   今日 質問 を さ せ て いただく に 当たっ て 、 前回 の 委員 会 の 質疑 で あっ たり 、 本 会議 は 当然 私 も 参加 を し て おり まし た の で 、 いろいろ と これ まで の 議事 録 も 改めて 確認 も さ せ て いただき まし た 。 発議 者 の 皆 さん は しっかり と 理解 を 求めよう と 真摯 に 御 答弁 も さ れ て いる 、 努力 を さ れ て いる と いう ふう に は 私 も 受け止め させ て いただい て おり ます 。 ただ 、 内容 が 内容 な だけ に 、 懸念 の 声 と いう の が いまだ に 払拭 でき て い ない と いう こと も 事実 だ と いう ふう に 思い ます 。   今日 四十 分 と いう 限ら れ た 時間 で かなり 多く の 通告 を 出さ せ て いただい て おり ます から 、 私 も 自分 が 持っ て いる 懸念 の 部分 に つい て この 限ら れ た 時間 で 十分 に 御 答弁 いただける か どう か 分かり ませ ん けれど も 、 そう いう 面 で は 、 ちょっと 数 が 多い の で 、 臨機 応変 に 質問 内容 も 飛ばさ せ て いただい たり も し ます の で 、 御 対応 を いただけれ ば と いう ふう に 思い ます 。   まず 、 先ほど ちょっと 高野 委員 の 方 が 質問 も さ れ た の で 少し 重複 する か も しれ ませ ん けれど も 、 今回 この 法案 と いう もの を 提案 を さ れ て 、 今後 実施 法 なり 、 その 後 各 自治 体 の 方 が いろいろ と 計画 を 立て られ て 、 認可 を 受け て 整備 に 入る と いう 手順 が 生まれる わけ で ござい ます けれど も 、 相当 、 数 年 規模 で 掛かる と 、 年数 が 掛かる と いう こと は これ まで も 御 説明 も あっ た わけ で ござい ます 。   そう いう 中 で 、 先ほど 西村 先生 も 御 答弁 さ れ まし た けれど も 、 二 〇 二 〇 に は なかなか ちょっと 厳しい ん で は ない か と いう よう な 御 答弁 も あっ た 中 で 、 ただ 、 その 一方 で 、 やはり 国際 イベント と 相乗 効果 と いう こと も この ＩＲ は 意義 が ある の だろう と いう こと で 今 御 答弁 も あっ た わけ で ござい まし て 、 そう する と 、 例えば 、 これ ちょっと 再 確認 に なり ます けれど も 、 二 〇 二 五 年 の 大阪 万博 で あっ たり と か 、 これ まで の 答弁 も 、 例えば 二 〇 三 〇 年 六千 万 人 と いう 目標 、 イン バウンド 、 そう いう 目標 に 向かっ て 何 と し て も 進め て いき たい と いう 、 そう いう 思い は 持た れ て いる と いう こと で よろしい ん でしょう か 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   はい 、 御 指摘 の とおり で ござい ます 。   訪日 観光 客 、 順調 に 伸び て おり ます けれど も 、 これ は 地方 の 努力 も ある と 思い ます が 、 更に その 魅力 を 増し て いく ため の 施設 の 一 つ と し て この ＩＲ 施設 を 考え て おり まし て 、 一定 規模 以上 の もの で 、 そして 日本 なら で は の そう し た エンターテインメント 、 クール ジャパン の 発信 と いう こと に なれ ば 更に 多く の 観光 客 を 引き付ける もの に なる と 思い ます の で 、 二 〇 二 〇 年 に は 間に合い ませ ん けれど も 、 二 〇 二 〇 年 前後 の 切れ目 ない 観光 対策 、 経済 対策 、 そして 六千 万 人 の 目標 に 向け て 、 私 ども と し て は 是非 推進 を し て いき たい と いう ふう に 考え て いる ところ で ござい ます 。 <br/> ○ 森本 真治 君   あと 、 この ＩＲ の 設置 箇所 と いう 部分 で いえ ば 、 これ まで も 二、三 か所 ぐらい が まず は 現実 的 で は ない か な と いう お 話 も ござい まし て 、 その 具体 的 な 場所 に つい て は 当然 まだ 分から ない わけ で ござい ます けれど も 、 いろんな 報道 など を 見 て いる と 、 例えば 都市 部 で あっ たり と か 、 これ まで の 答弁 で あれ ば 、 地方 創生 の 観点 から 地方 と いう こと も 意味 は ある の で は ない か と いう ふう に 言わ れ て おり ます けれど も 、 実際 に これ 都市 部 に 設置 する こと が 、 相当 これ 経済 効果 と いう か 、 相乗 効果 と いう か 、 そう いう 部分 で 意義 が ある と いう ふう に 理解 を し て いい ん でしょう か 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   基本 的 に 、 まず 地元 の 自治 体 が 地元 住民 の 理解 も 得 て 、 例えば 議会 の 同意 も 得 て 手 を 挙げ て いく と いう こと が あり 、 さらに 一定 規模 以上 で あり 、 さらに 民間 事業 者 が 投資 を する と いう こと に なっ て いき ます の で 、 これ は どこ で も かしこ で も できる と いう こと で は ない と 思い ます の で 、 人口 規模 と か 空港 の 立地 場所 と か それぞれ の 地域 の 持つ 観光 資源 と か 、 そう いっ た こと 全体 を 考え て 立地 場所 は 決まっ て いく もの と いう ふう に 思い ます し 、 具体 的 な 基準 は 実施 法案 の 中 で 定め られる もの で あり ます けれど も 、 もちろん 大 都市 で やる こと も 魅力 ある 、 そう いう 施設 を 造る の も 一 つ の 可能 性 、 大きな 可能 性 ある と 思い ます し 、 一方 で 、 地方 で も 可能 性 の ある ところ は 幾 つ か ある と いう ふう に 認識 を いたし て おり ます 。 <br/> ○ 森本 真治 君   先ほど 、 例えば 財源 の 問題 と か 都市 規模 の 問題 で 、 大 都市 と いう の は ぴん と くる ん です ね 、 相乗 効果 。 地方 で の 、 これ も 可能 性 が ある と 言わ れ て い ます けれど も 、 諸 外国 なんか で も 、 しっかり と これ 地方 で でき て いる ところ って ある ん です か 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   世界 百 二十 七 か国 で カジノ は あり ます の で 、 それ を 見 て いき ます と 様々 な タイプ が あっ て 、 大 都市 圏 に ある もの も あれ ば 地方 に 立地 し て いる もの も あり ます し 、 地方 の 温泉 地 に 立地 し て いる よう な もの も あり ます の で 、 そう いっ た もの も 参考 に し ながら 実施 法案 は 考え て いく こと に なる と 思い ます が 、 ただ 、 私 ども 、 十 か所 も 二十 か所 も 造る よう な こと は 想定 し て おり ませ ん の で 、 国内 で 一定 規模 以上 で あり 、 まさに 御 指摘 の あっ た 経済 へ の 大きな プラス 、 財政 へ の 寄与 、 こう いっ た こと も 考え て 一定 規模 以上 の もの で ある と いう ふう に 認識 を いたし て おり ます の で 、 そう いっ た 観点 から する と 、 大 都市 部 と それ から 地方 で 、 意欲 の あり 、 可能 性 の ある ところ が 幾 つ か ある の か な と いう ふう に 思っ て いる ところ で ござい ます 。 <br/> ○ 森本 真治 君   今日 は 時間 が ない の で 、 本当 に 地方 で 成功 できる か と いう よう な いろんな そう いう 根拠 など に つい て も いろいろ 教え て もらい たい ん です けれど も 、 今日 は もう 、 ちょっと 時間 限ら れ て い ます から 、 また 改めて この よう な 機会 が ある と 思っ て おり ます の で 、 いろんな 議論 も し なけれ ば いけ ない と 思っ て おり ます 。   なぜ 私 が この よう な 質問 を する か と いう と 、 やはり 地方 創生 、 地方 創生 と か って 言い ます けれど も 、 結局 、 今 の いろんな 地方 創生 の 政策 と いう の は ことごとく 成功 し て いる か どう か と いう の は 疑問 に 思っ て いる とき に 、 本当 に この よう な ＩＲ が しっかり と 地方 で できる の か と いう よう な こと です よ 。 そう いう こと も しっかり と 、 私 は 疑問 に 思わ ざる も 得 ない と いう よう な ところ が ある わけ です ね 。   それ と もう 一 つ 、 例えば 二 〇二五大阪万博 と いう よう な こと も 報道 さ れ て い ます 。 もう 既に 候補 地 、 大阪 あり き と か 、 そう いう こと が あっ て しまっ たら 、 今後 これ 選定 を する とき に 客観 的 な 条件 の 中 で 選定 さ れ ない と いう 、 やはり これ 公平 性 の 問題 など も 非常 に 問題 に なっ て くる ん で は ない か と いう ふう に 思い ます ね 。   これ は 、 認可 を する の は 推進 会議 です か ね 、 推進 本部 です か ね 、 その 辺り の 公平 性 、 どの よう に 担保 を 取っ て いく の か 、 しっかり と その 情報 公開 など も 含め て その 辺り の 担保 と いう の が 取れ て いる の か と いう こと も お 伺い し たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   たまたま 現在 、 先生 おっしゃる よう に 、 非常 に 意欲 を 持っ た 地域 で 一生 懸命 調査 研究 を さ れ て いる ところ の 名前 が どう し て も 表 に 出 て き て おり ます の で 、 あたかも 何 か そこ に 決まっ て いく か の よう に 報じ られる こと も あり ます が 、 それ は そう いう こと で は ござい ませ ん で 、 今 から の 選定 に なっ て いく わけ で ござい ます 。   その 場合 は 、 先生 おっしゃる よう に 、 手続 、 プロセス が 透明 性 を 持っ た もの で なけれ ば なら ない と いう の は もう 全く おっしゃる とおり で ござい まし て 、 そう で なけれ ば 国民 の 皆 さん の 信頼 も 得 られ ませ ん の で 、 どう いう 具体 的 な 詳細 な 選定 プロセス を たどっ て いく の か と いう こと に つい て は 、 これ は 実施 法 で しっかり 定め た 上 で 国会 の 御 判断 を 再び 仰ぐ と いう こと に なる わけ で ござい ます 。 <br/> ○ 森本 真治 君   この よう な 大 規模 プロジェクト の 場合 は 、 常 に やはり そう いう 利益 誘導 と いう か 、 そう いう 部分 が 付きまとう わけ です よ 。 やはり そこ ら 辺 の しっかり と し た 公平 性 の 部分 に つい て が 本当 に 進ん で いく の か と いう 部分 に つい て は 、 しっかり と チェック を し て いか なけれ ば なら ない 部分 だ と いう ふう に も 思い ます 。   今日 は 観光 庁 お 越し いただい て おる と 思う ん です けれど も 、 観光 庁 と し て 、 これ まで イン バウンド の 中 で ＭＩ ＣＥ の 推進 と いう こと を 取り組ま れ て おる と いう ふう に 思い ます 。 実際 に これ Ｍ ＩＣＥ の 、 グローバル Ｍ ＩＣＥ 都市 と いう の が 指定 さ れ て おり まし て 、 七 自治 体 、 東京 、 横浜 、 名古屋 、 京都 、 大阪 、 神戸 、 福岡 。 強化 都市 と いう の も あり ます ね 、 札幌 、 仙台 、 千葉 、 そして 我が 広島 も あり ます 、 北九州 。   今後 の この ＭＩ ＣＥ 戦略 、 ＩＲ が 今後 もし 進ん だ 場合 に 、 この 戦略 に つい て は 方針 転換 と か 、 また 、 この Ｍ ＩＣＥ 都市 、 強化 都市 など が この 候補 地 と し て 、 ＩＲ の 都市 と し て やはり 有力 な 候補 地 に なっ て いく の か と いう よう な こと も 含め て 、 お 考え を お 伺い し たい と 思い ます 。 <br/> ○ 政府 参考 人 （ 瓦林 康人 君 ）   お 答え 申し上げ ます 。   ＩＲ の 推進 は 、 ＭＩ ＣＥ も 含め た 観光 振興 、 地域 振興 、 産業 振興 に 資する こと が 期待 さ れ て おり ます 。 議員 立法 で ある 本 法案 が 現在 御 審議 中 で ござい ます の で 、 私 ども 観光 庁 と いたし まし て 、 御 審議 、 御 議論 等 を 注視 いたし まし て 、 これ を 踏まえ て 適切 に 対応 し て いく 必要 が ある と いう ふう に 考え て ござい ます 。 <br/> ○ 森本 真治 君   日本 再興 戦略 で は ＩＲ に つい て は 関係 省庁 に おい て 検討 を 進める と いう こと で 、 これ は もう 政府 の 方針 と し て ある と 思い ます 。 当然 もう 検討 は さ れ て いる と 思い ます 。 今 の 御 答弁 で は ちょっと これ から みたい な 、 答弁 に あり まし た けれど も 、 当然 これ 、 例えば 実施 法 なんか も 、 もう 一 年 しか あり ませ ん よ 、 僅か 。 その 中 で 今 から やる と 言っ て 、 本当 に これ 一 年 で できる の か どう か と いう 問題 ある じゃ ない です か 。   実際 に 今 何 も 検討 し ない と いう こと で いい ん です か 。 <br/> ○ 政府 参考 人 （ 瓦林 康人 君 ）   私 ども と いたし まし て 、 この 御 審議 、 御 議論 等 を 注視 し て しっかり 検討 し て まいり ます 。 対応 方針 考え て まいり ます の で 、 よろしく お 願い いたし ます 。 <br/> ○ 政府 参考 人 （ 中川 真 君 ）   御 答弁 申し上げ ます 。   政府 と いたし まし て は 、 今 観光 庁 から も ござい まし た よう に 、 まず は 、 この 議員 立法 で あり ます **ＩＲ** **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に つき まし て の 国会 で の 御 審議 の 推移 を 見極める 必要 が ある と は 思っ て おり ます 。 その 上 で 、 今 、 この 法案 が 成立 し た 場合 に 一 年 以内 に 政府 は 対応 が できる の か と いう お 尋ね で ござい まし た 。   この 法 案 五条 に おき まし て 、 政府 は 、 特定 複合 観光 区域 施設 の 整備 の 推進 に 必要 と なる 法制 上 の 措置 に つい て は 、 **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) の 施行 後 一 年 以内 を 目途 と し て 講じ なけれ ば なら ない と さ れ て いる こと は 承知 し て ござい ます 。   この 点 に つき まし て は 、 先日 この 当 委員 会 に おき まし て も 白 委員 から 御 質問 を いただき まし て 、 官房 長官 から 、 国会 で 議員 立法 と し て 結論 を 出さ れ た の で あれ ば 、 それ を やはり 私 ども 政府 の 立場 で は しっかり 受け止め て 、 その 趣旨 に 従っ て 、 国民 の 皆 さん に 理解 を し て いただき 、 また 不安 の ない よう な 形 に し て いく の が 政府 の 役割 で ある と いう ふう に 思い ます と いう ふう に 御 答弁 申し上げ まし た 。   したがい まし て 、 この **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) が 施行 さ れ た 場合 に は 、 政府 と いたし まし て は 、 カジノ 施設 関係 者 に 対する 規制 です と か 入場 規制 など に つい て 、 海外 の 、 諸 外国 の 事例 も 参考 に し つつ 、 衆議 院 段階 で の 附帯 決議 の 御 趣旨 も 含め 、 また 、 この 法 案 に 係る 国会 で の 御 議論 です と か 国民 的 な 議論 を 踏まえ 、 検討 を 進める こと に なる と 承知 し て おり ます 。   今 観光 庁 の 方 に お 尋ね が あり まし た 点 に つき まし て も 、 この よう な 御 趣旨 で 推進 を 検討 を し て いく こと に なる と いう ふう に 理解 し て ござい ます 。 <br/> ○ 森本 真治 君   これ 、 衆議 院 の 十 二 月 二 日 の 議事 録 で 細田 先生 が 、 政府 の 中 で も 、 もう 二 年 有余 の 間 、 どう いう 問題 が あり 、 世界 的 に も どう いう 対策 が 行わ れ て いる と いう こと も 行わ れ て おり ます と いう 答弁 を さ れ て いる の で 、 もう しっかり と やら れ て いる ん だ と いう 理解 の 下 で 今 質問 を さ せ て いただい た ん です けれど も 、 これ から 一 年間 の 間 で やる ん だ と いう よう な 答弁 で 、 ちょっと その 辺り 、 少し 不安 に 思い まし た よ 、 今 の 答弁 で は 。   そう いう 中 で 、 今日 厚労 省 も 来 て いただい て おり ます 。 先ほど も いろんな ギャンブル 依存 症 対策 など に つい て の ちょっと 御 答弁 あっ た ん です けれど も 、 改めて 今 の 検討 状況 、 この ＩＲ 導入 を 踏まえ 、 前提 と し て 、 どの よう な 厚労 省 と し て の 検討 を し て いる の か と いう こと を 御 答弁 ください 。 <br/> ○ 政府 参考 人 （ 堀江 裕 君 ）   お 答え 申し上げ ます 。   先ほど も 上月 委員 の 方 から の 御 質問 ござい まし た けれど も 、 現在 、 厚生 労働 科学 研究 の 方 で より 正確 、 より 詳しい その 実態 把握 と いう こと を 進め て ござい まし て 、 三 年 前 に 行い まし た 調査 に おい て 御 本人 に アンケート の よう な 形 で 問う だけ の もの より も 少し 進化 さ せ て 、 しっかり と 状況 把握 が できる よう に する と いう こと が まず 一 点 。   それ から 、 あと 、 拠点 医療 機関 と いう の を 今 全国 に 五 か所 持っ て おり ます けれど も 、 やはり 、 それ は 依存 症 全体 に つい て で ござい ます けれど も 、 それ を 来 年度 に 、 全国 の 都 道 府 県 ・ 政令 指定 都市 六十 七 に 向け まし て 、 より 広く 、 もう 全県 に 指定 し て いく よう な 形 を 進め て いき たい と 思っ て おり まし て 、 大幅 な 予算 要求 も 増額 を さ せ て いただい て おり ます 。   要 は 、 各県 なり で 中核 と なる 医療 機関 に お 願い いたし まし て 、 その 県 の 中 で の ほか の 精神 科 医療 を 持つ よう な 医療 機関 に 対し て 支援 が できる よう な 医療 機関 と いう の を 各県 に 拠点 を 置い て いこう と いう こと で ござい まし て 、 全国 拠点 と いたし まし て は 今 、 神奈川 県 の 方 の 久里浜 の 医療 センター の 方 で 、 そう し た 五 県 なり 、 あるいは 、 今後 指定 さ れ て くれ ば 全国 の 拠点 と なる 医療 機関 に 対し て この 研修 なり 情報 提供 なり を 行っ て いき 、 また 情報 を 集約 し て いく よう な 仕組み に し て いこう と いう ふう な 形 で 進め て いる ところ で ござい ます 。   また 、 先ほど 上月 委員 の とき に お 話し いたし まし た けれど も 、 各 治療 法 と いう の が ある わけ です けれど も 、 それ が もう 少し 整理 さ れ て 、 検証 さ れ て 、 こう し た 場合 に は こう し た 治療 法 が 適当 な ん だ と いう よう な もの が 広く 伝わっ て いく よう な 形 に し て いけれ ば と いう ふう に 考え て ござい まし て 、 要 は 、 医療 機関 は 各地 に たくさん ある わけ で ござい ます けれど も 、 そう し た ところ で 適切 に 、 こう いう とき に は どこ に 相談 し たら いい か な 、 それ から どう いう 治療 に し て いっ たら いい だろう か と いう よう な もの が 整理 さ れ て いく よう な 形 を 進め て いる ところ で ござい ます 。 <br/> ○ 森本 真治 君   今 は 、 状況 把握 は 限ら れ た 区域 と いう か 地域 を 選定 し て 、 これ 神奈川 の 実態 を 把握 し て いる と いう 意味 です か 、 今 の は 。 来 年度 以降 、 全国 を 調べる と いう こと です か 。 <br/> ○ 政府 参考 人 （ 堀江 裕 君 ）   それ は 、 済み ませ ん 、 私 の 説明 が 少し いろんな もの が 混じっ て 混線 さ せ て しまい まし た けれど も 、 全国 に 、 神奈川 県 と いう こと で は なく て 、 無 作為 に 成人 の 方 を 対象 に 調査 を 行っ て いる ところ で ござい ます 。   全国 の 拠点 と なる 医療 機関 を 今 、 神奈川 県 の 国立 病院 機構 久里浜 病院 の 方 に お 願い し て い て 、 そこ で 、 アルコール です と か 依存 症 、 ギャンブル 、 そう し た もの の 言わば 拠点 に なっ て いる ところ な もの です から 、 この ギャンブル に つい て も しっかり と 情報 が 全国 に 共有 し て 流布 し て いく よう に し て いこう と 、 こう いう ふう に 考え て いる ところ で ござい ます 。 <br/> ○ 森本 真治 君   今 の 無 作為 で 調査 し て いる この 結果 なり 、 それ に 対し て の 対策 と いう か 、 その 方針 です ね 、 それ は いつ 出る ん です か 。 <br/> ○ 政府 参考 人 （ 堀江 裕 君 ）   この 調査 研究 自体 は 二十 八 年度 から の 三 か年 計画 で ござい ます の で 、 全部 を 完結 さ せる の に は 三 年 を 要し ます けれど も 、 この 実態 把握 と いう ところ に つい て は 特に 急 を 要し て おり ます の で 、 年度 内 に できる だけ の 形 で 、 少なく とも 、 少し 、 何 と いい ます か 、 確定 に は なら ない か も しれ ませ ん けれど も 、 ギャンブル 依存 症 、 その 中 に 、 どう いう こと を 契機 と し た 依存 を 感じ て い られる か 、 要する に どの 種別 の 、 公営 競技 な の か あるいは 遊技 な の か と いっ た よう な こと も 含め まし て 分かる よう に し て まいり たい と いう ふう に 考え て ござい ます 。 <br/> ○ 森本 真治 君   速報 値 と いう か 、 今 年度 中 に は まず は 一 回 出し て いただける と いう こと の よう で ござい ます から 、 委員 長 、 内閣 委員 会 へ の 資料 の 提出 の 要求 を お 願い し たい と 、 求め たい と 思い ます の で 、 よろしく お 願い し ます 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   後刻 理事 会 で 協議 いたし ます 。 <br/> ○ 森本 真治 君   その 後 に 細か な 分析 が なさ れ て 、 これ 三 年 です か 、 今 言わ れ た やつ 、 三 年 後 に 出る と いう こと です ね 。 そう する と 、 新た な カジノ の 解禁 など に つい て も 、 これ を 進める の は やはり それ を 待っ て から で ない と 私 は 到底 でき ない な と いう ふう に も 思い ます ね 。   今 これ だけ この 委員 会 で も いろんな ギャンブル 依存 症 の 問題 が 大きく 、 議論 が この 国会 で も できる よう に なり まし た の で 、 しっかり と まず は 政府 と し て この 対策 を どの よう に 取っ て いく か と いう こと が 明確 に なっ て 、 そして その 中 で しっかり と し た 防止 策 と いう こと が 担保 を 取れ ない 限り は 、 私 は やはり この 問題 は 前 に 進め ない の で は ない か な と いう ふう に も 思っ て おり ます 。   来 年度 予算 の 要求 も さ れ て いる と いう こと で ござい ます けれど も 、 もう これ は 本当 に 、 例えば これ 発議 者 の 皆 さん なんか は 、 もう 一 日 で も 早く と いう よう な こと で あれ ば 、 これ は 別 に 私 は 進める と いう 立場 で は あり ませ ん けれど も 、 百 歩 譲っ て 、 まず この 依存 症 対策 と いう こと を スピード を 上げ て いく の で あれ ば 、 それ こそ 今 年度 補正 予算 なんか で も どんどん 組ん で 取り組む べき 問題 だ と いう ふう に 思い ます けれど も 、 お 考え を お 願い し ます 。 <br/> ○ 政府 参考 人 （ 堀江 裕 君 ）   この 当初 予算 に おい て しっかり 盛り込ん で 、 今 その 大事 な 局面 に 来 て いる と 思い ます の で 、 また 依存 症 対策 全体 と いたし まし て は 、 先ほど の よう な 、 治療 拠点 あるいは 相談 の 拠点 に なる よう な 、 各 都 道 府 県 、 政令 市 に 備え て いく こと 、 それ から 全国 の 拠点 病院 、 久里浜 医療 センター 、 こちら の 中 で の 研修 、 指導 者 の 養成 です と か 情報 収集 、 普及 啓発 、 それ から 依存 症 回復 施設 職員 の 資質 の 向上 と いっ た よう な こと 、 そして 全国 で の その 普及 啓発 と いっ た よう な こと を 幅広 に 、 あるいは 体系 的 に 行っ て まいり たい と 考え て ござい ます 。 <br/> ○ 森本 真治 君   あと 、 この 依存 症 対策 で ちょっと 発議 者 の 方 に も お 伺い し たい ん です けれど も 、 今回 の この カジノ の 収益 で 、 納付 金 で この 依存 症 対策 など に も 充てる と いう よう な 御 答弁 が あっ た と 思い ます けれど も 、 非常 に 不 安定 です よ ね 、 それ です と 。 やはり その とき の 収益 の 波 と いう か 、 今後 どの よう に それ が 安定 的 に 確保 できる か も 分から ない よう な 状況 だっ た 場合 に は 。   そう する と 、 この 依存 症 対策 と いう の は やはり 国費 で しっかり と 対策 を 取る と いう こと が 大 前提 だ と 思う し 、 今 そう いう よう な 説明 は さ れ て いる けれど も 、 私 は 非常 に その 辺り に つい て は 、 何 と いう か な 、 しっかり と その 依存 症 対策 費 の 担保 は 取れ て い ない と 、 この 納付 金 で の その 対応 と いう こと が です ね 、 思い ます けれど も 、 それ に つい て どの よう に お 考え に なる でしょう か 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   まず は 実態 を しっかり 把握 する こと が 大事 で ござい まし て 、 それ は 今 厚労 省 から 説明 が あり まし た よう に 、 具体 的 な 調査 が 開始 さ れ た ところ で ござい まし て 、 そう いう ところ は 現行 の 国費 を 投入 し て しっかり 行っ て いく と いう こと だ と 思い ます が 、 依存 症 対策 で は 、 教育 、 予防 、 調査 、 治療 、 更生 支援 、 いろんな ステージ に 分かれる ん だ と 思い ます が 、 そう いう こと を 既存 の 公営 競技 や 遊技 から 発し て いる もの も 含め て 包括 的 に 講じ て いこう と すれ ば かなり の 一定 規模 の 予算 が 必要 に なっ て くる と いう ふう に 思い ます 。   諸 外国 の 事例 を 見 て も 、 カジノ から の 収益 を その 依存 症 対策 に 充当 し て いる 事例 など も 見 られ ます の で 、 それ ら も よく 研究 し て 、 やはり その 依存 症 対策 に も 一定 額 を 投入 し て いく 仕組み を つくる と いう こと も 大事 で は ない か と いう ふう に 考え て おり ます 。 <br/> ○ 森本 真治 君   昨日 の 参考 人 の やり取り も 私 は インターネット で ちょっと 見 させ て もらっ たり も し て 、 その 中 で 、 恐らく 進める べき だ と いう 意見 を 述べ られ た 参考 人 の 方 の 意見 だっ た と いう ふう に 思う ん です けれど も 、 ただ 、 今 の 例えば この ギャンブル 依存 症 で あっ たり 、 それ に 派生 し て 様々 な いろんな 社会 的 な 問題 が 発生 し て いる 部分 に つい て は 、 例えば 刑法 百 八十 五 条 の 関係 の 違法 性 の 阻却 の 部分 で も 、 やはり 副次 的 弊害 の 防止 と いう 部分 に つい て は 、 今 の 公営 ギャンブル なり パチンコ なんか も 含め た 中 で いっ た とき に は 、 やはり これ は 問題 が ある と いう か 課題 が 多い の で は ない か と いう よう な こと を 意見 陳述 さ れ て い た の を 私 は 昨日 ちょっと 確認 を さ せ て いただき まし た 。   そう いう ところ で 、 ちょっと これ は 法務 省 の 方 に 確認 を し たい と 思う ん です けれど も 、 今 の 公営 ギャンブル なり も そう です けれど も 、 よく 違法 性 の 阻却 の 八 要件 と か と 言わ れ ます けれど も 、 そもそも この 副次 的 弊害 の 防止 と いう ところ が クリア さ れ て いる の か と いう こと を 私 は 素朴 に 思う わけ です 、 現行 の 公営 ギャンブル に おい て も 。 その 辺り に つい て 法務 省 の 見解 を お 伺い し たい と 思い ます 。 <br/> ○ 政府 参考 人 （ 加藤 俊治 君 ）   お 答え を 申し上げ ます 。   賭博 行為 は 国民 の 射幸 心 を 助長 し 、 勤労 の 美風 を 害する ばかり で なく 、 副次 的 な 犯罪 を 誘発 し 、 さらに 国民 経済 の 機能 に 重大 な 障害 を 与える おそれ すら ある こと から 、 社会 の 風俗 を 害する 行為 と し て 処罰 さ れる こと と さ れ て おり ます 。   基本 法 で ある 刑法 が 賭博 を この よう に 犯罪 と 規定 し て いる 趣旨 を 没却 する よう な 立法 が なさ れ ます と 、 法 秩序 全体 の 整合 性 を 害する こと に も なり かね ない こと から 、 既存 の いわゆる 公営 競技 等 の 特別 法 に おき まし て は 事業 の 公正 性 、 公益 性 等 を 制度 上 十分 に 担保 する よう に 努め て おる もの と 承知 し て おり ます し 、 刑法 を 所管 する 法務 省 と いたし まし て は 、 既存 の 公営 競技 等 に 係る 特別 法 の 立法 に 当たっ て は 、 御 指摘 の 副次 的 弊害 の 防止 等 を 含む 様々 な 考慮 要素 に 着目 し 意見 を 述べ て き た ところ で あり まし て 、 これ ら の 特別 法 に おい て は 、 副次 的 弊害 の 防止 等 と いう 観点 を 含め て 、 現に 刑法 が 犯罪 と 規定 し て いる 趣旨 を 没却 し ない よう な 制度 上 の 配慮 が なさ れ て いる もの と 認識 し て おり ます 。 <br/> ○ 森本 真治 君   正式 な 法務 省 の 見解 だ と 思い ます 。 なさ れ て いる と いう こと です ね 、 今 、 副次 的 な 防止 が 。   その こと に つい て は 非常 に 私 は 疑問 に 思う 今 の 答弁 で あり まし た けれど も 、 そもそも 、 これ は ちょっと 問題 提起 に も なり ます けれど も 、 今回 新た な やはり カジノ と いう の を 解禁 する 前段 と し て 、 本当 に これ だけ 今 ギャンブル 依存 症 の 問題 なんか が 国会 の 場 に おい て も 大きく 取り上げ られる よう な 状況 に なっ て き た 中 で いえ ば 、 そもそも論 の 、 公営 ギャンブル や パチンコ など も 含め て 、 もう 一 度 この 議論 と いう もの も し て いく 必要 が ある ん で は ない か と いう よう な こと は ちょっと 意見 と し て 述べ させ て いただき たい と 思い ます 。   ちょっと 百 八十 五 条 の 関係 で もう 一 つ だけ 確認 を し たい ん です けれど も 、 この 賭博 と いう もの が 、 「 賭博 を し た 者 は 、 五十 万 円 以下 の 罰金 又 は 科料 に 処する 。 」 と いう こと で あり ます が 、 その 後 の ところ 、 ちょっと 私 、 これ 気 に なっ た ん です けど 、 ただし 、 一時 の 娯楽 に 供する 物 を 賭け た に とどまる とき は この 限り で は ない 。 と いう こと は 、 例えば 違法 カジノ なんか で も 、 一 回 だけ そこ に 行っ た と いう よう な こと で は 、 これ は 賭博 罪 の 適用 に なら ない と いう こと な ん です か 、 これ は 。 <br/> ○ 政府 参考 人 （ 加藤 俊治 君 ）   お 答え 申し上げ ます 。   具体 的 な 犯罪 の 成否 に つき まし て は 、 捜査 機関 が 捜査 に より 収集 し た 証拠 に より 判断 さ れる 事柄 で ござい ます の で 、 この 場 で 一律 に お 答え する こと は 困難 で ござい ます 。   ただ 、 刑法 百 八十 五 条 ただし書き の 「 一時 の 娯楽 に 供する 物 を 賭け た に とどまる とき 」 と いう の は 、 一般 に は 、 関係 者 が 即時 娯楽 の ため に 費消 する よう な 寡少 な 物 を 賭け た に とどまる とき を いう と 解さ れ て おり ます 。 そして 、 この よう な 規定 が 設け られ た 趣旨 は 、 賭博 行為 に 該当 する とき で あっ て も 、 単に 一時 的 な 娯楽 の ため に 費消 する 物 を 賭け た に すぎ ない よう な 場合 に は 、 賭博 罪 の 保護 法益 を 侵害 する 程度 は 軽微 で あり 、 又 は 社会 的 に 相当 と 認め られる こと に ある と 考え られ て おり ます 。   お 尋ね の 、 一 回 だけ カジノ で ゲーム を し た と いう 事案 に つい て 、 具体 的 事案 を 前提 に お 答え する こと は 先ほど 申し上げ た よう に 困難 で は ござい ます が 、 金銭 を 賭け た と いう 場合 に は 、 その 金額 の 多少 に かかわら ず 、 一時 の 娯楽 に 供する 物 に は 当たら ない と する の が 判例 で ある と 理解 し て おり ます 。 <br/> ○ 森本 真治 君   そもそも の 刑法 の 部分 に つい て も 少し 、 今回 の を 私 も いろいろ と 調べる 中 で 腑 に 落ち ない よう な ところ も あり まし て 、 これ に つい て も ちょっと 引き続き 研究 も さ せ て いただき たい と いう ふう に 思い ます 。   時間 が あり ませ ん の で 先 に 進め させ て いただき たい と いう ふう に 思い ます けれど も 、 まず 、 順番 が いろいろ 行っ たり 来 たり で 大変 申し訳 ない ん です けれど も 、 ＩＲ 開設 に よる 雇用 創出 効果 の こと に つい て ちょっと お 伺い し たい と 思い ます けれど も 、 これ 、 具体 的 な 雇用 創出 効果 の 数字 と いう の は 、 研究 と いう か 、 示さ れ て いる ん です か 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   私 ども 、 国際 観光 の 振興 、 あるいは 地域 経済 の 振興 、 そして また 財政 へ の 寄与 と 、 こう いっ た こと から ＩＲ を 整備 を す べき だ と いう ふう に 考え て おり ます が 、 具体 的 な 基準 は これ から 実施 法 の 中 で 決め られ て いき ます の で 、 規模 で ある と か いろんな 条件 が 決め られ て いき ます から 、 それ を 待た ない と なかなか 判断 は し にくい ところ で ござい ます が 、 私 ども 参考 に し て おり ます の は 、 もう 既に よく 議論 に なっ て おり ます シンガポール で あり まし て 、 シンガポール の 場合 に は 、 約 五千 億 円 の 投資 が あっ て 、 雇用 は 、 二 つ の ＩＲ 施設 に よる カジノ の 直接 雇用 は 二 万 三千 人 と いう ふう に 認識 を いたし て おり ます の で 、 それ なり の 雇用 の 効果 を もたらす もの と いう ふう に 認識 を いたし て おり ます 。 <br/> ○ 森本 真治 君   具体 的 な ところ は ちょっと 分から ない ところ も ある と 思い ます けれど も 、 海外 の 例 で いう と 万 単位 の 雇用 創出 効果 と いう よう な こと が 実際 に ある と いう こと で ござい ます けれど も 。   我が 国 の 、 今 、 雇用 環境 の 問題 と し て 、 特に この 少子 高齢 化 の 中 で 生産 年齢 人口 が どんどん と 減っ て いる よう な 状況 が ある 。 そして 、 現在 も 、 特に サービス 業 を 中心 に 深刻 な 人手 不足 問題 と 、 そして 労働 環境 の 悪化 と いう こと が 言わ れ て おり ます ね 。 これ だけ 大きな 例えば この ＩＲ の 関連 施設 の 中 に 労働 者 が 集中 を し て しまっ た 場合 に 、 他 の 産業 に 影響 する 、 労働 環境 に 与える 影響 と いう の は 非常 に 深刻 な 問題 に 私 なる と 思い ます よ 。 その 辺り に つい て は 、 周り へ の 、 他 産業 へ の 影響 など に つい て は どの よう に 考え て いらっしゃる ん です か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   ＩＲ が 実際 に 営業 開始 に なる 時期 に 関し て は 、 先ほど 来 答弁 が あり ます よう に 、 まず 一 年 以内 の 実施 法 が あっ て 、 それ から 更に また 数 年 と 、 こう いう 話 に なる わけ です ね 。 です から 、 そう いっ た 意味 で は これ から 数 年 掛かる と いう の は まず 第 一 点 。   そして 、 昨今 、 今 先生 御 指摘 の よう に 、 サービス 産業 に おけ る 人手 不足 と いう 話 が あり 、 人件 費 が 高騰 し て いる と 、 こう いう 報道 も ある わけ で あり ます が 、 そう いっ た 数 年 の 期間 が ある と いう こと の 中 で 、 適正 に ある 意味 で は 各 産業 と も それ に 対応 し て いく と いう こと も 考え られ ます し 、 さらに また 、 認定 区域 は 二、三 か所 で ござい ます の で 、 必ず しも そう いっ た 他 の サービス 産業 に 二 、 三 か所 で あれ ば 影響 を 与える と いう こと で は ない と いう ふう に 思っ て おり ます 。 <br/> ○ 森本 真治 君   前向き な 御 答弁 しか 多分 なかなか でき ない ん だろう な と いう ふう に 思い ます けれど も 、 やはり この 人手 不足 問題 が 与える 様々 な 影響 の 中 で の 労働 環境 です よ 。   例えば 、 今回 の いろんな これ まで の 話 で も 、 入場 規制 と いう こと は いろいろ と 言わ れ て いる の か な と 思い ます けれど も 、 そこ で 働く 皆 さん の 雇用 環境 を どう し て いく の か と いう 議論 は ちょっと なかっ た の で は ない か な と いう ふう に 思い ます ね 。 例えば 、 働く 人 の 年齢 制限 は どう なる の か 、 健康 問題 は どう なる の か 。 特に 、 諸 外国 の 例 で いえ ば 、 これ 二十 四 時間 営業 と か が 多い ん でしょう 、 大体 カジノ と か と いう の は 。 そう する と 、 長 時間 労働 の 問題 も あり ます ね 。 さらに は 、 例えば 外国 人 労働 者 を どう する の か と いう よう な 問題 も あり ます ね 。 その 辺り の 働く 側 の 環境 に つい て 、 どの よう に しっかり と 守っ て いく の か と いう 部分 に つい て は 全く 今 の 段階 で は 示さ れ て い ない 。   今 、 その こと に つい て しっかり と 示し て いただき たい と 思う ん です けれど も 、 お 考え を お 伺い し ます 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   民間 事業 者 に カジノ の 免許 が 付与 さ れる 場合 に おい て は 、 カジノ 管理 委員 会 に よる 免許 審査 手続 に おい て 厳格 な 適格 性 審査 が 行わ れる こと に なる わけ です ね 。 そこ に おい て 、 カジノ に おけ る 労働 環境 の 整備 だ と か 、 例えば 健康 対策 だ と か 勤務 時間 等 に つい て 関係 諸 法令 を 遵守 する もの で ある こと は 言う まで も あり ませ ん 。   また 一方 、 労働 行政 当局 に おい て も 、 **労働** **基準** **法** ( `労働基準法` ) 等 に 基づい て 、 カジノ 管理 委員 会 と とも に 連携 し て 、 適切 に カジノ に おけ る 労働 環境 に つい て 必要 な 監督 が 行わ れる と いう こと で あり まし て 、 我が 国 に おけ る しっかり と し た そう いっ た 雇用 関係 、 労働 関係 の 基準 が 守ら れ て いく もの と 思っ て おり ます 。 <br/> ○ 森本 真治 君   期待 だ と いう ふう に 思い ます が 、 しっかり と その 辺り が 本当 に 不安 な 部分 と し て も 残っ て いる わけ で ござい ます 。   もう 一 つ 、 ちょっと そもそも論 に なり ます けれど も 、 ＩＲ の 定義 と し て 、 これ 幾 つ の 施設 が あれ ば ＩＲ に なる の か 。 カジノ と ホテル は 多分 ある ん です よ 、 一体 と し て 。 それ 以外 に 幾 つ の 、 例えば 会議 場 と か いろいろ 言い ます けれど も 、 アミューズメント 施設 と か と 言い ます けれど も 、 どこ まで だっ たら これ Ｉ Ｒ と し て と いう よう な 何 か 定義 って ある ん です か 、 これ は 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   法案 に その 点 は 書か せ て いただい て おり ます が 、 ＩＲ 施設 、 特定 複合 観光 施設 と は 、 会議 場 施設 、 レクリエーション 施設 、 展示 施設 、 宿泊 施設 その 他 の 観光 の 振興 に 寄与 する 施設 及び カジノ 施設 が 一体 と なっ て いる 施設 を いう と **推進** **法** ( `特定複合観光施設区域の整備の推進に関する法律` ) に も 書か せ て いただい て おり ます が 、 当然 、 実施 法 の 段階 で は 更に 明確 に 定義 を さ れ て いく もの と いう ふう に 思っ て おり ます 。 <br/> ○ 森本 真治 君   例えば 、 一 つ で も 欠け たら 、 これ は もう ＩＲ 施設 と し て の 認可 が 取り消さ れ て しまう の か と いう 話 、 要 は 、 例えば 不 採算 施設 なんか を 閉鎖 し た 場合 に は どう なる の か と いう こと です 。 その まま で も 営業 は できる の か と いう こと の 確認 です 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   今 申し上げ た よう な 、 あくまで も 統合 型 の 施設 を しっかり と 運営 できる 事業 者 を まず 選定 し なけれ ば なら ない と いう こと だ と 思い ます 。   先生 御 指摘 の よう な 、 カジノ 以外 の 不 採算 施設 を 閉鎖 する よう な 運営 事業 者 が まず 選定 さ れ ない よう に 、 免許 の 付与 、 ライセンス の 付与 に 当たっ て は 事業 計画 の 実現 性 など に つい て 厳格 に 審査 を する こと に なる と 考え て おり ます 。 <br/> ○ 森本 真治 君   ちょっと 今 の 答弁 よく 分から ない ん です けれど も 。   閉鎖 を する よう な 業者 は そもそも 選定 さ れ ない と いう こと の 答弁 です けれど も 、 実際 に 、 で も 、 もう これ 海外 の 事例 で も 不 採算 施設 を 閉鎖 し て いる と いう よう な こと は ＩＲ の 中 で あっ て 、 そもそも でき ない ん です か ね 、 それ は 。 だ から 、 閉鎖 自体 が でき ない と いう 理解 な ん です か 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   ＩＲ 施設 の 中 に は 、 その 他 観光 の 振興 に 寄与 する 施設 と あり ます が 、 例えば いろんな タイプ の エンターテインメント 施設 が あっ た とき に 、 その 中 で 、 言っ て みれ ば 営業 成績 が 余り 振るわ ない もの は 他 の エンターテインメント に 替える と いう こと は 、 それ は ある ん だろう と 思い ます が 、 いわゆる 採算 が 取り にくい 大型 の 会議 場 で ある と か 展示 場 、 これ は ＭＩ ＣＥ の 機能 を 強化 する ため に そもそも ＩＲ と いう もの を 限定 的 に 認めよう と し て いる わけ で ござい ます から 、 そう いう 契約 上 と いう か 、 ライセンス を 付与 する に 当たっ て の 条件 に 反する よう な 行為 が 出 て き た 場合 に は 例えば 免許 を 取り消す 等 の 措置 を とる こと など も 含め て 、 しっかり と 規定 を 設け て おか なけれ ば いけ ない の で は ない か と 考え て おり ます 。 <br/> ○ 森本 真治 君   ちょっと 時間 の 方 も 残り が 少なく なっ た の で 、 あと の 質問 に つい て は もう 割愛 を さ せ て いただき たい と いう ふう に 思い ます けれど も 、 これ まで 発議 者 の 方々 も 実施 法 など に 向け て の 中 で も しっかり と 議論 を やら なけれ ば いけ ない と いう よう な お 話 も さ れ て おり ます けれど も 、 そもそも 今回 の 法案 の 議論 の 入口 の 部分 から し て いろいろ と ちょっと 禍根 を 残し て しまっ た 中 で 進ん で しまっ た 。 その 結果 と し て 、 やはり マスコミ なんか の 論調 など も 含め て 非常 に この よう な 懸念 の 声 が 大きく 上がっ て しまっ て いる と いう 部分 に いえ ば 、 本当 に 発議 者 の 皆 さん と いう の は 、 やはり この 良 さ と いう こと の 部分 で 一生 懸命 前 に 進めよう と 努力 さ れ て も 、 もう そう いう 環境 を つくれ て い ない と いう 状況 の 中 で いえ ば 、 今後 実施 法 なんか の 議論 に おい て も 、 もし この まま 進ん で いけ ば 非常 に 大きな 禍根 を 残し て いく と いう か 、 その よう な こと と いう の は 、 これ は 発議 者 の 皆 さん に とっ て も 私 は 不幸 な こと だ と いう ふう に 思う ん です よ ね 。   そう する と 、 本当 に 、 今 国会 は あした が もう 最終 日 と いう こと で 、 この 委員 会 で 議論 を し て いく と いう こと が 物理 的 に も もう これ 非常 に 厳しく なっ た と いう 部分 に いえ ば 、 今回 の 様々 な 委員 会 なんか の 議論 も 含め て 、 もう 一 度 この 基本 法 の 中身 に つい て も しっかり と 見直し を する 、 一 回 この 法 案 に つい て は 撤回 を し て 、 そして その 中 で 、 冷静 な 環境 の 中 で もう 一 度 丁寧 に 議論 を し て いく と いう こと が 私 は 必要 に なっ て くる と いう ふう に も 思っ て おり ます 。   その こと に つい て 、 発議 者 の 皆 さん 、 本当 に これ を 前 に 進め て コンセンサス を 得よう と する の で あれ ば その よう な 対応 を 取る べき だ と 私 は 思い ます けれど も 、 最後 に その お 考え を お 伺い し ます 。 <br/> ○ 衆議 院 議員 （ 細田 博之 君 ）   我が 国 で 初めて 、 史上 初めて この カジノ と いう もの を 解禁 し て 、 それ を 含む ＩＲ 施設 を 認め て いこう と いう わけ で ござい ます から 、 もちろん 批判 を する 方 も 多い わけ で ござい ます 。 今 まで 世界 約 百 三十 か国 が やっ て い て も 日本 だけ やっ て い なかっ た 、 それ を どう し て 今更 やる ん だ と いう 御 意見 の 方 も おら れ ます 。   しかし 、 この 点 で いえ ば 、 再三 申し上げ て いる よう に 、 日本 に 対し て 大きな 投資 を し て 大きな 雇用 を 発生 する よう な プロジェクト 、 先ほど も お 話 ござい まし た けど 、 やはり 大きな 集会 施設 や ホテル や アミューズメント 、 そして この カジノ も 含め て 投資 を しよう と いう 人 は 特に 海外 を 中心 に 極めて 多い と いう 実態 が ある と いう こと 。 それ から 、 現に それ が ギャンブル だ から いけ ない と いう 論 は 、 むしろ これ まで ある 依存 症 等 、 非常 に 大きな 社会 問題 が あっ て 、 これ は 何 か 対応 し なけれ ば なら ない の に 、 その 問題 と 関連 さ せ て 、 それ な の に これ を やる の は おかしい と 、 そう いう こと が 言わ れ て いる わけ で ござい ます が 、 私 ども は 、 政府 と の 関係 で いえ ば 、 政府 に 対し て 、 今 まで おかしい 部分 は 必ず やら なけれ ば なら ない 、 そう いう 時期 が 来 た 、 しかし 、 カジノ 施設 を 含む ＩＲ 施設 も 経済 の 発展 の ため に 必要 な 時期 が 来 た 、 そう いう ふう に 我が 国 自身 も 転換 を する とき が 来 た と いう 判断 で 提案 を し て いる と 。   その こと は 国民 の 皆 様 方 に も 広く 理解 し て いただき たい わけ で ござい ます し 、 テレビ 等 を 見る と 、 番組 の 中 で は だんだん そう いう 推進 論 者 と いう もの も 、 これ は きっちり 管理 さ れる ん だ から 大丈夫 だ と 言う 方 も 出 て おら れ ます の で 、 これ から 一 年間 で 、 世論 形成 と いう もの は 当然 必要 で ある と 思っ て おり ます の で 、 我々 も 努力 し て まいり たい と 思い ます 。 <br/> ○ 森本 真治 君   終わり ます 。 <br/> ○ 矢田 わか子 君   引き続き 、 民進 党 ・ 新緑 風 会 の 矢田 わか子 より 質問 を さ せ て いただき たい と 思い ます 。   私 の 方 から は 、 **ＩＲ** **法** ( `特定複合観光施設区域の整備の推進に関する法律` ) の 導入 に 伴う 変化 を 一 つ の キー ワード に 三 つ の 質問 を さ せ て いただき たい と 思っ て おり ます 。 まず は 、 導入 の 大 前提 と なる 、 少し 重なる ん です けれど も 、 違法 性 の 阻却 に つい て 、 二 つ 目 に は 、 その 導入 に よっ て 皆 様 方 が 主張 さ れ て いる 経済 効果 の 検証 に つい て 、 三 つ 目 に は 、 多く の 国民 の 皆 さん の 不安 も 払拭 でき て い ない 依存 症 の 対策 に つい て お 伺い を し て いき たい と いう ふう に 思い ます 。   本当 は この 順番 に お 聞き し たかっ た ん です けれど も 、 答弁 者 の 御 都合 に よっ て 、 まず 最初 に 、 ギャンブル 依存 症 へ の 対応 に つい て お 伺い を し て いき たい と 思い ます 。   カジノ に 関し て 、 皆 様 から も 、 お 声 が たくさん 委員 から も あり ます と おり 、 ギャンブル 依存 症 の 対策 、 反 社会 的 組織 の 介入 対策 を 考える と 、 最も 有効 と 思わ れる こと の 一 つ に カジノ へ の 日本 人 の 入場 の 制限 と いう もの が ある と 思い ます 。 これ は ハード 面 で の 対策 と いう ふう に お 受取 を いただけれ ば と 思い ます 。 ただ 、 今回 、 大変 残念 な こと に 、 皆 様 方 から 御 提示 さ れ た 資料 の 中 に は 、 必要 な 措置 を 講ずる と いう 表現 に とどまっ て おり ます 。 恐らく 、 カジノ へ の 入場 を 外国 人 のみ と すれ ば 経営 と し て 成り立た ない と 思わ れ ます し 、 日本 に 進出 しよう と する カジノ の 資本 も 、 やはり ギャンブル の 志向 の 強い 日本 人 を 狙っ た と いう か 対象 と し た 経営 を 考え て いる と 思わ れ ます 。   今日 で は 、 高齢 者 を 狙う 振り込み 詐欺 と いう よう な もの も 横行 し て い ます が 、 だまさ れ やすい 、 若しくは のめり込み やすい 、 そう し た お 年 を 召し た 方々 が カジノ へ 通う と いう ふう な こと も 想定 さ れる 。 そう する と 、 また 新た な 高齢 者 の 問題 に も 発展 いたし ます 。 加え て 、 この 問題 と いう の は 、 間接 的 に で は あり ます けれど も 、 家族 を 巻き込む と いう こと を 考えれ ば 、 子供 に 対する 影響 も 大きく 懸念 さ れる こと の 一 つ だ と 思い ます 。   こう し た こと を 踏まえ て 、 日本 人 の 入場 制限 に つい て 、 まず 発議 者 の 皆 さん に つい て は 、 この 部分 、 本音 で どの よう に お 考え な の か 、 お 聞か せ いただけ ます でしょう か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   この 問題 は 衆議 院 の 内閣 委員 会 で も 出さ れ て おり まし て 、 私 も 答弁 を さ せ て いただき まし た が 、 私 ども も 、 そう いっ た 外国 人 に 特化 し た いわゆる ＩＲ と いう の が できる の か どう か と いう 観点 は 真剣 に 議論 を さ せ て いただき まし た 。 その 上 で 、 以下 申し上げる 四 点 等 を 懸念 と し て お 伝え し て おき たい と 思い ます 。   まず 、 日本 人 の 入場 を 禁止 し 、 外国 人 のみ を 対象 と する カジノ 施設 の 設置 、 運営 を 推進 する 政策 に つい て は 、 nan 我が 国 の 刑法 が 、 その 適用 範囲 に おい て 、 日本 の 領域 内 で 罪 を 犯し た 全て の 者 に 適用 する と いう 属地 主義 を 原則 と し て いる こと 、 nan 規制 の 目的 が 依存 症 など の 弊害 から 日本 人 を 保護 する ため だ と し て も 、 現行 刑法 は 、 日本 人 の 外国 で の 賭博 行為 を 国外 犯 と し て 処罰 し て おら ず 、 これ と 整合 性 が 図れる の か 、 nan 外国 で も 取ら れ て いる 入場 料 、 排除 プログラム 等 の 入場 規制 を 課す の で は なく 、 一律 に 入場 を 禁止 する こと は 過度 の 規制 と なら ない か 、 nan 国内 観光 客 を 排除 する と 公益 還元 が 不 十分 な もの と なり 、 そもそも 本法 案 の 目的 が 達成 でき ない の で は ない か と いっ た 、 以上 四 点 の 懸念 が あり まし て 、 委員 おっしゃら れる よう に 、 本 法案 で は 日本 人 の 入場 を 一律 に 禁止 する の で は なく て 、 入場 規制 など 一定 の ギャンブル 依存 症 対策 を 講じ た 上 で 日本 人 に も 入場 を 認める と いう こと に いたし まし た 。   そう いっ た 過程 が あっ た こと を 御 報告 を 申し上げ たい と 思い ます 。 <br/> ○ 矢田 わか子 君   今 まで の 論議 の 中 で も 、 やはり 、 そう し た 日本 人 、 一定 の 制限 と いう 、 一定 と いう 言葉 の 中 に 、 何 を その 根拠 と し た 論拠 を 立て て いく の か と いう こと を 考え て いか なけれ ば いけ ない と いう ふう に 思っ て おり ます 。 例えば 、 収入 で ある と か 負債 の 額 、 それ から 財産 、 こう し た こと も 、 今 で あれ ば 、 まあ 個人 情報 の こと は あり ます けれど も 、 ＩＤ 等 を 使え ば 管理 が できる と いう ふう な 側面 も あり ます の で 、 その よう な こと も 含め た 、 是非 と も 踏み込ん だ 入場 制限 に つい て も 御 審議 を お 願い し たい な と いう ふう に 思っ て おり ます 。   続い て 、 皮肉 に も 、 こう し た 論議 が 深まる 中 で 特に 依存 症 の 問題 が 大きく クローズ アップ を さ れ まし て 、 今 問題 が 深刻 で ある こと を 国民 の 皆 さん も 含め て 大変 懸念 を し て いる と いう こと で も あり ます 。 予算 上 の 措置 講じる と いう ふう な 答弁 も 先 ほど あり まし た けれど も 、 単に お 金 を 確保 し た だけ で は なく て 、 より 実効 性 の ある そう し た 政策 を 打ち出す ため に は 何 が 必要 な の か 。   先ほど 、 ハード 面 の 対策 と し て 入場 規制 と いう こと を 申し上げ まし た が 、 二 つ 目 と し て 、 教育 の 問題 が ある の で は ない か と 思っ て おり ます 。 シンガポール で は 今 十 四 歳 以上 、 いわゆる 日本 で いう と 中学 生 ぐらい の 方々 に 対し て は 義務 教育 で カジノ と いう か ギャンブル 依存 に つい て の 教育 が 行わ れ て いる と いう こと も 、 昨日 の 参考 人 の お 話 の 中 に 含ま れ て おり まし た 。   加え て 、 もう 一 つ の 観点 、 病気 と いう 位置 付け で こう し た 依存 症 の 解明 を 行っ て いく こと 、 こんな こと も 必要 で は ない か と 。 例えば 、 心理 学 、 精神 医学 、 それ から 脳 科学 まで 含め た 研究 を 行い 、 こう し た こと の 防止 策 を 取っ て いく 、 その よう な こと など も 一定 の 効果 が 上がる の で は ない か と いう ふう に 思い ます 。   今日 は 官房 長官 いらっしゃっ て いただい て おり ます の で 、 政府 と し て も こう し た 問題 に 対し て 総合 的 な 政策 を 打ち出す べき と いう ふう に 考え て おり ます が 、 その よう な 御 意思 が ある の か どう か 、 是非 御 答弁 を お 願い し たい と 思い ます 。 <br/> ○ 国務 大臣 （ 菅 義偉 君 ）   この ギャンブル 依存 症 に つい て は 、 適切 な 治療 と 支援 に よっ て 回復 が 可能 で ある と いう ふう に 考え て おり ます 。 依存 症 の 方 が 必要 な とき に 早期 に 治療 を 受け られる こと の できる 環境 と いう もの を 整備 する こと が 重要 で ある と いう ふう に 思い ます 。   この ため 、 政府 と し て は 、 ギャンブル 等 依存 症 を 始め と する 依存 症 へ の 対策 と し て 、 拠点 と なる 医療 機関 に おけ る 専門 治療 や 相談 支援 、 依存 症 の 方 の 支援 に 当たる 人材 に 対する 研修 の 実施 、 こう し た こと に 取り組ん で いる わけ で あり ます けれど も 、 今回 の この 法 案 に つき まし て は 、 衆議 院 の 委員 会 採決 時 の 附帯 決議 に も そう し た こと が しっかり うたわ れ て い ます 。   引き続き 、 政府 と し て は 、 正確 な 実態 把握 に 努め ながら しっかり 対応 し て いき たい 、 この よう に 思い ます 。 <br/> ○ 矢田 わか子 君   ありがとう ござい まし た 。   是非 と も 、 政府 と し て も 踏み込ん だ 御 対応 を お 願い し たい と いう ふう に 思い ます 。   ここ で 、 官房 長官 に は 御 退席 いただい て も 結構 で ござい ます 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   菅 官房 長官 、 御 退席 し て いただい て 結構 で ござい ます 。 <br/> ○ 矢田 わか子 君   続い て 、 違法 性 の 阻却 に 戻り たい と いう ふう に 思い ます 。   この 件 は 森本 委員 から も かなり 出 まし た の で 、 私 から は 、 昨日 の 参考 人 の 答弁 踏まえ て 、 法 の 専門 家 で ある 弁護 士 の 方々 の 意見 が 二 つ に 昨日 の 時点 で 分かれ て おり ます 。 そう いっ た こと から 、 素直 な 疑問 と し て 、 発議 者 の 皆 さん は いつ の 時点 で この 違法 性 の 阻却 と いう 言わば 最大 の 難関 に つい て 議員 間 で 納得 性 が 得 られる よう な 、 要する に これ は 大丈夫 な ん だ と いう 確信 を 持て た の か どう か 。 どんな 論議 経過 の 下 で この 論議 を し て き て 、 そして いつ の 段階 で この 違法 性 の 阻却 は 可能 で ある と いう こと を 確信 さ れ た の か 。 例えば 、 先ほど 法務 省 の 御 見解 あり まし た けれど も 、 衆議 院 の 法制 局 の 説明 で 確証 さ れ た の か 、 あるいは 内閣 法制 局 とも 調整 さ れ た の か 、 その 辺り 、 お 聞か せ いただき たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   お 答え を 申し上げ ます 。   私 ども 、 公営 ギャンブル に おけ る 政府 の 考え 方 等 、 これ は 国会 の 審議 の 中 で も 過去 に も 何 度 か 審議 が 行わ れ て おり ます し 、 今日 も 法務 省 から も 説明 が あっ た ところ で ござい ます 。 そう し た 政府 の 考え 方 、 国会 で の 審議 、 こう し た もの を 踏まえ て この 法 案 を 作成 を さ せ て いただい た ところ で ござい ます 。 当然 、 法案 を 作成 、 条文 を 作る に 当たっ て は 、 衆議 院 の 法制 局 に 議論 を し て もらい 、 相談 を し ながら 条文 を 私 ども 作っ て いっ た わけ で あり ます が 、 基本 的 に は これ まで の 政府 の 考え 方 や 国会 の 審議 を 踏まえ て この 条文 を 作っ た ところ で ござい ます 。   もう 委員 よく 御 存じ の とおり で あり ます が 、 この 法案 で 違法 性 が 阻却 さ れる わけ で は ござい ませ ん の で 、 私 ども と し て は 、 八 つ の 項目 に つい て 政府 が 実施 法案 の 中 で しっかり と 規定 を し て もらっ て 、 そして 賭博 罪 の 違法 性 が 阻却 さ れる 、 例外 と し て 認め られる と いう ふう に なる よう に 、 私 ども と し て は 、 基本 法 案 で あり ます の で 、 八 つ の 項目 に つい て の 基本 的 な 方向 性 を 示し て ござい ます 。 です の で 、 この 法 案 に 則し て 、 そして また 、 私 ども の 答弁 も 申し上げ て おり ます の で 、 答弁 や 衆議 院 で 付さ れ まし た 附帯 決議 、 こう し た もの を 踏まえ て 、 政府 に おい て は 違法 性 が 阻却 さ れる よう な 実施 法案 を しっかり と 作っ て いただく と いう こと に なり ます し 、 それ が また 国会 に おい て 審議 を さ れ ます の で 、 その 時点 で も また 御 議論 を いただけれ ば と いう ふう に 考え て いる ところ で ござい ます 。 <br/> ○ 矢田 わか子 君   刑法 上 の 違法 性 を 阻却 する 、 もちろん 皆 さん 御 存じ だ と 思い ます が 、 その 第 一 の 要件 が 国民 が 納得 し 得る 公益 が 存在 する こと と いう ふう に あり ます 。 国民 が 納得 し 得る と いう ところ 、 ポイント だ と 思い ます の で 、 是非 そこ を 含め た 御 検討 の 推進 を お 願い し たい と 思い ます 。 二 つ 目 は 、 守ら れる べき 法益 に 配慮 し た 措置 、 そして 三 つ 目 が 、 これ ら を 主張 する に 足る 政策 的 理由 が 存在 する こと と いう ふう に なっ て おり ます の で 、 いま 一 度 ここ に 照らし合わせ を し た 、 具体 的 な 検討 を 進め て いただく 上 に おい て は 加味 を いただき たい と いう ふう に 思っ て おり ます 。   続け て 、 経済 効果 に つい て それ で は 触れ て いき ます 。 公益 の 確保 と いう これ は 観点 から で あり ます 。   カジノ の 経済 的 効果 、 当然 とても 重要 な 要素 の 一 つ と いう こと に なり ます が 、 この 点 に 関し て も 専門 家 の 意見 は 今 現在 で も 分かれ て おり ます 。 楽観 的 な 推測 で は 、 何 兆 円 も の 経済 効果 を 生み出す と いう 分析 も あり ます 。 一方 で 、 様々 な コスト 等 を 計算 すれ ば プラス 効果 は 出 て こ ない と いう 、 そう いう 見 方 も あり ます 。   例えば 、 これ から 四、五 年 後 の 日本 経済 、 近隣 諸国 の 経済 情勢 が どの よう に なっ て いる の か 、 これ は 的確 に は 予測 は でき ませ ん 。 また 、 東京 オリンピック 後 も 中国 、 東南 アジア 諸国 を 始め 各国 から 多く の 富裕 層 が 日本 に 来 て お 金 を 落とし て もらえる か どう か 、 これ も 予測 は でき ませ ん 。 一方 、 カジノ で 射幸 心 を 抑制 する よう な ルール 作り や 管理 監督 が 行わ れる と 、 これ 裏腹 でし て 、 集客 が 落ち ます 、 売上げ も 落ちる 。 既に パチンコ 業 界 で は 射幸 心 を あおる パチンコ 台 を 撤去 し た 、 その こと に よっ て 、 反面 、 最近 で は 売上げ が 落ち て いる と いう ふう な 、 そう いう 負 の 効果 が 出 て き て い ます 。 裏腹 な わけ です 、 つまり 。   この 裏腹 な 現象 に どう 対応 策 を 持っ て いく の か と いう こと が ポイント に なる の か と 思っ て おり ます 。 つまり は 、 集客 力 の 見通し を 含め 経済 効果 に つい て 読み 切れ ない 部分 が 大きい 中 で 、 特に カジノ の 建設 の 段階 から 健全 な 運営 の ため の コスト に つい て も 考慮 し て おく べき で ある と 考え ます 。   まず 一 つ 目 に は 、 カジノ の 管理 監督 や 治安 維持 の ため に 必要 な 行政 の 組織 を まず つくり 、 そこ に 公務 員 を 当然 配置 し なけれ ば いけ ない 、 コスト も 加味 し て いただか なけれ ば いけ ませ ん 。 二 つ 目 に は 、 ＩＲ の 建設 の ため 、 これ は 民間 が する と いっ て い ます けれど も 、 それ に 対する 社会 インフラ の 整備 の ため の 公的 な 負担 、 これ も 当然 考慮 し て いただか なけれ ば いけ ない 。 そして 三 つ 目 に は 、 出 て き て いる 依存 症 対策 、 お 幾ら 掛かる か 分から ない その 莫大 な 費用 に つい て も 、 少し なり とも やっぱり 考慮 を いただか なけれ ば いけ ない と いう ふう に 思い ます 。 発議 者 の 皆 さん の 説明 で は 、 経済 効果 に つい て 非常 に 、 大変 申し訳 ない ん です が 、 楽観 的 に 考え て いらっしゃる よう な 気 が し ます 。   こう し た 社会 的 な コスト に つい て どの よう に 考え て いらっしゃる の か 、 今 の 段階 で の 御 答弁 を お 願い し ます 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   お 答え 申し上げ ます 。   社会 コスト に つい て な ん です けれど も 、 まず 、 この カジノ の 運営 、 建設 と いっ た もの に つい て な ん です けれど も 、 建設 は 、 御 承知 の とおり 二 条 で 、 「 民間 事業 者 が 設置 及び 運営 を する もの を いう 。 」 と いう こと に なっ て おり まし て 、 公営 ギャンブル など と 違い まし て 、 これ は 本当 に 民設 民営 を 基本 と し て いる 、 まあ 民 で できる もの は 民 で で は ない です けれど も 、 この よう に 、 国 や 地方 が これ の 部分 に つい て は リスク を 負わ ない と 、 まさに 民間 活力 を 活用 し て いる と いう ところ が 今回 の 法案 の 大きな ポイント で あろう と いう ふう に 我々 は 考え て おり ます 。 その 上 で 、 カジノ の 建設 と いう か 、 カジノ で は なく て ＩＲ の 建設 です よ ね 、 御 質問 の 趣旨 は 。 ＩＲ の 建設 に おい て も 建設 需要 、 それ から 運営 に おい て は 雇用 と いう もの を 生み出す 。   そこ で 、 この 法 案 の もう 一 つ の ポイント は 、 やはり 私 ども は 八条 で あろう と 思っ て おり ます 。 地方 公共 団体 の 構想 を 尊重 する と いう こと を 八条 に も しっかり 書か せ て いただい て いる わけ で あり まし て 、 地方 が やる気 が なけれ ば 今回 の 構想 は やら なく て いい わけ で あり まし て 、 そこ で やはり 地方 が こう いう こと を やり たい ん だ と いう その 構想 を やはり 我々 と し て は 現実 の もの と しよう と いう 哲学 が 流れ て いる と いう ところ で あり ます 。   その 上 で 、 公共 インフラ の 整備 に つい て は 衆議 院 で も もろもろ 御 質問 が あり まし た が 、 基本 的 に は 、 公共 インフラ と いう もの は 通常 は 国 や 地方 公共 団体 が 行っ て いく と いう こと が 基本 と なっ て いる と は 思い ます が 、 実際 に ＩＲ を 実現 する ため に 必要 不可欠 な 公共 インフラ の 整備 と いう こと に 当たっ て は 、 国 や 地方 公共 団体 だけ が やる ん で は なく て 、 いわゆる ＰＰＰ と か ＰＦＩ と いっ た 、 こう し た 手法 を 採用 する こと も 考え られる と いう ふう に 考え て おり ます し 、 また 、 特に 我々 シンガポール を 随分 と 参考 に さ せ て いただい て おり まし て 、 シンガポール で は 本島 と セントーサ を 結ぶ 連絡 橋 と いう の が あり ます けれど も 、 この 公共 インフラ 整備 を 一部 民間 事業 者 が 更に 負担 を し て いる と いう 例 も ある わけ で あり まし て 、 いずれ に し て も 、 この 地方 の 構想 を 国 が 審査 を する と いう 形 を 取っ て おり ます の で 、 これ まで の そう し た 税金 だけ を 突っ込む と いう よう な 形 に は 私 たち は なっ て い ない と いう ふう に 承知 を いたし て おり ます 。 <br/> ○ 矢田 わか子 君   そう いう こと も 含め て な ん です が 、 カンフル 的 に ぼん と 建て た とき に 一定 の 利益 が 上がる 、 そして 物珍しく て 人 が 来る 、 そう いう カンフル 剤 で は なく て 、 持続 可能 な ＩＲ を 推進 し て いく ため に は やはり 中長期 を 含め た プランニング が 必要 だ と いう ふう に 思い ます の で 、 是非 と も そこ は 加味 し て いただき たい と 思い ます 。   それ と 、 ＩＲ 導入 の 効果 を いつ どの よう に 判断 する の か 、 これ も 一 つ 大きな ポイント だ と 思っ て おり ます 。 事業 側面 を 重視 する 余り 、 なし崩し 的 に 規制 が 緩和 さ れる と いう こと は 好ましく あり ませ ん 。 だ から こそ 、 事業 評価 は どう いう 時間 軸 で どの よう な タイミング で どんな ふう に 行う の か 、 是非 と も これ も 実行 法 の 段階 で 詰め て いただき たい と いう ふう に 思っ て おり ます の で 、 よろしく お 願い を し ます 。   時間 が なくなっ て き まし た 。 最後 に もう 一 つ だけ お 願い し ます 。 法益 の 確保 に つい て と いう こと で お 伺い を し て いき ます 。   特に 法益 の 確保 に つい て は 、 カジノ 施設 に おけ る 不正 行為 の 防止 、 カジノ の 運営 に 伴う 有害 な 影響 の 排除 と いう こと で 法案 の 第 十 条 に 規制 項目 が 列挙 さ れ て おり ます が 、 ちょっと 気 に なる 項目 の 一 つ は 、 昨日 の 中 で も 実 は あっ た ん です けれど も 、 第 五 号 に ある 風俗 環境 の 保持 等 の ため に 必要 な 規制 と いう もの で あり ます 。   これ は 他国 の 事例 で は あり ます けれど も 、 ここ は カジノ の ほか に サッカー 、 競馬 、 ドッグ レース 、 マージャン など の 賭け事 が 町 じゅう で 行わ れ 、 併せ て 性 風俗 産業 が 大きな ウエート を 占め 、 サウナ 、 ナイトクラブ など で は 売春 も 公然 と 行わ れ て いる と いう 実態 が あり ます 。   カジノ の 健全 性 を 強調 さ れる の で あれ ば 、 やはり 性 風俗 に つい て も 適正 かつ 厳しい 規制 を す べき だ と 考え ます 。 カジノ の 内部 そして 周辺 に 対し どの よう な 規制 を 想定 さ れる の か 、 お 伺い し たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   先生 御 指摘 の よう に 、 法案 第 十 条 、 一 から 八 まで の 観点 から 必要 な 措置 を 講ぜよ と いう ふう に 政府 に 命じ た 第 十 条 で ござい ます が 、 その 中 の 五 番 目 に 「 風俗 環境 の 保持 等 の ため に 必要 な 規制 に 関する 事項 」 と いう の が ござい ます 。 今 先生 が 指摘 を さ れ た よう な 事柄 が 行わ れ ない よう に 、 カジノ 施設 の 設置 及び 運営 に 関し て 、 この 有害 な 影響 の 排除 を 適切 に 行う 観点 から 風俗 環境 を 適切 に 保持 する と いう こと が 必要 で ある こと は 当然 の こと と 我々 は 考え て おり ます 。   これ に つき まし て は 、 地方 公共 団体 、 特に 都 道 府 県 警察 、 カジノ 管理 委員 会 、 しっかり と 連携 を し て 、 また 事業 者 の 協力 も 得 て 、 この ＩＲ 内部 並び に 周辺 の 風俗 環境 が 適切 に 保持 さ れる ため の 措置 を 講じ て いく こと に なる と いう ふう に 考え て おり ます 。 <br/> ○ 矢田 わか子 君   ありがとう ござい まし た 。   今回 の 法案 は 基本 法 、 プログラム 法 と いう こと で あり ます けれど も 、 社会 に 大きな 影響 を 与える 法案 で あり ます 。 一 年 も ある 、 一 年 しか ない 、 どちら な の でしょう か 。 あと は 政府 に お 任せ し ます で は やっぱり 困る と 思い ます 。 やはり 、 提案 者 、 発議 者 と し て の 責任 は 重たい と いう 御 認識 を お 願い し たい と 思い ます 。   この 法案 が 可決 さ れる と 、 次 は 実施 法案 の 検討 の 段階 に なり ます けれど も 、 国会 審議 に 至る まで 、 皆 さん が この 国会 の 中 で まさに 答弁 さ れ て き た こと が きちんと 本当 に 生かさ れる の か どう か 、 私 たち も 見極め て いく 責任 あり ます けれど も 、 是非 と も 、 発議 者 の 皆 さん や 与党 の 皆 さん も いらっしゃい ます の で 、 そう し た プロセス に 深く 関わる こと が できる と 思い ます 。 審議 は 実際 時間 と し て 足ら ない と いう ふう な 思い も あり ます けれど も 、 その こと も お 願い を 申し上げ まし て 、 私 の 質問 を 終わり ます 。   ありがとう ござい まし た 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   午後 一時 に 再開 する こと と し 、 休憩 いたし ます 。       午後 零 時 二 分 休憩           ─ ─ ─ ─ ─ ・ ─ ─ ─ ─ ─       午後 一時 開会 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   ただいま から 内閣 委員 会 を 再開 いたし ます 。   委員 の 異動 に つい て 御 報告 いたし ます 。   本日 、 森本 真治 君 が 委員 を 辞任 さ れ 、 その 補欠 と し て 神本 美恵子 さん が 選任 さ れ まし た 。         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 委員 長 （ 難波 奨 二 君 ）   休憩 前 に 引き続き 、 **特定** **複合** **観光** **施設** **区域** **の** **整備** **の** **推進** **に** **関する** **法律** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) を 議題 と し 、 質疑 を 行い ます 。   質疑 の ある 方 は 順次 御 発言 願い ます 。 <br/> ○ 大門 実紀史 君   大門 です 。   この 間 の 世論 調査 です ね 、 国民 多数 が 反対 と いう こと と 、 マスコミ の 社説 も 全て が 批判 的 論調 と 。 そんな 下 で 何 と し て も 通し たい と いう こと で 今日 も 行わ れ て いる わけ です けれど も 、 議員 立法 で こんな 異常 な 強行 ぶり と いう の は 見 た こと あり ませ ん 。 これ だけ 世論 が 反対 な の に 、 国会 は どう し た の か な と 、 特に 参議 院 は どう なっ た の か な と 本当 に 思い ながら 、 今日 も 質問 を する と いう こと に なる か と 思い ます けれど も 。 やっぱり 、 もっと もっと 国民 の 声 を 真摯 に 、 先ほど も あり まし た けれど 、 ちゃんと 受け止め て 立ち止まる べき で は ない か と 思い ます けれど も 、 質問 に 入り ます 。   今 は 、 特定 の 企業 と か 特定 の 業界 から 献金 を 受け たり パーティー 券 買っ て もらっ て 、 そして その 企業 や 業界 の 利益 に なる よう な 質問 を し て も 、 国会 質問 を し て も 受託 収賄 に 問わ れる よう な 時代 に なっ て き て おり ます 。 そんな 下 で 、 まして や 議員 立法 で 、 特定 の 業界 、 企業 の 利益 誘導 に なる よう な 議員 立法 を 通そう と 、 利益 誘導 を 図る と いう こと は 、 一 つ の 質問 や 何 か より も よっぽど 私 は 危ない 話 じゃ ない か と いう こと で 、 この カジノ の 問題 を ずっと 国会 で 取り上げる たび に 、 テレビ の 前 で も 申し上げ まし た けど 、 そう いう 指摘 を し て き た ところ で あり ます 。   そこ で 、 今日 は まず その 問題 聞き たい ん です けれど 、 前回 の 質問 の 前 に 発議 者 の 皆 さん に 既に 通告 は し て ござい ます 。 カジノ の 関連 企業 、 いろいろ 幅 が 広い ん です けど 、 すなわち 、 直接 、 パチスロ や ゲーム 機器 です ね 、 あるいは 観光 業 、 パチンコ 業 界 も 絡む と 思い ます けれど も 、 そう いう カジノ の 関連 企業 から 政治 献金 を 受け取っ たり パーティー 券 を 買っ て もらっ た こと が ある か どう か と いう こと は 発議 者 の 皆 さん 全員 に 通告 し て あり ます けれど 、 まず 西村 さん 、 いかが です か 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   先般 も どなた か の 質問 を 、 衆議 院 でし た か 、 お 答え を 申し上げ まし た けれど も 、 私 、 **政治** **資金** **規正** **法** ( `政治資金規正法` ) に のっとり まし て 適正 に 処理 を いたし て おる ところ で ござい ます の で 、 それ 以上 の お 答え は 差し控え たい と 思い ます 。 <br/> ○ 大門 実紀史 君   **政治** **資金** **規正** **法** ( `政治資金規正法` ) に のっとり と いう の は 当たり前 で 、 のっとら なかっ たら 大変 な こと に なる わけ です ね 。 私 が 言っ て いる の は 受け取っ た こと が あり ます か と いう こと な ん です けれど 。   まあ 、 もう 資料 を 配っ て いる から 私 の 方 で 申し上げ ます けれど 、 資料 の 四 枚 目 で ござい ます けれど も 、 これ は 兵庫 県 が 発表 し た 政治 資金 規正 報告 書 の 概要 と いう もの で ござい ます 。 その 一 年 分 だけ 資料 に し まし た けど 、 要する に 二 〇 一 三 年 、 一 四 年 、 一 五 年 の 三 年間 に 大阪 府 の あり ます スロット マシン の 製造 販売 業 の パイオニア 社 と いう ところ から 各々 三十 七 万 円 ずつ 、 三 年間 で 百 十 一 万 円 の 献金 を 受け取っ て おら れ ます 。   この 会社 と いう の は どう いう 会社 です か 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   機械 製造 の 会社 と 認識 を いたし て おり ます 。 <br/> ○ 大門 実紀史 君   私 は 、 恐らく 地元 の 中 小 企業 さん です か ね 、 何 か 直接 この カジノ の 今 推進 の 中心 に なっ て いる と か そう いう こと で は ない と 思う ん です けれど も 、 今後 これ が 無理やり 強行 さ れ て 、 大阪 で カジノ が 始まる と いう ふう に なり ます と 、 こう いう ゲーム 機器 の メーカー の 皆 さん は こぞっ て 参加 し たい と 、 こう なる よう な 流れ に あり ます の で 、 気 を 付け られる べき だ と いう こと だけ 、 西村 さん の 場合 は です ね 、 申し上げ て おき たい と いう ふう に 思い ます 。   もう 一 つ は 、 小沢 さん に 同じ こと を お 聞き いたし ます けれど 、 **政治** **資金** **規正** **法** ( `政治資金規正法` ) で 処理 し て 、 きちっと し て いる の は 当たり前 です 。 その 上 で 、 カジノ 関連 企業 です ね 、 パチンコ 、 パチスロ 、 ゲーム 機器 など の 企業 、 業界 など から パーティー 券 買っ て もらっ たり 政治 献金 を 受け た こと は 、 小沢 さん 、 ござい ます か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   適正 に 処理 を し て いる の は その とおり な ん です が 、 もう 既に 資料 に 付け て いただい て おり ます こと は 事実 で ござい ます 。 <br/> ○ 大門 実紀史 君   この 資料 、 収支 報告 書 を 用意 、 これ も 全部 じゃ なく て 一部 で ござい ます けれど も 、 パチンコ ホール の 全国 チェーン 展開 する ダイ ナム 社 です ね 、 これ から 二 〇 一 三 年 に 三十 万 円 、 二 〇 一 五 年 に 百 万 円 の パーティー 券 を 買っ て いただい て いる と 、 買っ て いる と いう こと です ね 。   この ダイ ナム と いう 会社 は どう いう 会社 です か 、 小沢 さん 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   パチンコ の ホール を 営業 し て いる 会社 だ と いう ふう に 承知 し て い ます 。 <br/> ○ 大門 実紀史 君   先ほど の 大阪 の 中 小 企業 と は 違っ て 、 この ダイ ナム と いう 会社 は 、 特に 都市 部 など の パチンコ ホール を たくさん 経営 し て き た パチンコ ホール 企業 の 大手 で ござい ます 。 なおかつ 今回 の カジノ を 積極 的 に 推進 し て き た 企業 で あり まし て 、 ダイ ナム の 会社 の ホーム ページ に は 、 投資 家 向け 情報 と いう こと で 、 カジノ ・ ジャパン へ の 挑戦 を 表明 し ます と いう こと で 、 とにかく マカオ 、 シンガポール 、 そう いう もの に 関心 持っ て いる 云々 等 、 当社 グループ は その 事業 の 柱 に なっ て いく と いう よう な こと を 宣言 さ れ て おり ます し 、 ダイ ナム ジャパン の 佐藤 取締 役 会 議長 と いう ん です か ね 、 は 、 この アジア 各国 の カジノ の 運営 会社 と 度々 接触 さ れ て おり まし て 、 これ も 新聞 報道 に も 出 て いる ところ で ござい ます 。   後 で お 話 も し て もらい ます けれど も 、 外国 資本 と ダイ ナム ジャパン は 大変 、 この カジノ が 解禁 さ れる と いう こと を 見込ん で ずっと 折衝 し て き て 、 なおかつ 国会 で この 法案 が 通る こと を 望む と いう こと も はっきり 掲げ て こ られ て おり ます し 、 ちなみ に 、 この 法案 が 衆議 院 で 可決 さ れ た とき に 香港 市場 で 何 が 起き た か と いい ます と 、 この ダイ ナム ジャパン の 株 が 急 上昇 し た と いう こと が ある わけ で あり ます 。 ダイ ナム が カジノ に 食い込ん で いく だろう と 、 株 は 買っ て おい た 方 が いい と いう こと で 急 上昇 し た 。 香港 市場 です から ね 、 なかなか 報道 は なかっ た わけ です けれど も 、 目立た ない ん です けれど も 、 急 上昇 し て いる と 、 こう いう 会社 で あり ます 。   これ は 、 小沢 さん ね 、 先ほど の 西村 さん の 中 小 企業 と は 違っ て 、 もう 直接 、 **カジノ** **推進** **、** **カジノ** **解禁** **法** ( `特定複合観光施設区域の整備の推進に関する法律` ) を 求め て き た もう 中心 の 企業 グループ から こう いう パーティー 券 を 買っ て いただい て 、 個々 で 議員 立法 で 提案 し て 、 カジノ は 前 に やる べき だ 、 進める べき だ と 。 これ は 、 判断 する の は いずれ 司法 の 場 か も 分かり ませ ん けれど も 、 大変 、 大変 危ない 話 じゃ ない ん です か 。 大変 国会 議員 と し て 疑わ れる 話 じゃ ない ん です か 。 疑惑 が 持た れる よう な 話 じゃ ない ん です か 。 そう いう こと を 自覚 さ れ て やっ て こ られ まし た か 、 小沢 さん 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   こう いう 形 で 適正 に 処理 を し て 、 そして 、 私 自身 も ある 意味 で は そう いう 思い が ない もの です から 、 きちっと 処理 を さ せ て いただい て いる わけ で ござい ます 。   ここ の 佐藤 会長 、 議長 って 今 おっしゃい まし た けれど も 、 最近 ずっと 日本 に いらっしゃい ませ ん の で 、 こう し た 話 、 私 は 全く し て おり ませ ん 。 で あり ます の で 、 元々 パチンコ ホール の 会社 だ と いう こと は 事実 で あり ます けれど も 、 この カジノ に 関し て 私 が 頼ま れ て 云々 など と いう こと は 全く ござい ませ ん 。 <br/> ○ 大門 実紀史 君   この 佐藤 社長 と お 会い に なっ た こと も ない です か 。 あっ 、 議長 です か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   いえ 、 会っ た こと は もちろん あり ます けれど も 、 最近 は あり ませ ん と 先ほど 申し上げ た わけ で あり ます 。 <br/> ○ 大門 実紀史 君   これ 以上 は 、 私 は と いう より も 、 まあ 場合 に よっ て は 国会 で しかる べき 場 が 必要 か も 分かり ませ ん けれど 、 司法 の 方 で 調べ て もらう こと に なる 疑惑 の ある 、 今日 の 時点 で は 申し上げ て おき たい と 思い ます けれど 。   この 前 も 申し上げ まし た けど 、 決して 与党 の 議員 の 皆 さん も 全て が 最初 から カジノ が いい と 、 ＩＲ 必要 だ と 思っ て いる 方 、 余り 、 そんな に たくさん いらっしゃら なかっ た です よ ね 、 最初 は 。 ＩＲ と いう 観光 戦略 で 、 名前 だけ 貸し て くれ と 言わ れ て 議員 連盟 に 入っ た 方 と か 、 民主 党 も おら れ た ん です けど 。 ただ 、 非常 に 中心 で 頑張っ て こ られ て 、 どう し て も これ を 粘り強く やっ て こ られ た 方 と いう の は 、 国会 で も 取り上げ まし た けれど 、 パチンコ ・ チェーン ストア 協会 と か 、 ずっと 一貫 し て そう いう 業界 と 、 政治 分野 アドバイザー と か に なっ て やら れ た 方 が 多い もの です から 。 それ と 、 地方 で カジノ を やり たい と いう 声 を 受け て 、 地元 の 企業 の 声 を 受け て 推進 と いう 方 も いらっしゃい ます けれど も 、 大体 そう いう 何 か 支援 し て いただい て いる 方々 が いる から 頑張っ て いる と 。   ちょっと ほか の 問題 と 違っ て 、 これ は やっぱり 賭博 問題 です から 、 なかなか 本当 は 余り 関わり たく ない と いう 方 が 多い ん です けれど も 、 そう いう こと が ずっと この 問題 取り上げ て き て 色 濃く 感じる もの です から 、 やっぱり 小沢 さん の 場合 は 一番 、 この ダイ ナム と いう カジノ 推進 の 中心 の 企業 グループ と こう いう 関係 に ある と いう の は 大変 私 は まずい と 思い ます 、 こう いう 議員 立法 を 提案 する 方 と し て 。   実 は 、 パーティー 券 と か 政治 献金 と いう の は 、 今 は なかなか 、 パーティー 券 その もの も まとまっ て 買わ なきゃ 名前 も 出 ませ ん し 、 分散 さ れ て い ます から ブラック ボックス な ん です けれど 、 私 は やっぱり 株 の 問題 、 カジノ 関連 株 、 あと 、 今 大手 の 銀行 なんか も 絡ん で カジノ 投資 ファンド と いう の が でき 始め て おり ます 。 投資 し て リターン と いう よう な こと から 、 決して 政治 献金 と か パーティー 券 買っ て もらう よう な 原始 的 な 手法 だけ で マネー が 動い て いる わけ で は ない ん です よ ね 。   そんな こと も 含め て 、 もちろん インフラ 整備 で も お 金 が 動く と 思い ます から 、 これ は 巨額 の マネー が 動い て いる と いう こと を 前 から 見 て き て いる ところ で ござい ます けれど 、 我が 党 も こう なっ たら 全面 的 に この 調査 に 入ろう と いう ふう に 思っ て おり ます けれど も 、 本当 に 小沢 さん 、 少なく とも 今 の 時点 で 受け取っ た パーティー 券 代 と か 表面 化 し て いる 分 だけ で も お 返し に なる べき じゃ ない です か 。 いかが です か 。 立法 の 提案 者 と し て 、 それ ぐらい 当たり前 だ と 思い ます が 、 どう です か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   検討 は し たい と 思っ て おり ます が 、 後ろめたい 思い は ない もの です から 、 この 後 、 何 と いい ます か 、 李下 に 冠 を 正さ ず と 、 こう いう 話 も あろう か と 思い ます の で 、 検討 し たい と は 思い ます 。 <br/> ○ 大門 実紀史 君   今日 は もう その 指摘 だけ に とどめ て おき ます 。   午前 中 も あり まし た けれど 、 例 の 、 例 の と いい ます か 、 違法 性 の 阻却 の 問題 です ね 。 午前 中 、 上月 さん から も 十 二 月 七 日 の 、 お 手元 の 資料 の 一 枚 目 です ね 、 前 も 配り まし た が 、 十 二 月 七 日 、 私 の 質疑 に 合わせ て 法務 省 が 出し て くれ た この 文書 です 。 誤解 の ない よう に と いう こと で 上月 さん 、 質問 し て いただい た ん です けど 、 私 も 誤解 の ない よう に 整理 し 直し たい と 思っ て おり ます 。   前回 の 答弁 を 改めて 整理 し て もらい たい と 、 確認 し たい と 思う ん です けれど 、 まず 、 法務 省 は 、 競馬 や 競艇 など の 公営 競技 に 関わる 特別 立法 に 当たっ て は 、 この 文書 に ある よう な 立場 から 意見 を 言っ て き た と いう こと で いい です か 、 まず 。 <br/> ○ 政府 参考 人 （ 加藤 俊治 君 ）   お 答え 申し上げ ます 。   基本 的 に 御 指摘 の とおり で ござい ます が 、 既存 の 公営 競技 に 係る 特別 法 の 立法 に 当たっ て は 、 ここ に 書か れ て ある よう な 点 に 着目 し て 意見 を 申し述べ て き た と いう こと で ござい ます 。 <br/> ○ 大門 実紀史 君   それ で 、 こう いう 御 意見 を 述べ て こ られ て 、 結果 と し て 競馬 や 競艇 など の 運営 主体 と いう の は 官 又 は それ に 準じる 団体 と さ れ て き た と 、 まあ 結果 と し て です けど ね 。 そう いう 意見 を 基 に 検討 し て 、 結果 と し て 競馬 、 競艇 など の 運営 主体 は 官 又 は それ に 準じる 団体 と さ れ て き た と いう こと で よろしい です か 。 <br/> ○ 政府 参考 人 （ 加藤 俊治 君 ）   お 答え 申し上げ ます 。   現行 の 公営 競技 等 の うち 、 競馬 の 施行 主体 は 日本 中央 競馬 会 あるいは 都 道 府 県 又 は 指定 市 町 村 と さ れ て おり 、 競輪 、 小型 自動 車 競走 及び モーターボート 競走 の 施行 主体 は 都 道 府 県 、 指定 市 町 村 等 と さ れ て おり ます 。 これ ら の 施行 主体 は 、 政府 が 全額 出資 する 特殊 法人 で ある か 又 は 地方 公共 団体 に 限ら れ て おり 、 その 意味 で 官 又 は これ に 準じる 団体 に 限ら れ て いる と いう 議員 の 御 指摘 は その とおり で あろう と 認識 し て おり ます 。 <br/> ○ 大門 実紀史 君   そう です ね 。 午前 中 も 加藤 さん は 正確 に 答弁 さ れ て おり まし て 、 上月 さん の 質問 に 対し て です ね 、 八 項目 で 意見 を 申し上げ て き て 、 その 結果 、 現に ある こと の 例示 が この 括弧 の 中 に ある もの です と 、 その とおり です よ ね 。 今 まで 結果 と し て 公営 主体 しか 認め て こ なかっ た と 、 ただし 、 最初 から 公営 主体 じゃ なきゃ 駄目 です よ と いう の は どこ に も 書い て い ない と 、 それ は その とおり です ね 。 検討 し た 、 意見 を 申し上げ た 結果 と し て 公営 主体 しか 認め られ て こ なかっ た と 、 今 あっ た とおり で ござい ます 。   問題 は 、 今回 の **ＩＲ** **、** **カジノ** **解禁** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) は 民営 賭博 で ござい まし て 、 歴史 上 初めて 民営 賭博 を 認める か どう か と 、 その 違法 性 が 阻却 できる か どう か と いう の が 最大 の 焦点 の 一 つ で あり ます 。 あと は 、 もう 具体 的 に 実施 法 が 出 て こ ない と 法務 省 も 意見 の 言い よう も ない と いう ふう に 思い ます し 、 結果 と し て どう なる か 別 です から ね 。   ただ 、 ちょっと 一般 論 と し て 、 なぜ そもそも この 八 項目 な の か と いう こと も 含め て 、 一般 論 と し て ちょっと お 聞き し たい ん です けど 。   まず 最初 の 目的 の 公益 性 です ね 、 これ が 八 項目 の 一 番 目 で あり ます 。 これ 、 収益 の 使途 を 公益 性 の ある もの に 限る こと を 含む と 。 これ 、 日本 語 を その まま 取り ます と 、 収益 の 使途 を 公益 性 の ある もの に 限る と 、 これ を 含ん で もらわ なきゃ 困る ん だ と 。 だ から 、 これ 例示 じゃ なく て 、 例えば じゃ なく て 、 そう いう ふう に 日本 語 で 読め ば 読める わけ です ね 。 もう それ しか 読め ない と 思う ん です けれど も 。 です から 、 民営 で は どう 考え て も 当然 自分 たち の 企業 利益 出さ なきゃ いけ ませ ん から 、 公益 性 の ある もの に 限る と いう こと は 不 可能 じゃ ない か と いう こと を 申し上げ た わけ です ね 。   二 つ 目 の 、 これ は 後 で 申し上げ ます けど 、 この 背景 は 、 官 又 は それ に 準じる 団体 に 限る など です から 、 限る など と いう の も 変 な 話 な ん です けど 、 一応 、 矛盾 し た 表現 で は あり ます けれど 、 官 又 は それ に 準じる 団体 に 限っ て き た けれど 、 それ が 全て で ない と いう の は 、 おっしゃる よう に 例示 的 に 解釈 でき なく も ない と いう ふう に 思い ます ね 。   三 番 目 の 収益 の 扱い 。 これ は もう 完全 に 、 これ は もう 業務 委託 の 話 です から 、 主体 は 公 で 、 公的 主体 で 、 業務 を 民間 委託 し た 場合 、 今 の 競馬 あるいは 競輪 です ね 、 そう いう こと を 想定 、 その とき の 話 だ から 、 これ は 民間 主体 など 想定 し て い ない そもそも 話 な ん です ね 。   四 点 目 の 射幸 性 の 程度 です けれど も 、 これ は ちょっと 法務 省 に 聞き たい ん です けど 、 これ が 意味 する ところ と いう の は 、 これ 一般 論 で いい です よ 、 射幸 性 の 程度 と いう こと が 要件 の 一 つ に なっ て いる と いう の は 、 射幸 性 を 異常 に 高く し て は いけ ない 、 コントロール できる よう に し なさい と いう 意味 が この 射幸 性 の 程度 と いう ところ に 含ま れ て いる と いう こと で いい ん でしょう か 。 <br/> ○ 政府 参考 人 （ 加藤 俊治 君 ）   お 答え を 申し上げ ます 。   賭博 罪 が 犯罪 と さ れ て いる 趣旨 の 中 に は 、 その 賭博 と いう 行為 が 射幸 心 を あおる 行為 で ある と いう こと が 含ま れ て いる わけ で ござい ます 。 その ため 、 特別 法 に おい て その 賭博 に 当たる 行為 を 許容 する と いっ た 場合 に は 、 射幸 性 の 程度 、 射幸 性 が 高い 、 低い と いっ た 要素 が 刑法 と の 整合 性 を 保つ 上 で 一 つ の 考慮 要素 と なる と いう こと で ここ に 挙げ させ て いただい て いる もの で ござい ます 。 <br/> ○ 大門 実紀史 君   発議 者 の 岩屋 さん に お 聞き し たい と いう ふう に 思い ます 。   カジノ は 一 晩 で 何百 万 もうける 、 ほとんど レア ケース です けど 、 そう いう 場合 も ある し 、 ほとんど そんな もうから ない ん です けど 、 する 場合 は 何十 万 、 何百 万 と する 。 だ から みんな カジノ に 行っ ちゃう と いい ます か 、 と いう ところ が ある わけ です ね 。 それ で 百 億 以上 すっ ちゃっ た と いう 人 も 日本 の 経営 者 で いる わけ です けれど も 、 それ が カジノ な ん です よ ね 。 パチンコ と は 違う 。 パチンコ の 場合 は 、 余り に も 射幸 性 が 高く なっ て 、 爆裂 何 と か 機 と いう の を 警察 が 指導 を し て 射幸 性 を 落とさ せ まし た けれど 、 そう いう こと が できる 世界 と 違っ て 、 だ から 賭博 、 だ から カジノ と いう こと に なっ て いる ん です よ ね 。   したがっ て 、 民間 の 賭博 場 、 民営 賭博 に この 射幸 性 を コントロール さ せる と いう こと その もの が 難しい ん じゃ ない か と 思う ん です よ ね 。 いかが です か 。 どう やっ て やる ん です か 。 民間 業 者 に 射幸 性 コントロール しろ 、 抑えろ と 言う の は できる ん です か 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   射幸 性 と いう もの を どう いう ふう に 考える か と いう こと に つい て は いろんな 考え 方 が ある ん だろう と 思い ます 。 また 、 それ も ゲーム の 種類 等 に よっ て 射幸 性 と いう もの を どう 考え たら いい の か と いう の は 様々 な 意見 や 議論 が ある ん だろう と 思い ます 。 今 の 例えば 公営 競技 や 富くじ 等 に おい て も 、 例えば キャリー オーバー で 賞金 が 十 億 を 超す なん と いう ケース も 間々 出 て き て いる わけ で ござい まし て 、 そう いっ た こと と 射幸 性 と いう の は どう 考える べき か と いう 議論 も ある ん だろう と 思い ます 。   カジノ の 場合 は 、 先生 は ちょっと 極端 な 例 を 今 挙げ られ まし た が 、 基本 的 に は やっぱり 時間 消費 型 の 一 つ の エンターテインメント な ん だろう と 私 は 思っ て おり ます の で 、 しかも 、 大体 、 客 に 対する 還元 率 と いう こと で いい ます と 、 他 の ゲーミング に 比べれ ば 最も カジノ の 還元 率 が 高い と いう ふう に さ れ て おり ます の で 、 その 射幸 性 と いっ た 観点 から 見 て も 、 私 は 特段 問題 に さ れる ゲーミング の 業態 、 姿 で は ない の で は ない か な と 考え て い ます 。 <br/> ○ 大門 実紀史 君   射幸 性 が いろいろ と 言わ れ ちゃっ たら もう 何 が 何 だ か な ん です けど 、 射幸 性 は 、 今 まで 言わ れ て き た とおり 、 何 と いい ます か ね 、 当たり外れ の 格差 の 大き さ です よ ね 。 だ から 、 する とき は むちゃくちゃ する と 。 その 代わり 、 もうかる とき に は むちゃくちゃ もうかる 。 ただ 、 その 確率 は ほとんど 少ない と いう ところ の 確率 の 問題 の こと を 射幸 性 で 表し て いる わけ です ね 。   それ を コントロール できる の か と いう こと です けれど も 、 コントロール する と いう 世界 じゃ ない ん です よ ね 。 ルーレット で どう やっ て コントロール する の か と 。 スロット マシン だ と 、 あの 機械 を 調整 し て 出る 確率 でき ます けど 、 ルーレット は 、 あれ は 回す だけ です から 、 どう やっ て 射幸 性 を 落とす の か と いう こと と か に なる わけ で 、 賭博 の 世界 は そもそも 無理 な ん です よ ね 、 コントロール は 。 だ から 、 法務 省 は そこ の ところ を 非常 に 留意 し て いる と いう こと で あり ます 。   実 は 、 この 文書 その もの は 、 私 も 、 最初 は 十 二 月 七 日 、 私 が 法務 省 に 要求 し た から この 括弧 付き の 、 注釈 付き の もの を 出し て くれ た の か な と 、 前回 の 審議 の とき まで そう 思っ て い た ん です 。 しかし 、 これ は そう じゃ なかっ た ん です ね 。   実 は この 法務 省 の 八 要件 、 前回 の 答弁 で 、 私 が 最初 に 示さ れ た の は いつ です か と 聞い たら 、 二 〇 一 三 年 の 十 一 月 二十 日 の 衆議 院 内閣 委員 会 で の 答弁 でし た と いう ふう に おっしゃっ た ん です ね 。 その 議事 録 も 前回 配付 し た ん です けれど も 、 実 は それ は 虚偽 だっ た わけ で あり まし て 、 最初 に 示さ れ た の は その 一 年 半 前 の 二 〇 一 二 年 四 月 六 日 、 資料 の 二 枚 目 に あり ます けれど も 、 当時 の 民主 党 の 内閣 、 法務 、 国土 交通 の 合同 部門 会議 、 ここ で 実 は 同じ もの 、 ほとんど 同じ もの が 示さ れ て い た わけ です ね 。 この 前 の 十 二 月 七 日 が 最初 じゃ なかっ た ん です 。 既に ここ で もう 示さ れ て い た ん です 。   この 二 〇 一 二 年 四 月 六 日 の 民主 党 の 内閣 、 法務 、 国交 の 三 部門 合同 会議 と いう の は どう いう 会議 だっ た か と いい ます と 、 当時 、 民主 党 を 中心 に 今 の よう な **ＩＲ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) が まとめ られ て 、 古賀 一成 さん と か です ね 、 会長 で やら れ て き て 、 その 当時 の **カジノ** **法** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) 、 **カジノ** **解禁** **法** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) 、 **ＩＲ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) が あっ た ん です よ ね 。 それ を 民主 党 の 中 で 部門 会議 で 検討 する と いう こと が 行わ れ て 、 三 部門 合同 で 検討 、 そして この とき は 政府 の ヒアリング 、 政府 の 意見 を 聞こう と いう 会議 だっ た ん です 。 その 三 部門 会議 で 法務 省 の 意見 を 聞こう と なっ た とき に 実 は この 文書 が 、 もう その とき に 法務 省 の 考え 方 と し て 文書 と し て 配ら れ た と いう こと で ござい ます 。   それ が 資料 三 です ね 。 これ が 平成 二十 四 年 四 月 六 日 です ね 。 ほとんど 一緒 な ん です けれど も 、 違う の は 、 一番 最後 の 「 その 方針 に 変更 は ない 。 」 と いう 言い 方 を し て いる ん です ね 。 十 二 月 七 日 の 場合 は 、 一番 最後 は 「 同様 で ある 。 」 と 。 この 意味 と か 、 あるいは 、 午前 中 、 上月 先生 が 、 これ 誤解 を 受ける と 、 よく 分から ない と おっしゃっ た 意味 は 、 実 は 今回 の 審議 に 当たっ て 出し た ん じゃ なく て この 二 〇 一 二 年 の 四 月 六 日 の とき に 出し た 文書 だ から 、 ちょっと 今 読む と 誤解 と いう か 、 意味 が 分から ない ところ が いろいろ ある わけ で あり ます 。   その 意味 を 説明 いたし ます けれど も 、 当時 、 民主 党 の 法務 部門 の 座長 さん だっ た 松野 信夫 さん 、 今 熊本 で 弁護 士 やっ て いらっしゃい ます よ ね 。 大変 正義 感 の 強い 弁護 士 さん で 、 当時 民主 党 の 中 で 出 て き た カジノ の 法案 も 、 私 たち と 同じ 考え 方 でし た から 、 あれ は 良く ない と いう ふう に おっしゃっ て い た 方 で あり ます 。 その 松野 信夫 さん 、 元 参議 院 議員 が 当時 民主 党 の 法務 部門 の 座長 さん を やっ て いらし た ん です ね 。 昨日 も 電話 で 直接 お 話 を 聞き まし た 。   法務 省 は この とき に 、 二 〇 一 二 年 の 四 月 六 日 の この 会議 で 明確 に 民営 賭博 は 難しい と いう 見解 を 示し た と 明言 を さ れ て おり ます 、 当時 の この 民主 党 の 法務 部門 の 責任 者 、 座長 さん だっ た 松野 さん が 、 この とき は 法務 省 は 民営 賭博 は 難しい と の 見解 を 示し た と 明確 に 教え て いただき まし た 。   法務 省 、 そう じゃ なかっ た ん です か 。 <br/> ○ 政府 参考 人 （ 加藤 俊治 君 ）   お 答え を 申し上げ ます 。   突然 の お 尋ね で ござい ます の で 、 二十 四 年 の 四 月 の 会議 で 具体 的 に いかなる 発言 を し た か 、 法務 省 の 職員 が いかなる 発言 を し た か と いう こと は 確認 し なけれ ば 正確 に お 答え する こと は でき ませ ん が 、 当時 お 出し し て いる 資料 も 、 現在 委員 会 に 提出 、 大門 委員 に 提出 を し て おり ます 資料 と 同種 の もの で ござい ます 。 この 趣旨 も 現在 私 が 説明 し て いる の と 同 趣旨 で ござい ます の で 、 その 当時 から 法務 省 の 立場 、 考え 方 と いう もの は 変わっ て い ない と いう ふう に 申し上げる こと が でき ます 。   以上 で ござい ます 。 <br/> ○ 大門 実紀史 君   いや 、 変わっ て いる ん です よ 。 変わっ て いる ん です よ 。   そもそも 私 自身 も 最初 、 カジノ の 問題 を 国会 で 取り上げる とき に 一番 最初 に 勉強 さ せ て もらっ た の は 、 法務 省 の 方々 、 検事 さん です ね 。 賭博 罪 の 意味 と か 、 百 八十 五 条 、 百 八十 六 条 の 成り立ち と か 、 三十 五 条 と どう 関わる か と か です ね 。 最初 、 法務 省 の 特に 検事 さん に 私 は 勉強 さ せ て もらっ て 、 教え て もらっ て この 問題 を 取り上げ 始め た ん です 。   最初 の とき は 、 もう 別 に 今更 その とき の 人 の 名称 と か 出す こと は やり ませ ん けれど 、 明確 に 法務 省 は 賭博 の 解禁 に つい て は 大変 厳しい 姿勢 を 取っ て おら れ まし た し 、 カジノ は そう 簡単 に 認め ない し 、 民営 賭博 と いう の は もう 想定 外 です と いう よう な こと 。 あるいは 、 当時 いろんな こと が あっ た ん です よ 、 構造 改革 特区 で の 民営 賭博 あるいは 地方 で の 賭博 が でき ない か 、 あるいは 公営 ギャンブル 、 今 ある 競馬 と か 競輪 も 民間 で でき ない か と いう よう な こと いろいろ あっ た 中 で 、 そう いう よう な 中 で も 、 民間 賭博 は 、 民営 賭博 は 無理 です と 、 想定 外 です と いう こと を ずっと 私 は 聞い て き た もの です から 、 最初 から この 話 は 非常 に 違和 感 あっ た ん です けど 、 この 違和 感 が 松野 さん に 昨日 聞い て よく 分かっ た わけ で あり ます 。 当時 は 明確 に 民間 賭博 を 法務 省 は 否定 さ れ た と 、 それ を 複数 の 部門 会議 です から 、 一 対 一 じゃ ない です から 、 部門 会議 の 責任 者 が おっしゃっ て いる わけ で ござい ます 。 です から 、 この 当時 の 文書 に わざわざ 「 その 方針 に 変更 は ない 。 」 と 強く 、 強く 語尾 に 書い た の は そう いう こと が ある わけ な ん です ね 、 今 と の 違い は 。   なおかつ 、 なぜ 括弧 を 付け て 解説 し た の か と 。 一 個 一 個 、 収益 は どう の こう の と 、 官 又 は 官 に 準じる と 、 わざわざ 個々 で 何 で こう いう 括弧 付け を し た か と いう と 、 それ は 、 当時 の ＩＲ 、 カジノ 推進 の 特に 民主 党 の 議員 さん に 対し て 、 民営 は 無理 です よ と 、 民営 賭博 は 無理 です よ と 分かっ て もらう ため の 解説 と し て 書き込ん だ わけ で あり まし て 、 それ が そもそも この 括弧 付け の 意味 な ん です よ ね 、 意味 な ん です よ ね 。   です から 、 先 ほど 申し まし た 、 収益 の 使途 を 公益 性 の ある もの に 限る こと を 含む と いう 、 これ を 、 強い 表現 を 入れ た の は 、 そう いう こと を やろう と し て いる 当時 の 特に 民主 党 の 内閣 部門 、 内閣 の 方 は やり たがっ て い まし た から 、 その 人 たち に 無理 です よ と 言う ため に 強い 表現 を 入れ た わけ で あり ます 。 「 （ 官 又 は それ に 準じる 団体 に 限る など ） 」 も 、 まあ 頭 から 限る と 言っ ちゃう と 、 そもそも 目的 の 、 運営 主体 の 性格 など 言わ なく て 、 官 又 は 民 に 限る と 最初 から 書け ば いい わけ です から 、 そう なる と 身 も 蓋 も ない ん で 、 一応 「 など 」 を 付け た と 。 だ けれど も 、 これ は 分かっ て くれ と いう こと です ね 。   です から 、 この 文書 の 意味 は 、 そんな 一般 的 な 話 で は なく て 、 法務 省 と し て は 民営 賭博 に つい て 厳しい 考え 方 を 持っ て い ます と いう こと を 当時 の 民主 党 の 三 部門 会議 で 示す ため に 作ら れ た 文書 と いう の が 最初 な ん です 。 それ が たまたま 、 どう いう 考え 方 な の と いっ たら 、 十 二 月 七 日 、 私 の 質問 に 対し て ぱっ と 出し て き まし た けど 、 あれ が 初めて で は なかっ た と いう こと で あり ます 。   背景 も 今 と 違い まし て 、 二 〇 一 二 年 、 この とき は 民主 党 政権 で あり ます し 、 この とき の 法務 大臣 は 、 今 も 頑張っ て おら れ ます 小川 敏夫 さん で ござい ます 、 弁護 士 さん です ね 。 やはり 正義 感 の 強い 、 カジノ に は 反対 の 弁護 士 さん で 議員 さん で あり ます し 、 民主 党 の 法務 部門 と いう の は 元々 カジノ 反対 が 多かっ た ん です ね 。 当時 、 この とき も 、 この 前 申し上げ まし た 、 被災 地 に カジノ と いう 話 が あっ て 、 私 が 質問 で やり まし た けど 、 それ が 最後 阻止 さ れ た 、 潰さ れ た の は 、 この 民主 党 の 部門 会議 で 駄目 だ と いう こと に なっ て 潰れ た わけ で あり まし て 、 そう いう 、 民主 党 の 法務 部門 は きちっと し た 方 が 多かっ た と 。 なおかつ 、 民主 党 政権 で あり 、 法務 大臣 が 小川 敏夫 さん だっ た と 。 だ から 、 今 と 違っ て 、 法務 省 は 自分 たち の 思う こと を ずばっ と 、 ずばっ と 書く こと が でき た と いう こと で あり ます 。   そう いう こと で 思う と 、 上月 さん が 誤解 する よう な こと 、 何 が 言い たい ん だ と おっしゃっ た の は 、 もう その 部分 は その とおり で あり まし て 、 実 は これ が 二 〇 一 二 年 四 月 の その とき の 状況 下 で 書か れ た 文書 と 思う と すっきり と 分かる わけ で あり ます 、 要する に 民営 賭博 は 駄目 です よ と いう こと で 。   もう 一 つ は 、 当時 も プログラム 法 と いい ます か 基本 法 だっ た ん です ね 。 今 と 同じ だっ た ん です ね 。 今 と 同じ だっ た ん です ね 。 その 枠組み に 対し て 、 当時 の 法務 省 は 、 プログラム 法 の 段階 で 民営 、 民間 賭博 だ から 駄目 です と いう 意見 を 表明 し た と いう こと で あり ます 。 実施 法 が 出 て くる まで じゃ ない ん です ね 。 民営 賭博 その もの が 無理 です よ と いう こと を 当時 法務 省 は 公 の 政党 の 三 部門 の 大きな 会議 の 場 で 示さ れ た と いう こと で あり ます 。   だ から 、 それ で いき ます と 、 今回 も 、 本来 なら ば 今 の 段階 で 法務 省 は 民営 賭博 無理 です よ と いう こと を 表明 さ れる べき な ん です ね 、 本当 は 、 です よ ね 、 と 思い ます 。 それ は やっぱり 、 法務 省 の この 法 の 判断 の 、 違法 性 の 阻却 に 対する 判断 の 姿勢 が 、 やっぱり 一貫 性 が なきゃ 駄目 です よ ね 。 どこ か で 拡大 解釈 し たり し て い たら 駄目 です よ ね 。 その 一貫 性 も 問わ れ ます し 、 この 法 案 の 提案 その もの に 関わる 根本 問題 だ と 私 は 思う わけ で あり ます 。 実施 法 を 待っ て じゃ なく て 、 法務 省 は 今 ストップ 掛け なかっ た と いう こと に なる わけ で あり ます ね 。 二 〇 一 二 年 の 当時 は 掛け た の に と いう こと に なる わけ です 。   いずれ に せよ 、 それ は 恐らく 加藤 さん も その とき に は 携わっ て おら れ なかっ た し 、 先ほど の 名簿 の メンバー で いき ます と 、 内藤 惣一郎 さん 、 渡邊 真知子 さん です か ね 、 同じ 刑事 局 です けれど も 、 そう いう 方々 が 実際 に 意見 表明 さ れ て 、 それ を 座長 の 松野 さん 含め て たくさん の 民主 党 の 議員 が 聞い て い た と いう こと に なる わけ で あり ます 。   したがっ て 、 これ 大変 大事 な 問題 です の で 、 委員 長 に お 願い し たい ん です けれど も 、 この 法 の 基本 的 な 違法 性 の 阻却 の 法務 省 の 姿勢 に 関わる 重要 な 問題 です の で 、 当時 の 法務 省 の 刑事 局 の 内藤 さん 、 渡邊 さん 、 そして 民主 党 法務 部門 の 座長 だっ た 松野 信夫 さん 、 弁護 士 さん を 参考 人 と し て 委員 会 に 呼ん で いただい て 、 やっぱり 一番 法 の 根幹 の 判断 に 関わり ます の で 、 参考 人 と し て 呼ん で いただい て 意見 陳述 を 求め たい と 思い ます が 、 委員 長 、 いかが です か 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   後刻 理事 会 で 協議 いたし ます 。 <br/> ○ 大門 実紀史 君   と いう こと が この 文書 の 意味 だっ た と いう こと で あり ます 。 これ を おい て おい て 先 の 議論 は ない だろう と 思い ます の で 、 きちっと し た 参考 人 の 意見 陳述 を 求め たい と 、 はっきり さ せ たい と いう ふう に 思い ます 。 法務 省 に とっ て も 大事 な こと です ね 。 法務 省 が 途中 で 政治 的 な 状況 が 変わっ た から と いっ て こう いろんな 判断 を 変え たら 駄目 です よ 、 法務 省 は 。   それ で 、 そう いう こと の 上 に 、 これ に 、 今 の こと に 関わる こと で 質問 し たい と 思い ます けれど も 、 元々 、 二 〇 〇 六 年 、 平成 十 八 年 の 当時 は 、 自民 党 の 、 ここ に あり ます けれど も 、 我が 国 に おけ る カジノ ・ エンターテイメント 導入 に 向け て の 基本 方針 と いう の が ある ん です けれど も 、 自由 民主 党 政務 調査 会 観光 特別 委員 会 カジノ ・ エンターテイメント 検討 小委 員 会 、 平成 十 八 年 六 月 十 六 日 と いう の が ある ん です ね 。 カジノ に 、 この とき は まだ ＩＲ と いう 言葉 は あり ませ ん でし た けれど も 、 カジノ 導入 の 基本 方針 と いう こと です と 。 この とき は 、 この 案 と いう の は 民営 賭博 じゃ なく て 公設 民営 で の 案 で ござい まし た 。 主体 は 地方 自治 体 で 、 運営 を 民間 業 者 に 委託 する と いう 仕組み でし た よ ね 。   岩屋 さん が 一番 御 存じ だ と 思い ます の で 、 なぜ この とき は 公設 民営 と いう 形 だっ た ん でしょう か 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   その とき の 小 委員 長 は 私 で ござい まし た 。 当初 、 私 ども は 、 今 まで の 公営 競技 の スキーム を この ＩＲ あるいは カジノ に 準用 する と いう か 適用 する こと が でき ない だろう か と いう 発想 で 勉強 を 開始 を いたし まし た 。 しかし 、 その 勉強 を すれ ば する ほど に 、 公営 競技 等 の スキーム と カジノ の 施行 者 が プレーヤー と なっ て 客 と 向き合う 、 ゲーム の リスク を 一時 的 に 取ら なく て は いけ ない と いう この ゲーミング は 、 やっぱり 種類 の 違う もの だ な と 。 これ を 公的 主体 に 任せる と いう の は 適切 で は ない と いう 判断 に 至り まし た 。   その 後 、 自民 党 だけ で は なく て 、 先生 おっしゃっ た よう に 、 当時 の 民主 党 さん も 、 あるいは 公明 党 さん も 維新 の 会 の 皆 さん も 一緒 に なっ て 超 党派 の 勉強 会 を つくっ て 、 ＩＲ 、 カジノ の スキーム は 公営 競技 の スキーム と は やっぱり 変え て いか なきゃ いけ ない よ ね と 。 その 代わり 、 民間 事業 者 に 施行 を 委ねる 以上 は 、 今 まで に ない 厳格 な 監視 管理 体制 を つくら なく ちゃ いけ ない よ ね と 。 それ が カジノ 管理 委員 会 と いう 発想 に つながっ て いく わけ です が 、 そう いう 議論 の 過程 を 経 て 今日 の 案 に なっ て いる と いう こと を 御 理解 いただけれ ば と 思い ます 。 <br/> ○ 大門 実紀史 君   その 後 変わっ て いっ た と いう の は 承知 し て おり ます けれど 、 なぜ 最初 は 公設 民営 だっ た の か と いう ところ な ん です ね 。   二 〇 〇 六 年 当時 と いう の は どう いう とき だっ た か と いい ます と 、 先ほど 申し上げ まし た 、 前回 も 申し上げ まし た けど 、 小泉 構造 改革 、 竹中 さん が 出 て き て がんがん やり合い まし た けれど も 、 あの 中 の 一 つ が 何 で も 官 から 民 へ と いう よう な こと が あっ て 、 財政 諮問 会議 で も 、 特区 で カジノ が でき ない か 、 あるいは 公営 ギャンブル を 民営 化 でき ない か と いう 議論 が あっ て 、 それ が ちょっと 落ち着い た とき だっ た ん です よ ね 。   つまり 、 二 〇 〇 三 年 から 二 〇 〇 七 年 に かけ て 、 **競馬** **法** ( `競馬法` ) と か **競輪** **法** ( `自転車競技法` ) と か の いわゆる 公営 ギャンブル が 最初 は 民営 化 でき ない か と いう 話 が 、 やっぱり 無理 だ と 、 公的 主体 で なきゃ 無理 だ と 、 業務 委託 だけ と いう こと に なっ て 、 あの とき に ずっと 法 が 改正 さ れ て 、 まさに 公設 民営 しか 駄目 です よ と いう の が あの とき の 法務 省 の 歯止め だっ た から 、 この 案 も 最初 は 公設 民営 化 から スタート し た の で は ない か と 。 当時 は そう いう 状況 でし た から 、 恐らく 、 法務 省 に 聞い て み たら 、 形 は 公設 民営 と か 、 そこ まで 具体 的 に 言っ た か どう か は 別 です けど 、 当時 の 議論 から いう と 、 最初 は とにかく 公設 民営 に し た の は やっぱり 法務 省 の その とき の 刑法 の 解釈 が あっ た から で は ない ん です か 、 最初 の 公設 民営 と いう 意味 は 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   必ず しも 正確 な 記憶 で は ない か も しれ ませ ん が 、 先生 おっしゃる よう に 、 構造 改革 特区 の とき に 、 もう 既に その 段階 で 、 うち の 地域 で カジノ を 合法 化 さ せ て もらえ ない か と いう 地域 から の 要請 が 随分 国 に 上げ られ て おり まし た 。 二 回 に わたっ て 当然 それ は 門 前払い を さ れ た わけ で ござい ます 。 と いう の は 、 刑法 の 一部 を 一定 の 地域 だけ 穴 を 空ける など と いう 措置 は それ は とり 得 ない と いう こと で 、 全 部門 前払い に し た わけ で ござい ます 。   そこ で 、 今 まで の 違法 性 が 阻却 さ れ た 仕組み で ある 公営 競技 と いう スキーム を 何 と か それ に 準用 する こと が でき ない か と いう 発想 で 、 そう いっ た 地域 の 要望 に も 応え たい と いう 思い も 私 ども あり まし て 勉強 を 開始 し た ん です が 、 その 後 は さっき 申し上げ た とおり で ござい まし て 、 これ は どう も 公的 主体 に 委ねる と いう の は 適切 で は ない な と いう 判断 に 至っ た と いう こと で ござい ます 。 <br/> ○ 大門 実紀史 君   私 、 共産 党 です から 、 当時 の 民主 党 の 中 の 議論 と いう の は 直接 知ら ない ん です けれど 、 この 点 に つい て も 聞い て みる と 、 民主 党 政権 に その 後 替わっ て き ます よ ね 、 その とき に 、 やっぱり 、 何 と いい ます か ね 、 当時 、 独特 の あの 、 役所 は 敵 だ みたい な 、 役所 は もう 駄目 な ん だ と いう の が 最初 の 民主 党 に わあっ と あっ た じゃ ない です か 。 ああ いう ところ で 、 この 公設 民営 だ と 天下り 先 に なる 、 何 に なる と いう こと で 、 もう 民間 で と いう の が 強く 出 て 民営 論 に なっ た と いう こと は 聞い た こと あり ます けれど も 、 私 は 、 実 は それ は そう か も 分から ない けれど も 、 それ は 分かり ませ ん 。   私 が 思う の は 、 ちょうど その 頃 から 、 二 〇 一 〇 年 前後 し て やっぱり スポンサー が 、 特に 海外 の 投資 家 等々 の スポンサー が 具体 的 に 現れ 始め た と 。 日本 の カジノ に 投資 を する 場合 、 公設 民営 なんて 面倒 くさい こと を 言っ て くれる と 投資 効率 が 落ちる と 、 やる なら ば 民営 で やら せ て もらい たい と いう よう な こと が 強く 働い て 、 そう いう 意見 を 言わ れ た 当時 の 外資 の 方々 も たくさん 、 資料 取っ て おり ます けれど 、 あっ て 、 そう いう こと から 、 彼 ら は 、 民営 なら ば 、 完全 民営 なら ば 投資 を する と 、 公設 民営 だ と やり にくい と いう よう な こと も あっ て 、 民営 、 民営 と 。 実 は これ が 一番 大きい 民営 の 方向 に 来 た 背景 で は ない か と 。   それ が 、 前回 の とき に 細田 さん が 私 の 質問 に 対し て 言わ れ た 、 世界 中 の 投資 家 が 投資 意欲 が 非常 に ある と おっしゃっ た の は 、 やっぱり もう そもそも 民営 と いう 枠組み を 前提 と し た 話 で は ない か と いう ふう に 思う ん です けれど 、 その 海外 投資 家 に とっ て 完全 民営 で なきゃ いけ ない ん じゃ ない です か 、 普通 に 当たり前 の よう な 話 に なっ て き た ん じゃ ない です か ね 。 いかが です か ね 。 <br/> ○ 衆議 院 議員 （ 細田 博之 君 ）   私 も 、 議連 の 会長 は 当時 の 民主 党 の 古賀 一成 さん の 後任 と いう こと で 、 政権 交代 に 伴っ て 議連 の 会長 を 受け た わけ で ござい ます 。 そして 、 様々 な 外国 の 、 例えば オイル マネー と か その 他 過去 に 実績 の ある 人 たち が 日本 を 訪れ て は 、 法律 が できる の か でき ない の か と いう よう な こと を 聞い て き まし た 。 彼 ら は 、 膨大 な ホテル 、 アミューズメント その 他 の もう 何 兆 円 に も なる よう な 絵 を 描い て き たり 、 あるいは 、 まあ それ より 多少 小さい けれど も 、 いずれ も シンガポール を 上回る よう な 膨大 な 投資 を し て 、 観光 振興 に も 日本 の 雇用 振興 に も 役 に 立つ と 、 そう いう 、 特に オリンピック を 目指し て い た 人 たち が 多かっ た です けれど も 、 そう いう 話 を し て おっ た わけ で ござい ます 。   それ は それ と し て 、 単なる 希望 と し て 来日 し て い た わけ で ござい ます が 、 その 中 で 、 当然 カジノ 施設 と いう の は 非常 に 大きな ノウハウ が あっ て 、 日本 に は ノウハウ が 余り ない わけ です ね 。 やり 始めれ ば 、 これ から ノウハウ を 従業 員 と か 管理 者 と し て 学ん で いか なきゃ なら ない 。 そこ の ノウハウ が まだ 行き届い て い ない の で 、 言わば 日本 企業 だけ に よる 運営 と いう の は 実際 上 難しい だろう と いう こと は 分かる わけ で ござい ます ね 。   したがっ て 、 投資 は 歓迎 する 、 しかし 、 その 投資 家 の 中 に は 日本 企業 と ジョイント で やろう と 。 それ は 、 訳 も 分から ない 日本 に 投資 し て も 、 それ で 本当 に 観光 客 が 日本 文化 の 中 で 溶け込ん で 喜ん で くれる か どう か は 、 じゃ 、 どう いう シアター を 造る か と か 、 どう いう ミーティング を やっ たら いい の か 、 どう いう 規模 で やっ たら いい の か 、 どこ に 造っ たら いい の か 、 これ は 日本 の 大きな 企業 と 連携 を し たい と いう こと で 来 て いる わけ で ござい ます 。 したがっ て 、 我々 は 、 やはり 大きな 施設 と し て は 民間 に よる ジョイント ベンチャー と いう こと に なる で あろう と 、 それ は やむ を 得 ない と 。   しかし 、 この 常習 賭博 罪 等 の 関係 で いえ ば 、 がっちり 、 従来 の 公営 ギャンブル と 同じ よう に きちっと 納付 金 を 取っ て 、 そして その 納付 金 は 、 ギャンブル 依存 症 は ともかく 、 日本 の 観光 振興 と か 、 特に 文化 の 面 で は まだまだ 整備 が 遅れ て いる わけ で ござい まし て 、 伝統 的 な 建築 、 建造 物 を 改築 し 、 そして 保存 する 事業 と か 、 あるいは 文化 を 担う 伝統 芸能 の 人 たち 、 あるいは オペラ に し て も オーケストラ に し て も 、 様々 な 芸術 分野 の 人 は 文部 科学 省 から ほとんど お 金 を もらわ ず に 、 もう 貧しい 俳優 と か たくさん いる わけ です ね 。   したがっ て 、 文化 の 振興 、 そして 観光 の 振興 に 役 に 立つ 、 そして 、 さっき ちょっと 最初 に 申し上げ た よう な 、 日本 中 の 観光 に つなげる よう な 、 お 金 を 納付 金 と し て 納め て もらっ て 、 それ を 活用 する と いう こと を 一 つ の 条件 設定 と し て 考え て いこう じゃ ない か 。 当然 、 しかし 、 それ で 利益 が 上がる 部分 に つい て 法人 税 が 掛かる と か 、 そう いう こと は 当たり前 の 話 で ござい ます が 、 そう いう こと に なっ て いる 、 そう いう 経緯 と いう か 実態 が ござい ます 。 <br/> ○ 大門 実紀史 君   とにかく 法務 省 が 今 まで の ずっと 取っ て き た 姿勢 を 変え ない 限り 、 法的 に 言え ば 私 は 民営 賭博 は 不 可能 だ と 。 それ は ちゃんと 、 さっき 言っ た 参考 人 を 、 開い て 明らか に し て もらい たい な と 思い ます けれど も 。   そして 、 実 は 外資 が やっぱり 鍵 を 握っ て いる と 。 その 外資 が 言っ て いる の は 、 決して 観光 客 の 話 じゃ ない ん です よ ね 。 今 いろんな 外資 の 名前 が 出 て い ます けれど も 、 ＭＧＭ リゾート ・ インターナショナル 、 マリーナ ・ ベイ ・ サンズ 、 あと 、 サンズ は 、 あれ です か ね 、 細田 さん と 懇談 さ れ た こと も ある ん です か ね 、 まあ 分かり ませ ん けれど も 、 いろんな 名前 出 て い ます けれど も 、 彼 ら が いろんな ところ で 言っ て いる の は 、 日本 に 海外 の お 客 さん が 来 て じゃ なく て 、 日本 人 が 持っ て いる 日本 の 金融 資産 が 大きい と 、 こう いう こと を 言っ て いる わけ な ん です よ ね 。 こう いう こと を 言っ て いる ん です ね 。   そこ の ところ と 、 ついで に 言っ て おき ます けれど も 、 先ほど 岩屋 さん が 、 外資 入っ たら どう なる ん だ と 、 ちゃんと 日本 に しかる べき 責任 を 果たす の か と 、 納税 の 義務 と か おっしゃっ た けれど も 、 今 納税 の 義務 を 果たさ ない の が 大 問題 に なっ て おり まし て 、 私 も 財政 金融 委員 会 の 中 で 、 そう いう 外国 資本 が 納税 の 義務 を 果たさ ない から 、 だ から 国際 課税 頑張ろう と いう こと で 麻生 大臣 と 一緒 に 提案 し て いる ぐらい で 、 今回 これ 、 外資 が 入っ て くる わけ でしょう 。 そんな の 、 納税 の 義務 なんか 、 もう 世界 中 の どこ か タックス ヘイブン で やる わけ です から 、 納税 の 義務 なんか 果たさ れる わけ が ない と 、 これ は 申し上げ て おき たい と 思い ます けれど も 。   話 を 戻し て 、 国際 資本 が 狙っ て き て いる の も 日本 の 金融 資産 で あり ます し 、 一番 驚い た の は 、 大阪 商業 大学 、 ある 意味 で は カジノ 推進 派 の 皆 さん の シンク タンク の よう な 大学 、 研究 所 で あり ます けれど も 、 その 谷岡 さん 、 谷岡 一郎 さん です か ね 、 学長 さん が 、 前回 の 資料 と か 参考 人 の 資料 に も あり まし た けれど も 、 この カジノ に よっ て 高齢 者 の たんす 預金 など 、 高齢 者 の たんす 預金 など が 、 世の中 に 出 にくい お 金 が 回り 始める 、 これ が 期待 さ れる と 。 もう 正直 な 方 です よ ね 、 あからさま です よ ね 。 さらに 、 マカオ で は 、 マカオ で は です よ 、 マカオ で は カジノ の 稼ぎ が 去年 、 二 〇 一 三 年 、 この とき だ と 三 兆 八千 億 円 あり まし た 、 これ は 売上げ で は あり ませ ん 、 収益 です と 。 つまり 人々 が 失っ た 金 の 合計 が 三 兆 八千 億 円 です と 。 これ 、 カジノ 推進 派 の 皆 さん の シンク タンク と 言わ れ て いる 大阪 商業 大学 の 学長 さん が 言っ て いる の で 、 私 たち が 人 の 金 を 巻き上げ て いる と 言っ て いる だけ じゃ なく て 、 皆 さん の シンク タンク が おっしゃっ て いる ん です よ 、 人々 が 失っ た お 金 です ね 。   ハイ ローラー と 呼ば れる ギャンブル 愛好 家 は カジノ の お 得意 様 です けれど も 、 この 種 の ギャンブラー だけ を 相手 に し て い て は 経営 が 安定 し ない 、 一般 の 方々 が 参加 できる 広い 裾野 が 必要 です と 。 一定 の 所得 と 貯蓄 を 持つ 分厚い 中間 層 、 今 は 大分 少なく なっ た と 思っ て い ます けれど も 、 が いる 日本 の 大 都市 圏 は 大変 魅力 ある 市場 です と 。   つまり 、 ギャンブラー だけ を 相手 に し て いる と 波 が ある 、 経営 に リスク が 伴う と 。 だ から 、 安定 収益 に なる の が 、 小銭 を 持っ て いる 、 一定 の 貯蓄 を 持っ て いる 高齢 者 と か 一般 の 家庭 だ と いう こと を おっしゃっ て いる わけ で あり ます 。 そう いう 方 が たくさん いる の は 地方 じゃ なく て 、 これ 都市 部 に なり ます から 、 それ で 大阪 だ の 横浜 だ の 、 話 が こう なっ て いる わけ で あり まし て 、 外資 も 言っ て いる し 、 皆 さん の シンク タンク の 大阪 商業 大学 の トップ も 言っ て いる の は 、 共通 する の は 、 今回 の ＩＲ で やろう と し て いる の は 、 日本 の 個人 金融 資産 、 これ を 出そう と いう こと な ん です ね 。 これ が 、 ある 意味 で 、 私 たち が 何 度 も 指摘 し て き た よう な 、 今回 の 、 きれい 事 ばっかり 、 バラ色 ばっかり 並べ ます けど 、 ＩＲ の 本質 で は ない か と いう ふう に 思う わけ で あり ます 。   昨日 、 鳥畑 先生 が おっしゃっ て い まし た けど 、 シンガポール と 日本 、 ＩＲ の 、 カジノ の 違い は ターゲット が 国内 だ と 、 鳥畑 先生 も 明確 に 分析 さ れ て おり まし た 。 過当 競争 が ある 、 ない と 言い ます けれど も 、 もちろん 国内 で は ない か も 分かり ませ ん けど 、 普通 なら アジア の ほか の カジノ と の 競争 が ある はず です よ ね 。 それ で 考える と 、 マカオ が あり シンガポール が あり 、 さらに その 人 たち を 日本 に 連れ て くる と いう の は かなり しんどい 話 です けれど も 、 そもそも 国内 が ターゲット だっ たら そんな 心配 は ない わけ です ね 。   実 は 、 経済 同友 会 、 関西 経済 同友 会 が 試算 し て いる 、 試算 が あり ます けれど も 、 時間 の 関係 で もう 結論 だけ 言い ます けれど も 、 カジノ の 収入 が 約 六千 億 です ね 。 ＩＲ 全体 の 八 割 は やっぱり カジノ で 稼ぐ と 。 それ を 、 大阪 商業 大学 は 、 大阪 に カジノ を 造っ た 場合 、 大体 来場 者 数 を 百 万 人 と いう こと を よく 出さ れ て おり ます の で 、 六千 億 を 百 万 人 で 割り ます と 一人 年間 六十 万 円 お 金 を 落とし て もらう 、 そう いう 計算 を 関西 経済 同友 会 が 出さ れ て おり ます 。 これ は リピーター を 想定 し ない と そう なら ない です ね 。   と いう こと で あり まし て 、 ＩＲ と いう と 家族 で 来 て ください と いう ふう に 何 か ふわっ と し た もの で おっしゃい ます けれど も 、 鳥畑 先生 が 指摘 さ れ た よう に 、 家族 で 来 て もらっ て 、 お 父 さん 、 お 母 さん 、 まず やっ て もらっ て 、 次 また 来 て もらう 、 リピーター に し て いく と いう こと じゃ ない か と 、 まさに 依存 症 製造 計画 じゃ ない か と 私 は 前 も 質問 で 言い まし た けど 、 そう いう もの じゃ ない か と 思い ます 。   それ から 、 大体 家族 連れ で 行っ て です よ 、 家族 連れ で 行っ て 親 が カジノ に 夢中 に なっ て い たら 、 子供 たち 、 もう 迷子 だらけ に なっ ちゃい ます よ ね 。 今 の パチンコ が そう でしょう 。 パチンコ 、 親 が 夢中 に なっ て 犯罪 まで 起き て い ます よ ね 。 子供 が 亡くなっ て い ます よ ね 。 そんな こと も ある わけ で 、 余り 家族 連れ だ と か 、 何 か そんな いい こと ばっかり 言わ ない で もらい たい な と いう ふう に 思い ます 。   ちょっと 時間 が あれ です けれど も 、 岩屋 さん 、 何 か 言い たい でしょう から 、 一言 どうぞ 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   ちょうど 夕べ 、 東大 で ＩＲ に 関する シンポジウム が あり まし て 、 そこ で 今 お 話 が 出 た 谷岡 先生 とも お 話し する 機会 が あり まし た 。 さき の 発言 の 真意 に つい て お 話し し て おら れ まし た が 、 今 、 何 と いい ます か 、 成熟 社会 に なっ て き て 、 成長 戦略 一生 懸命 やっ て い ます が 、 投資 が なかなか 思っ た ほど は 伸び ない 、 消費 も 伸び ない と いう 中 に ある わけ で ござい ます 。 今 、 ある 意味 で いう と 、 もう 団塊 の 世代 の 方々 が 高齢 者 に なっ て おら れる わけ で ござい まし て 、 ある 意味 で は 、 物 は 持っ て いる けれど も 、 物 の 消費 から 事 の 消費 と いう 、 観光 ニーズ が 高まっ て き て いる とき な の で 、 そう いう もの を 掘り起こし て いか なきゃ いけ ない よ ね と いう 意味 で 言っ た ん だ けど 、 非常 に 言葉 足ら ず に 、 誤解 を さ れ て 申し訳 なかっ た と いう 趣旨 の 発言 を 谷岡 先生 は し て おら れ まし た 。   それ から 、 我が 国 の 観光 ビジョン 目標 は 、 先生 、 今 二千 万 人 に なっ た ばっかり です が 、 オリンピック まで に 四千 万 人 、 その 十 年 後 に 六千 万 人 と いう こと で ござい まし て 、 私 は 、 それ は 実現 できる し 、 実現 し なきゃ いけ ない と 思っ て い ます が 、 多く の やっぱり 海外 の お 客 様 を 我が 国 に お 招き を し て いく と いう の が これ から の 観光 政策 の 大きな 柱 で ある と 思い ます の で 、 我が 国 に おけ る ＩＲ が 日本 人 だけ を ターゲット に し て いる と いう 指摘 は 当たら ない し 、 そう いう こと で は 目的 を 達する こと も でき ない と 思っ て い ます 。   それ から 、 ある 意味 、 先進 地 の ラスベガス で は 、 今 ゲーミング の 収入 は 三 五 ％ ぐらい に 落ち て き て おり まし て 、 ノン ゲーミング が 六 五 ％ ぐらい の 収入 を 生み出し て おり ます 。 我々 が 目指し て いく べき 方向 も そう いう 方向 で は ない か と いう ふう に 思っ て いる ところ です 。 <br/> ○ 大門 実紀史 君   とにかく この 法 案 の 最大 の 問題 点 は 、 民営 賭博 を 歴史 上 初めて 、 一千 三百 年 の 歴史 を 破っ て 解禁 する と いう 話 で あり ます 。 それ が 無理 だ と いう こと を 、 普通 の 解釈 なら ば 無理 だ と いう こと を ちゃんと 検証 も し ない で やっ て いる こと その もの が おかしい と いう こと と 、 ギャンブル 依存 症 は 、 この 前 申し上げ た よう に 、 事後 の 治療 と か 処置 は あっ て も 、 賭博 を 開い て おい て 依存 症 を 増やさ ない 対策 なんか 一 つ も あり ませ ん の で 、 依存 症 対策 なん と いう 言葉 を 使う べき で は ない と 。   経済 効果 と 言い ます けれど も 、 そもそも これ は 経済 で は あり ませ ん 。 サミュエルソン が 言っ て いる とおり 、 これ ゼロ サム ゲーム です から 、 人 の 金 を こっち へ 持っ て くる と いう 話 だ から 、 そう いう もの を 経済 効果 論 で 語る こと その もの が もう 恥ずかしい し 、 地 に 落ち て いる 、 国会 の 議論 で は ない と いう こと を 申し上げ て 、 私 の 質問 を 終わり ます 。 <br/> ○ 山本 太郎 君   ありがとう ござい ます 。 自由 党 の 共同 代表 、 山本 太郎 です 。   会派 を 代表 し まし て 、 **ＩＲ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) 、 いわゆる **カジノ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に つい て お 聞き いたし ます 。   今回 の 法案 で 、 衆議 院 で 附帯 決議 が 付い た と 。 で 、 ある 文言 が 多く 見 られ まし た 。 依存 症 、 依存 症 です 。 ギャンブル 依存 症 に 対する 対策 が 非常 に 大切 で ある と いう 認識 、 衆議 院 で も やはり 共有 さ れ て い た と いう こと だ と 思い ます 。 もちろん この 参議 院 の 議論 に おき まし て も 、 ギャンブル 依存 症 対策 に つい て は 、 与え られ た 時間 の 中 で 、 本当 に 多く の 時間 が この 依存 症 に 対し て 割か れ た の か な と いう ふう に 思い ます 。 まず 、 この 依存 症 の 問題 に つい て お 聞き を し たい と 思い ます 。   法案 提出 者 の 先生 方 それぞれ に お 聞き し たい ん です けれど も 、 その 後 に 、 もし よろしけれ ば 菅 官房 長官 に お 答え いただい て 締め て いただく よう な 形 で お 願い し たい と 思い ます 。   質問 いたし ます 。   あなた が ギャンブル 依存 症 問題 を 知っ た の は 、 ギャンブル 依存 症 問題 に 出会っ た の は いつ頃 でし た か と いう こと を 教え て いただけ ます か 。 ギャンブル 依存 症 問題 を 初めて 知っ た の は いつ でし た か と いう こと を お 聞き し たい です 。 順番 に お 願い し ます 。 <br/> ○ 衆議 院 議員 （ 細田 博之 君 ）   現在 、 非常 に 多く の 人 が そう いう 問題 に 当面 し て いる と いう こと は 、 私 も 数 年 以上 前 から 、 朝 、 パチンコ 屋 の 前 に 行け ば 分かる わけ です 。 九 時 から 並ん で 、 十 時 に 整理 券 を もらっ た の で 入っ て 、 台 を 選ん で 一 日 中 やっ て いる と 、 それ で 景品 と 換える と か です ね 。 競馬 に おい て も 競輪 に おい て も 競艇 に おい て も そう いう 人 たち が いる と 。 毎日 行こう が 行く まい が 、 自分 の 稼い だ 金 の 範囲 内 で 、 貯蓄 の ある 範囲 内 で 使う の は 誰 で も 自由 だ と 思う ん です 、 自分 の 遊び です から 。 それ しか 趣味 の ない 人 で 、 かわいそう だ と し て も 、 それ は 別 に 問題 は ない 。   だ から 、 依存 症 の 問題 と いう の は 、 その こと に よっ て 家族 が 泣い たり 、 先般 の 山本 議員 の いろんな 例示 で ある よう に 、 非常 に 問題 が 生じ て いる 、 困窮 も し て 闇金 に 走っ たり 、 何 か 売春 を し たり 、 売る 方 の です ね 、 そう いう こと を し たり 、 大変 な 悲劇 が 起こっ て 、 自殺 者 が 出る と か 、 そう いう こと に つい て 、 これ は もう 何 と か し なきゃ なら ない と いう 社会 問題 に なっ て いる と いう こと な ん で ござい ます が 、 どこ で 線 を 引く か が でき ない が ため に 、 これ まで の 政府 は 、 私 が 勝手 に 見る ところ で は 、 これ を 全部 を ギャンブル 依存 症 と いう わけ に は いか ない し 、 つまり 、 いじめ や ＤＶ と 同じ で 、 家族 の 訴え が あっ て 、 あるいは 自分 の 訴え が あっ て 、 それ は かわいそう だ 、 社会 で 面倒 見よう と いう こと に なら ない と なかなか お 世話 が でき ない よう な こと が 同じ よう に 起こっ て いる の で は ない か と 。   それ を 、 ただ 毎日 パチンコ を やっ たり 競馬 に 行っ たり する 人 は 全部 が ギャンブル 依存 症 と いっ て も しよう が ない の で 、 そう いう 問題 が なけれ ば 毎日 行っ たら いい ん です 。 何 万 円 で も 何百 万 円 で も 貯金 が ある 人 は 使っ たら いい ん です 。 だ から 、 そう いう 意味 で 、 私 は 、 そこ に は ギャンブル 依存 症 と いう 問題 で は ない の で 、 毎日 マージャン やる 人 と か 、 世の中 に は 毎日 テレビ ゲーム やっ て いる 人 も 、 スマホ ゲーム やっ て いる 人 も みんな 似 た よう な もの です 。   それ で 、 そう いう 依存 症 的 傾向 が 実 生活 に 、 あるいは 家族 を 崩壊 さ せる よう な 意味 で 問題 が 生じ た とき に 、 どう 救済 し て 、 国家 が どう いう 予算 を 作る か 、 取る か と いう 問題 で ある の に かかわら ず 、 一 億 円 と か そう いう お 金 で 、 アルコール 依存 症 や あるいは 麻薬 依存 症 や そう いう 人 と つっ くるみ で 対策 が 取ら れ て いる と 言っ て いる けれど も 、 これ は 社会 問題 と し て どう やっ て 取り上げ て 、 ＤＶ や いじめ の よう に 社会 が 取り組ん で いく か と いう こと の 方 が 大事 な わけ で ござい ます 。   だ から 、 カジノ の 法案 と いう の は 、 一 つ の 契機 で あり ます けれど も 、 これ まで ある もの の 延長 で あり ます から 、 逆 に 言う と 、 それ を どう 捉える か と いう こと が 今 問題 に なっ て い て 、 その こと は この 法 案 を 審議 する 上 で は 私 は 非常 な 進歩 を し て いる と 思い ます し 、 衆議 院 で の 附帯 決議 で も ござい まし た よう に 、 政府 側 も こう いう こと に つい て は 責任 を 持っ て やる と いう ところ まで は 来 た わけ です が 、 それ に 対し て 、 じゃ 、 パチンコ と いう の は やめ たら どう か と 、 事業 を 廃止 し たら どう か と か 、 競馬 と いう の は やめ たら どう か と いう 議論 は ない わけ です よ ね 。 それ は 、 そこ は でき ない だろう と いう ふう に 思っ て いる わけ です から 。   そう する と 、 やっぱり 被害 者 を ピックアップ し て 助ける しか ない ん です 。 だ から 、 そう いう こと を 私 は 実感 と し て 持っ て いる の で 、 これ は みんな 、 我々 は 挙げ て 何 か を し なけれ ば なら ない 、 そう いう 問題 認識 で ござい ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   今 、 細田 提出 者 から 詳細 に わたっ て お 話 が あり まし た の で 、 いつ 知っ た か と いう こと で 簡潔 に お 答え し たい と 思い ます が 、 ギャンブル と は 認め て い ない もの の 、 遊技 と いう の は 我々 子供 の 頃 から 身近 に あっ た 存在 で ござい まし た 。 私 も 、 子供 の 頃 は 父 に 連れ られ て 、 一緒 に 煙 もくもく の 音 ジャラ ジャラ の 中 で 何 時間 も 過ごし た こと も ござい まし た 。   遊技 も 、 時間 消費 型 の 娯楽 を 国民 に 提供 し て いただい て い て 、 ほとんど の 愛好 者 と いう か 利用 者 は 健全 に 接し て いただい て いる と は 思い ます が 、 一部 に そう で ない と いう 方 が いる と いう こと は 子供 心 に 感じ て おり まし た 。   それ から 、 私 、 大分 県 別 府市 が 出身 地 です が 、 別府 競輪 と いう の が ござい まし て 、 開設 し て から 六十 六 年 に なり ます 。 幸い 一 度 も 赤字 に なっ た こと が なく て 、 この 六十 六 年間 で 約 四百 八十 億 ほど の 市 の 財政 に 対する 貢献 を し て いただい て い ます の で 、 そう いう 意味 で は まさに 公益 に しっかり 資し て いただい て いる な と 思っ て いる ん です が 、 しかし 、 ここ の 利用 客 の 中 に も 、 残念 ながら ごく 一部 に は 毎日 行っ て おら れる よう な 方 も いる と いう こと も かなり 早い 段階 から 承知 を し て おり まし た 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   私 も 、 かなり 以前 から この 依存 症 の 問題 は 認識 を し て おり まし た けれど も 、 明確 に ギャンブル 依存 症 に 対し て 対策 含め て やら なきゃ いけ ない と 認識 し た の は 、 これ 七、八 年 前 、 あるいは もう ちょっと 前 か も しれ ませ ん 、 議連 で 超 党派 で シンガポール に 訪問 し て 、 シンガポール が カジノ 解禁 、 ＩＲ を 整備 を する と いう こと に 踏み切っ た 当時 の 議論 なり その 後 の 対策 を 聞い た とき に 改めて 認識 を し まし た し 、 今回 、 この 法 案 の 審議 の 過程 で 、 先生 方 から いろんな 議論 を いただき まし た し 、 また 、 多く の 団体 、 取り組ん で おら れる 方々 から も いろんな 意見 を いただき まし た の で 、 改めて これ は 我々 と し て も 取り組ま なきゃ いけ ない 大きな 問題 だ な と いう こと を 再 認識 し た ところ で あり ます 。   是非 、 今回 の 審議 、 それ から 、 私 ども は 法案 が 成立 する こと を 望ん で おり ます が 、 これ を 機 に 、 後々 に 依存 症 対策 は この とき が ターニング ポイント だっ た と 言わ れる よう に 是非 政府 に おい て も 充実 し て ほしい と 思い ます し 、 我々 と し て も しっかり と 取り組ん で いき たい と いう ふう に 思っ て いる ところ で ござい ます 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   私 も 、 ギャンブル 依存 症 を 初めて 認識 し た の は 、 この **ＩＲ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) を 作る と いう とき に いろんな 勉強 を し た 中 で ござい ます 。   そう し た 中 で 、 ギャンブル 依存 症 問題 を 考える 会 の 田中 代表 と か 、 あるいは また ラスベガス の 市長 経験 の 方 と か 、 そう いう 方々 を 議連 の ところ に お 招き を し て お 話 を 聞い たり 、 あるいは また 現地 の ところ に 行っ て その 自助 グループ の 様子 を 見学 さ せ て いただい たり と いう よう な こと が 私 に とっ て の きっかけ で ござい ます 。   今 も 西村 委員 から お 話 が あり まし た が 、 この ギャンブル 依存 症 問題 が これ まで なかなか 十分 で は なかっ た と いう 話 は 、 私 も 今 そう 認識 し て いる ところ で あり まし て 、 この **ＩＲ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) を きっかけ に この 問題 が 大きく クローズ アップ さ れ て 、 そして それ に 対する 対応 策 が 今 議論 さ れ て いる こと 、 この こと は 大変 うれしい と 依存 症 対策 の 方々 から も 聞く 機会 が ある わけ で あり まし て 、 そう いっ た 意味 で は 、 これ から の カジノ の ギャンブル 依存 と いう こと だけ で は なく て 、 既存 の ギャンブル 依存 症 対策 、 さらに は また 、 省庁 を またぐ 問題 を しっかり 考え て いこう と いう 話 を 衆議 院 の 内閣 委員 会 で 附帯 決議 と し て まとめ て いただい た こと は 大変 有 意義 だっ た と 、 こう 思っ て おり ます 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   いつ 依存 症 を 知っ た か と いう 御 質問 だっ た と 思い ます けれど も 、 私 も 大学 生 時代 、 友人 ら が 、 私 も 行き まし た けれど も 、 よく マージャン を いたし まし て 、 雀荘 に 行く 、 そして マージャン 関係 で は 特に マージャン 専門 の 漫画 まで 出 て おり まし て 、 そう し た 中 で 、 確か に ギャンブル 依存 の 皆 さん と いう の は こう やっ て 存在 する もの だ な と いう の は 実地 で 、 体 で 教え て いただい た な と 思う 次第 で あり ます けれど も 。   それ に 加え まし て 、 最近 は 私 、 デジタル ヘロイン の 問題 を 随分 研究 し て おり まし て 、 子供 たち が 今 、 先生 この 間 ＥＭＧ の 問題 に 触れ られ まし た けれど も 、 今 の 子供 たち が スマホ で ゲーム を する と 、 これ は 大人 も そう です けれど も 、 脳 の 中 の 報酬 系 に これ が 語りかける と いう こと で 報酬 系 が 異常 を 起こす と 。 子供 たち は 特に ああ いう ゲーム を する と ドーパミン が 麻薬 並み に 多く 出さ れ たり と か する と いう 知見 を 研究 する に 従っ て 、 やはり ギャンブル の 依存 だけ で は なく て 、 我々 国会 議員 と いたし まし て は 、 もう 本当 に 、 今 ＳＮＳ で も これ に はまる 女性 、 女性 の 方 が ＳＮＳ は はまる 率 が 高い ん です けれど も 、 こう し た もの は 、 例えば 自分 へ の いい ね が 多い から 、 それ が ない と 自己 承認 欲求 が 満たさ れ ない と か 、 これ また ギャンブル は お 金 が もうかる から あれ です けれど も 、 引きこもり なんか の 人 たち に よっ て は 、 自分 たち が 、 無 報酬 です から 、 無 報酬 だ けれど も 、 その 点数 だけ で 快感 を 得 て しまう 報酬 系 が 脳 の 中 に できる と か 、 そう いっ た よう な 様々 な 、 ギャンブル だけ で は なく て 、 国会 議員 と いたし まし て は 、 今 我々 が 新た に 子供 たち に も 迫っ て いる こう し た 刺激 と の 関係 と いう もの に つい て も 広く 取り組ん で いく べき で は ない か と いう ふう に 思っ て 、 この 間 も 先生 の 質問 を 聞か せ て いただき まし た 。 <br/> ○ 国務 大臣 （ 菅 義偉 君 ）   私 、 依存 症 を いつ 知っ た か と いう こと です けれど も 、 新聞 の 人生 相談 みたい な の が よく あり ます けれど も 、 そう いう 中 で 、 ああ 、 こう いう 人 も いる ん だ な と いう 、 そう いう 意識 し た の が 高校 生 の とき だっ た の か な と いう ふう に 思っ て い ます 。   私 たち 政府 の 立場 で 、 衆議 院 で 附帯 決議 が あり まし た 。 そこ の 中 で 、 ギャンブル 依存 症 対策 に つい て は 従来 の もの も 含め て と いう 趣旨 だっ た と いう ふう に 思っ て おり ます 。 また 、 参議 院 の 委員 会 で も そう し た 議論 が 度々 さ れ て いる こと も 承知 を いたし て おり ます 。 そう いう 中 で 、 そこ は しっかり 受け止め て 対応 し なきゃ なら ない と 、 そう いう 思い で あり ます 。 <br/> ○ 山本 太郎 君   あなた が ギャンブル 依存 症 を 知っ た の は いつ です か 、 僕 は 頭 の 中 で この 質問 を 作っ て いる とき に 、 五 年 前 に この 法 案 に 関わっ た とき と か と いう ふう に あっさり いく の か な と 思っ た ん です けれど も 、 かなり 皆 さん 熱 を 帯び た と いい ます か 、 この ギャンブル 依存 症 に 対する 理解 と 、 そして これ から の 取組 を やっ て いく と いう 熱 と いい ます か 意気 込み みたい な もの も 感じる よう な お 答え を いただい た と 思う ん です けれど も 、 重ね て もう 一 度 お 願い する ため に 、 質問 と いい ます か 、 短め に 答え て いただき たい ん です けど 、 特に 法案 提出 者 の 方々 、 今回 は 法案 提出 者 の 方々 に お 聞き し ます 。   ギャンブル 依存 症 対策 は 重要 な 問題 で あり 、 この 先 も 自分 自身 が 熱心 に 取り組ん で いく と 、 これ 約束 し て いただけ ます か 。 約束 する 、 若しくは それ は 約束 でき ない 、 どちら で も 結構 です 。 短く お 願い し ます 。 ありがとう ござい ます 。 <br/> ○ 衆議 院 議員 （ 細田 博之 君 ）   まさに 社会 的 な 問題 、 潜む 問題 が 、 元々 認識 は あり ます けれど も 、 表 に 出 て いる わけ です から 、 そして 、 先ほど も スマホ ゲーム に つい て も そう 言っ て い まし た 、 子供 たち が テレビ ゲーム や スマホ ゲーム に 溺れ て 勉強 も し ない 、 何 か 思考 能力 が 落ち て いる 、 大学 生 も 勉強 し て い ない 、 スマホ だけ 見 て いる 。 そう いう 現象 全体 が 本当 は 社会 に とっ て は 大変 な マイナス の 時間 の 浪費 で あり 、 それ に 加え て お 金 の 浪費 を し て 困窮 する 人 たち も 出 て いる 。 だ から 、 単なる 困窮 し た 人 、 それ から 家族 が 困っ て いる 人 など だけ を 対象 に する ん で は なく て 全体 と し て 、 日本 全体 が 一種 の 依存 症 に なっ て き て いる こと を どう 取り組ん で いく か と いう こと は 非常 に 大事 だ と 思う 。   かつ 、 被害 の ある 人 、 誰 か から 訴え が あれ ば 、 本人 の 訴え で も いい 、 家族 の 訴え で も いい 、 そう いう こと を きちっと 面倒 を 見 られる 、 お 医者 さん で も いい し 、 コンサルタント で も いい し 、 社会 全体 で も いい 、 何 か 隠れ家 を 用意 し て も いい ん だ けれど も 、 何 か そう いう 手だて を 、 具体 的 に 予算 を 構築 し て いか なけれ ば 。 今 まで 何に も 言わば ない と いう こと は 、 その こと が 政治 的 に 大きな 問題 に なっ て い なかっ た ん です 。 非常 に タッチ ー な 問題 と いう か 、 微妙 な 問題 と し て 放棄 さ れ て き まし た から 、 この 際 、 しっかり これ は 対応 す べき で ある と 、 私 は 信念 を 持っ て そう 思っ て おり ます 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   全員 に 、 山本 君 、 聞き ます か 。 <br/> ○ 山本 太郎 君   はい 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   私 は 、 最初 に 自民 党 の 中 で 勉強 会 を つくっ た とき から もう 十 五 年間 、 一貫 し て この 問題 に 関わっ て まいり まし た 。 それ は あくまで も 国際 観光 振興 を し て 日本 を 観光 立国 に し たい と いう 思い から で あり ます が 、 一方 、 これ まで 国 が 特別 法 に よっ て 幾 つ か の 公営 競技 を 認め 、 あるいは **風** **適法** ( `風俗営業等の規制及び業務の適正化等に関する法律` ) の 中 で 遊技 と いう の を 認め て き て いる に も かかわら ず 、 ギャンブル 依存 症 の 問題 に 必ず しも 真 正面 から 取り組ん で こ なかっ た と いう 問題 意識 は 一貫 し て 持っ て まいり まし た 。   したがい まし て 、 もし **推進** **法** ( `特定複合観光施設区域の整備の推進に関する法律` ) を 通し て いただけれ ば 、 制度 設計 は 政府 に 委ね られる わけ で あり ます が 、 その 中 で しっかり と 包括 的 な ギャンブル 依存 症 対策 が 取ら れ て いく よう に しっかり と ウオッチ を し て 物 を 言っ て いく 責任 が ある と 。 今度 の 衆参 の 審議 を 通じ て 先生 方 の 御 意見 を 承り ながら 、 その 責任 を ひしひし と 痛感 を し て いる ところ で ござい ます 。 その 決意 を 持っ て これ から も やっ て いき たい と いう ふう に 考え て おり ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   私 も 提案 者 の 一人 と し て この 法 案 を 提出 さ せ て いただい て 、 そして 、 いろんな 質疑 の 中 で 私 も 答弁 も さ せ て いただき まし た 。 責任 を 持っ て 依存 症 対策 に 取り組む と いう 決意 で ござい ます 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   私 も 、 個人 的 に は しっかり 取り組ん で いき たい と 思っ て おり ます 。   先週 たまたま ある シンポジウム が あり まし て 、 この ギャンブル 依存 症 対策 の 議員 立法 を 作っ たら どう か と 、 こう いう 話 が あっ て 、 私 は 個人 的 に 是非 やら せ て いただき たい と 、 こう 皆 さん の 前 で 申し上げ まし た し 、 昨日 は 維新 の 会 の 中 の 政調 会長 を 始め 皆 さん 方 と そう いう 具体 的 な 話 を し て 、 維新 と し て は もちろん やる と 、 できれ ば 超 党派 で やれ たら いい ね みたい な 議論 を し て おる ところ で あり まし て 、 しっかり 取り組ん で いき たい と 思っ て おり ます 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   先ほど デジタル ヘロイン の 問題 で 取り上げ まし た けれど も 、 事 ギャンブル に つい て も 最近 は 科学 的 な 知見 が どんどん どんどん と スピード を 持っ て 発展 し て おり ます の で 、 これ より も 広く 、 先ほど 申し上げ まし た よう に 、 ギャンブル だけ で は なく て 、 お 金 が もうから ない の に 課金 を する と いう よう な 脳 の メカニズム と いう の も 動き ます の で 、 私 も この 問題 に 興味 を 持っ て から 自分 で こう いう スマホ ゲーム を やっ て み たら 自分 が はまっ て しまい まし て 、 本当 に ミイラ 取り が ミイラ に なる よう な 、 やっぱり これ は すごい 巧妙 な 仕組み だ と いう こと は 体感 し て おり ます の で 、 また 先生 も 御 一緒 に やっ て いただき たい と 思い ます 。 <br/> ○ 山本 太郎 君   ありがとう ござい まし た 。   本当 に 熱 を 持っ て この 依存 症 対策 に は これ から も 取り組ん で いく と 。 これ 、 あと 政府 に 渡す の で は なく 、 その 旗振り 役 と し て も 頑張っ て いく と いう よう な 雰囲気 が 受け取れる よう な 答弁 で ござい まし た 。   次 の 質問 な ん です けれど も 、 少し 答え づらかっ たら 本当 に 申し訳 ござい ませ ん 。 しかし 、 これ 違法 で なけれ ば 堂々 と お 答え いただける 内容 だ と 思い ます 。   皆 さん の 中 で 、 パチンコ ホール 運営 企業 、 又 は パチンコ 、 スロット 機器 関係 の 製造 、 納入 など を 行っ て いる 企業 、 若しくは 、 先ほど 出 て き まし た デジタル ヘロイン 、 ＳＮＳ 系 の 企業 など から 政治 献金 を 受け た こと が ある 、 又 は パーティー 券 を 購入 し て もらっ た こと が ある 方 は この 中 に いらっしゃい ます でしょう か 。 ある か ない か で 、 提出 者 、 順番 に お 答え いただけ ます か 。 もし 政治 献金 若しくは パーティー 券 を 買っ て もらっ た こと が ある 方 は 、 覚え て いれ ば その 金額 も よろしく お 願い いたし ます 。 <br/> ○ 衆議 院 議員 （ 細田 博之 君 ）   **政治** **資金** **規正** **法** ( `政治資金規正法` ) の 範囲 で きっちり 処理 し て おり ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   私 、 政治 活動 を 始め て 三十 周年 と いう こと で 、 先々 週 、 また 先週 、 東京 で 一 か所 、 地元 二 か所 で 政治 資金 パーティー を 開催 を さ せ て いただき まし た 。 もちろん 、 長く 応援 し て いただい た 様々 な 職種 、 業種 の 方 が お 見え いただい て おっ た ん です が 、 地元 の 商工 会議 所 の 会頭 、 二 代 続け て パチンコ 店 を 営む 業態 の 会長 さん で あっ た と いう こと など も ござい まし て 、 来 て いただい た 方 の 中 に は そう いう 関連 業種 の 方 も いらっしゃい まし た 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   私 も 、 先ほど 答弁 し まし た とおり **政治** **資金** **規正** **法** ( `政治資金規正法` ) に のっとっ て 適正 に 処理 を し て おり ます が 、 先 ほど 御 指摘 が あり まし た 、 関係 する 機械 を 作っ て いる 大阪 の 中 小 企業 から 寄附 を いただい て おり ます が 、 これ は 私 の 地元 淡路 島 御 出身 の 社長 さん で あり まし て 、 純粋 な 気持ち で 応援 を し て くださっ て いる と いう ふう に 理解 を し て おり ます し 、 カジノ あるいは ＩＲ に つい て お 話し し た こと は 一切 ござい ませ ん 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   先ほど 公表 を し て いただき まし た ん です から 、 そう いっ た もの も 含め て そう いっ た ホール の 皆 さん たち から パーティー 券 等 を 購入 し て いただい た こと は ござい ます 。 適正 に 処理 を し て おり ます 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   私 も 長年 政治 活動 を し て おり ます の で 、 本当 に 様々 な 、 恐らく 全て の 業態 と 言っ て いい ぐらい の 業界 から いろんな 御 縁 で いただい て おり ます が 、 残念 ながら 五 万 円 以上 の 寄附 と か 二十 万 円 以上 の パーティー 券 購入 と いう 例 は なかっ た か と 思い ます 。 <br/> ○ 山本 太郎 君   済み ませ ん 、 これ は 糾弾 し たい わけ じゃ なく て 、 合法 な 献金 で あれ ば 何 の 問題 も ない お 話 だ と 思う ん です ね 。 政治 に は いろいろ お 金 が 掛かる 、 これ は しよう が ない 話 だ と 思う ん です 。 いろんな 人 や 企業 が つながり 支援 者 を 増やし て いく こと も 大切 な 政治 活動 だ と 思い ます 。 ちょっと そう いう 意味 で 心配 し て いる 部分 が ある の で 、 それ は 後ほど お 話し し たい と 思い ます 。   昨日 の 参考 人 質疑 で 私 が 参考 人 に お 聞き し た 質問 です 。 国会 審議 で 依存 症 の 話 に なる と 、 治療 や 相談 体制 の 必要 性 は 認める もの の 、 今 存在 する 依存 症 は カジノ に よっ て つくら れ た もの で は ない と いっ た よう な 雰囲気 に なり ます 、 パチンコ や 公営 ギャンブル など と 比べ て カジノ で は 依存 症 が そこ まで 増え ない の でしょう か と いう 私 の 質問 に 対し まし て 、 参考 人 の 日本 弁護 士 連合 会 多重 債務 問題 ワーキング グループ 座長 、 弁護 士 の 新里 宏二 先生 、 そして 静岡 大学 人文 社会 科学 部 の 教授 、 鳥畑 与一 先生 は こう いう ふう に おっしゃっ た ん です ね 。   鳥畑 先生 は 、 シンガポール の ＮＣ ＰＧ も オーストラリア の 生産 性 委員 会 も 、 カジノ は 既存 の ギャンブル 、 例えば 宝くじ で ある と か 競馬 など に 比べれ ば 依存 症 を 誘発 する 危険 性 が 高い と いう ふう に 言っ て いる と 。 パチンコ 以上 の 、 要する に 無 制限 の 射幸 性 が そこ に 与え られ た 場合 に は 、 既存 の ギャンブル 依存 症 プラス アルファ 新しい ギャンブル 依存 症 が 生まれ て くる ん だろう と 思っ て おり ます と 。   新里 先生 も 、 カジノ は ゲーム の 仕組み から する と 非常 に 遊技 性 が 高い 、 依存 症 に つい て は きちっと し た 対策 を 取ら ない と 、 入場 規制 と か し ない 限り は 非常 に 大きな 問題 に なっ て くる の で は ない か と 。 もちろん 、 入場 規制 と いう の も これ から 考え て いく と いう 部分 は ある と 思う ん です ね 。 入場 料 を 取っ たり と いう こと も 考え て いらっしゃる と 思い ます 。 で も 、 その 入場 料 と いう 部分 に 関し まし て も 、 新里 先生 は 、 払っ た もの の 元 を 取ろう と し て より 深み に はまっ て しまう と いう ケース も ある 、 この 入場 料 と いう 部分 に 関し て も いろいろ と 課題 が ある だろう と いう お 話 を さ れ て い まし た 。   カジノ が できれ ば 、 今 まで の 公営 ギャンブル など で は 捕捉 さ れ なかっ た 新た な 人々 を 依存 症 に いざなう だけ で なく 、 依存 手前 だっ た プレ 依存 の 人々 に も 火 を 付ける 、 その 可能 性 は 十分 に ある と 思い ます 。   そして 、 何 より 、 現在 進行 形 で 依存 症 を 広げ て いる 事象 に も 対処 し なけれ ば なり ませ ん 。 先生 方 も 先ほど お 話 の 中 で あり まし た よ ね 、 今 ある もの に 対し て も しっかり と やっ て いか なきゃ いけ ない と 。 現在 は 日本 に おけ る カジノ で の 賭博 行為 は 違法 です から 、 現在 ギャンブル 依存 症 と さ れる 方々 は 、 公営 ギャンブル 又 は パチンコ 、 スロット など に よっ て 依存 症 に なっ た と いう 話 か と 思い ます 。   先日 、 私 が 資料 と し て 委員 の 皆 さん に も お 配り いたし まし た ビッグ イシュー 基金 と ギャンブル 依存 症 問題 研究 グループ に よる 冊子 、 「 疑似 カジノ 化 し て いる 日本   ギャンブル 依存 症 は どう いう かたち の 社会 問題 か ？ 」 から 引用 し ます と 、 のめり込む ギャンブル の 種類 と いう もの が あり ます 。   日本 に は 六 つ の 公営 ギャンブル と 、 法的 に は ギャンブル と みなさ れ て い ない 摩訶 不思議 な 隠れ ギャンブル が あり ます 。 公営 ギャンブル は 、 年商 の 多い 順 に 列挙 する と 、 競馬 三 兆 円 弱 、 宝くじ 九千 五百 億 円 ほど 、 競艇 九千 億 円 、 競輪 六千 億 円 、 スポーツ 振興 くじ 一千 億 円 、 オート レース 七百 億 円 、 そして 隠れ ギャンブル で ある パチンコ と スロット の 年商 は およそ 二十 兆 円 です 。   ギャンブル 症 者 、 ギャンブル 依存 症 者 の こと です よ ね 、 ギャンブル 症 者 の はまる ギャンブル は 、 五 割 から 六 割 が パチンコ 、 スロット です 。 パチンコ 、 スロット 絡み で ない の は 五 ％ 以下 で しか あり ませ ん 。 しかも 、 女性 で は ほぼ 全例 が パチンコ 、 スロット です 。 日本 の ギャンブル の 一大 問題 は 、 この パチンコ 、 スロット が ギャンブル と みなさ れ て い ない 点 に ある の です 。 （ 発言 する 者 あり ） ありがとう ござい ます 。 違法 ギャンブル と し て は 、 賭け マージャン や 野球 賭博 、 裏 カジノ が ある もの の 、 あくまで 少数 派 に とどまっ て いる と 。   ギャンブル 障害 の 日本 的 な 特徴 。 ギャンブル 障害 に は 誰 で も 陥る 可能 性 が ある と おっしゃっ て いる ん です ね 。 陥る か 陥ら ない か の 差 は 、 ほぼ 環境 要因 で 決まる と 言っ て も 過言 で は ない 。   ギャンブル へ の アクセス の 良 さ 。 環境 その もの に ギャンブル し やす さ が 整っ て いる と ギャンブル 症 者 は 確実 に 増え ます 。 これ を 決定 する の が ギャンブル 場 の 場所 や 立地 条件 、 開催 時期 、 年齢 制限 の 有無 、 ギャンブル の 害 へ の 無知 など です 。 公営 ギャンブル の 競馬 、 競艇 、 競輪 、 オート レース で は 、 開催 場所 や 開催 日時 は 制限 さ れ て いる と は いえ 、 場外 売場 や ネット で も ギャンブル が できる 至便 性 が ある 。 宝くじ や スポーツ 振興 くじ 売場 は 人 の 集まる ところ に は 必ず あり 、 一 年 中 売り出さ れ て いる 。 年齢 制限 など ない に 等しい の です 。 そして 、 ギャンブル と さ れ て い ない パチンコ 、 スロット の ホール は コンビニ の ローソン より も 多く 、 全国 に 一 万 二千 館 あり ます 。 ギャンブル の 機器 の 台数 で は 、 世界 七百 二十 万 台 の 三 分 の 二 が 日本 に 集中 し て い ます 。 朝 は 十 時 から 夜 も 十 時 まで 開店 し て い て 、 冷蔵 庫 付き の ロッカー や 託児 所 を 備え て いる ホール も 珍しく あり ませ ん 。 ＡＴＭ の 設置 も ほぼ 行き 渡っ て い ます 。 究極 の 至便 性 と 安楽 性 が 実現 さ れ て いる の が パチンコ 、 スロット です 。   ギャンブル 機器 の 射幸 性 。 パチンコ や スロット など の 機器 は 、 エレクトロニック ・ ギャンブリング・マシン 、 Ｅ Ｇ Ｍ と いい ます 。 この Ｅ Ｇ Ｍ は 、 人 を ギャンブル に のめり込ま せる ため に あらゆる 技術 改良 と 工夫 が 可能 です 。 古来 から ある さいころ ゲーム や 花札 、 闘鶏 など の 比 で は あり ませ ん 。 もう 少し で 当たる 錯覚 を 生み出す ニア ミス 、 ニア ゲイン の 細工 や 、 大当たり の 前触れ を 知ら せる リーチ 表示 だけ で なく 、 画像 に なじみ の アニメ や 映画 を 登場 さ せ 、 物語 性 を 付加 し ます 。 これ ら の 脳 刺激 は 、 派手 な 映像 と 音響 に よっ て 何 倍 に も 増強 さ れる 。 危険 ドラッグ 同様 、 数 回 これ ら の 脳 刺激 を 受ける と もう 抜け出せ なく なる と 。 その よう に おっしゃっ て いる ん です ね 。   そして 、 借金 の し やす さ 。 パチンコ 、 スロット ホール に ＡＴＭ が ある くらい です から 、 銀行 も ギャンブル に 費消 さ れる お 金 を 虎視 眈々 と 狙っ て いる の は 確か です 。 一昔 まで は 消費 者 金融 が 大 流行 、 無人 の 貸金 所 まで あっ た ほど です 。 当人 に 返済 能力 が ある か など の 調査 は 、 爪 の あか ほど も なさ れ ませ ん でし た 。 しかも 、 その 金利 は 年率 二 割 を 超え て い た の です から 、 高利 貸し と 大差 あり ませ ん 。   しかし 、 消費 者 金融 から の 借金 が 膨らん で 自殺 者 と 自己 破産 者 が 増え た ため 、 二 〇 〇 六 年 、 **貸金** **業** **法** ( `貸金業法` ) が 改正 。 過剰 融資 を 防止 する べく 総量 規制 が 定め られ 、 年収 の 三 分 の 一 を 超える 融資 は 禁止 さ れ まし た 。 ところ が 、 銀行 は 、 この 過剰 貸付け の 規制 から は 対象 外 に さ れ まし た 。 銀行 の カード ローン 広告 に も 、 総量 規制 なし 、 年収 の 三 分 の 一 以上 借入れ 可 と うたっ て いる ほど です 。   しかも 、 この 銀行 の 融資 に 消費 者 金融 が 返済 保証 を し て いる 。 今 で は 、 大手 消費 者 金融 は 大手 銀行 グループ の 子 会社 。 アコム は 三菱 東京 ＵＦＪ フィナンシャル ・ グループ の 子 会社 、 プロミス は 三井 住友 フィナンシャル グループ の 子 会社 。 これ で は 、 **貸金** **業** **法** ( `貸金業法` ) の 総量 規制 の 脱法 行為 と 非難 さ れ て も 当然 。   まだまだ ある ん です けれど も 、 取りあえず 引用 は ここ まで に し たい と 思い ます 。   依存 症 対策 する と 言う ん だっ たら 、 もちろん 銀行 の この 乱暴 な お 金 の 貸し方 と いう こと も 直ちに 改め て いか なきゃ いけ ない と いう の は 当然 な ん です けれど も 、 現在 の ギャンブル の 中 で 一番 依存 症 を 生み出し て いる パチンコ 、 スロット へ の 規制 、 これ 必要 な こと 明らか です よ ね 。 依存 症 対策 を 講ずる と 言う なら ば 、 パチンコ 、 スロット の 規制 が 今後 喫緊 の 課題 と し て 進め て いく 必要 、 絶対 的 に なっ て くる と 思う ん です けれど も 、 話 が 戻り ます 、 ここ で 。   その とき に 、 規制 を し ます と いう とき に 、 パチンコ 、 スロット 関係 企業 から 献金 を 受け た 身 で 厳しい 規制 を 実行 する 旗振り は 果たして できる の でしょう か 。 政治 献金 を 受け た こと が ある 又 は パーティー 券 を 買っ て もらっ た こと が ある と お 答え に なっ た 方 に お 答え いただき たい ん です 。 それ で も やれる 、 若しくは なかなか 難しい 、 その 二 択 で お 願い いたし ます 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   山本 君 、 指名 を お 願い いたし ます 。 <br/> ○ 山本 太郎 君   済み ませ ん 。   先ほど お 答え いただい た 方 で 、 政治 献金 を 受け た こと が ある と おっしゃっ た 方 、 若しくは パーティー 券 を 買っ て もらっ た こと が ある と 先ほど お 答え いただい た 方 に お 答え いただけ ます か 。 お 願い し ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   当然 だ と 思い ます 。 私 ども 、 様々 な 政策 決定 に 関わる わけ で ござい ます が 、 時 に は 支援 者 あるいは 献金 を いただい た 方 の 意 に 沿わ ない 意思 決定 、 政策 決定 も し なく て は いけ ない と 、 特に 政権 与党 は そう だ と 思っ て おり ます が 、 当然 、 そう いう 場合 が 出 て き た とき に も 自ら の 政治 信念 に 従っ て 行動 する 所存 で ござい ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   まず 、 パチンコ や いわゆる スロット と いう 今 ある **風営** **法** ( `風俗営業等の規制及び業務の適正化等に関する法律` ) 上 の 遊技 に つい て は 、 これ は いろんな 議論 が ある こと は 承知 を し て おり ます が 、 この **ＩＲ** **法** ( `特定複合観光施設区域の整備の推進に関する法律` ) と は 別 の 枠組み で 、 **風俗** **適正** **化** **法** ( `風俗営業等の規制及び業務の適正化等に関する法律` ) の 下 で その 規制 に 従っ て 運営 を さ れ て いる と 認識 を し て おり ます の で 、 その **風営** **法** ( `風俗営業等の規制及び業務の適正化等に関する法律` ) の 在り 方 に つい て 別途 それ は 議論 を し て いく べき 話 だ と いう ふう に 思い ます 。   ただ 、 依存 症 に つい て は 、 この 遊技 も 含め て 多く の 方 が 依存 症 に なっ て いる と いう の は 認識 を し て おり ます の で 、 依存 症 に つい て は 、 今回 の 法案 を 機 に 、 既存 の ギャンブル 、 公営 ギャンブル あるいは 遊技 に つい て も 含め て 、 全体 と し て 取り組ん で いく こと が 大事 だ と いう ふう に 思っ て おり ます 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   私 も 、 今 、 西村 議員 と 同じ 思い な ん です が 、 きちっと 分け て 、 この **ＩＲ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) は **ＩＲ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) なり に 議論 を いただける と 有り難い な と いう 話 を まず 申し上げ た 上 で 、 自分 自身 で 、 献金 を 、 あるいは パーティー 券 を 買っ て もらっ て い て そう いう 話 が できる の か と いう こと に 関し て は 、 十分 、 私 自身 も やっ て まいり まし た し 、 できる と 、 こう いう ふう に 思っ て おり ます 。   具体 的 に は 、 依存 症 の 問題 を 実 は そう いう 方 と も 話 を し た こと が あり ます し 、 依存 症 に 対する 対応 を 御 検討 を いただい た こと も ござい ます 。 あるいは また 、 **風営** **法** ( `風俗営業等の規制及び業務の適正化等に関する法律` ) の 在り 方 その もの に つい て の 議論 も さ せ て いただい て き た ところ で ござい まし て 、 そう いっ た 意味 で は 、 献金 や パーティー 券 を もらっ て いる から 何 も 言え ない の で は ない か と いう こと は あり 得 ない と いう ふう に 思っ て おり ます 。 <br/> ○ 山本 太郎 君   ありがとう ござい まし た 。   別個 議論 を し て いく べき だ と いう お 話 も あり まし た けど 、 これ 地続き の 話 だ と 思う ん です よ ね 。 結局 、 今回 の プログラム 法案 の 先 に 実施 法 が ある 、 結局 それ に よっ て 何 が 実現 できる か と いっ たら 、 今 まで 違法 だっ た もの が 合法 に なっ て いく と いう 方向 性 です よ ね 。 だ って 、 普通 に 賭場 を 開張 でき ない わけ です よ ね 、 普通 に は 、 賭け事 する 場所 を 提供 でき ない わけ です よ ね 、 普通 は 、 合法 で は ない わけ だ から 。 この 二 段階 に よっ て 、 その 先 に は 賭場 を 開く こと が オーケー に なる 地域 で あっ たり と か 主体 と いう もの を 決め て いか れる わけ だ から 、 これ 地続き です よ 。   今 まで 国 が 抜け道 的 に と いう か 、 いろんな 公営 ギャンブル だっ たり と か パチンコ 的 な もの 、 スロット と いう もの に 対し て 認め て き た こと の 今 まで 依存 症 、 これ だけ 広がっ て き て いる 。 そして もう 一 つ 、 ギャンブル と いう もの に 扉 を 開く と いう なら ば 、 これ は 当然 、 この **ＩＲ** **法** ( `特定複合観光施設区域の整備の推進に関する法律` ) 、 プログラム 法 の 中 で も この 依存 症 に 対し て どう し て いく べき な の か と いう こと が 一番 厚く 担保 さ れ なきゃ いけ ない 私 は 審議 の 内容 だ と 思い ます 。   少し 話 が 変わる ん です けれど も 、 皆 さん に 、 皆 さん に と 言っ たら あ かんな 、 また あれ な ん で 、 代表 者 一 名 の 方 に 、 皆 さん お 聞き し たい ん です けれど も 。   ギャンブル で 一 億 円 を 失い まし た と いう の と 事業 に 投資 し て 一 億 円 を 、 何 です か ね 、 損 し て しまい まし た 、 ギャンブル で 一 億 円 すっ ちゃっ た と いう の と 事業 で 一 億 円 なくなっ ちゃい まし た と いう の と 、 これ 経済 効果 的 に は どちら が ある と 思わ れ ます か 。 どちら か で 、 二 択 で お 答え いただける と 助かり ます 。 済み ませ ん 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   山本 君 、 答弁 者 指名 し て ください 。 <br/> ○ 山本 太郎 君   そう です ね 、 じゃ 、 一番 の 若手 に 。 松浪 先生 、 いい です か 。 済み ませ ん 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   経済 効果 と いう の が 何 を もっ て 経済 効果 と いう の か と 、 これ は ちょっと 質問 、 お 答え 、 非常 に つらい と 思い ます 。   ついで に 言え ば 、 さっき 僕 だけ お 答え が なかっ た の で 、 パチンコ の 献金 と か そう いう あれ の 問題 で 僕 だけ あれ だっ た ん です けど 、 私 自身 も 実際 、 生活 保護 の 問題 で 、 生活 保護 者 の 方 が こう いう パチンコ 等 を する こと に つい て の 問題 、 予算 委員 会 等 で も 取り上げ た こと が ある と いう こと を まず 付言 を し て おき たい と 思い ます 。   その 上 で 、 経済 効果 と いう と 、 なくなれ ば ダメージ は 当然 、 我々 も 、 自分 ら の 支持 者 で 連帯 保証 人 に なっ て い て 失っ た と か いう こと も ある の で 、 その 周り へ の 波及 と いう の は やっぱり 、 それ は 事業 で あれ 、 自分 の 信頼 する 人 たち から お 金 を 借り て い て 、 自分 だけ で は なく て 周り に 波及 する と いう こと 、 これ は マイナス の 経済 効果 が 大きい と 思い ます し 、 ギャンブル の 場合 は 家族 等 に 大変 な 迷惑 を 掛ける と いう こと に おい て は これ も 同じ で あり ます けれど も 。   経済 効果 と 言う と あれ です けど 、 道義 的 に 考えれ ば 、 当然 ギャンブル は やっぱり 自分 の 私利 私欲 と いう の が 前 に 出る と 思い ます し 、 事業 と いう の は 、 もっと 仕事 で 家族 を 養い たい と いう 思い から 損 を し た 、 結果 的 に 損 を し て しまっ た と いう こと に なる の で 、 ここ は 一概 に 比べる べき だ と は 私 は 思い ませ ん 。 <br/> ○ 山本 太郎 君   失礼 し まし た 。 ちょっと 言い 方 が 悪かっ た です ね 。   一 億 円 と いう お 金 を 持っ て い まし た と 。 それ を 事業 に 投資 し まし た と いう 人 と 、 カジノ で すり まし た 、 で 、 事業 は 失敗 し た 。 この 二人 に おい て 国内 の 経済 活動 が より 活発 に なる の は どちら でしょう か と いう こと を お 聞き し たかっ た ん です ね 。 そんな こと は ちょっと 中身 を 見 て み なきゃ 分から ない と いう 、 すごく 、 何 です か ね 、 センシティブ な お 答え を し て いただい た と 思う ん です けれど も 。   例えば です けど 、 事業 で 一 億 円 を 投資 し た 人 が 失敗 。 しかし 、 本人 は 損し た と し て も 、 取引 先 が 国内 企業 事業 者 で あれ ば 、 従業 員 の 給料 だけ で なく 、 関連 する 下請 の 売上げ に なっ たり と か いろいろ 、 何 と いう ん です か ね 、 波及 し て いく と いう よう な 、 事実 上 、 日本 社会 の 経済 活動 に 貢献 し た と いう 見 方 も できる か な と 思う ん です ね 。   一方 で 、 カジノ で 一 億 円 を すっ た 場合 、 基本 的 に は 、 一 億 円 の 所有 権 が お 金 を すっ た 人 から カジノ 運営 者 に 移動 し た だけ 。 つまり 、 その 場 の 運営 費 、 経費 以外 は 経済 効果 が 生まれ づらい ん じゃ ない か な 、 日本 社会 の 経済 活動 に は なかなか 寄与 し ない 状態 が 生まれる ん じゃ ない か な と いう ふう に 思う ん です よ ね 。   じゃ 、 その 収益 、 カジノ 側 が 、 もし も 一 億 円 すっ て くれ た お 客 が い た と し て 、 カジノ 以外 の 日本 国内 の 事業 に 対し て 投資 を し て くれる と いう こと が ある なら ば 、 何 です か ね 、 経済 効果 、 日本 の 経済 に 寄与 し た と いう よう な 形 に つながっ て いく の か な と 思う ん です けれど も 、 別 に 再 投資 し て くれる よう な 取決め も ある わけ じゃ ない です もん ね 。 ごめん なさい ね 、 話し て いる の は 、 外資 の そう いう カジノ の 運営 会社 が と いう 話 を し たかっ た と いう こと な ん です よ ね 。   結局 、 外資 の カジノ 運営 会社 が 得 た 収益 を 積極 的 に 日本 に 再 投資 し て くれる と いう よう な 何 か 可能 性 みたい な もの って 余り 感じ られ ない な と いう ふう に 思う ん です よ ね 。 結局 、 日本 に 存在 する 資産 、 所得 を 外資 の カジノ 事業 者 が 吸い上げ て 資産 を 移転 する と いう 話 に なっ て いか ない か な と いう ふう に 思う ん です 。   日本 の 会社 が やれ ば いい じゃ ない か と いう 話 も ある と 思い ます けど 、 運営 の 方 。 で も 、 海外 の 企業 が やっ た 場合 に 、 より 国内 に 経済 効果 が 生まれる よう な 回し 方 、 再 投資 で あっ たり いろんな こと と いう の は なかなか 望め ない ん じゃ ない か な と いう ふう に 思う ん です よ ね 。   カジノ で 経済 効果 が 上がる と いう ん だっ たら 、 客 から 吸い上げ た お 金 は ちゃんと 国内 の 生産 活動 に 再 投資 さ れ ない と 経済 効果 と いう の は なかなか 出 にくい だろう と 。 海外 の 企業 が 客 から 莫大 な 金 を 吸い上げ て 、 それ を 自分 自身 の 国 に 移転 し て しまう よう な こと 、 これ が 連続 的 に 行わ れ たら 、 これ 国富 の 流出 に つながる ん じゃ ない か な と 。   私 は カジノ 反対 と いう 立場 です けど 、 百 歩 譲っ て と いう 考え 方 で 、 カジノ が 日本 で 解禁 し たら と いう 視点 で 見 て み た とき に 、 海外 の 企業 の 参入 を 許す と いう こと を 考え たら 、 日本 に とっ て の プラス が 余り ない ん じゃ ない か な 、 薄く なっ て いく ん じゃ ない か な と いう ふう に 思う ん です よ ね 。   外資 事業 者 、 短期 的 な 利益 を 追求 する と いう 傾向 が ある と 。 その 結果 、 公共 性 、 周辺 産業 へ の 波及 効果 が 抑制 さ れ て 、 超過 利益 の 海外 流出 、 つまり 日本 国内 に 再 投資 さ れ ない こと は 何 か 想像 でき そう な 感じ だ な と 思う ん です よ ね 。 結局 、 外資 の カジノ 関連 会社 に 対し て 日本 を 草刈り 場 と し て 提供 する よう な こと に なっ ちゃっ たら 嫌 だ な と 思う ん です よ 。 何 か そんな 心配 は ない ん です か ね 。 ─ ─ 何 か 、 あっ 、 ちょっと 待っ て ください 、 短め に 答え て いただける ん だっ たら ここ で お 渡し し ます 。 そう で ない なら 、 ちょっと 待っ て ください 、 一番 お 話 が 濃く なり そう な 方 です ね 。 大丈夫 です か 。 <br/> ○ 衆議 院 議員 （ 細田 博之 君 ）   やっぱり 、 今 、 例えば 外資 と 国内 資本 が 組ん で 投資 し たい と いう 、 投資 規模 は 一 兆 円 ぐらい ある ん です ね 、 大体 。 ホテル から 何 から 全て を 組み上げ て 投資 を しよう と 言っ て いる の は 大体 一 兆 円 以上 を 言っ て い ます 。 三 兆 、 四 兆 と 言っ て い た ところ も あり ます けど 、 それ は ちょっと 大きい と 思う ん です ね 。   そう する と 、 一 兆 を 投資 する こと の 効果 、 これ は 、 もう そこ で 大変 大きな 効果 が 生ずる わけ です から 、 今度 、 あと は カジノ も 含め た 運営 を し て 、 今度 は 回収 が どの ぐらい で できる か と いう こと だ と 思う ん です ね 。 回収 と いう の は 、 宿泊 に よっ て 払う 人 も あれ ば エンターテインメント で 払う 人 も ある 、 ゲーミング で 払う 人 も い ます から 。 その うち 、 上がり の 何 ％ か は 国 が 、 あるいは 地方 公共 団体 が 納付 金 で いただく わけ です から 。 一種 の 税金 で いただく 。 もうけ が 出れ ば そこ で 法人 税 等 を いただく し 、 残り は 投資 収益 と し て 彼 ら が 再 投資 に 向かう か 、 あるいは 資金 を 元 の 国元 に 持っ て 帰っ て また 別 の 運用 を する か 、 それ は 分かり ませ ん けど 、 投資 と いう の は そう いう 性格 が ある の で 、 それ は 、 まず 投資 を し て くれる と いう 企業 が あれ ば それ は 有り難い と いう 気持ち は する ん です ね 、 それ は 立派 な こと です から 。 今 、 民間 設備 投資 が もう どんどん 縮小 し て 困っ て いる よう な 状態 です から ね 。   以上 で あり ます 。 <br/> ○ 山本 太郎 君   ありがとう ござい ます 。   まあ 最初 の 一番 大きな 投資 と いう の が 何 より も 大きい ん だ と 、 その 後 は 法人 税 だっ たり いろんな もの で 、 納付 金 だっ たり と いう もの で いただい て いく ん だ から 心配 に は 当たら ない よ と いう お 話 を し て いただい た と 思い ます 。   二 〇 一 四 年 、 都内 で 開催 さ れ まし た カジノ の 国際 会議 、 ジャパン ・ ゲーミング ・ コングレス 、 これ ちょっと 話 の 内容 が 変わっ て くる ん です けれど も 、 その 中 で プレゼンテーション が あっ た と 。 主な 顧客 は 、 外国 人 観光 客 で は なく 、 日本 人 の 富裕 層 と いう 内容 だっ た 。 日本 人 富裕 層 の 個人 金融 資産 量 を 日本 に できる 推定 カジノ 数 三 から 十 で 割る と 、 海外 に 比べ て 日本 の 一 つ の カジノ 当たり の 個人 金融 資産 量 は 突出 し て いる と 。 日本 の カジノ は 莫大 な 利益 、 これ が 確実 に なる と 投資 を 呼びかけ られ て い た そう です 。   この 会議 に 、 スペクトラム ・ ゲーミング 、 もう 御 存じ です よ ね 、 皆 さん 、 世界 四十 六 か国 、 アメリカ で は 三十 六 州 に おい て ギャンブリング に 対する 幅広い コンサルティング サービス を 提供 する 国際 的 な コン サルト 会社 、 その 代表 取締 役 を 始め 、 海外 の カジノ 業 界 の 大物 が 多数 参加 し て い た そう な ん です ね 。   で も 、 はっきり 言っ て い ます もん ね 、 日本 人 富裕 層 の 個人 金融 資産 量 を カジノ の 数 で 割っ たら 、 どう 考え て も 、 日本 一 つ の カジノ 考え た だけ で も 個人 金融 資産 量 は 突出 し て いる から 、 莫大 な 利益 は 確実 だ と 。 それ は そう です よ ね 、 手付 か ず な ん です もん ね 。 新た な 草刈り 場 が ここ に あり ます と いう こと を そう いう ところ で 、 来 ませ ん か 、 日本 に と いう よう な 話合い に なっ て い た と いう 話 な ん です よ ね 。 この 会議 に は そう いう 方々 が 参加 さ れ て い た と 。 海外 から 日本 に 来る 観光 客 が 目当て で は ない ん だ と 。 日本 国内 の 資産 を いかに 吸い上げる か と いう 部分 が 、 恐らく 、 商売 です から 、 当然 考え られる こと だ と 思う ん です よ ね 。 もう 皆 さん 御 存じ の とおり です よ ね 、 海外 の 事業 者 、 日本 で の カジノ に 乗り乗り の よう です と 。   日本 で の カジノ 、 ＩＲ 解禁 を 見据え て 、 多く の 海外 の カジノ 運営 事業 者 が 投資 を 表明 し て いる と 。 特に ＭＧＭ リゾーツ ・ インターナショナル なんか は 、 会長 さん です か 、 十 一 月 一日 の インタビュー に 答え て いる ん です ね 、 これ 。 大 都市 型 と 地方 都市 型 の 二 種類 の うち 、 ＭＧＭ と し て は 大 都市 型 に フォーカス を 当て て いる 。 それ は そう です よ ね 、 東京 と か 大阪 で やれ たら 、 より 個人 金融 資産 と いう もの は 広がっ て いき ます もん ね 。 吸い上げ られる 量 が 全然 変わる と いう 話 です よ ね 。 ほか に も 、 ラスベガス ・ サンズ など も 積極 的 だ と 。   一方 、 昨年 二 月 、 ビル ・ ホーン バックル 社長 会見 で は 、 永遠 に は 待て ない と いう 発言 も あっ た と 。 こんな こと を 言わ れ たら 焦り ます よ ね 、 早く ＩＲ を 、 早く カジノ を と いう 話 に なる だろう な 、 早く 成立 さ せ なきゃ いけ ない よ と いう 話 に なる だろう な と 。 いつ まで も 待て ない よ 、 俺 たち って 。 いつ まで も やる と いう わけ じゃ ない から ね 、 日本 で って 。 早め に 決め て ね と いう 話 な ん です か ね 。   ほか に も 、 マカオ の メルコ 社 、 オーストラリア の クラウン ・ リゾート 社 の 合弁 会社 、 メルコ ・ クラウン ・ エンターテインメント も 、 最低 五十 億 ドル の 投資 など 、 市場 参入 へ の 意欲 を 示し て いる と 。 クラウン ・ リゾート は メルコ へ の 投資 比率 を 減らし て いる けれど も 、 法案 が 成立 すれ ば 日本 へ の 投資 意欲 が 再燃 する の で は ない か と いう 観測 も ある そう です 。 海外 事業 者 は やる気 満々 の よう な ん です よ ね 。   これ 、 カジノ の 運営 に 関わる 事業 者 の 選定 って 、 選定 まで 行か ない と し て も 、 大体 の めど と いう の は もう 付い て いる ん です か ね 。 これ は 政府 参考 人 に お 願い し た と 思う ん です けれど も 。 <br/> ○ 政府 参考 人 （ 中川 真 君 ）   御 答弁 申し上げ ます 。   政府 に おき まし て は 、 二 年 前 の 成長 戦略 の 中 で 、 この 法 案 に つい て 、 国会 で の 御 審議 の 動向 及び 国民 的 な 議論 の 動向 を 見据え て 関係 省庁 で 検討 し て まいる と いう こと に し て おり ます けれど も 、 これ まで 政府 に おき まし て は 、 海外 の ＩＲ 事業 者 など の 実態 です と か 、 あるいは カジノ 規制 当局 の やり 方 など を いろいろ 勉強 を し て 積み上げ て き た ところ で ござい まし て 、 今 お 尋ね の 日本 に おけ る ＩＲ の 具体 像 、 海外 事業 者 の 内容 など に つい て 、 そう いう 具体 像 の 検討 は 一切 し て ござい ませ ん 。 <br/> ○ 山本 太郎 君   ありがとう ござい ます 。   まあ そう はいっ て も 、 水面 下 で やっ て いる でしょう 。 だ って 、 もう ここ まで 来 た ん だ し 、 もう 実施 法 も 先 見え て いる わけ だ から 、 もう やっ て い ない と 間に合い ませ ん もん ね 。   私 、 何 か やる気 満々 みたい です けど 、 基本 的 に は **カジノ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に は 反対 と 。 で 、 一応 解禁 し た 場合 と いう 視点 で お 聞き し て い ます 。   で も 、 日本 で カジノ 運営 の ノウハウ を 持っ た 事業 者 が い なかっ たら 、 自然 と ノウハウ を 持つ 外資 の 事業 者 が カジノ の 運営 を 手 に 入れる こと に なる と 思う ん です よ ね 。 もちろん 一緒 に やる と いう 話 は ある か も しれ ない けれど も 、 ノウハウ を 持っ て いる の は 向こう な ん だ から と いう 話 です よ ね 。 これ 、 非常 に 危険 な 状態 と 言える ん じゃ ない か な と 思う ん です 。   これ 、 国内 企業 に 優先 的 に カジノ の 運営 やら せる と いう 考え 方 は ない ん でしょう か 。 実際 に 今 現在 日本 で は カジノ の 運営 の ノウハウ を 持っ た 企業 と いう の は ない でしょう けれど も 、 官房 長官 、 日本 の 企業 って 、 これ 、 この 運営 の ところ から 排除 さ れる と いう 可能 性 は ある ん です か ね 。 済み ませ ん 、 何 か 急 に こんな こと を お 聞き し て 。 <br/> ○ 国務 大臣 （ 菅 義偉 君 ）   まだ どう する か と いう こと は 決め て い ない と いう 状況 だ そう です 。 <br/> ○ 山本 太郎 君   なるほど 。 今 も 審議 中 です から ね 。 ここ で いろいろ 考え て い ます と か と 言っ たら 、 どう いう こと だ と いう 話 に なっ ちゃい ます もん ね 。 済み ませ ん 。 ありがとう ござい ます 。   日本 企業 に は ノウハウ は ない と いう お 話し まし た けれど も 、 もう すぐ ノウハウ を 手 に 入れる 企業 は 存在 し て いる と 。 セガサミーホールディングス 、 韓国 カジノ 最 大手 の パラダイス グループ と の 合弁 で 立ち上がる 統合 型 リゾート 、 ＩＲ が 二 〇 一 七 年 五 月 開業 を 目指し て 既に 建設 中 だ と 。 この 韓国 へ の 投資 は 、 日本 で カジノ が 解禁 さ れ 、 カジノ 事業 に 本格 参入 を 果たす こと を 前提 に 、 運営 事業 者 と し て の ノウハウ を 習得 する ため の もの だ と いう 趣旨 の 報道 も 見 られ ます 。   やはり 国内 企業 を 参入 さ せる ため に は 、 セガ サミー の よう に ノウハウ を 積ん だ ところ が 選ば れる 可能 性 と いう の は 高い の でしょう か 。 高い だろう と いう 答え な の か 、 そう で は ない だろう と いう こと な の か 、 分から ない と いう 答え な の か 、 短め に 答え て いただけ ます か 。 ─ ─ あっ 、 ちょっと 待っ て ください 。 いや 、 違い ます 。 提案 者 の 方 で さくっ と 答え て いただける 方 。 申し訳 ない です 。 時間 の 関係 です 。 済み ませ ん 。 申し訳 ない 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   いや 、 簡単 に 答えれ ば 、 分から ない と いう こと だ と 思い ます 。   いずれ に し て も 、 事業 者 に 対し て は 厳格 な 審査 が 行わ れ て 、 ライセンス を 得 た 者 で しか この 事業 に 参画 を でき ない と いう 仕組み に なる わけ です から 、 海外 企業 で あろう が 日本 の 企業 で あろう が その 審査 を パス する こと が 大 前提 です から 、 今 の 時点 で は 分から ない と 申し上げる しか ない と 思い ます 。 <br/> ○ 山本 太郎 君   ありがとう ござい ます 。   ライセンス 発行 する と いう こと は 先々 ある ん だろう けど 、 今 何 も まだ 決まっ て い ない 段階 で そんな こと を 細かく 言え ない 、 今 は 分から ない と しか 言え ない ん だ と いう 話 な ん です けれど も 、 その ライセンス の 発行 に 関し て 、 何 か こう 、 便宜 を 図る みたい な 話 に なっ たら 嫌 だ な と 思う ん です よ ね 。   例えば です けれど も 、 安倍 総理 は セガ サミー の 会長 さん の 里見 さん と つながり が 深い と いう 、 お 聞き し ます よ ね 。 セガ サミー の 会長 の 娘 さん の 結婚 式 に まで 出席 さ れ た と いう こと は 、 二 〇 一 三 年 十 月 四 日 の 写真 週刊 誌 フライデー に も 出席 さ れ て いる 様子 が 写真 で 紹介 さ れ て い た と 。   こう いっ た きずな で お 友達 人事 的 に カジノ の 運営 権 が 手 に 入り やすい と いっ た よう な こと は 行わ れ ない です よ ね 。 これ は まだ 決まっ て い ない から 言え ない こと と は また 別 の 話 だ と 思う ん です よ ね 。 官房 長官 、 こう いう こと ない です よ ね 。 <br/> ○ 国務 大臣 （ 菅 義偉 君 ）   まだ 決まっ て い ない です から 、 私 から いい 悪い と 言う こと は 控え たい と 思い ます 。 <br/> ○ 山本 太郎 君   いや 、 これ 、 決まっ て い なく て も やっ ちゃ 駄目 な こと です よ ね 、 つながり が ある から と いう ところ の 企業 に 便宜 を 図る と いう の は 。 もちろん 、 そう いう こと は 起こら れ ない です よ ね 。 官房 長官 、 いかが でしょう 。 <br/> ○ 国務 大臣 （ 菅 義偉 君 ）   先ほど 提案 者 から 答弁 さ れ て い まし た けれど も 、 それ は 公正 厳格 な 審査 と いう 形 に なる だろう と いう ふう に 思っ て い ます 。 <br/> ○ 山本 太郎 君   済み ませ ん 、 お 忙しい 中 来 て いただい て 。 もう これ 以上 官房 長官 に は ござい ませ ん の で 、 退席 し て いただい て 結構 です 。 若しくは 、 ゆっくり コーヒー 飲ま れ て いただい て も 結構 です けれど も 、 お 話 を 聞い て いただい て も 。 ありがとう ござい ます 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   菅 官房 長官 、 御 退席 いただい て 結構 で ござい ます 。 <br/> ○ 山本 太郎 君   で は 、 先 に 進め ます 。   済み ませ ん 、 ありがとう ござい ます 。   これ 、 セガ サミー に 限っ た 話 じゃ ない よ な と 思う ん です よ ね 。 パチンコ 、 スロット に 関連 する 企業 が カジノ の 運営 権 を 手 に 入れ た 場合 、 その 母体 の 収益 に 影響 する よう な 規制 が 、 ギャンブル 依存 対策 が 本当 に 行う こと できる の か な って 。 依存 症 対策 、 特に 喫緊 の 課題 で ある パチンコ 、 スロット の 規制 が 進み にくく なる おそれ って ない ん です か ね 。 これ 、 どなた か 。 ありがとう ござい ます 、 お 答え ください 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   これ まで ずっと 私 ども お 答え し て き た 、 また 議論 し て き た よう に 、 当面 数 か所 で しか 我々 この ＩＲ と いう の を 想定 し て い ない わけ です 。   どう いう 地域 が 選ば れ て どう いう 事業 者 が 選定 さ れ て いく か と いう こと は 、 もう 全 国民 、 全 世界 注視 の 下 で 行わ れる 作業 に なる わけ です から 、 当然 、 透明 感 を 持っ た 公正 厳正 な プロセス で なけれ ば 、 国民 の 信頼 も 世界 の 、 国際 社会 の 信頼 も 得る こと が でき ませ ん し 、 そう で なけれ ば 成功 する こと も でき ない と いう ふう に 考え て おり ます 。 <br/> ○ 山本 太郎 君   ありがとう ござい ます 。   以前 、 国家 戦略 特区 や 構造 改革 特区 に おい て 何 件 か の カジノ 事業 の 特区 申請 が 上がっ て き た と 思う ん です ね 。 現実 に は 、 戦略 特区 で の カジノ 構想 は 立ち消え に なっ た と 。 それ で 、 カジノ は 国家 戦略 特区 内 で は でき ない と いう 趣旨 の 回答 が 警察 庁 と 法務 省 から 出 た そう な ん です 。   申し訳 ない です 、 時間 が 短い の で 、 どちら か 一方 で も いい です か 、 多分 法務 省 の 見解 の 方 が 短かっ た です か ね 、 でき なかっ た 理由 と いう の は 。 申し訳 ない です 、 警察 庁 の 方 来 て いただい た の に 。 その 理由 を お 聞か せ ください 。 <br/> ○ 政府 参考 人 （ 加藤 俊治 君 ）   お 答え を 申し上げ ます 。   御 指摘 の 検討 要請 に 対する 回答 に おき まし て 法務 省 は 、 刑法 に より 禁止 さ れ て いる カジノ を 合法 化 する 措置 が 要望 さ れ た こと に 対し て 、 措置 の 分類 と し て は 、 特区 と し て の 対応 が 不 可能 で ある もの で ある 旨 の 回答 を し て おり ます 。 措置 の 内容 と し て は 、 法律 上 の 手当て を 必要 と する もの で ある 旨 回答 し て おり ます 。   その 理由 と いたし まし て は 、 刑法 百 八十 五 条 及び 百 八十 六 条 は 社会 の 風俗 を 害する 行為 と し て 規定 さ れ て おり ます ところ 、 刑罰 法規 の 基本 法 で ある 刑法 を 改正 し て 、 カジノ のみ を これ ら の 構成 要件 から 外す こと は 、 その 趣旨 に 鑑み 、 でき ない と 考える こと を 挙げ て おり ます 。   また 、 その 際 の 回答 に 明記 し て は おり ませ ん が 、 特区 と し て の 対応 が 不 可能 で ある もの と し た 理由 と し て は 、 全国 的 な 統一 性 の 観点 から 、 特定 の 地域 に おい て 基本 法 で ある 刑法 の 適用 を 一律 に 排除 する こと は でき ない と 考え た ところ で ござい ます 。   もっとも 、 同じ 回答 に おき まし て 、 カジノ に 関する 特別 法 の 制定 に 関し 、 御 指摘 の 回答 に おい て も 、 法務 省 が 積極 的 に 検討 する 主体 で は ない もの の 、 いずれ か の 省庁 に おい て カジノ を 法制 化 する 法律 を 立案 する こと と なれ ば 、 その 内容 に つい て 個別 に 当該 省庁 と の 協議 に 応じる 用意 は ある と し て いる ところ で ござい まし た 。   以上 で ござい ます 。 <br/> ○ 山本 太郎 君   ありがとう ござい ます 。 要 は 、 賭博 を 許す わけ に いか ない と いう こと です よ ね 、 特区 で 。 そう いう 考え 方 です よ ね 。 百 八十 五 、 百 八十 六 に 該当 する 話 です よ ね 、 それ ね 、 要 は 。 そこ は 引っかかる から 国家 戦略 特区 で は でき ない よ と いう こと で 立ち消え に なっ て いっ た と いう 話 です よ ね 、 国家 戦略 特区 で カジノ は と いう 話 です ね 。   お 聞き し ます 、 内閣 府 地方 創生 、 藤原 さん に 次 お 聞き し ます ね 。 できれ ば 端的 に お 答え いただける と 助かり ます 。   国家 戦略 特区 で は 今後 規制 改革 の メニュー を 増やす お 気持ち が ある と の お 話 を 以前 聞い た こと が ある ん です けれど も 、 今後 どの よう な 分野 に おい て どの よう な メニュー を 増やし て いく お つもり な ん でしょう か 。 <br/> ○ 政府 参考 人 （ 藤原 豊 君 ）   お 答え 申し上げ ます 。   本年 六 月 に 閣議 決定 さ れ まし た 日本 再興 戦略 二 〇 一 六 に おき まし て 、 国家 戦略 特区 に おい て は 、 今後 二 年間 を 集中 改革 強化 期間 と いたし まし て 、 重点 的 に 取り組む べき 、 これ は 外国 人 の 受入れ で ござい ます と か あるいは イン バウンド の 振興 、 そう いっ た 部分 に つき まし て 、 六 つ の 分野 を 始め と し まし て 規制 改革 を 進め て いく と いう ふう に さ れ て おり ます 。   自治 体 や 事業 者 から 具体 的 な ニーズ に 基づい て 規制 改革 の 御 提案 を 幅広く いただき ながら 、 今後 必要 な 規制 改革 を 実現 し て まいり たい と 思っ て おり ます 。 <br/> ○ 山本 太郎 君   今回 の プログラム 法案 が 可決 さ れ て 、 先 の 実施 法案 が 可決 すれ ば 、 これ 、 カジノ が 違法 と いう 縛り 外れる わけ です よ ね 。 と いう こと は 、 つまり 、 カジノ の 違法 性 の 阻却 事由 が でき た 場合 、 特区 事業 と し て カジノ も 国家 戦略 特区 に おい て 認定 さ れる こと 、 十分 に あり 得る と 思う ん です よ 。 だ って 、 元々 国家 戦略 特区 で でき なかっ た 理由 が 、 要 は カジノ イコール 賭博 だっ た から 。 これ が 別 の ライン で プログラム 法 が 通り 、 実施 法 が 通っ た 後 に は 、 これ 、 元々 国家 戦略 特区 で 拒ま れ て い た こと は 解消 さ れる わけ です よ ね 。 これ 認定 さ れる 可能 性 と いう の は 特区 に おい て ある ん です か ね 。 藤原 さん 、 いい です か 。 <br/> ○ 政府 参考 人 （ 藤原 豊 君 ）   規制 改革 の 実現 が ござい まし て 初めて 経済 効果 が 発揮 さ れる と いう こと で ござい ます 。   今回 の 場合 、 刑法 の 特例 措置 を 伴う と いう こと を 国家 戦略 特区 で 行う か どう か と いう 判断 に なる と 思い ます けれど も 、 これ まで も 、 例えば 構造 改革 特区 で あれ ば 刑法 の 特例 措置 と いう の は 慎重 な 判断 が 必要 だ と 。 通常 の 法律 で は なく 基本 法 で ござい ます の で 、 そう いっ た 判断 に は 慎重 で ござい まし た 。 国家 戦略 特区 で 仮 に この **カジノ** **、** **ＩＲ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に 関し まし て 議論 を 進める と いう 場合 に は 、 そう いっ た 刑法 の 特例 措置 、 法務 省 その 他 と 議論 が 慎重 に 行わ れる 必要 が ある と 考え て ござい ます 。 <br/> ○ 山本 太郎 君   これ 十分 あり 得る 話 だ と 思う ん です 。 だ って 、 どう し て 国家 戦略 特区 で カジノ が でき なかっ た ん です か って 。 違法 性 の 阻却 の 問題 が あっ た と 、 阻却 事由 の 問題 が あっ た と 。 だ けど 、 これ 、 結局 そっち に 向かっ て やっ て いる わけ です もの ね 、 プログラム 法 、 その 後 の 実施 法 。 と する なら ば 、 国家 戦略 特区 で でき なく なる 理由 が ない ん です よ ね 。 その 先 の こと は その 先 に 考える と いう お 話 だ と 思い ます 。 これ 、 十分 あり 得る と 思う ん です 。   国家 戦略 特区 の 枠組み で カジノ を 含む ＩＲ を やる と いう 話 、 これ 私 、 ちょっと 資料 を 付け させ て いただい た ん です けれど も 、 国家 戦略 特区 指定 する とき の 基準 って こんな もの が ある よ って 。 これ 赤線 引い て ある ところ だけ 読ん で みる と 、 大きな 経済 的 社会 的 効果 が 生じる よう な ところ 、 実施 する こと に より 、 産業 の 国際 競争 力 の 強化 又 は 国際 的 な 経済 活動 の 拠点 の 形成 を 通じ て 、 全国 的 な 社会 的 経済 的 効果 も 含め て 広く 波及 効果 を 及ぼす もの 、 ほか に は 先進 性 、 革新 性 を 有する もの 、 従来 なかっ た 取組 を 新しく 行う 場合 も 含む と 書か れ て いる 。 ほか に も 、 区域 内 の 地方 公共 団体 が 取り組む 意欲 が 高い と か 。 これ 、 で も 、 カジノ で 、 この ＩＲ で 言っ て いる こと 、 ほとんど 当てはまら ない ん です か 。 観光 業 です よ ね 。   特区 やっ て 何 が 悪い ん だ って 話 です けれど も 、 これ やら れる こと で 損する の 国民 な ん です よ 。 だ って 、 皆 さん 、 ずっと おっしゃっ て い た じゃ ない です か 、 税金 を 掛け ない ん です と 。 皆 さん の 税金 は 使い ませ ん 、 負担 は さ せ ませ ん と 。 民間 が やり ます から と うたっ て おい て 、 これ 戦略 特区 内 で やる こと に よっ て 固定 資産 税 、 設備 投資 、 所得 税 など の 控除 も 受け られる と 。 これ 、 事実 上 税金 で 補助 する 事業 に なる わけ です よ ね 。   もし も 、 これ が 本当 に 特区 で やら れる と いう 話 に なっ た 場合 、 だ って もう 縛り 何 も ない です よ 。 縛り は この プログラム 法案 と この 先 の 実施 法 で 外れる ん です から 。 この 法 案 の 説明 で 言い 続け て き た こと と 全く 違う 話 に なる 。 しかも 、 これ 外資 で あれ ば 、 税 控除 を 受け て 安く 事業 が やれ た 上 に 、 再 投資 も ほとんど する こと なく 、 日本 人 の 富裕 層 から 大きく 、 そう で も ない 方々 から は それ なり に 資産 を 巻き上げる 国富 の 流出 を 税金 まで 使っ て お 手伝い する 話 です か と いう こと に なっ て いく 。   で も 、 今 聞い て も 分から ない です もの ね 、 今 聞い て も 分から ない 。 ─ ─ いや 、 手 挙がら ない です から 。 それ 、 おかしい 、 おかしい 。 だ って 、 分から ない って 言っ て いる ん だ から 、 まだ 決まる まで 。   だ けど 、 間違い なく この 特区 と いう もの で カジノ が 成立 し なかっ た 理由 は 刑法 の 百 八十 五 条 、 百 八十 六 条 です よ ね 、 と いう 説明 あり まし た よ ね 、 法務 省 。 だ から 、 ギャンブル 、 この カジノ が 違法 だ と いう 考え 方 、 賭博 は 開け ませ ん と いう 考え 方 の 下 です もの ね 。 （ 発言 する 者 あり ） いや いや いや 、 ちょっと 待っ て ください 。 いやいや 、 ちょっと 待っ て ください 。 まだ 先 が ある ん です よ 。 だ って さんざん 答え た じゃ ない です か 、 皆 さん 。 始まっ て から 二十 分 全部 皆 さん で お 話し に なっ た ん です よ 。 済み ませ ん 、 先 に 進ま せ て ください ね 、 まだ 続き が あり ます から 。   これ 、 認定 さ れる 可能 性 は まだ 分から ない と 。 で も 、 それ は 排除 さ れ て い ない わけ です よ ね 。 特区 で でき なかっ た 理由 が 、 平たく 言い ます 、 ばくち 場 を 開く こと を 特区 だけ 合法 化 する に は 無理 が ある と いう こと です よ ね 。 特区 だけ で ばくち 場 を 開く こと を 合法 化 する に は 無理 が ある と 。 じゃ 、 違う ルート で と いう こと で プログラム 法 と 実施 法 と いう 二 段階 に 分かり にくく 、 ばくち 場 を 開ける 場所 や 主体 を 合法 化 し て しまえ ば 特区 と なじむ と いう 話 に なら ない か って 。 法務 省 と 警察 庁 の 懸念 を 、 今回 と 先 の 立法 、 実施 法 で 例外 を 作っ て 払拭 なさる ん でしょう 、 だ って 。 一 度 迂回 さ せ て ばくち 場 を 開ける よう に し た 上 で 、 運営 する 企業 に 税 控除 と いう インセンティブ まで 与える って 、 随分 大胆 な お 話 だ な と いう ふう に 思う ん です よ 。 これ 、 絶対 に 特区 で は やら ない と いう よう な 、 絶対 的 な 理由 って 何 か あり ます か 。   ちょっと 待っ て ください 、 そんな に 長く なら ない ん だっ たら お 答え 、 大丈夫 です か 。 じゃ 、 三十 秒 で お 願い し ます 。 ありがとう ござい ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   ありがとう ござい ます 。   特区 に おい て 刑法 の 特例 を 求め た 国家 戦略 特区 は これ は 認め られ ない と いう こと で もう 既に 解決 さ れ て いる 答え が ある わけ で あり ます 。 私 ども は 、 それ は 無理 だ と 私 ども も 思っ て おり ます の で 、 刑法 の 違法 性 を 阻却 する ため に 今回 の 特別 法 を 作っ て 、 カジノ 単体 で なく ＩＲ と いう 形 で 八 項目 を クリア し て 、 これ は 阻却 できる と いう こと で 政府 に それ を 求め て 、 実施 を 求め て いる わけ です が 、 実施 法 が でき た 暁 に 、 仮 に 特区 で やろう と 思え ば 、 それ は 特区 の 中 で 実施 法 の 特例 を 求める と いう こと に なり ます 。 それ は 私 ども 全く 、 これ は 厳しい 管理 の 下 に 、 カジノ 管理 委員 会 の 管理 の 下 に 置か れ ます から 、 そんな こと は 一切 考え て おり ませ ん し 、 それ は 私 ども も でき ない と いう ふう に 思っ て おり ます 。 <br/> ○ 山本 太郎 君   その お 言葉 を 信用 し たい 。 なぜ なら ば 、 こんな こと で やら れる カジノ に は 何 の 意味 も ない 。 それ は そう です よ ね 。 それ で 喜ぶ の は 誰 です か って 、 事業 者 だけ です よ って 、 あと ゼネコン と 、 あと 地上げ し て 土地 が もうかる 不 動産 屋 、 それ に 関連 する 人 たち だけ です よ 。 こう いう 税金 の 使い 方 が なされ ない よう に 、 何 か また この 先 変更 が ある か も しれ ない 、 国家 戦略 特区 に よっ て 、 それ に 関わる こと に よっ て 大きな 控除 が 与え られる よう な インセンティブ が 与え られ ない よう な 監視 も 是非 先生 たち に お 願い し たい と 申し上げ て 、 質問 を 終わり ます 。 そろそろ 終わり です よ ね 。   ありがとう ござい まし た 。         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 委員 長 （ 難波 奨 二 君 ）   この 際 、 委員 の 異動 に つい て 御 報告 いたし ます 。   本日 、 大門 実紀史 君 が 委員 を 辞任 さ れ 、 その 補欠 と し て 田村 智子 さん が 選任 さ れ まし た 。         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 里見 隆治 君   公明 党 の 里見 隆治 で ござい ます 。 本日 は 、 質問 の お 時間 を いただき まし て ありがとう ござい ます 。   まず 、 質問 を さ せ て いただき たい の が 、 依存 症 対策 に つい て で ござい ます 。 本 法案 を 議論 し て いる 中 で 、 既存 の ギャンブル 依存 症 に つい て 、 その 実態 が 必ず しも 正確 に 把握 さ れ て おら ず 、 また 、 その 対策 を 拡充 し て いく こと の 必要 性 が 明らか に なっ て まいり まし た 。   私 は 、 地元 愛知 県 で 、 カジノ あるいは ギャンブル で は ござい ませ ん けれど も 、 薬物 依存 症 の 回復 施設 と し て 活動 し て いる ダルク と いう 団体 の 拠点 を 複数 訪れ た こと が ござい ます 。 この 施設 で は 、 以前 依存 症 だっ た 方 が 先輩 と し て 、 自分 の 体験 を 基 に 今 依存 症 から 脱しよう と し て いる 方々 の 支援 を さ れ て いる と いう 、 非常 に ため に なる 視察 で ござい まし て 、 こう し た 経験 を ほか の 依存 症 に も 生かし て いけ ない か と 、 そんな こと を 伺っ て まいり まし た 。   こう し た 薬物 依存 あるいは アルコール 依存 に つい て は ある 程度 全国 的 に も 拠点 が ある 中 で 、 まだまだ この ギャンブル 依存 症 、 これ に つい て は 拠点 が 全国 的 に は 広がっ て い ない と 。 そう いう 中 で 、 まず この 拠点 を 整備 し て いく こと 、 そして 、 この 拠点 で どの よう に 相談 支援 を し て いっ たら いい の か 、 こう し た 点 も 、 まだまだ 今後 整備 す べき 点 が 多い か と 思い ます 。   今回 の 法案 の 議論 の 過程 で 、 一致 し て 御 提案 を いただい て いる 発議 者 の 皆 様 、 そして 、 与野党 に かかわら ず この 依存 症 対策 、 これ に つい て は 一致 する ところ と いう ふう に 私 も 考え て おり ます し 、 また 、 衆議 院 の 内閣 委員 会 に おい て も 附帯 決議 が なさ れ た と いう ふう に 承知 を し て おり ます 。   この 点 に つき まし て 、 まず 政府 に おか れ て は 、 ギャンブル 依存 症 に つい て 正確 な 実態 を 把握 いただく べき こと 、 また 、 カジノ に とどまら ず ギャンブル 依存 症 対策 に つい て 、 治療 のみ なら ず 、 教育 など 予防 、 家族 へ の ケア など 必要 な 予算 を 確保 し て 、 関係 省庁 が 十分 連携 し て 対策 強化 を 図る べき と 考え ます 。   政府 全体 と し て 、 今後 どの よう な 対策 を お 取り組み に なる か 、 こう し た これ まで の 審議 を 踏まえ て の 今後 の 方針 に つい て お 伺い を いたし ます 。 <br/> ○ 政府 参考 人 （ 中川 真 君 ）   政府 と いたし まし て は 、 まず は 議員 立法 で あり ます この **ＩＲ** **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に つい て の 国会 で の 御 審議 の 推移 を 見極める こと が 必要 か と いう ふう に は 承知 し て おり ます 。   その 上 で 、 この **ＩＲ** **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) が 成立 し た 場合 に 、 依存 症 対策 など に つい て の 、 政府 に つい て の 今後 の 対応 方針 に つい て の お 尋ね が ござい まし た 。   政府 と いたし まし て は 、 **ＩＲ** **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) を めぐる これ まで の 国会 の 御 審議 の 中 で 、 カジノ 事業 者 へ の 規制 の 在り 方 、 反 社会 勢力 の 排除 の 徹底 、 マネー ロンダリング 対策 の 徹底 、 ギャンブル 等 依存 症 対策 の 包括 的 な 強化 の 必要 性 など 、 多 方面 に わたる 論点 が 御 議論 さ れ た と 承知 し て おり ます 。   これ ら の 対策 の 必要 性 に つい て は 、 法 第 十 条 で 政府 に 必要 な 措置 を 講ずる こと が 求め られ て いる だけ で は なく 、 衆議 院 に おか れ まし て は 十 五 項目 に わたる 附帯 決議 も 付さ れ て いる と 承知 し て おり ます 。 十条 で 求め られ て いる 必要 な 措置 や 、 これ まで 国会 で 御 審議 いただい た 論点 は 、 政府 に おい て は 多数 の 省庁 の 所管 事項 に またがっ て おり まし て 、 仮 に この **ＩＲ** **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) が 成立 する 場合 に は 、 政府 と いたし まし て は 、 関係 省庁 が 十分 連携 し て 、 また 、 衆議 院 の 附帯 決議 に も あり ます よう に 十分 に 国民 的 議論 を 尽くし ながら 実施 法案 を 準備 する こと に なる と 承知 し て おり ます 。   また 、 今 ギャンブル 依存 症 に つい て の 個別 の 具体 的 な 取組 事項 に つい て の 御 指摘 が ござい まし た 。   これ まで 厚生 労働 省 の 政府 参考 人 も この 場 で 御 答弁 申し上げ て おり ます よう に 、 既に 今 年度 から この ギャンブル 依存 症 に 焦点 を 絞り まし た 実態 の 把握 に 関する 調査 、 しかも 、 前回 の 調査 と は 違い まし て 、 個人 が アンケート に 記入 する と いう 形 で は なく て 、 お 医者 さん など が 、 専門 家 が きちんと インタビュー など を し ながら 的確 な 判断 を する 人 が 答える と 、 そう いう フィードバック の ある よう な 調査 を し て おり ます 。 これ ら に つい て は 今 年度 末 まで に 何 ら か の きちんと し た 報告 を する と いう こと で の 答弁 も 厚生 労働 省 の 方 から これ まで 再々 ござい まし た 。   そう いう こと を 踏まえ まし て 、 政府 と いたし まし て は 、 ギャンブル のみ なら ず 、 ギャンブル 、 遊技 等 を 含む 既存 の 依存 症 対策 に つい て も しっかり 取組 を せよ と いう 御 議論 が この 国会 の 場 で ござい まし た の で 、 その 趣旨 を 踏まえ まし て きちんと 対応 し て いく こと が 必要 で ある と 、 かよう に 認識 を し て おり ます 。 <br/> ○ 里見 隆治 君   どう も ありがとう ござい ます 。   まさに この 依存 症 対策 、 これ もう 、 法律 通る 通ら ない 、 その 時期 に かかわら ず 、 早急 に 御 対応 を お 願い し たい と 。 公明 党 と し まし て も 、 党内 で しっかり 検討 チーム を 設置 し 、 政府 に も 提言 を し て いく 予定 で ござい ます 。 どうぞ 取組 に つい て よろしく お 願い いたし ます 。   次 に 、 法案 に 関し まし て 、 発議 者 に 御 質問 を さ せ て いただき ます 。   私 は 、 本 法案 に 関し まし て は 、 地域 の 創意 工夫 及び 民間 の 活力 を 生かし た 国際 競争 力 の 高い 魅力 ある 滞在 型 観光 を 実現 し て 、 観光 や 地域 経済 の 振興 を 進める と いう 考え 方 に つい て 、 我が 国 が 観光 立国 を 目指す と いう 上 で 大変 重要 な こと で ある と いう ふう に 考え て おり ます 。 一方 で 、 本 法律 案 の 特定 複合 観光 施設 に 含ま れる カジノ 施設 に つい て は 、 刑法 百 八十 五 条 が 賭博 を 禁じ て いる こと から 、 しっかり これ は 整理 し なけれ ば なら ない と いう ふう に 考え て おり ます 。 こう し た メリット 、 デメリット の 両者 の 観点 から 質問 を し て まいり ます の で 、 国民 の 皆 様 に この 法 案 の 趣旨 が 伝わる よう 御 答弁 を いただき ます よう お 願い を いたし ます 。   まず 、 昨日 も 様々 参考 人 の 先生 方 から 御 意見 を いただき まし た 。 その 中 で 、 私 まだまだ 、 国民 の 皆 様 も そう です が 、 専門 家 の 皆 様 で も 相当 この ＩＲ に 対し て の 立ち 位置 と いい ます か 、 見 方 が 相当 幅 が 広い と いう ふう に 感じ て おり ます 。 市場 が 飽和 し て いる 中 で 後発 の 日本 が 成功 する の か と いう 意見 も 一方 で ござい ます 中 で 、 観光 立国 推進 政策 と し て カジノ を 含む ＩＲ を 取り入れ て いこう と いう の が 今回 の 法案 の 中核 で ある と いう ふう に 受け止め て おり ます が 、 その 意義 や 必要 性 、 この 感じ 方 が 、 捉え 方 が 人 に よっ て 大きく 異なり ます 。 その 理由 は 、 この ＩＲ に つい て どう いう ふう に 捉える か 、 それ が 人 に よっ て 相当 イメージ が 異なっ て いる から で あっ て 、 共通 認識 が なかなか でき て い ない の で は ない か と 。   したがい まし て 、 そこ で 改めて この 時点 で に なり ます けれど も 、 この ＩＲ に つい て どの よう な イメージ を 発議 者 の 皆 様 は お 持ち で ある か 。 具体 的 に は 、 箇所 数 、 これ は 二、三 程度 と いう お 話 、 御 答弁 が ござい まし た 。 あるいは 、 都市 型 か 地方 型 か 。 あるいは 、 三 点 目 と し て 、 ターゲット 、 集客 する べき 対象 と し て 外国 人 富裕 層 な の か 外国 人 一般 顧客 層 な の か 、 あるいは 日本 人 富裕 層 、 日本 人 の 一般 顧客 層 の どれ が 多い の か 、 あるいは 混合 の 場合 、 その 割合 を どう いう ふう に 見 て いく の か 。 あるいは 、 何 を もっ て 利益 を 上げ て いく の か 。 こう し た 点 に つい て 、 改めて 御 答弁 を お 願い いたし ます 。 <br/> ○ 衆議 院 議員 （ 細田 博之 君 ）   カジノ を 含む ＩＲ に つき まし て は 、 国際 会議 場 、 そして 展示 場 、 レクリエーション 施設 、 宿泊 施設 、 文化 施設 、 カジノ 施設 など から 構成 さ れる 複合 観光 施設 で ある と いう こと が 定義 さ れ て おり 、 シンガポール など の アジア の 各 地域 に おい て 戦略 的 な 国際 観光 の 振興 を 図る ため に 導入 さ れ 、 あるいは 導入 が 計画 さ れ て いる もの で あり ます 。   我々 は 、 シンガポール が 成功 を 収め て 、 リー ・ クアンユー さん が 最初 は 反対 し て い た 、 しかし きっちり し た 管理 の 下 で やれ ば 、 大変 大勢 の 国際 的 な 観光 客 が やっ て き て 大 成功 を 収め た と 、 これ は 世界 的 に 衝撃 を 与え て おり まし て 、 所得 の 高い 日本 に おい て 、 国民 所得 が 高い です から 、 様々 な 意味 に おい て 外国 人 客 と 合わさっ て 成功 を 収める の で は ない か と いう 期待 が 大きい と 。 そして 、 大きな 投資 規模 が ある 、 そして 、 観光 地 の 形成 です と か 地域 経済 の 振興 に も 貢献 する と いう こと で ござい まし て 、 また 、 カジノ の 収益 が ある こと に よっ て 全体 が 潤い 、 かつ 財政 の 改善 に 資する と いう こと で 我々 も 取り上げ て いる わけ で ござい ます 。   日本 は 、 日本 再興 戦略 二 〇 一 六 など に おい て 、 政府 が 二 〇 三 〇 年 まで に 六千 万 人 を 目標 と する と いう 海外 観光 客 の ツーリズム の 創出 に つい て は 、 東京 オリンピック の 後 は 必ず しも 平たん で は ない と 思っ て おり まし て 、 今 や 三 年 ほど は 徒過 いたし まし た の で 、 これ から どこ を 造っ て も オリンピック より 後 に なる 。 その とき に 、 世界 から できる だけ 多く の お 客 さん を 迎え 、 また 日本 人 も 平均 所得 、 貯蓄 など は 大きい わけ です から 、 本当 に 楽しい 施設 が できれ ば 、 それ で かなり にぎわう 楽しい 施設 が できる の で は ない か 。   しかし 、 これ だけ 大きな 施設 で ござい ます から 、 数 と し て は 、 従来 から 言わ れ て おり ます よう に そんな に どこ に で も できる わけ で は なく て 、 まず 、 多分 二 つ 、 三 つ で やっ て 、 それ で 成功 する か どう か と いう の を 見極める と いう こと に なる の で は ない か と 思っ て おり ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   補足 を さ せ て いただき ます 。   まさに 、 今 答弁 が あっ た ところ で ござい ます が 、 一定 規模 以上 の もの を 求め て いく こと に なる と 思い ます 。 これ は 日本 経済 全体 へ の プラス の 効果 と いう こと を 考え て 、 温泉 旅館 の 横 に ちょこっと カジノ を 造る よう な こと は 想定 し て おり ませ ん の で 、 一定 の 宿泊 施設 、 展示 場 、 国際 会議 場 、 劇場 、 こう いっ た もの を 総合 的 に 整備 し て いく こと に なり ます の で 、 一定 規模 以上 の 民間 投資 が 必要 と なり ます し 、 地方 は 地元 議会 の 同意 も 必要 と いう こと に なっ て いく と 思い ます の で 、 そう いっ た こと を 考える と 、 十 も 二十 も 日本 全国 できる わけ で は なく て 、 一定 の 数 、 これ は 衆議 院 の 附帯 決議 で も 、 厳格 に 少数 に 限り 、 区域 認定 数 の 上限 を 法定 する と いう ふう に さ れ て いる ところ で ござい ます が 、 今 お 話 が あり まし た とおり 二 つ 、 せいぜい 二 つ 、 三 つ から 始め て 、 そして その 効果 あるいは 課題 を 検証 し ながら 、 増やす と し て も 段階 的 に 増やし て いく と いう こと に なり ます 。   それ から 、 大 都市 か 地方 か と いう こと で あり ます が 、 私 ども 大 都市 に 限る と は 全く 思っ て おり ませ ん で 、 地域 が 地域 の 特徴 を 生かし て 、 日本 なら で は の 地域 の 観光 資源 を 生かし た 、 そう し た ＩＲ 施設 も 是非 造っ て もらい たい と 思っ て おり ます が 、 先ほど 申し上げ まし た と おり 、 一定 規模 以上 で 投資 が 必要 に なっ て まいり ます し 、 地元 議会 の 同意 も 必要 と いう こと で あり ます の で 、 なかなか 、 そう あちこち で 造る と いう こと で は なく て 、 幾 つ か の 地方 で 是非 こう し た もの も 整備 が さ れる と いい な と いう ふう に 思っ て いる ところ で あり ます が 、 具体 的 な 基準 は 実施 法 に おい て 定め られ て いく こと に なり ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   ターゲット は 富裕 層 か 一般 層 か 、 あるいは 外国 人 か 日本 人 か と いう お 尋ね で ござい ます が 、 提出 者 と いたし まし て は 、 言う まで も なく 、 内外 問わ ず 幅広い 層 の 誘客 を 目指す 、 また それ が できる ＩＲ の 構築 を 考え て いる ところ で ござい ます 。   どこ に どう いう 規模 の どう いう 内容 の ＩＲ が できる か に よっ て そこ は 微妙 に 変わっ て くる ん だ と 思い ます が 、 先生 御 承知 の とおり 、 今年 十 一 月 に イン バウンド が 二千 万 人 を 突破 し まし た 。 これ を 四千 万 人 、 六千 万 人 に 引き上げ て いく と いう の が 政府 の 目標 で ござい ます 。 目下 の ところ 、 二 五 ％ が 中国 、 二 〇 ％ が 韓国 、 一 八 ％ が 台湾 と いう こと で 、 やっぱり かなり アジア に 集中 し て いる 向き が あり ます が 、 四千 万 人 、 六千 万 人 と いう ふう に し て いき ます と 、 かなり 世界 から 幅広く お 客 様 を 迎え て いか なく て は なり ませ ん 。 したがっ て 、 まさに 国 の 内外 を 問わ ず 、 幅広い 方 に 来 て いただける 施設 、 ＩＲ と いう もの を 目指し て いる ところ で ござい ます 。   それ から 、 何 を もっ て 利益 を 上げ て いく か と いう お 尋ね で ござい ます が 、 さっき も 申し上げ まし た よう に 、 今 ネバダ 州 で は ノン ゲーミング の 収入 が 六 割 を 超える 収益 構造 に なっ て おり ます 。 したがっ て 、 ゲーミング 部分 だけ に 頼る ＩＲ で は なく て 、 この ノン ゲーミング の 収益 部分 が 大きく なっ て いく よう な ＩＲ を 目指し て いく べき だ と 考え て いる ところ で ござい ます 。 <br/> ○ 里見 隆治 君   ありがとう ござい ます 。   こう し た イメージ を 前提 と し て 、 ＩＲ に よる 経済 効果 、 また それ に 付随 し て 、 負 の 側面 、 社会 的 コスト 、 例えば 治安 です と か 依存 症 に 与える 社会 的 影響 、 これ に つい て お 伺い を し て おき たい と 思い ます 。   これ に つき まし て も 、 昨日 の 参考 人 から の 御 説明 、 これ も 様々 、 もう 百 八十 度 違う よう な 捉え 方 も ござい まし て 、 これ も 国民 の 皆 様 、 審議 を 御覧 に なっ て ちょっと 不安 に なっ て おら れる の か な と 。 そう いう 意味 で 、 これ も 、 御 提案 を さ れ て いる 発議 者 の 皆 様 の お 立場 から 、 この 経済 効果 、 これ を どの よう に 見 て おら れる の か 。   また 、 この 社会 的 コスト 、 これ も 定量 的 に 測れる か どう か 昨日 議論 が あっ た ところ で ござい ます が 、 こう し た 点 も 、 私 、 昨日 も 意見 と し て 申し上げ た とおり 、 こう し た もの を 極力 明確 化 し て いく 、 明示 し て いく こと で 具体 的 な 、 冷静 な 議論 が 、 検討 が なさ れ て いく と いう 意味 で 、 是非 と も 発議 者 の お 立場 で これ を より 分かり やすく 明示 化 できる よう な 、 そう いっ た 御 説明 を 試み て いただき たい と いう ふう に 考え て おり ます が 、 この 点 、 御 答弁 を お 願い いたし ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   経済 効果 、 社会 的 な コスト 共 に 、 具体 的 に どの 地域 に どの 規模 の どう いう スタイル の ＩＲ が できる か と いう こと が 定まっ て き ませ ん と 、 この 段階 で 正確 に はじき出す と いう こと は 困難 で ある と いう ふう に 思っ て おり ます が 、 我々 が 参考 に し て おり ます 、 度々 出 て おり ます シンガポール の 事例 で は 、 この 数 年間 、 二 つ の ＩＲ の 開業 で 観光 客 数 が 六 割 増 、 観光 収入 は 九 割 増 と いう 実績 を 上げ て おり ます の で 、 十分 それ に 匹敵 する か 上回る 効果 を 上げる こと が できる ん で は ない か と 期待 を し て いる ところ で ござい ます 。   なお 、 具体 的 に は 、 この 実施 法 の 制定 後 に 地方 公共 団体 の 具体 的 な 申請 に 基づい て ＩＲ 区域 が 認定 さ れ 、 計画 が 具体 化 し て いく 中 で 当然 、 その 経済 効果 は いか ばかり か 、 社会 的 コスト は いか ばかり か と いう よう な 点 も しっかり と 審査 を さ れる こと に なる と いう ふう に 考え て おり ます 。 <br/> ○ 里見 隆治 君   今 、 社会 的 コスト に つい て は 余り 触れ られ て おり ませ ん でし た けれど も 、 今後 、 この プログラム 法 、 さらに 実施 法 、 さらに 地方 で と いう ふう に 検討 が 進み ます と 、 これ 、 私 ども 国会 、 委員 会 の 中 で も 大変 な 議論 に なっ て おり ます し 、 まだまだ 世論 調査 等 で も 非常 に 数字 が 割れ て いる と いう 中 で 、 地方 で 御 検討 いただく 際 に も 余り 感情 的 に 賛成 、 反対 と ぶつけ合っ て も 生産 的 で ない わけ です から 、 中央 レベル で こう し た 社会 的 な 便益 また コスト に つい て しっかり と 客観 的 に 明記 できる よう な 、 そう いっ た 工夫 を しっかり 地方 に 御 提示 いただく 必要 が ある ん で は ない か と 、 その よう に 考え て おり ます の で 、 その 点 、 御 検討 を 引き続き お 願い し たい と 思い ます 。   次 に 、 本 法案 に つい て なかなか 理解 が 得 られ ない 背景 の 一 つ に 、 カジノ が 犯罪 あるいは 治安 悪化 、 青少年 へ の 悪 影響 、 依存 症 患者 の 増加 など に つながる の で は ない か と いっ た 不安 が まだまだ 拭い 切れ て い ない と いう こと が 背景 に ござい ます 。 この 点 、 衆議 院 内閣 委員 会 に おい て も 、 犯罪 防止 ・ 治安 維持 、 青少年 の 健全 育成 、 依存 症 防止 の 観点 から 問題 を 生じ させ ない よう に と の 附帯 決議 も なさ れ て いる ところ で ござい ます 。   したがい まし て 、 いま 一 度 、 こう し た 点 から 厳格 な 規制 を 構築 し て いく と の 観点 で 、 反対 さ れ て いる 国民 の 皆 様 に も 、 あるいは 判断 に 迷っ て おら れる 国民 の 皆 様 に も 安心 、 御 理解 いただける よう な 発議 者 の 御 所見 を いただけれ ば と 思い ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   私 ども 、 法案 の 十条 で 、 まさに 、 この カジノ 施設 の 設置 、 運営 に 伴う 有害 な 影響 の 排除 を 適切 に 行う 観点 から 次 に 掲げる 事項 に つい て 必要 な 措置 を 講ずる もの と する と いう こと で 、 幾 つ か の 項目 、 まさに 犯罪 の 予防 で あっ たり 、 風俗 環境 の 保持 で あっ たり 、 青少年 の 保護 で あっ たり 、 こう いっ た こと を 列挙 を さ せ て いただい て 、 政府 に 対し て 必要 な 措置 を 講ずる と いう こと で その 対応 を 求め て いる ところ で ござい ます けれど も 、 今般 、 衆議 院 、 参議 院 の 議論 を 通じ て 様々 な 御 議論 を いただき まし た の で 、 私 ども も 真摯 に 受け止め て 、 政府 が 作る べき その 実施 法案 の 方向 性 を 示そう と いう こと で しっかり と 答弁 を し て き た ところ で ござい ます し 、 衆議 院 の 附帯 決議 に も その 旨 記載 を さ れ て いる ところ で ござい ます けれど も 、 まさに 御 指摘 の とおり 、 カジノ 施設 が 社会 に 及ぼす 影響 や リスク を これ 十分 に 考慮 し て 、 犯罪 防止 ・ 治安 維持 、 そして ギャンブル 依存 症 防止 、 こう し た 対策 を 適切 に 講じ て いく ため に も 、 世界 最高 水準 の 厳格 な カジノ 営業 規制 等 を 構築 する と いう こと が 必要 で ある と 認識 を し て おり ます 。   具体 的 な その 内容 に つき まし て は 、 附帯 決議 も なさ れ て おり ます の で 、 諸 外国 に おけ る カジノ 規制 の 現状 も 踏まえ て 政府 に おい て 実施 法案 の 立法 の 過程 で 十分 な 検討 が 加え られ て 、 まさに 違法 性 を 阻却 する に ふさわしい そう し た 世界 最高 水準 の 厳格 な 規制 、 こう し た もの が 適切 に 規定 を さ れる と いう こと を 求め たい と いう ふう に 思っ て おり ます 。 <br/> ○ 里見 隆治 君   ありがとう ござい ます 。   そう し た 意味 で 、 今回 の プログラム 法 の 限界 と いい ます か 、 ある 程度 政府 の 実施 法案 に 委ね て いく と いう こと で 今後 の 検討 に 任さ れ て いる 、 そう いう 意味 で は 今後 政府 と し て どの よう に 検討 を し て いく か と 。 これ は 法案 成立 が 前提 で ござい ます けれど も 、 仮 に と いう こと で 政府 に おけ る 対応 に つい て 御 質問 を し たい と 思い ます 。   まず 、 政府 に おか れ て の この カジノ に 関する 規制 、 これ は 先ほど 来 質疑 応答 ござい まし た と おり 、 これ まで も 様々 検討 が なさ れ て い た と いう ふう に 承知 し て おり ます 。   まず 、 従来 、 構造 改革 特区 や 国家 戦略 特区 、 こう し た 制度 に おい て も カジノ を 含む 特区 を 認め て こ られ なかっ た と いう 経緯 が ござい ます けれど も 、 その 理由 に つい て 改めて お 伺い を いたし ます 。 <br/> ○ 政府 参考 人 （ 藤原 豊 君 ）   お 答え 申し上げ ます 。   カジノ を 含み ます ＩＲ の これ まで の 特区 制度 に おき ます 検討 経緯 に つい て 御 指摘 を 頂戴 し まし た 。   こちら の 、 まず 平成 十 四 年 に 成立 を いたし まし た 構造 改革 特区 の 制度 の 下 で の 経緯 で ござい ます けれど も 、 平成 十 四 年 から 二十 二 年 に かけ まし て 計 十 九 件 の 提案 、 これ が 自治 体 や 事業 者 から いただき まし た 。 他方 、 平成 十 四 年 に 、 これ は 政府 の 方針 で 、 刑法 に 関する もの に つい て は 選定 対象 外 と す べき で は ない か と いう 方針 、 それ から 、 先ほど から 御 議論 ござい ます 関係 省庁 、 これ は 法務 省 や 警察 庁 で ござい ます が 、 こちら から 慎重 に 検討 す べき と いう 意見 も ござい まし て 、 実現 を いたし ませ ん でし た 。   続き まし て 、 これ は 三 年 前 に なり ます が 、 平成 二十 五 年 に 国家 戦略 特区 、 この 法律 を 制度 化 する 際 で ござい ます が 、 これ も 様々 な 規制 改革 提案 の 要望 が あっ た 中 に 、 カジノ を 含む ＩＲ を 含め た 提案 、 十 件 ほど ござい まし た 。 各省 と の 間 で 引き続き 、 ここ は やはり 議論 が 煮詰まっ て い ない こと 、 また 、 その 頃 に なり ます と 超 党派 で 議員 立法 の 動き 、 まさに 今回 の 動き で ござい ます けれど も 、 ある こと 等 から 、 まず これ ら の 動向 を 注視 し て いく と いう 判断 を 当時 さ せ て いただい た ところ で ござい ます 。   この よう な 経緯 に 基づき まし て 、 カジノ を 含む ＩＲ に つき まし て は 、 構造 改革 特区 、 国家 戦略 特区 の 規制 改革 の メニュー と は 現在 至っ て い ない と いう 経緯 が ござい ます 。   以上 で ござい ます 。 <br/> ○ 里見 隆治 君   どう も ありがとう ござい ます 。   私 も 実 は 、 先ほど 他 の 委員 から 質問 が あり まし た 平成 二十 四 年 当時 、 内閣 官房 の 地域 活性 化 事務 局 に おり まし て 、 法務 省 の 所管 する 法律 を 担当 し て おり まし た の で 、 今 御 指摘 の あっ た よう な 答弁 、 記憶 に ござい ます し 、 確か に その よう な スタンス で 対応 し て い た と いう ふう に 思い出し て おる ところ で ござい ます 。   そして 、 私 自身 、 こう し た 特区 、 非常 に この 特区 制度 、 いい ところ は 、 規制 緩和 を 一旦 行っ た 後 に 、 期日 を 定め て 、 その 規制 緩和 が 社会 的 に どの よう な 効果 を もたらし た か 、 あるいは その 弊害 が どの よう で あっ た か と いう こと を きちんと 定期 的 に 検証 し て 、 その 検証 し た 結果 、 チューニング を し て いく と 。 厳し め に する こと も 当然 ある でしょう し 、 もう これ で 大丈夫 だ と いう こと で あれ ば 全国 的 に 展開 を する と 、 そう いっ た いわゆる 、 言い 方 は 悪い です けど 、 社会 的 実験 と いう よう な こと も 、 これ は 内容 に よっ て 実験 し て いい もの 悪い もの 当然 ござい ます けれど も 、 そう いっ た 意味 で 、 この カジノ に つい て 、 これ は 別 に 特区 制度 で やる と いう こと で は ござい ませ ん けれど も 、 まず 箇所 数 を 限っ て 実施 を し て いく と いう こと で ござい ます の で 、 そう いう 意味 で は 、 特区 制度 と 同様 な 、 この 初期 段階 で 数 年 時点 で の 検証 、 また 検証 結果 を 踏まえ て の 更なる 規制 の 在り 方 の 見直し と いっ た こと が 重要 か と 思い ます 。   この 点 、 提案 者 に 対し まし て 、 こう し た ステップ ・ バイ ・ ステップ の 取組 に つき まし て 、 この 規制 を 検証 し て 、 また その 次 に 緩める の か あるいは 厳しく する の か と いう 点 に つい て どの よう に お 考え か 、 その 点 、 御 答弁 を いただき たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   特区 制度 の 中 で 、 過去 に かなり 多く の 地域 から カジノ を 認め て くれ ない か と いう 話 が あっ て 、 政府 と し て は 却下 し た と いう 経緯 は 先生 から も 今 お 話 あり まし た し 、 政府 から も 説明 が あっ た ところ で ござい ます 。   私 ども が 考え て おり ます の は 、 そう いう 特区 と いう やり 方 で は なく て 、 特別 法 を 設け て 違法 性 を 阻却 を し て いく わけ です けれど も 、 それ に は やっぱり 様々 な 条件 を 付さ なけれ ば いけ ない よ ね と 、 それ が 今回 の **ＩＲ** **推進** **法** ( `特定複合観光施設区域の整備の推進に関する法律` ) の 中 に 盛ら れ て いる わけ で 、 例えば カジノ 管理 委員 会 と いう 厳格 な 今 まで なかっ た よう な 管理 委員 会 を つくる と か いう よう な こと も 含め て 、 特別 に 当面 二、三 か所 程度 で 限定 的 に 施行 し 、 その 効果 、 課題 を 十分 に 評価 、 検証 し ながら 、 安全 に 適切 に 運営 さ れ て いる か どう か と いう こと を 検証 し 、 国民 が この 構想 に つい て 、 事業 に つい て 信頼 を し て いただける と いう 状況 を 確認 し た 上 で 初めて 段階 的 に 次 に 進ん で いく べき だ と いう 考え 方 を 持っ て いる ところ で ござい ます 。   衆議 院 内閣 委員 会 の 附帯 決議 に おい て は 、 ＩＲ 区域 の 数 に つい て は 、 「 国際 的 競争 力 の 観点 及び ギャンブル 等 依存 症 予防 等 の 観点 から 、 厳格 に 少数 に 限る こと と し 、 区域 認定 数 の 上限 を 法定 する こと 。 」 と さ れ た ところ で ござい ます 。   したがっ て 、 先生 おっしゃる よう に 、 提出 者 と いたし まし て は 、 ステップ ・ バイ ・ ステップ の 手法 を 用い て 慎重 に 進め て いく 必要 が ある と いう ふう に 考え て いる ところ で ござい ます 。 <br/> ○ 里見 隆治 君   どう も ありがとう ござい まし た 。   こう し た 特区 の 議論 を 経 て 、 二 年 前 に 政府 と し て は ＩＲ に つい て 内閣 官房 に 新しく 検討 体制 を 構築 し た と いう ふう に 承知 を し て おり ます 。 二 年 前 に ＩＲ に つい て 政府 で 検討 を 始める こと と し た 背景 、 また 、 それ 以来 どの よう な 体制 、 部局 、 人数 等 どう いっ た 体制 で 検討 を さ れ て いる か 、 また 、 その 検討 状況 、 先ほど 海外 の 事例 等 、 ＩＲ に つい て 、 カジノ に つい て も 勉強 さ れ て き た と いう こと です けれど も 、 もし 何 か 御 紹介 いただける よう な こと が あり まし たら 、 その 点 も お 願い いたし ます 。 <br/> ○ 政府 参考 人 （ 中川 真 君 ）   ＩＲ に つき まし て は 、 先ほど 政府 参考 人 から も 紹介 あり まし た けれど も 、 平成 二十 五 年 末 に この 議員 立法 の **ＩＲ** **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) が 提案 さ れ た こと など 、 そして 、 ちょうど 、 衆議 院 内閣 委員 会 で あっ た と 思い ます けれど も 、 この **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に つい て の 御 審議 の 機会 も あり まし た こと など も 踏まえ まし て 、 ちょうど 二 年 前 の 政府 の 成長 戦略 、 日本 再興 戦略 改訂 二 〇 一 四 の 中 で 、 先ほど も 御 答弁 さ せ て いただき まし た よう に 、 この **ＩＲ** **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) の 状況 や ＩＲ に 関する 国民 的 な 議論 を 踏まえ 、 関係 省庁 に おい て 検討 を 進める と いう こと で 進め て まいっ た 次第 で ござい ます 。   その 政府 の 中 で の 検討 体制 で ござい ます が 、 これ まで も 国会 審議 の 中 で 御 答弁 申し上げ た こと も ござい ます けれど も 、 現 時点 に おき まし て は 、 内閣 官房 に おい て 、 内閣 審議 官 三 名 を 含め 、 各 省庁 から の 出向 者 など 約 二十 名 弱 に よっ て 必要 な 検討 など を 進め て いる ところ で ござい ます 。   これ まで 、 委託 調査 事業 など を 行い まし て 、 諸 外国 に おけ る ＩＲ 制度 、 カジノ 規制 の 在り 方 など に つい て の 調査 報告 書 も まとめ たり 、 また 、 これ ら は 官邸 の ホーム ページ を 通じ まし て これ まで も 公表 し て おり ます し 、 また 、 ここ の 場 で 何 度 も 御 議論 いただい て おり ます 各国 の この 依存 症 対策 の 取組 に つい て も 、 別 の そう いう 調査 事業 を 掛け て 報告 書 を まとめ て もらい 、 これ ら の 報告 書 に つき まし て は 昨年 の 春 と 秋 に まとめ て おり ます 。 いずれ も ホーム ページ を 通じ て 公表 さ せ て いただい て おり ます 。   無論 、 我々 政府 の 職員 自身 も 、 各 カジノ 、 ＩＲ を 持っ て いる 国 に 出かけ 、 それぞれ の 御 当局 から 、 どの よう な 観点 で どの よう な 規制 を し て いる の か 、 あるいは どの よう な 取組 を この 依存 症 対策 の ため に し て いる の か 、 そう いう こと に つい て の ヒアリング など を 行っ て これ まで 蓄積 を し て き た つもり で ござい ます 。 <br/> ○ 里見 隆治 君   ありがとう ござい ます 。   その 意味 で 、 二 年間 、 相当 検討 も し 、 また 様々 調査 も さ れ て いる と いう こと で ござい ます が 、 今回 仮 に この 法律 が 成立 を し た 場合 に 、 そう し た 現在 の 検討 体制 あるいは 検討 内容 に つい て どの よう な 影響 を 受ける こと に なる の か と 。 これ は 法案 の 状況 に かかわら ず ある 程度 検討 を 更に 引き続き 進める こと が できる の か 、 それ と も 法案 が なけれ ば 、 法案 が 成立 し なけれ ば 進め られ ない 部分 が ある の か 、 そう いっ た 点 も 含め て 御 答弁 を いただけれ ば と 思い ます 。 <br/> ○ 政府 参考 人 （ 中川 真 君 ）   政府 と いたし まし て は 、 まず 、 この 議員 立法 で あり ます **ＩＲ** **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に つい て の この 国会 で の 御 審議 の 推移 を 見極め て いか なけれ ば なら ない と いう ふう に 思っ て おり ます 。   その 上 で 、 今 の お 尋ね は 、 この **ＩＲ** **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) が 成立 し た 場合 に どの よう な 形 で 今後 政府 が 検討 体制 を 組ん で いく の か と いう お 尋ね で あっ た と いう ふう に 承知 し て おり ます けれど も 、 御 指摘 の とおり 、 法 第 十 四 条 など に おき まし て 、 まず 、 内閣 に 内閣 総理 大臣 を 本部 長 と し ます 特定 複合 観光 施設 区域 整備 推進 本部 を 置く と とも に 、 法 第 二十 一 条 に より 、 その 本部 に 学識 経験 者 から 成る 特定 複合 観光 施設 区域 整備 推進 会議 を 置く こと と さ れ て おり ます 。   また 、 この **推進** **法** ( `特定複合観光施設区域の整備の推進に関する法律` ) 第 二十 二 条 に より 、 本部 の 事務 を 処理 さ せる ため に 事務 局 を 置く と いう こと に なっ て おり まし て 、 また 、 これ ら の 政府 の 検討 体制 に 関し ます この **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) の 規定 は 、 法 附則 に おき まし て 、 この 法 の 公布 の 日 から 起算 し て 三 か月 を 超え ない 範囲 内 に おい て 政令 で 定める 日 から 施行 さ れる こと と さ れ て おり ます の で 、 この 法案 が 成立 し た 場合 に は です けれど も 、 政府 と いたし まし て は 、 これ ら の 規定 を 踏まえ て 適切 に 対応 を し て いく こと に なる と 認識 し て おり ます 。 <br/> ○ 里見 隆治 君   ありがとう ござい ます 。   今後 、 政府 で 、 これ は 成立 し た 場合 に と いう 仮定 を 置い た 上 で です けれど も 、 実施 法 の 検討 に 当たっ て 法律 上 焦点 に なり ます の が 、 ずっと 審議 で 出 て き て おり ます 賭博 罪 の 違法 性 の 阻却 に つい て で あろう か と 思い ます 。   これ まで 発議 者 から は 、 プログラム 法 で 、 刑法 の 賭博 罪 の 違法 性 を 阻却 する もの で は ない と いう 御 説明 を いただい て おり ます けれど も 、 これ を 実施 法 に 委ねる 理由 に つい て まず お 伺い を し たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   お 答え 申し上げ ます 。   もう これ まで 議論 を し て いただい て いる とおり 、 ＩＲ の 導入 は 国際 観光 の 振興 と か 地域 経済 の 活性 化 と いう 大きな メリット も ある 一方 で 、 いろんな 負 の 側面 、 社会 に 与える 問題 、 リスク 、 こう し た こと に つい て の 国民 の 不安 、 懸念 も ござい ます 。 こう いっ た こと を 我々 踏まえ て 、 やはり 丁寧 に 議論 を 深める と 、 そして 国民 の 理解 、 信頼 を 得る 必要 が ある と いう ふう に 思っ て き た ところ で ござい ます 。   この ため 、 今回 の **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) は 、 基本 理念 、 方針 、 ＩＲ を 実現 する ため の その 方向 性 、 枠組み を 定め た もの で ござい まし て 、 いわゆる 二 段階 論 で ござい ます 。 この 法案 が もし 成立 を すれ ば 、 その 後 一 年 掛け て 政府 が 、 まさに 先ほど 来 答弁 に あり まし た よう に 、 これ まで 検討 し 、 考え て き た こと を 更に 深め て 、 日本 で ＩＲ を 実現 する ため の 、 まさに 賭博 罪 の 違法 性 阻却 の 点 も 含め て 具体 的 な 制度 設計 、 規制 、 こう し た もの を 実施 法案 の 中 で 定め て いく と いう こと に し た もの で ござい ます 。   いろんな 議論 が ござい まし た けれど も 、 議員 立法 で こう し た 規制 法 、 細かい 規制 に つい て 、 特に 御 議論 の あり まし た 、 まさに 民間 主体 で 賭博 罪 の 例外 を 認める と いう もの で ござい ます の で 、 違法 性 の 阻却 八 項目 に つい て やはり きちっと し た 規制 の 下 で これ は 行わ れ ない と 阻却 さ れ ない と いう こと で あり ます の で 、 これ は やはり 政府 に おい て しっかり と 具体 的 な 制度 設計 やっ て もらっ た 方 が いい だろう と いう 判断 で 、 私 ども は 、 基本 理念 、 方針 、 その 方向 性 を 今回 この 法案 で 定め て 、 そして 政府 に 実施 法案 で しっかり と 規定 を し て もらう と いう 、 そう いう 考え 方 に 立っ た もの で ござい ます 。 <br/> ○ 里見 隆治 君   今 、 実施 法 に 委ねる と いう 点 に つい て は 、 非常 に 政府 と し て は 重い 宿題 を 抱える こと に なる と 思い ます 。 これ は 政府 と し て 、 検討 し た けれど も やはり 阻却 する の は 非常 に 難しかっ た です と 、 法案 が 提出 でき ませ ん と いう わけ に は いか ない わけ で ござい ます 。   そう いう 意味 で 、 これ は 非常 に 、 政府 と し て 委ね られる と いう 点 に つい て どう お 考え に なる か 、 この 点 、 政府 に 確認 を さ せ て いただき たい と 思い ます 。 <br/> ○ 政府 参考 人 （ 中川 真 君 ）   違法 性 阻却 の 措置 を 含む 実施 法案 の 策定 を 委任 さ れる こと に つい て の 政府 と し て の 見解 の お 尋ね だ と いう ふう に 御 理解 し て おり ます 。   言う まで も なく 、 **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) の 第 五 条 に おき まし て 、 政府 は ＩＲ の 整備 の 推進 に 必要 な 法制 上 の 措置 を 、 この 法 の 施行 後 一 年 以内 を 目途 と し て 法的 措置 を 講じ なけれ ば なら ない と さ れ て いる ところ で ござい ます けれど も 、 この 点 に つき まし て は 、 先般 の 当 委員 会 に おき まし て も 、 たしか 白 委員 から 御 質問 いただき まし て 、 菅 官房 長官 から 、 国会 で 議員 立法 と し て 結論 を 出し た の で あれ ば 、 それ を やはり 私 ども 政府 の 立場 で は しっかり 受け止め て 、 その 趣旨 に 従っ て 、 国民 の 皆 さん に 、 理解 を し 、 また 不安 の ない よう な 形 に し て いく の が これ は 政府 の 役割 で ある と いう ふう に 思い ます と いう ふう に 御 答弁 を 申し上げ て おり ます 。   この ため 、 同法 案 が 施行 さ れ た 場合 に は 、 カジノ 関係 者 に 対する カジノ 規制 です と か 諸 外国 で の 例 です と か 、 入場 規制 に つい て の 海外 の 事例 など も 参考 に 、 衆議 院 で の 附帯 決議 の 御 趣旨 も 含め て 、 同法 案 に 関する 国会 で の 御 議論 や 国民 的 な 議論 も 踏まえ 、 検討 を 進める こと に なる と いう ふう に 考え て おり ます 。 <br/> ○ 里見 隆治 君   ありがとう ござい ます 。 この 点 、 非常 に 重要 な 検討 事項 か と 思い ます 。   また 、 もし これ が 通っ て 、 そして 政府 で と いう こと に なり ます と 、 様々 重要 な 事項 を 審議 する こと に なろう か と 思い ます 。 例えば 、 特定 複合 観光 施設 区域 整備 推進 本部 に おい て 重要 事項 を 調査 審議 する ため の 有識 者 で 組織 する 推進 会議 を 設置 する と 。 そして 、 この 有識 者 に つい て は 、 様々 な 観点 から 公平 に 偏り なく 調査 審議 を いただく 上 で 公正 な 人事 が 求め られる と いう ふう に 考え て おり ます 。   そう し た 意味 で 、 この 推進 会議 の 学識 経験 者 、 この 皆 さん を どう いう ふう に 人選 し て いく か 、 その 基準 等 に つい て 、 これ は 発議 者 の 方 に お 尋ね を し たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   ＩＲ の 導入 に 当たり まし て は 、 海外 の 事例 等 を 参考 に 、 観光 や 地域 経済 の 振興 に 寄与 する もの と する と とも に 、 カジノ 施設 の 導入 に 伴う 社会 的 問題 に つい て も 丁寧 に 議論 を 深め て いただく 必要 が ある と いう ふう に 思っ て おり ます 。   したがい まし て 、 推進 会議 の 委員 と いたし まし て は 、 海外 の カジノ や ＩＲ に つい て の 専門 家 、 観光 産業 、 観光 振興 に 関する 専門 家 、 地域 経済 振興 に 関する 専門 家 、 ギャンブル 依存 症 に 関する 専門 家 、 マネー ロンダリング 対策 や 反 社会 勢力 排除 対策 に 関する 専門 家 、 青少年 の 健全 育成 に 関する 専門 家 等 が 考え られ ます 。   具体 的 に は 、 こう いう 方向 性 で 政府 に おい て 適切 に 人選 が なさ れる もの と いう ふう に 考え て おり ます 。 <br/> ○ 里見 隆治 君   これ は 具体 的 に は 政府 で と いう お 話 です けれど も 、 先 ほど 来 申し上げ て いる とおり 、 これ は プラス の 面 、 マイナス の 面 ござい ます の で 、 しっかり 両面 を 見渡せる よう に 幅広く 、 また 公平 な 人選 を 行う べき だ と いう ふう に 申し上げ て おき たい と 思い ます 。   次 に 、 カジノ 管理 委員 会 に つい て お 伺い を いたし ます 。   これ は 、 カジノ 施設 の 設置 、 運営 に 関する 秩序 の 維持 、 安全 の 確保 を 図る ため に カジノ 施設 関係 者 に 対する 規制 を 行う と いう もの で 、 そう いう 意味 で は 、 犯罪 防止 、 治安 の 維持 、 マネー ロンダリング 対策 と いう 点 に おい て は 、 国家 公安 委員 会 あるいは 都 道 府 県 警察 本部 に おい て も もちろん 体制 整備 を 図る こと が 必要 で ござい ます し 、 カジノ 管理 委員 会 と の 連携 と いう こと も 必要 か と 思い ます 。   一方 で 、 カジノ 管理 委員 会 、 また 国家 公安 委員 会 、 それぞれ いわゆる 三 条 委員 会 と いう こと で 独立 性 を 確保 と いう こと で ござい ます けれど も 、 余り に 独立 性 と いう こと を 強調 し 過ぎ ます と この 連携 と いう こと は 取れ ない わけ で ござい まし て 、 そう いう 意味 で 、 この カジノ 管理 委員 会 、 また 国家 公安 委員 会 、 警察 、 都 道 府 県 警察 本部 と の 関係 、 いかに ある べき か 、 この 点 、 提出 者 に お 伺い を いたし ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   これ は 先生 おっしゃる とおり だ と いう ふう に 私 ども も 思っ て おり ます 。   カジノ 管理 委員 会 は 、 カジノ 施設 の 設置 、 運営 に 関する 秩序 維持 や 安全 の 確保 を 図る ため に カジノ 施設 関係 者 に 対する 規制 を 行う 委員 会 で ござい ます けれど も 、 カジノ と いう もの が これ まで 我が 国 に なかっ た 新た な 形態 で 特殊 な 事業 内容 で あり ます こと から 、 これ を 適切 に 規制 、 監督 を し て いく ため に は 、 この カジノ 管理 委員 会 と いう カジノ 専門 の 規制 機関 のみ なら ず 、 先生 おっしゃっ た 国家 公安 委員 会 、 都 道 府 県 警察 本部 と 適切 に 協力 、 連携 を し て いく こと が 不可欠 だ と いう ふう に 考え て おり ます 。 <br/> ○ 里見 隆治 君   ありがとう ござい ます 。   しっかり この 治安 の 維持 、 また 犯罪 防止 、 この 観点 は 決して 忘れ ない で その 体制 整備 を と いう ふう に お 願い し ます けれど も 、 結局 、 この 法律 、 実際 に 施行 また 運用 さ れる の は それぞれ の 地域 と いう こと で ござい ます 。 その 上 で 、 これ も 再三 、 私 、 昨日 来 申し上げ て おり ます けれど も 、 この ＩＲ が 地方 公共 団体 の 申請 に 基づい て 国 の 認定 を 受け た 区域 に 設置 さ れる と いう 、 そう いう 意味 で は 地方 公共 団体 の 役割 は 大変 重要 で ある と いう ふう に 考え て おり ます 。   その 意味 で 、 この 地方 公共 団体 の 役割 を どの よう に 考える か 、 この 点 を お 伺い いたし ます の と 、 加え まし て 、 この 地方 公共 団体 、 それ なり の 位置 付け を 与え られる と いう こと は 、 これ は 内閣 委員 会 で も 、 「 地方 議会 の 同意 を 要件 と する こと 。 」 と いう 附帯 決議 が なさ れ て おり ます けれど も 、 この ＩＲ を 受ける その 地域 に おい て の 住民 コンセンサス の 形成 プロセス 、 これ に つい て 大変 重要 で あろう か と 思い ます 。   こう し た 地方 公共 団体 の 役割 あるいは 申請 に 向け て の 地方 に おけ る 住民 の 皆 さん へ の 理解 を どの よう に 進め て いく か 、 その 手続 に つい て 発議 者 の 御 所見 を お 伺い し たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   地方 公共 団体 が 果たす 役割 も 極めて 重要 で ござい ます 。   国 に 推進 本部 、 政府 に 推進 本部 が でき まし た 場合 に は 、 その 推進 本部 から 基本 的 な 方針 が 定め られる と 思い ます が 、 その 方針 に 沿う よう に 、 地域 の インフラ の 整備 状況 あるいは 周辺 環境 の 現況 等 を 総合 的 に 地方 公共 団体 に おい て 勘案 を し て プラン を 作っ て もらう と いう こと が 必要 で ござい ます 。 様々 な 民間 事業 者 の 企画 提案 を 検討 し た 上 で 最も 効果 の 高い ＩＲ 施設 整備 計画 を 作成 し 、 国 に 対し て 区域 認定 の 申請 を する と いう の が まず 地方 公共 団体 の 役割 と いう こと に なり ます 。   また 、 先生 御 指摘 の 地域 の 同意 を 得る と いう こと も 地方 公共 団体 の 重要 な 役割 に なっ て いく と 思い ます 。 これ に つき まし て は 、 衆議 院 の 内閣 委員 会 の 附帯 決議 に おき まし て 、 やはり その 当該 地域 の 議会 の 同意 と いう もの が 必要 で あろう と いう こと が 附帯 決議 の 中 に 付さ れ た ところ で ござい ます 。 地域 で コンセンサス を 得る ため に は 、 やはり 説明 会 、 公聴 会 を 開く 等 、 しっかり と し た 取組 を し て いただく 必要 が ある と いう ふう に 考え て おり ます 。 <br/> ○ 里見 隆治 君   どう も ありがとう ござい ます 。   これ は しっかり と 地方 で 、 もし これ が 実施 法 まで 行っ て 通っ た と いう 場合 、 地方 で 混乱 を 起こさ せ て は いけ ない 。 この 目指す べき 観光 立国 と いう 点 は 非常 に いい こと だ と 思い ます けれど も 、 手順 を 間違う 、 あるいは コンセンサス づくり を 誤る と 、 なかなか 合意 も 得 られ ず 、 せっかく つくっ た 制度 も 動か なく なる と いう こと も ござい ます の で 、 どう か この 手続 あるいは 合意 形成 と いう 点 に つい て は 丁寧 に し て いただける よう な 、 そう し た 制度 設計 を お 願い し たい と 思い ます し 、 また 、 私 ども の この 場 で も そう し た 丁寧 な 議論 と いう もの を 引き続き お 願い を し たい と 思い ます 。   以上 で 終わり ます 。 <br/> ○ 西田 実仁 君   関連 し て 質問 を さ せ て いただき たい と 思い ます 。 公明 党 の 西田 です 。   もう ずっと この 審議 を し て き て 、 各党 各 会派 が 指摘 し 、 多く の 人 たち が 共通 し て 指摘 し て いる 問題 は 、 この ギャンブル 依存 症 の 問題 な ん です ね 。 ちょっと 質問 通告 し て おり ませ ん けれど も 、 質問 を し たい と 思い ます が 、 法律 を 提出 さ れ た 皆 様 方 です から すぐ お 答え に なれる と 思い ます の で 。   この 条文 を 全部 見る と 、 あれ だけ 皆 さん の ギャンブル 依存 症 に 対する 対策 と いう こと に 熱意 を 先ほど も 表明 さ れ まし た が 、 この 法文 の 中 に ギャンブル 依存 症 と いう 言葉 が どこ に も ない わけ です ね 。 カジノ と いう 言葉 は 数える と 二十 五 か所 ぐらい 、 当然 カジノ 施設 と か いろいろ ある わけ です けれど も 。   この ギャンブル 依存 症 対策 、 しかし しっかり やっ て いく ん だ と 、 こう いう 思い を 先 ほど 述べ られ まし た 。 この 法文 の 中 で どこ で それ を 読む の か を 改め て 御 指摘 いただき たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   お 答え 申し上げ ます 。   第 十 条 に 、 カジノ 設置 、 運営 に 伴う 有害 な 影響 の 排除 を 適切 に 行う 観点 から 、 必要 な 事項 、 措置 を 講じる と いう こと で 条文 を 設け て おり ます が 、 私 ども 、 その 第 一 項 の 八 号 に 、 「 カジノ 施設 の 入場 者 が カジノ 施設 を 利用 し た こと に 伴い 悪 影響 を 受ける こと を 防止 する ため に 必要 な 措置 に 関する 事項 」 と いう ところ で 、 ここ で 私 ども 依存 症 の こと も 認識 を し て 明記 を し て いる ところ で ござい ます が 。   法制 局 と の いろんな 議論 の 中 で 、 まだ カジノ が ない もの です から 、 日本 に は 、 その カジノ 依存 症 と いう もの が 、 そもそも カジノ が ない もの です から カジノ 依存 症 と いう もの が ない と いう こと で 、 法制 的 に いろいろ 議論 し た 結果 、 依存 症 と いう 言葉 は 、 カジノ 依存 症 と いう 言葉 は 、 あるいは ギャンブル 依存 症 と いう 言葉 は 、 ここ に は 法制 的 な 整理 の 下 で 私 ども そう 書か ず に 、 この よう な 形 で 「 悪 影響 を 受ける こと を 防止 する ため に 必要 な 措置 」 と いう ふう に 書か せ て いただい た 次第 で ござい ます 。 <br/> ○ 西田 実仁 君   確か に カジノ は ない ん です けれど も 、 その 悪 影響 の 中 に は ギャンブル 依存 症 と いう こと を 主に 想定 し て ここ に 込め て いる と 、 こう いう 話 でしょう か 。   附帯 決議 に は 、 ギャンブル 依存 症 と いう 言葉 は 、 依存 症 対策 と いう こと は 衆議 院 の 内閣 委員 会 の 附帯 に も 随分 項目 を 割い て 出 て ござい まし た 。 先ほど の 皆 さん の 思い 、 また この 委員 会 で の 共通 する 恐らく 思い と し て は 、 この ギャンブル 依存 症 対策 と いう もの を 、 どう あれ 、 これ もう 、 すぐ に もっと 拡充 し なきゃ いけ ない と 、 こう いう こと で あろう と いう ふう に は 思い ます 。   その 上 で 質問 を さ せ て いただき ます が 、 先ほど も ちょっと お 話し に なり まし た が 、 この プログラム 法 と 言わ れる 法律 が 通っ た 場合 に は 、 第 五条 で 、 この 法律 の 施行 後 一 年 以内 を 目途 と し て 必要 と なる 法制 上 の 措置 を 講じ なけれ ば なら ない と 、 こう いう ふう に 定め て ござい ます 。 義務 が 掛かっ て いる わけ です ね 。 閣法 と し て 提出 を する と いう こと だろう と 思い ます 。   一方 で 、 衆議 院 の 附帯 決議 に おき まし て は 、 この 必要 と なる 法制 上 の 措置 の 検討 に 当たっ て は 十分 に 国民 的 な 議論 を 尽くす べき で ある と 、 この よう に も 書か れ て ござい まし て 、 当然 、 これ が 仮 に 通っ た と すれ ば 実施 法 を 作る 際 は 閣法 で 出す わけ です から 、 与党 と し て の プロセス と いう こと も ある でしょう し 、 また 、 もっと 広く 国民 的 な 議論 を 尽くす と いう 附帯 に 載っ て いる こと を その まま 完璧 に こなす と すれ ば かなり 幅広い 議論 が なさ れ なけれ ば なら ない と 、 この よう に 思う わけ で あり ます けれど も 、 そこ の 提案 者 と し て の 思い を お 聞か せ いただき たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 細田 博之 君 ）   ＩＲ の 導入 は 、 国際 観光 の 振興 、 地域 経済 の 活性 化 など 、 メリット が 大きい わけ で は ござい ます が 、 一方 で 、 ＩＲ の 施設 の 一部 で ある カジノ 施設 に つい て は 、 社会 に 与える 問題 、 リスク に 対する 国民 の 不安 、 懸念 が 大きい ところ で あり ます 。 したがっ て 、 我が 国 に おけ る カジノ 施設 の 在り 方 に つい て 丁寧 に 議論 を 深める と とも に 、 国民 の 理解 、 信頼 を 得る 必要 が あり ます 。 その ため 、 本 法案 で は 、 基本 理念 、 方針 など 、 ＩＲ を 実現 する ため の 枠組み を 定める こと と し 、 カジノ 施設 の 在り 方 、 具体 的 な 規制 など の 詳細 に つい て は 政府 に おい て 実施 法案 で 定める と いう ふう に し た わけ で ござい ます 。   また 、 本 法案 成立 後 の 実施 法案 の 策定 、 審議 の プロセス に おき まし て は 、 政府 と Ｉ Ｒ 議連 、 地方 公共 団体 、 民間 、 経済 団体 など が 一体 と なっ て 国民 の 各界 各層 、 各 世代 を 幅広く 巻き込ん だ 議論 が 必要 で ある と 考え て おり ます 。 まだまだ 時間 を 掛け て しっかり と 議論 し て いく 必要 が ある と 考え て おり ます 。 <br/> ○ 西田 実仁 君   先般 、 私 の 質問 を さ せ て いただい た 際 に 、 様々 な ギャンブル 依存 症 も あり ます し 、 様々 な 負 の 側面 を なかなか この 日本 の 社会 で 受け入れ られ ない と すれ ば 、 当分 の 間 に 限っ て 日本 人 の 入場 を 禁止 し て は どう か と いう こと を 質問 さ せ て いただい た 際 に 、 岩屋 提出 者 の 方 から 、 それ は もう 随分 初期 の 段階 で 議連 で も 議論 を し た けれど も 、 やはり 刑法 の 属地 主義 と いう 、 その 場所 です ね 、 土地 と いう か 、 その 場所 に 属する 属地 主義 を 原則 と し て いる 以上 なかなか それ は 難しい ん で は ない か と いう 、 そう いう 御 答弁 を お 聞き を いたし まし た 。   改めて お 聞き し たい と 思う ん です けれど も 、 結局 、 この ギャンブル 依存 症 と いう こと が これ だけ みんな 問題 意識 を 持っ て い て 、 それ に 対する 対策 と いう こと を きちんと 取ら れ て 、 ああ 、 これ なら いける か も しれ ない と 国民 の 多く が 思う 状況 に なれ ば 、 それ は まさに 必要 は ない の か も しれ ませ ん けれど も 、 なかなか そう は なら ない と すれ ば 、 この ギャンブル 依存 症 対策 と いう こと が きちんと できる まで の 当分 の 間 、 例えば 日本 人 の 入場 を 禁止 する と いう よう な 議論 と いう の は なされ なかっ た の か 。   前回 の 議事 録 を もう 一 度 読み ます と 、 慎重 に 岩屋 議員 も 答弁 さ れ て おら れ まし て 、 こう いう 議論 も 行っ た と か 、 あるいは こう いう ふう に も 考え まし た だ と か 、 考え た ところ で ござい ます と いう ふう に お 答え いただい て おり ます の で 、 全く その 余地 が ない と も 思え ない 、 そう いう 議論 の 余地 を 残し た 御 答弁 だっ た よう に も 思っ て おり ます の で 、 改めて お 聞き し たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   先生 に お 答え し た よう に 、 刑法 の 属地 主義 等 の 様々 な 観点 から 、 日本 人 の 入場 を 一律 に 禁止 する の で は なく て 、 日本 人 、 内国 人 、 これ は だ から 定住 外国 人 など は 内国 人扱い と いう こと に なろう か と 思い ます 、 日本 人 と 同様 に アクセス が 外国 人 旅行 者 より も 容易 に なり ます の で 、 そう いう 意味 で 内国 人 と 、 こう 言っ たり し て いる わけ で ござい ます が 、 これ を 一律 に 入場 を 禁止 に する の で は なく て 、 やはり 一定 の 入場 管理 政策 、 入場 規制 政策 を 取る こと が ギャンブル 依存 症 対策 の 上 で も 適切 で は ない か と 判断 を 我々 し た ところ で ござい ます 。   仮 に 、 私 ども が 想定 し て いる よう に 、 この **ＩＲ** **の** **推進** **法** ( `特定複合観光施設区域の整備の推進に関する法律` ) 、 実施 法 、 あるいは その 後 の 、 その 地域 の 選定 、 事業 者 の 審査 、 事業 者 へ の 免許 の 付与 、 ＩＲ 施設 の 建設 、 完成 、 最終 的 な 許諾 、 そして 実際 の 施行 と いう ふう に 進ん で いっ て も 、 やはり 数 年 これ から 掛かっ て いく こと に なる ん だろう と 思い ます 。   依存 症 対策 と いう の は 、 やはり この 国会 で も 、 衆参 で も いろいろ 指摘 を し て いただき まし た よう に 、 今 厚労 省 も 最新 の 調査 を やっ て いただい て おり ます が 、 したがっ て 、 その ＩＲ の 本当 の その 実施 以前 から この 依存 症 対策 が スタート し て いく と いう こと に なる と 思い ます し 、 また そう で なけれ ば なら ない と いう ふう に 考え て いる ところ で ござい ます 。 <br/> ○ 西田 実仁 君   そう する と 、 法案 の 第 十 条 の 第 二 項 に あり ます よう に 、 この カジノ 施設 に 入場 する こと が できる 者 の 範囲 の 設定 と いう こと の 措置 と いう ふう に 定め られ て おり ます 。 今 の お 話 です と 、 外国 人 の 観光 客 と 、 内国 人 も 含め て 日本 人 の 入場 者 と 、 その 範囲 は 異なる と いう こと は 十分 に あり 得る と 、 その 範囲 の 中 に 、 例えば 当分 の 間 入場 を 禁止 する と いう よう な こと も 場合 に よっ て は あり 得る と 、 と いう か 、 全く それ が 排除 さ れ ない と いう ふう に 考え て よろしい ん でしょう か 。 <br/> ○ 衆議 院 議員 （ 岩屋 毅 君 ）   私 ども は 、 先ほど 申し上げ た よう な 適切 な 入場 管理 政策 を 取る こと に よっ て 、 内国 人 、 日本 人 も 利用 できる 施設 で ある こと が 望ましい と いう ふう に 考え て おり ます の で 、 先生 の 御 提案 は 、 当面 内国 人 は ストップ と いう か 入場 さ せ ない よう に し た 方 が いい ん で は ない か と いう 御 提案 だ と 思い ます が 、 それ は 、 先般 も お 答え 申し上げ まし た 様々 な 理由 に より まし て 適切 で は ない の で は ない か と 考え て いる ところ で ござい ます 。   そもそも 、 内国 人 を 排除 する と いう こと に なり ます と 、 本来 この 法律 が 目的 と し て おり ます 公益 還元 と いう もの が 不 十分 な もの と なっ て 、 その 目的 が 達成 でき ない おそれ が ある と いう ふう に も 考え て いる ところ で ござい ます 。 <br/> ○ 西田 実仁 君   公益 と いう の は 非常 に 幅広い 概念 でし て 、 ギャンブル 依存 症 を 助長 さ せ ない と いう の は もう 物すごく 大きな 公益 な わけ です から 。 当然 、 私 も 未来 永劫 に わたっ て 日本 人 は 絶対 入れ ない と 言っ て いる わけ じゃ ない ん です よ 。 こう いう 依存 症 対策 と いう こと が 、 これ から 更に やっ て いく に し て も 、 それ で いよいよ 営業 開始 と いう とき に それ が まだ 十分 で は ない と いう 事態 に なっ た とき に は 、 当分 の 間 、 それ が きちんと できる まで は 、 そう いう 意味 の 広い 本当 に 公益 です けれど も ね 。   お 金 が 還元 し て いく と いう 公益 も もちろん あり ます けれど も 、 お 金 が 還元 し て いく と いう 公益 も 、 今 おっしゃっ て いる ＩＲ 全体 の 中 で 、 日本 人 が 入ら ない と 、 要 は 簡単 に 言え ば もうから ない と 、 もうから なけれ ば それ を 納付 金 で も 納め られ ない と 、 納め られ なけれ ば それ が 様々 な 施策 に 使う こと も でき ない と いう 意味 で の 公益 を おっしゃっ て いる ん だ と 思い ます 。   それ は そう だ と 思う ん です けれど も 、 しかし 同時 に 、 そう いう ギャンブル 依存 症 と いう 人 を たくさん 生み出さ ない よう に する と いう の も 大変 重要 な 公益 で あり ます から 、 その 公益 と いう こと に つい て は 幅広く 見 て 、 その とき の 事態 に よっ て は そう いう 入場 制限 と いう か 、 当分 の 間 の 入場 の 禁止 と いう よう な こと も 、 検討 を する こと が なけれ ば 一番 いい ん です けれど も 、 そう いう こと も 全く 排除 す べき で は ない と いう 意見 を 申し上げ させ て いただい た わけ で ござい ます 。   もう 時間 が ない の で 、 あと 一 問 だけ 。   前回 、 西村 議員 に 御 質問 を さ せ て いただき まし た 地域 の 同意 と いう こと に つい て 改めて お 聞き し たい と 思い ます 。   この 地方 議会 の 同意 の 要件 化 と いう こと は 既に 衆議 院 の 附帯 決議 に も 入っ て ござい ます けれど も 、 しかし 、 そこ に 至る 地域 の 合意 に 向け た アクション と し て 、 例えば 市民 へ の アンケート 、 あるいは 公聴 会 と いう よう な こと を きちんと やる と いう こと を 当然 と し て 前提 に し なけれ ば なら ない し 、 当該 地域 から 、 また 周辺 の そう し た 方 に も ちゃんと 同意 を 得 て いく 、 合意 を つくっ て いく 、 そう いう プロセス が 重要 で は ない か と 思い ます けれど も 、 最後 に お 聞き し たい と 思い ます 。 <br/> ○ 衆議 院 議員 （ 西村 康 稔 君 ）   先般 も 御 指摘 を いただき まし た 。 まさに 西田 委員 おっしゃる とおり 、 地域 の 住民 の 合意 形成 と いう の は 非常 に 大事 な 点 だ と 思っ て おり まし て 、 御 指摘 の アンケート 調査 と か 公聴 会 を 開く と か 、 そう いっ た こと を 通じ て 地域 住民 の 合意 に 向け た 取組 、 これ は 大変 大事 だ と 思っ て おり ます 。   先般 も 申し上げ まし た けれど も 、 マサチューセッツ 州 で も 、 地域 住民 と 事業 者 が 協定 、 アコード の よう な もの を 結ぶ よう な 例 も ござい ます 。 いろんな 事例 も 参考 に し ながら 地域 が 住民 の 合意 形成 に 努力 を し て いる と 、 こう いっ た 点 も 認定 に 当たっ て は 重要 な 判断 材料 に なる と いう ふう に 考え て おり ます 。 <br/> ○ 西田 実仁 君   以上 です 。 終わり ます 。 <br/> ○ 清水 貴之 君   日本 維新 の 会 の 清水 と 申し ます 。 どうぞ よろしく お 願い いたし ます 。   訪日 外国 人 の 数 が 非常 に ハイ スピード で 伸び て いる と いう 話 、 岩屋 議員 から も あり まし た 。 二千 万 人 を 達成 し て 更に と いう こと で 、 その 二千 万 人 と いう 数字 、 その 数字 以上 に 、 もう ここ に いる 皆 さん が 、 もう 東京 だけ じゃ なく て 様々 な 町 、 皆 さん の 御 地元 歩い て いらっしゃっ て も 、 わあ 、 こんな に 多く の 方 が 訪れ て くださっ て いる ん だ な と いう の は もう 実感 と し て 感じ て いらっしゃる ん で は ない か な と いう ふう に 思い ます 。   その 訪日 外国 人 の 数 な ん です が 、 二千 万 人 から 四千 万 人 へ 、 六千 万 人 と 、 そう いう 目標 を 立て て いる と いう 話 が あり まし た 。 その 人数 の 目標 が ある 一方 で 、 じゃ 、 実際 に その 来 て くれ た 方々 が 日本 で どれ ぐらい の 消費 を し て くださっ た の か と 、 お 金 を 使っ て くれ て いる の か 、 こう いっ た 数字 も あっ て 、 目標 も 立て て いる と いう 話 を 聞き まし た 。   まず は 、 この 目標 に つい て 聞か せ て いただけ ます でしょう か 。 <br/> ○ 政府 参考 人 （ 瓦林 康人 君 ）   お 答え 申し上げ ます 。   本年 三 月 に 策定 いたし まし た 明日 の 日本 を 支える 観光 ビジョン 、 ここ に おき まし て は 、 従来 の 政府 目標 を 大幅 に 前倒し し 、 かつ 質 の 高い 観光 交流 を 加速 さ せる ため 、 新しい 目標 を 定め 、 広く 関係 者 と とも に 共通 の 目標 を 目指し て 取り組む こと と いたし まし た 。   具体 的 な 目標 と いたし まし て 、 まず 、 先生 が おっしゃら れ まし た 、 今 、 この 訪日 外国 人 旅行 者 数 に つき まし て は 、 二 〇 二 〇 年 、 平成 三十 二 年 に は 二 〇 一 五 年 実績 の 約 二 倍 と なる 四千 万 人 を 目指す こと と いたし まし た 。   また 、 数 だけ で は なく 質 を 高める と いう こと 、 この こと に より まし て 地方 創生 、 経済 の 活性 化 を 図る こと が 重要 で あり ます こと から 、 訪日 外国 人 旅行 消費 額 、 これ に つき まし て 、 観光 ビジョン に 盛り込ま れ た 施策 の 実施 に より まし て 一人 当たり の 単価 を 増やし て 、 二 〇 二 〇 年 に は 八 兆 円 と いう 消費 額 、 これ を 目標 と し て 設定 し た ところ で ござい ます 。   今後 、 世界 が 訪れ たく なる 日本 を 目指し まし て 、 観光 ビジョン に 盛り込ま れ た 施策 を 政府 一丸 と なっ て 着実 に 実施 し て まいり たい と 、 かよう に 考え て ござい ます 。 <br/> ○ 清水 貴之 君   二 〇 二 〇 年 で 八 兆 円 と いう こと です から 、 相当 高い 目標 で は ない か な と いう ふう に 思い ます 。   ただ 、 その 一方 で 、 中国 人 の 方々 の 爆 買い と いう ブーム が ちょっと 落ち着い て き た と いう よう な 報道 も よく 見かけ ます 。 個人 が 使う お 金 の その 買い 方 です ね 、 お 金 の 全体 額 と いう の も 減少 し て いる と いう よう な 報道 も 見 ます 。   最近 の この 数字 と いう の は 、 実際 の ところ は どう な ん でしょう か 。 <br/> ○ 政府 参考 人 （ 瓦林 康人 君 ）   昨年 の 訪日 外国 人 旅行 消費 額 、 これ は 外国 人 旅行 者 数 が 、 旅行 者 数 です ね 、 が 増加 し た こと に 伴い まし て 、 対 前年 比 七 一 ・ 五 ％ の 三 兆 四千 七百 七十 一 億 円 と なり まし た 。 また 、 本年 に 入り まし て も 、 訪日 外国 人 旅行 者 数 、 これ は 堅調 に 増加 し て おり まし て 、 これ に 伴い まし て 、 本年 一 月 から 九 月 まで の 期間 、 九 か月 間 の 外国 人 旅行 消費 額 、 これ は 前年 同期 比 一 〇 ％ 増 の 二 兆 八千 五百 五十 六 億 円 と 更に 拡大 し て ござい ます 。   一方 に おき まし て 、 本年 七 月 から 九 月 、 この 三 か月 間 のみ に 絞り ます と 、 外国 人 旅行 消費 額 は 、 同じ 時期 、 前年 同期 比 で 二 ・ 九 ％ 減少 し 、 九千 七百 十 七 億 円 と なり まし た 。 また 、 一人 当たり の 旅行 支出 、 これ も 前年 同期 比 で 一 七 ・ 一 ％ の 減少 と なっ て おり ます 。 この 主な 要因 で ござい ます が 、 何 より も 為替 レート の 円高 方向 へ の 動き 、 これ が この 前年 同期 と の 関係 で 動き が ござい まし た 。 これ に 加え まし て 、 今 先生 御 指摘 が ござい まし た 一部 旅行 者 の 高額 商品 購入 額 の 減少 と いう こと が 挙げ られ ます 。   しかし ながら 、 一人 当たり の 旅行 支出 、 これ は 現地 通貨 ベース 、 それぞれ の 発地 国 の 通貨 ベース で 見 ます と 、 主要 国 の 多く に おき まし て 、 これ は 中国 も 含め まし て 、 前年 同期 比 で 増加 し て ござい まし て 、 決して 消費 マインド が 冷え込ん で いる と いう ところ で は ない と いう ふう に 考え て ござい ます 。   観光 庁 と いたし まし て は 、 本年 三 月 に 策定 し た 先ほど の 観光 ビジョン 、 これ に 基づき まし た 施策 を 関係 省庁 と 連携 し て 実施 し 、 一層 の 消費 の 拡大 に 努め て まいり たい と 考え て おり ます 。 <br/> ○ 清水 貴之 君   今 話 あり まし た よう に 、 いろいろ な 数字 が あっ て 、 で も やはり 、 まあ 正直 な ところ 、 やっぱり 波 が ある と いう の も これ も 事実 だ と 思う ん です ね 。 今後 、 じゃ 、 ずっと 伸び 続ける か と いっ たら 、 そこ だけ に 頼る と いう の も 非常 に 不 安定 な ところ も ある と 思う の で 、 です から こう いっ た ＩＲ の よう な 、 こう いっ た 法案 を 通し て 、 そう いっ た 施設 を 造っ て 、 また 新た な 起爆 剤 を つくっ て いこう と いう 、 そう いう 意図 で は ない か な と いう ふう に 思う ん です けれど も 。   その 意味 で 、 この ＩＲ を 推進 し て いく こと に よっ て 、 様々 な 目標 が あり ます 、 これ に 向け て どの よう な 役割 を ＩＲ が 果たし て いく の か 、 そこ を お 聞か せ いただけ ます でしょう か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   お 答え 申し上げ ます 。   ＩＲ は 、 これ は 何 度 も 話 が 出 て おり ます けれど も 、 いわゆる 観光 立国 の 推進 と いう こと で 考え た とき に 大変 力強い エンジン に なる と いう ふう に 思っ て おり ます 。 今回 の **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) は 、 ＩＲ を 整備 する こと に より まし て 、 国際 競争 力 の ある 魅力 ある 観光 地 の 形成 、 地域 経済 の 振興 に 寄与 する と とも に 、 適切 かつ 厳格 な 国 の 規制 、 監督 の 下 で 運営 さ れる カジノ の 収益 に より 財政 の 改善 に 資する こと を 目的 と し て いる わけ で ござい ます 。   こう し た ＩＲ の 整備 は 、 これ も 何 度 も 言わ れ て いる こと で あり ます けれど も 、 税 負担 なき 経済 対策 、 都市 政策 と し て 、 そして 二 〇 二 〇 年 東京 オリンピック ・ パラリンピック 前後 の 切れ目 の ない 国際 観光 政策 と し て 位置 付け られる もの 、 人 を 呼び込む ため の 大変 大きな 起爆 剤 に なる と 、 そう 考え て おり ます 。 <br/> ○ 清水 貴之 君   その 今 の 話 の 中 で 、 カジノ の 収益 と いう 話 が あり まし た 。 今回 の 法案 の 中 で も 、 この カジノ が ある 部分 と いう の が 、 やはり 皆 さん 、 様々 な 依存 症 の 問題 は もちろん あり ます し 、 マネー ロンダリング の 話 も あり ます し 、 カジノ と いう ところ が やっぱり 、 ＩＲ と いえ ば ほか の 施設 も 含め て の 話 で は ある ん です が 、 やはり この カジノ が 一番 大きな ポイント に なる ところ だ と 思う ん です ね 。   そう いっ た 意味 で 、 そもそも な ん です けれど も 、 ＩＲ に カジノ は これ なく ちゃ いけ ない ん でしょう か 。 カジノ ない ＩＲ 、 複合 施設 、 ホテル や イベント 会場 で あっ たり と か アミューズメント 施設 だけ で これ 成立 し ない ん でしょう か 。 これ が うまく 進め ば 、 収益 が 上がれ ば 、 いろんな この 議論 と いう の も すんなり と 行く ん じゃ ない か な と 思う ん です が 、 ここ に つい て は いかが です か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   この 議論 も ずっと 何 度 も やっ て き た 議論 で ござい ます 。 カジノ が なく て も その 他 の 複合 施設 が でき て いく と いう こと で あれ ば 、 それ は それ で 大変 結構 な こと だ と 思い ます けれど も 、 その ため に は 法律 は 必要 ない わけ で あり ます から 、 大いに どんどん やっ て いただけれ ば いい わけ で あり ます 。   ただ 、 それ が 現実 的 に 行わ れる か どう か と いう こと に 関し て 言え ば 、 いわゆる 投資 家 の 皆 さん たち は そう いっ た こと に 投資 は し ない でしょう と いう 話 が 昨日 の 参考 人 質疑 で も ござい まし た 。 で あり ます の で 、 カジノ を 含む いわゆる 統合 リゾート と いう 話 が 、 今日 我々 が 注目 し 、 世界 で も 注目 さ れ て いる ビジネス モデル で あり まし て 、 そう いう 中 に あっ て 、 カジノ の いわゆる マイナス 面 を できる だけ 閉じ込め て 小さく し ながら 、 しかし 税 負担 なき まさに 都市 開発 を 行っ て いく 、 これ が 今回 の 法案 の 趣旨 で ござい ます 。 <br/> ○ 清水 貴之 君   じゃ 、 逆 に 言い ます と 、 カジノ が あっ て それ 以外 の 割合 を なるべく 高め て と いう よう な イメージ が ある の か も しれ ませ ん けれど も 、 じゃ 、 それ 以外 の 施設 です ね 、 カジノ 以外 の 施設 が 持つ 役割 、 やはり これ が 一体 と なっ て の ＩＲ だ と は 思う ん です けれど も 、 それ 以外 の 施設 の 役割 は どの よう に 考え て いる ん でしょう か 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   この 逆説 的 な 質問 は 今 まで の 長い これ まで の 審議 の 中 で も なかっ た か な と 思う ん です けれど も 、 これ が いわゆる 特定 複合 観光 施設 の 一部 だ と いう こと に なり ます と 、 当然 、 法案 の 中 で 、 観光 、 地域 経済 の 振興 、 財政 の 改善 を 図る ため と いう 位置 付け に なり ます 。 そこ で 、 カジノ 以外 の 施設 が ある こと で 国際 観光 振興 の 法 目的 に 大変 整合 する こと 、 カジノ が あっ て 、 そして その ほか に これ まで に なかっ た よう な 複合 施設 の 連携 と いう もの が 生まれる と いう こと で 、 我々 は この 法 目的 に 整合 的 だ と いう ふう に 考え て おり ます 。 そして 、 カジノ 以外 の 施設 の 民間 投資 規模 と いう の が それ に よっ て 大きく なっ て おり ます と 、 やはり 地域 経済 の 振興 の 効果 と いう の も 大きく 見込ん で いる と いう ところ で あり ます 。 <br/> ○ 清水 貴之 君   その カジノ 施設 以外 の 話 を お 聞き し まし た けれど も 、 実際 に 、 じゃ 、 国内 の そう いっ た 会議 場 で あっ たり と か 、 カジノ 以外 の ところ です ね 、 ＭＩ ＣＥ 施設 と いう の が 必要 だ から と いう 話 が あり ます 。 本当 に 、 じゃ 、 必要 な の か と いう ところ も ちゃんと 、 造っ たら 外国 から いっぱい お 客 さん が 来 て くれ て 、 国際 会議 が たくさん 開か れ て 、 一緒 に カジノ で も 遊ん で くれ て と いう よう な 何 と なく の イメージ で は やはり これ は 良く ない と 思う ん です ね 。 しっかり と 根拠 が あっ て 、 データ など も あっ て 、 今 、 日本 で は これ だけ 国際 会議 が 開か れ て い て 、 世界 で は これ ぐらい の 需要 が あっ て 、 もし こう いっ た 施設 が 造ら れ たら これ ぐらい の 方々 が 来 て くれ て 、 こう いっ た ちゃんと し た 、 しっかり し た 見通し が あっ て 初めて こう いっ た 法案 を 、 設備 を 進め て いく べき で は ない か な と いう ふう に 思い ます 。   今 の 日本 の 現状 と いう の は 、 その 施設 面 と か その 会議 の 現状 と か 、 足り て いる の か 足り て い ない の か 、 必要 な の か どう な の か 、 この 辺り に つい て は いかが でしょう か 。 <br/> ○ 政府 参考 人 （ 瓦林 康人 君 ）   お 答え 申し上げ ます 。   ＭＩ ＣＥ 施設 で ござい ます 。 Ｍ ＩＣＥ 施設 、 これ は 国際 会議 で あり ます と か 展示 会 と いっ た 、 いわゆる ＭＩ ＣＥ の 開催 を 主 目的 と し た 施設 で ござい まし て 、 国際 会議 場 、 展示 会場 等 ござい ます 。   我が 国 の 主要 な まず 国際 会議 場 で ござい ます が 、 これ は 収容 人数 約 五千 人 の 会議 場 が 首都 圏 に 二 か所 、 同 約 三千 人 の 会議 場 が 九州 二 か所 、 中部 に 一 か所 、 近畿 一 か所 、 計 四 か所 運営 さ れ て おり ます 。 海外 と 比べ ます と 、 シンガポール で は 収容 人数 一 万 人 を 超える 会議 場 が 二 か所 整備 さ れ て いる ほか 、 香港 で は 八千 人 規模 、 韓国 で は 七千 人 規模 の 会議 場 が 整備 さ れ て いる と いう よう に なっ て い ます 。   展示 会場 で ござい ます が 、 展示 会場 に つき まし て は 、 展示 面積 約 九 ・ 五 万 平方 メートル の 東京 ビッグ サイト 、 これ を 筆頭 に 、 二 万 平方 メートル 以上 の 施設 が 首都 圏 、 中部 、 近畿 の 計 五 か所 で 運営 さ れ 、 かつ 活用 さ れ て ござい ます 。 世界 と の 関係 で 見 ます と 、 この 面積 で ござい ます けれど も 、 全 世界 で 見 ます と 、 国別 の 総 展示 面積 で は アメリカ 、 中国 、 ドイツ が 上位 を 占め て おり まし て 、 日本 は 十 六 位 と なっ て おり ます 。 また 、 一 か所 当たり の 規模 で ござい ます が 、 やはり ドイツ 、 中国 の 施設 が 上位 と なり まし て 、 日本 最大 の 東京 ビッグ サイト 、 これ は 七十 三 位 と なっ て ござい ます 。 <br/> ○ 清水 貴之 君   その 活動 状況 と いう ん です か ね 、 実際 に どれ ぐらい 、 稼働 状況 と いう ん です か ね 、 どれ ぐらい 動い て いる 、 若しくは どれ ぐらい 不足 し て いる 、 そう いっ た データ と いう の は 。 日本 の 国内 です ね 。 <br/> ○ 政府 参考 人 （ 瓦林 康人 君 ）   手元 に 数字 と し て 稼働 率 の よう な 具体 的 な データ ござい ませ ん けれど も 、 この 国際 会議 場 に つき まし て は それぞれ 活発 に 活用 さ れ て おり ます 。 展示 会場 の 方 は 、 これ は また 投資 が 行わ れ て おり まし て 、 増設 の 動き も 出 て き て いる と いう ふう に 伺っ て おり ます 。 <br/> ○ 清水 貴之 君   済み ませ ん 、 しっかり 僕 が データ 最初 に 要求 し なかっ た の で 、 申し訳 ない ん です けれど も 。   海外 の 状況 を 見 て い て も 活発 に 動い て い て 、 これ が 日本 に もし 造ら れ たら 本当 に お 客 さん 日本 に 来 て くれる と いう 、 そう いう 見込み と いう の は しっかり 、 結構 、 やっぱり 地方 なんか でし たら その 規模 が 全然 違い ます けれど も 、 箱物 行政 って 何十 年 前 から 言わ れ て い て 、 いろんな 建物 が 建っ た けれど も 実際 活用 さ れ ず に 残念 ながら 朽ち 果て て いっ て しまっ て いる よう な 施設 も たくさん ある わけ です ね 。 そう やっぱり なっ て ほしく ない わけ です ね 。 もう 造る から に は 、 人 が あふれ て 常 に 動い て い て と いう よう な 、 そう いう 施設 に する べき だ と いう ふう に 思っ て い ます の で こう いっ た 質問 を さ せ て いただい て いる ん です けれど も 、 いかが でしょう か 。 <br/> ○ 政府 参考 人 （ 瓦林 康人 君 ）   この 国際 会議 で あれ 、 また 展示 会 で あれ 、 誘致 合戦 が アジア 太平 洋 地域 を 中心 に 大変 活発 に なっ て おり ます 。 そう いう 意味 で 、 これ は 誘致 する 上 で 規模 と か キャパシティー に 応じ て 誘致 を し て いる 現状 が ござい まし て 、 規模 が 拡大 すれ ば それ だけ 誘致 できる 国際 会議 で あり ます と か 展示 会 が 増える と 、 大きく なる と いう こと が ござい ます 。 <br/> ○ 清水 貴之 君   そう なっ た 場合 の 、 これ も これ まで 議論 出 て い ます が 、 経済 効果 です ね 、 これ も 新聞 など を 見 て い ます と 、 もう いろいろ な 団体 と か 業界 と か 若しくは 自治 体 など が それぞれ その 試算 など を 行っ て い ます 。 もし この データ が それぞれ 分かる よう なら ば お 答え いただけ たら と 、 若しくは 全体 の 効果 です ね 、 この 辺り も 分かる なら ば お 答え いただけ たら と 思い ます 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   これ も 各種 の 試算 の 話 が 何 度 か 出 て おり ます が 、 私 ども が 一番 念頭 に 置い た シンガポール の 例 を 改めて 申し上げ ます と 、 シンガポール の 例えば 海外 旅行 者 数 は 、 開業 前 の 二 〇 〇 九 年 は 九百 六十 八 万 人 、 開業 後 の 二 〇 一 三 年 は 一千 五百 四十 九 万 人 と いう こと で 、 六 〇 ％ 増 で ござい ます 。 それ から 、 ツーリズム 分野 の 売上げ に 関し まし て は 、 二 〇 〇 九 年 が 百 二十 四 億 シンガポール ・ ドル です ね 、 二 〇 一 三 年 が 二百 三十 五 億 、 八 九 ・ 五 ％ 増 と 、 こう いう 話 に なり ます 。   我が 国 に とっ て どう いう ふう な こと が 予想 さ れる か と 、 こう いう こと で あり ます が 、 これ は どこ に どの くらい の 規模 で 造る か と いう こと に よっ て 違っ て くる と 思い ます 。 朝 の 質疑 の 中 で は 、 横浜 の シミュレーション が たしか 提示 さ れ て おり まし た が 、 私 ども 議連 で 勉強 を さ せ て いただい た ところ で 一 つ 分かり やすく 申し上げる と 、 オックスフォード ・ エコノミクス と いう シンク タンク が 東京 と 大阪 の こと を シミュレーション を し て おり まし て 、 一 年間 で 合計 消費 支出 が 、 東京 で それ なり の 規模 の 想定 さ れる もの を 行っ た 場合 に は 二 ・ 二 兆 円 、 大阪 は 一 ・ 六 兆 円 、 雇用 創出 に 関し まし て は 、 東京 で 十 万 三千 人 、 それ から 大阪 で 七 万 七千 人 、 それ から 税 です ね 、 国税 、 地方 税 、 東京 で は 四千 七百 億 円 、 大阪 で は 三千 四百 億 円 と 、 こう いう シミュレーション 結果 を 出し て おり まし て 、 私 ども の 勉強 の 一 つ の データ と し て そう いう こと を 勉強 し て まいり まし た 。 <br/> ○ 清水 貴之 君   国内 の 状況 も お 話し いただき まし て 、 海外 の 例 と し まし て は シンガポール と いう 話 、 小沢 先生 から も 言わ れ まし た 、 岩屋 先生 も 常 に シンガポール を と いう こと を おっしゃっ て おり ます 。 ただ 、 先ほど も あり まし た 点 、 全 世界 で は 今 百 二十 を 超える 国 で カジノ が ある と いう 話 も あり まし た 。 ＩＲ と また 別 か も しれ ませ ん 、 カジノ だけ の ところ も ある でしょう から これ は 違う か も しれ ませ ん けど 、 そう いっ た ところ も 見 ながら 、 様々 な 海外 の 状況 と いう の を 参考 に し て 進め て いく 、 もう 先行 事例 が ある わけ です から 、 進め て いく べき で は ない か な と 思い ます 。   シンガポール 、 一 つ の 例 だ と いう の も 大変 よく 分かる ん です けれど も 、 ほか に ある か どう か 分かり ませ ん けれど も 、 海外 の 事例 を どの よう に 参考 に し て 、 どう やっ て この 法 案 を 作っ て いっ た の か 、 皆 さん の 議連 の 中 で は どう いっ た 議論 が あっ た の か 、 この 辺り 、 教え て いただけ ます でしょう か 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   今 シンガポール の 話 も 出 た ん です けれど も 、 議連 の 中 で は 、 例えば 、 ある カジノ が 、 先進 的 な 医療 機関 が 併設 さ れ て い て 、 これ を 医療 ツーリズム と 合致 さ せ た よう な 例 と か 、 ユニーク な もの は 世界 に 多く ある と いう こと も 我々 様々 勉強 さ せ て いただい た ん です が 、 やはり これ 、 成功 例 も あれ ば 失敗 例 も ある わけ で あり まし て 、 よく 関係 者 の 皆 さん と 意見 交換 する と 、 特に 韓国 なんか の 例 で 、 シンガポール と か 、 まだ でき て い ない けど 、 日本 の よう に やはり ゼロ から きっちり と この 施設 を 設計 できる と いう こと は 羨ましい な と いう よう な 声 と いう の は 、 やはり 関係 者 の 中 から も 多く いただい た と いう よう な 経緯 が あり ます 。   そう いう 意味 で 、 最近 でき て 、 そして 様々 な 規制 の 在り 方 等 も 踏まえれ ば 、 そして また 、 さらに ＧＤＰ の 伸び率 等 も 見れ ば 、 やはり シンガポール と いう もの が 我々 と し て は 非常 に 参考 に なる と いう の が ここ で 一致 し て いる の は 、 我々 も それ に 近い もの を やはり 描い て いる と いう こと の 証左 で あり ます 。 <br/> ○ 清水 貴之 君   今 お 話 伺っ て 、 ああ 、 面白い な と 思っ た の は 、 ゼロ から でき て 、 そこ が 非常 に ほか から し たら いい 話 だ な と いう こと な ん です けれど も 。 と いう こと は 、 ほか の ところ で うまく いっ て い ない 例 など を 見 ます と 、 そう いっ た ゼロ から その 施設 を 幅広く 設計 し て 造っ た の で は なく て 、 スペース が 空い て いる から そこ に カジノ を 造っ て み ましょう 、 ここ に 会議 場 を 造っ て み ましょう 、 ちょっと そう いっ た 施設 が ばらばらばら ばらし て い たり 、 そう いっ た 話 な ん でしょう か 。 この 辺り 、 やっぱり 失敗 例 と いう の も しっかり 見 て いく べき か と いう ふう に 思い ます 。 そう いっ た ところ も 反面 教師 に し ながら 、 日本 に 合っ た 、 成功 する ＩＲ 施設 と いう の を 模索 し て いく べき で は ない か と 思う ん です けれど も 、 いかが でしょう か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   失敗 例 と いう こと で 他国 の 話 を 申し上げる の は なかなか 、 適切 で は ない と いう か 難しい ところ が ある わけ で あり ます が 、 昨日 、 参考 人 質疑 の 中 で も この 話 が 出 まし て 、 美原 参考 人 から お 話 を いただき まし た の で 、 その こと を もう 一 回 リファー し て 申し上げ たい と 思い ます が 、 例えば 韓国 の 江原 ランド は 、 当初 から 、 江原 ランド を 造る 前 から 外国 人 向け の カジノ を ずっと 韓国 は やっ て き て い て 、 で あり ます から 、 いわゆる 内国 者 の 入場 制限 と か 依存 症 対策 と か 、 そう いっ た こと を 考え ない で も 済ん で ずっと 来 て い た 、 そこ に 江原 ランド を 造っ た ため に 、 適切 な 入場 管理 政策 が 取ら れ て い なかっ た と か 、 例えば 入場 料 は 五百 円 から 六百 円 程度 と いう よう な 低い 額 だっ たり と 、 あるいは また 監視 、 監督 する そう いっ た 機関 が きちっと なかっ た と 、 こう いう 話 が 昨日 は 御 紹介 を さ れ まし た 。   それ に 対し て 我が 国 は 、 今 こう やっ て 議論 を し て おり ます よう に 、 しっかり と し た 管理 政策 を 取っ て 行っ て いく と いう こと に 関し て は 、 美原 先生 が お 出 に なっ た シンポジウム で も 日本 は 羨ましい な と 、 そう いう お 話 が 出 た と 。 後発 で ある こと の マイナス も ある の か も しれ ませ ん が 、 後発 で ある が ゆえ の 、 そう いっ た 各国 の 政策 を しっかり と 取り込ん で 、 できる だけ いい もの を つくっ て いく と いう こと が できる の で は ない か と 思っ て おり ます 。 <br/> ○ 清水 貴之 君   今 お 話し いただい た とおり 、 是非 、 シンガポール と いう と やはり もう 成功 例 で 、 それ は それ で 参考 に する の は 有 意義 な こと だ と 思う ん です けれど も 、 やはり 、 うまく いっ て い ない ところ から 学ぶ べき こと も 、 人生 も 何 で も そう だ と 思い ます けれど も 、 そう いっ た ところ から 学ぶ べき こと も 多々 ある と 思い ます の で 、 幅広い 視点 で 見 て この 議論 は 是非 進め て いき たい な と いう ふう に 思っ て い ます 。   次 の 質問 は 、 外資 の 参入 に つい て 、 これ も これ まで に も 質問 と し て 出 た ん です が 、 気 に なる 部分 で ある の で お 聞き し たい と 思い ます 。   やはり 、 ある 程度 の 規模 を 持っ た 、 経験 を 持っ た 企業 の 参入 が 想定 さ れ て いる と いう こと です の で 、 経験 の ある 外資 が 入っ て くる 可能 性 と いう の は 十分 考え られる こと だ と 思い ます 。 そう なっ た 場合 に 、 やはり 皆 さん 、 日本 の お 金 が 、 もう けが 、 収益 が 海外 に 流れ て いっ て しまう ん じゃ ない か と 、 結局 日本 は 損 だけ し て 外国 が もうかっ て 、 それ だけ に なっ て しまう ん じゃ ない か と 、 こう いっ た 問題 意識 を 持っ て いる 方 も きっと 非常 に 多い ん じゃ ない か な と いう ふう に 思い ます 。 私 も そう 思い ます の で 。   これ に つい て の お 答え と いう の は 、 どの よう に お 答え に なる でしょう か 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   これ 再三 議論 に なっ て いる ところ で あり ます けれど も 、 外国 企業 が カジノ 運営 者 に なる か どう か に つい て は 実施 法 で 決め て いく と いう こと に なり ます 。 まず この 上 で 、 外国 企業 が この 運営 者 に なっ た 場合 に も 当然 我が 国 の 法令 に 従っ て いただく 、 さらに は 、 租税 に 関する 法令 に 基づい て 法人 税 、 消費 税 、 固定 資産 税 の 税金 を 負担 し た 上 で 、 実施 法 で 納付 金 を 納める こと と し て おり ます 。 当然 、 納付 金 、 高 過ぎ て もうから ない な と いう と こんな 参入 も ない でしょう し 、 低 過ぎ て は 意味 が ない と いう こと です の で 、 そこ は 適切 に 、 まず は 我々 二、三 か所 で やっ て いく わけ です から 、 そこ から で も いろいろ と 学ぶ こと あろう か と 思い ます 。   それ に いたし まし て も 、 衆議 院 で いただき まし た 附帯 決議 に おき まし て は 、 カジノ 運営 主体 が 民間 事業 者 に なる こと に まず 鑑み て 、 税制 ・ 会計 規則 に つい て は 諸 外国 の 制度 を 十分 に 勘案 の 上 、 検討 と いう こと に なっ て おり ます の で 、 最も 厳格 な 仕組み を つくる わけ で あり ます から 、 そこ は 明確 な もの が ある と 思い ます 。 <br/> ○ 清水 貴之 君   もう 一 つ 懸念 材料 と し て 、 やはり 日本 は バブル 経済 、 バブル 景気 と いう の を 良い も 悪い も 経験 を し て き まし た 。 あの とき に も たくさん の 建物 が 建っ て 、 **リゾート** **法** ( `総合保養地域整備法` ) と いう 法律 も でき て 、 しかし 最終 的 に は 経営 が うまく いか なく なっ て 倒れ て しまっ た 、 たくさん の 赤字 を 生ん で しまっ た 、 税金 の 投入 も あっ た 、 もう 様々 な 負 の 遺産 も あり ます 。   今回 、 こう いっ た 施設 を 造る こと が 、 その よう に 建設 ラッシュ に なる と いう か 、 建設 現場 が 活況 を 呈する など メリット も ある と 思う ん です が 、 一方 、 造っ た は いい けれど も 施設 が それ ほど はやら なく て 、 にぎわう こと なく て 、 建物 が 残っ て しまう 、 使わ れ ず に どんどん どんどん 廃れ て いっ て しまう 、 地元 経済 も 疲弊 し て いく 、 こう いっ た こと が 起きる 懸念 と いう の は 考え られ ない もの でしょう か 。 いかが でしょう か 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   当時 の こと を 思い返し ます と 、 当時 は **リゾート** **法** ( `総合保養地域整備法` ) が 施行 さ れ まし て 、 四十 二 も 基本 構想 を 作っ て しまっ た と 。 我が 党 は 元々 道州 制 なん と いう の を 進め て いる わけ です けれど も 、 都 道 府 県 に 一 つ なん と いう イメージ は 、 元々 議連 の 中 で も こう し た イメージ は なく て 、 もう ちょっと 大きな 、 都 道 府 県 を 越え た 地域 に ある べき だろう と 。 そこ で 、 数 は ある 程度 の イメージ で 、 まあ 、 そんな に 幾 つ も 造る 、 数十 個 と いう イメージ で は なく て 、 遠く に 、 最初 は 二、三 か所 で 実験 的 に 検証 し て と 、 先ほど から の 皆 さん の 議論 に も あっ た よう な 形 を 持っ て き て 、 我が 国 の 中 で まず 過当 な 競争 が 起き ない よう に 慎重 に これ を 入れ て いく と いう こと に 我々 は なっ て いる わけ で あり ます 。   その 上 で 、 地方 公共 団体 が できる だけ 魅力 的 な 構想 を 作っ て いく と いう こと を 優先 する 。 まさに これ が この 法 案 の 地方 創生 に も 資する と いう 意義 が ある と いう ふう に 考え て おり まし て 、 まさに 効果 の 高い 、 優れ た 構想 と いう の を 厳選 できる 仕組み を 入れ て いる と いう ところ で あり ます 。 <br/> ○ 清水 貴之 君   今 、 最後 に お 話 あり まし た 地方 創生 の 観点 と その 地方 公共 団体 の 関わり 、 ここ は 私 も 大変 重要 な ところ か な と いう ふう に 思っ て い ます 。 地域 と の 親和 性 と いう ん です か 、 こう いっ た もの も 大事 か な と いう ふう に 思っ て い ます 。   地方 公共 団体 が 様々 な アイデア を 出す こと に なっ て いく ん でしょう けれど も 、 地域 に やっぱり うまく なじん だ 施設 じゃ ない と 住ん で いる 皆 さん に とっ て も 良く ない と 思う ん です ね 。 カジノ と いい ます と ラスベガス の よう な にぎやか な ところ を 思い出し ます けれど も 、 あの にぎやか さ が 、 今 、 例えば 、 私 関西 です の で 、 大阪 の 南港 の 辺り に あれ が ぽん と でき た 場合 に 、 そこ だけ ちょっと どう か な と 、 浮い て しまう ん じゃ ない か な と 。 周り は 会議 室 や ホテル や と 考え た 場合 に 、 うまく やっぱり 地域 に 溶け込ん だ よう な 、 そう いっ た もの を 造っ て いか ない と 、 うまく 地元 の 方 から 愛さ れる よう な 施設 に は なら ない ん じゃ ない か な と いう ふう に 思う ん です ね 。   この 辺り 、 地方 創生 の 観点 と か 地域 と の 親和 性 に つい て は 、 もちろん 地方 公共 団体 が 率先 し て 考え て いく もの な の か も しれ ませ ん けれど も 、 皆 さん の 中 で その イメージ と か 考え が あれ ば お 聞か せ いただけ たら と 思い ます 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   それ に つい て は 、 衆議 院 で も 大阪 の 浦野 議員 なんか が 、 地方 議員 の 経験 から し て も 、 地元 の 商店 街 と か そう し た ところ と まず マーケット は 違う だろう と いう こと を 指摘 さ れ た 上 で 、 質問 の 中 で 、 その 代わり に 人 は 多く 来る わけ です から 、 地元 も せっかく 多く 来 た お 客 さん の 受皿 に なる よう な 工夫 が できる ん じゃ ない か 、 相乗 効果 が ある ん じゃ ない か と いう よう な 考え に 立っ て 、 やはり みんな が 、 民間 が 努力 し て いく よう な イメージ を 持っ て い ます と いう こと を おっしゃっ て い た の で 、 我々 も それ に 全く 同じ 考え で あり ます 。 <br/> ○ 清水 貴之 君   今 おっしゃら れ た とおり 、 確か に 本当 に 、 地方 の ショッピング モール の 例 なんか よく 皆 さん 感じ られ て いる と 思い ます けど 、 大きな ショッピング モール が 今 地方 へ 行け ば 大体 どこ に で も あっ て 、 で も やっぱり 、 それ は それ で 便利 な ん でしょう けど 、 周り の 商店 街 が どんどん どんどん 廃れ て いく と いう こと に なる わけ です ね 。   カジノ も そう なら ない で い て ほしい な と 、 そう いっ た 施設 に する べき だ な と 。 建設 業 界 が 一時 的 に 盛り上がる か も しれ ない けれど も 、 で も 、 大手 だけ が そこ の 仕事 を 受注 し て 、 周り の 本当 に 小さな 中 小 企業 の 方々 に は 全然 恩恵 が 行か ない 、 地元 に 対し て の メリット が ない と 、 こう いう こと が ない よう な 設計 で あっ たり と か 仕組み を 是非 つくっ て ほしい な と いう ふう に 思い ます 。   もう 一 つ 、 私 が 懸念 し て いる と いう か 、 こう なら ない で い て ほしい な と 思う の が 、 地域 の 格差 が 生まれる 懸念 も ある の で は ない か な と いう ふう に 考え ます 。   また 大阪 の 例 で 恐縮 な ん です が 、 大阪 は 非常 に 熱心 に これ を 推進 し て いこう 、 是非 大阪 で 造り たい と いう 意見 を 持っ て いる と いう ふう に 認識 し て い ます 。 一方 、 西村 先生 も 私 も そう な ん です が 、 地元 の 兵庫 県 は 、 知事 も 神戸 市 長 も それ ほど 積極 的 で は ない と いう こと です ね 。   大阪 に その 施設 が でき た 場合 に 、 大阪 は 大いに にぎわっ た けれど も 、 じゃ 、 兵庫 県 は どう な ん だ 、 南 の 奈良 県 や 和歌山 県 は どう な ん だ と 。 その 地域 経済 に 与える 、 大阪 だけ が 、 これ こそ 我々 東京 一 極 集中 を 是正 しよう と 言っ て いる わけ な ん です が 、 関西 で いう と 今度 逆 に 大阪 一 極 集中 に なっ て しまう ん じゃ ない か と 、 こう いっ た 懸念 も これ 考え られる わけ です ね 。 これ に つい て は いかが です か 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   まず 最初 に 我々 考え て おか ない と いけ ない の は 、 我が 国 は 今 人口 減少 し て 、 地方 も 大変 な スピード で 人口 減少 が 進ん で い て 、 地方 公共 団体 の 消滅 なん と いう 本 が ベスト セラー に なる よう な 現状 に あっ て 、 これ が 選択 と 集中 の 問題 に 帰結 を し ない よう に 対策 を 打つ と いう の は 、 私 は まず 一 つ 別 の 話 か な と 思い ます 。   その 中 で 、 我々 は 、 この ＩＲ で は 、 地方 公共 団体 も 広域 的 な 連携 を 図る こと と いう こと を 非常 に 重視 を し て いる わけ で あり ます 。 この 大阪 と 兵庫 の 問題 に 限ら ず 、 関西 で あれ ば 、 我々 は 両方 関西 人 です の で 、 関西 で あれ ば 、 大阪 に でき た 場合 に は 、 そこ に 魅力 的 な 施設 が ある 、 そして 魅力 的 な ホテル が ある 、 十分 な キャパシティー が ある 。 そこ に 来 た 皆 さん が 、 ここ を ハブ と し て 、 魅力 的 な 、 神戸 に 行こう 、 奈良 に 行こう 、 京都 に 行こう 、 和歌山 に 行こう と いう ふう に 、 周り に そもそも 、 ここ に 新た な 、 この 間 から ゴールデン ルート の 話 が でき て い ます けど 、 我々 関西 だっ たら 例えば そう いう プラチナ ルート を つくっ て しまう と いう よう な 形 で 、 様々 な 相乗 効果 と いう の が ない と 、 結局 この ＩＲ を 導入 する 我々 の 意義 が ない わけ で あり まし て 、 その とき に は 、 ＩＲ で 大阪 に 来 た 宿泊 客 を 次 は どこ へ 流し て いこう と 。   一 遍 ＩＲ に 来 た けれど も 、 実 は ほか の ところ に 、 魅力 的 だ から 、 相乗 効果 です ね 、 もう 一 回 、 今回 は ＩＲ に 来 て から 次 の 神戸 に 行こう 、 京都 に 行こう と いう よう な 関係 を つくり上げる と いう の が やはり ＩＲ の 大きな 意味 で あろう か と いう ふう に 考え て おり ます 。 <br/> ○ 清水 貴之 君   大阪 の 舞洲 、 夢 洲 に でき た こと を 考える と 、 松浪 先生 の 御 地元 の 高槻 なんか です と 、 ちょっと 微妙 な 距離 か な と 。 近 過ぎ ず 遠 過ぎ ず みたい な 、 その 辺り に も しっかり と いい 影響 は 広がる と いう ふう な 考え で いらっしゃい ます か 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   かなり 局地 的 な 質問 を いただい た ん です が 、 既に 大阪 で も 、 我々 の 地域 で も ホテル は もう いっぱい で あり まし て 、 よく スピルオーバー と いう ん です けれど も 、 決して 私 の 選挙 区 、 京都 と 大阪 の 間 で そこそこ 便利 な ん です けれど も 、 中途 半端 に 遠い ん です けれど も 、 その 地域 の ホテル が 既に 今 飽和 を し て いる と いう 状況 で あり ます の で 、 こう し た 拠点 を 整備 する と いう の は 、 私 たち の 地元 感覚 から いっ て も これ は 大変 意義 の ある こと で あろう と 感じ て おり ます 。 <br/> ○ 清水 貴之 君   この ＩＲ の 設備 の 範囲 に つい て も お 聞き し たい な と いう ふう に 思い ます 。   今 、 関西 の お 話 を 出さ せ て いただき まし た の で 、 引き続き 関西 で 聞か せ て いただけれ ば と 思う ん です けど 、 大阪 の 夢 洲 、 舞洲 、 南港 の エリア と いう の が 今 、 一 つ 大阪 で は 候補 に 挙がっ て い ます 。 私 の 兵庫 県 で いい ます と 、 神戸 の 南 の 方 です ね 、 ポートアイランド と いう 大きな 埋立 地 が あり まし て 、 神戸 空港 が ある ところ です 。 はっきり 言っ て 、 もう かなり の 空き地 が 売れ ず に 残っ て 、 空い て おり ます 。 こう いっ た ところ に 持っ て くる こと は 、 エリア と し て 可能 な ん でしょう か 。 大阪 から 見 ます と 、 もう 車 で 三、四十 分 やはり 掛かり ます の で 、 多少 距離 が あり ます 。 こう やっ て 、 広い 範囲 で ＩＲ 全体 の 施設 、 カジノ は 大阪 、 ホテル は 兵庫 、 神戸 みたい な こと が 可能 な ん でしょう か 。   これ 、 東京 で いっ て も 、 お 台場 に 造っ て 、 千葉 に 、 幕張 メッセ の ところ に ホテル と か 、 横浜 と 川崎 で と か 、 こう いっ た 連携 も 様々 な ところ で 考え られる の か な と 思う ん です が 、 これ は どの よう に 想定 を し て いる の でしょう か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   お 答え を し ます 。   今 、 基本 法 の 中 で は 、 そう いっ た 地理 的 な 範囲 を 規定 する よう な 話 は 一切 ない わけ です ね 。 実施 法 の 中 で 今後 、 申請 に 当たっ て の 基準 を 出し て くる と 思い ます が 、 しかし 、 それ に よっ て も 、 この ＩＲ の 提案 は 、 御 案内 の とおり 、 地方 自治 体 が 事業 者 と とも に 計画 を 作っ て 申請 を する と 、 こう いう 話 に なっ て い て 、 その 地方 自治 体 は 、 都 道 府 県 単位 で も いい し 、 それ が 二 県 に またがっ て も いい し 、 あるいは 市 町 村 だけ で やっ て も いい し と いう の が 基本 的 な 私 ども の 発想 で あり ます から 、 あくまで も それ は 地方 の 皆 さん 方 の 独自 、 自主 性 を 尊重 し て いく と いう の が 基本 で あり ます 。   で あり ます から 、 例えば 私 ども の ところ に いろいろ 御 相談 に 来る 中 で は 、 北海道 の よう な 大変 広い 地域 の ところ で は 大変 幅広い 地域 選択 と いう 話 も あり 得る か も しれ ませ ん し 、 今 、 清水 委員 の おっしゃっ た よう な いわゆる 関西 地区 の そう いう 話 も ある か も しれ ませ ん し 、 繰り返し に なっ て 恐縮 で あり ます が 、 あくまで も 地方 自治 体 と 事業 者 に よっ て 計画 が 立て られ て 申請 が 行わ れる と いう こと に なり ます 。 <br/> ○ 清水 貴之 君   そう なる と 、 やはり 地方 公共 団体 、 自治 体 の 役割 と いう の が 非常 に 大きく なっ て いく ん だろう な と いう ふう に 思い ます 。 先ほど の 兵庫 が 、 知事 も 神戸 市 長 も 余り 乗り気 、 積極 的 で は ない と いう お 話 を さ せ て いただい て 、 西村 先生 の 御 地元 の 淡路 島 など は 非常 に 関西 空港 から も 近い 、 神戸 空港 から も 近い 、 四国 から も 関西 から も 行ける 、 そう いっ た の に いい 場所 じゃ ない か と いう の を お 聞き しよう と 思っ た の です が 、 席 を 外さ れ た よう な の で 、 一緒 に 頑張っ て いき ましょう と いう の は また 個別 に これ は お 願い を 、 神戸 の ポートアイランド も そう な ん です けど 、 様々 、 あっ 、 また 、 で も お 願い を いたし ます 。   そう なる と 、 地方 公共 団体 の 役割 が 非常 に 大きく なる と 思う ん です が 、 地方 公共 団体 に よっ て やっぱり その 規模 が 違い ます よ ね 。 都 道 府 県 と か 大阪 市 、 神戸 市 など 政令 都市 でし たら ある 程度 の 規模 が あっ て 、 そう いっ た 職員 さん も い て 積極 的 に できる か も しれ ませ ん が 、 ちっちゃい 、 申し訳 ない けれど も それ ほど 大きく ない 自治 体 が やり たい な と 思っ た とき に 、 で も 大きな ある 程度 の 規模 の 企業 を 呼ん で こ なけれ ば いけ ない と なっ て くる と 、 これ また いろいろ 大変 な 作業 に なっ て くる と 思う ん です ね 。   そう なる と 、 地方 公共 団体 の 役割 と いう の が 本当 に 難しい な と いう ふう に 思う ん です が 、 どう いっ た 役割 を 地方 公共 団体 が 果たし て いく こと を 想定 し て いる ん でしょう か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   委員 御 指摘 の とおり 、 地方 公共 団体 の 役割 は 極めて 大きい 、 重要 だ と いう ふう に 思っ て い ます 。   地方 公共 団体 は 、 当該 地域 に おい て ＩＲ を 設置 しよう と する 場合 に は 、 国 の 方針 に 沿う よう に 、 地域 の インフラ の 整備 状況 、 周辺 環境 の 現況 等 を 総合 的 に 勘案 する と とも に 、 様々 な 民間 事業 者 の 企画 提案 を 検討 し た 上 で 最も 効果 の 高い ＩＲ 施設 整備 計画 を 作成 し 、 国 に 対し て ＩＲ 区域 の 認定 を 申請 する こと に なり ます 。 なお 、 ＩＲ 設置 後 も 、 地方 公共 団体 は 、 ＩＲ 区域 及び その 周辺 環境 の 健全 化 、 安全 化 など 、 カジノ が 社会 に 与える 影響 や リスク を 最小 限 に 抑制 する よう に 取り組ん で いく こと が 望ま れ ます 。   ＩＲ 設置 後 は 、 ＩＲ 事業 者 の 監督 は カジノ 管理 委員 会 が 行う こと に なり ます けれど も 、 地方 公共 団体 の 関与 の 在り 方 に つい て は 実施 法 の 中 で 定め られる こと に なり ます 。 極めて 需要 な 役割 を 果たす こと に なる と 思っ て おり ます 。 <br/> ○ 清水 貴之 君   この 後 は 、 衆議 院 の 方 で は 十 五 項目 の 附帯 決議 が 付い て い ます 、 この 中 を 中心 に 、 衆議 院 で の 議論 を 踏まえ た 上 で 、 こう いっ た こと を やっ て いく べき です と いう 話 だ と 思い ます の で 、 現 時点 で もし 決まっ て いる こと 、 もし 進めよう と 思っ て いる こと 、 イメージ さ れ て いる こと が あれ ば 是非 お 答え いただき たい です し 、 やはり 細かい こと は 実施 法 だ と いう こと なら ば そう いっ た お 答え に なる ん だ と は 思う ん です けれど も 、 お 聞き を し て いき たい と いう ふう に 思い ます 。   附帯 決議 の 中 に 、 我が 国 の 伝統 ・ 文化 ・ 芸術 を 生かし た 日本 らしい 観点 、 こう いっ た もの を 取り込ん で いく べき だ と いう よう な 話 が あり ます 。 これ も 非常 に 大事 な 視点 か な と 思う ん です が 、 一方 で バランス も 難しい な と 思っ て おり まし て 、 先ほど の 、 きんきん ぎらぎら し た 建物 も ちょっと 違和 感 が ある か も しれ ませ ん が 、 じゃ 、 急 に 瓦屋根 の 日本 家屋 みたい な もの で カジノ と 言わ れ て も 、 これ は これ で どう か な と いう ふう に も 思い ます 。   こう いっ た バランス と いう の は 非常 に 難しい な と 、 これ が 日本 の 伝統 です と 言わ れ て も どう な の か な と 思い ます の で 、 この 読み 方 と いう の は 非常 に いろんな 、 幅広く 読める の で 、 どう いっ た もの を 想定 し て いらっしゃる の か 、 お 聞か せ いただけ ます でしょう か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   議連 で いろいろ 議論 を し て いる 場合 に 、 私 なんか は やはり 発想 が 乏しい の で 、 こう いう もの 、 ああ いう もの と 具体 的 な 話 を 余り し て も しよう が ない ね と 、 もっと 豊か な 民間 の 皆 さん の 発想 で すばらしい もの を つくっ て もらい たい ん だ と 、 こう いう 話 を ずっと 一貫 し て 言っ て まいり まし た が 、 そう いう 中 に あっ て 、 今 委員 が 御 指摘 の よう に 、 附帯 決議 の 中 で は 、 日本 らしい 視点 と 、 こう いう 話 を 強調 し て いただい て おり ます 。   日本 で 造ら れる ＩＲ は 、 日本 で なく て は でき ない もの で なけれ ば なら ず 、 日本 独自 の 歴史 、 伝統 文化 や 地域 の 特色 を 反映 さ せ 、 訪れる 外国 人 観光 客 に 日本 の 魅力 を 効果 的 に 伝える こと が できる 施設 、 すなわち クール ジャパン の よう な 発信 基地 と なる こと が 必要 と 考え られる と 、 こう いう 一般 論 を まず 申し上げ た 上 で 、 具体 的 に は 、 じゃ 、 何 が ある か と いう こと で いえ ば 、 例示 と し て 考え て いただけれ ば と 思い ます が 、 シアター に おい て は 歌舞伎 の 上演 、 レストラン ゾーン に おい て は 和食 の 提供 、 ショッピング ゾーン に おい て は 伝統 工芸 品 を 展示 、 施設 に おけ る 最 先端 の ロボット や 自動 運転 車 の 配備 、 ホテル や ビラ に は 日本 庭園 が あり 、 わび寂 を 体験 、 それ から アニメ の 拠点 と いう よう な こと が 考え られる の で は ない でしょう か 。 <br/> ○ 清水 貴之 君   分かり まし た 。   そう いっ た 意味 で 、 文化 や 芸術 へ の 振興 に 、 ある 意味 、 そう いっ た ところ に 、 誘致 を する 、 招致 を する と いう こと で 貢献 も できる の か な と いう ふう に 思い ます が 、 一方 で 、 大変 大きな 収益 が 生まれ たり 、 税金 が 発生 する と いい ます か 、 税金 が 納め られる 施設 で も ある と 思い ます 。 こう いっ た ところ から 文化 芸術 へ の 振興 と いう の も 考え られ ない か な と いう ふう に 思う ん です ね 。 この 辺り に つい て は いかが でしょう か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   これ も 当初 から そう いっ た 議論 を ずっと 一貫 し て 続け て き て おり まし て 、 御 案内 の とおり 、 本 法案 は 納付 金 を 徴収 する こと が できる と いう こと で あり まし て 、 納付 金 の 具体 的 な 使途 に つい て は 附帯 決議 等 の 趣旨 を 踏まえ て 実施 法 の 立案 過程 に おい て 適切 に 検討 さ れる こと に なる と 、 こう 思っ て おり ます 。   今回 の **推進** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) の 目的 に は 、 ＩＲ 整備 の 推進 に 伴う 観光 や 地域 経済 の 振興 が 掲げ られ て いる ほか 、 広く 財政 の 改善 に 資する もの と さ れ て おり まし て 、 納付 金 の 使途 と し て は 、 国民 生活 の 安定 向上 に つながる 社会 福祉 、 文化 芸術 の 振興 等 、 広く 公益 に 還元 さ れる こと を 想定 し て おり ます 。 したがっ て 、 日本 文化 芸術 へ の 振興 に も 十分 寄与 し て いく こと が できる と 考え て おり ます 。 <br/> ○ 清水 貴之 君   続い て 、 次 に お 聞き しよう と し て い た 、 通告 さ せ て いただい た の は 規模 の 話 と 今後 の 法 整備 な ん です が 、 これ は これ まで に も 質問 に 出 まし た の で 、 ここ は 省か せ て いただき たい と 思い ます 。   次 の ＩＲ の 区域 の 数 の 制限 の 話 です ね 。 やはり 最初 は 二 つ 、 三 つ から スタート し て と いう 話 が あり まし た 。 確か に 、 いきなり 十 個 、 二十 個 って でき た 場合 に 共倒れ に なっ て しまう 可能 性 も ある でしょう し 、 そこ まで の 運営 が できる の か な と いう 懸念 も ある ん です が 、 理解 は できる ところ な ん です が 、 その 必要 性 と 、 一方 、 増やし て いく こと も 考え て いる の か 、 どれ ぐらい の 数 まで 上限 を 設定 する の か 、 どれ ぐらい 増やし て いく の か 、 この 辺り に つい て の 数 に つい て の 考え を お 聞か せ いただけ ます でしょう か 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   この 数 に つい て は 、 もう 本当 に 民主 党 政権 下 で 本格 化 し た この ＩＲ 議連 に おい て 長年 の 議論 が あっ て 、 この 二、三 か所 と いう ところ に いろんな 様々 な 議論 を 経 て 落ち着い た ところ で あり まし て 、 当然 二、三 か所 です から 競合 し ない 、 そして 効果 、 課題 を その 中 で 十分 に 評価 、 検証 する と 、 そして 着実 な 施行 を 確認 し た 上 で 段階 的 に 数 を 増やそう と いう こと に なっ て おり ます 。   そして 、 御 指摘 の よう に 、 衆議 院 の 附帯 決議 の 中 で も 、 厳格 に 少数 に 限る と いう こと と 区域 の 認定 数 の 上限 に つい て も 項目 が 書か れ て おる わけ で あり まし て 、 これ を 踏まえ て 、 政府 に おい て どの よう な 法令 で 定める か は 別 と いたし まし て 、 これ に つい て は 上限 の イメージ を しっかり つくっ て いく と いう こと に なろう か と 思い ます 。 です から 、 さっき 申し上げ た よう に 、 何十 に も なる よう な イメージ を 我々 は はな から 持っ て い ない と いう こと で あり ます 。 <br/> ○ 清水 貴之 君   その 最初 の 二 つ 、 三 つ と いい ます と 、 今 報道 で 出 て いる 話 でし たら 、 もっと 多く の ところ が 手 を 挙げ たい と いう よう な こと を 言っ て いる と いう ふう に 聞い て い ます 。 と なる と 、 非常 に 激しい 競争 が そこ で 起き て くる わけ です ね 。   しっかり と ここ は やはり フェア に 選別 を し て いか なけれ ば いけ ない と いう ふう に 思っ て い ます 。 その 地域 に いる 議員 の 力 が 強い から 選ば れ た ん だ と か 、 これ まで に も あり まし た と おり 、 そこ の 企業 と 深い 関係 が ある から 選ば れ た ん だ 、 こう いう こと が 絶対 に ない よう に 、 しっかり と し た 選定 方法 を 確立 し て 、 それ に 沿っ て 進め て いっ て いただき たい と 思い ます が 、 いかが でしょう か 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   おっしゃる とおり 、 これ は あくまで 手 挙げ 方式 で あり まし て 、 地方 公共 団体 が 国 に 対し て 申請 を する 、 その 前 に 魅力 的 な 計画 を 作っ て 、 事業 者 の 選定 等 も 行っ て いく と 。   この 事業 者 の 選定 は 非常 に 厳格 に やる と いう こと で 、 度々 ライセンス の 話 が 上っ て おり まし た 。 まず 国 が 事業 選定 者 に 対し て ライセンス を 下ろし て から 地方 公共 団体 と 話 を する の か 、 それ と も 、 地方 公共 団体 が 一緒 に なっ て 作っ た 、 計画 を やっ た 事業 者 に その 後 に ライセンス を 付与 する の か と 。 これ は 手間 の 問題 で 、 どちら に なる か と いう の は 、 さらに は 我々 は 実施 法 の 中 で これ を どう いう パターン に する の か と いう の は 詰め て いただき たい と 。 ここ まで は 論点 と し て 今 課題 に 、 この 審議 の 中 で 明らか に なっ た こと か な と 認識 を し て おり ます 。   その 上 で 、 具体 的 な 基準 は 、 国際 競争 力 の 、 先ほど から の 目的 に 、 法 目的 に 沿っ た 形 で 、 国内外 の 観光 客 の 増加 や 多様 な 魅力 ある サービス の 提供 に よる 雇用 の 拡大 など 大きな 経済 効果 が 認め られる こと 、 そして 当該 地域 の 地域 創生 、 町づくり に 資する こと など を 基本 と し て 、 基準 と し て 考え て いこう と いう こと で あり ます 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   ちょっと 速記 止め て ください 。       〔 速記 中止 〕 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   それ で は 、 速記 を 起こし て ください 。 <br/> ○ 清水 貴之 君   続い て お 聞き し たい の が 、 地方 議会 の 関与 、 これ も 重視 し て いく と いう 話 が これ まで に も あり まし た 。 それ に 至っ た 議論 が どの よう な 議論 が あっ た の か 、 そう いっ た ところ が あり まし たら 、 で 、 どう いっ た 考え で 地方 議会 と の 関わり を 深め て いく の か 、 この 辺り に つい て お 聞か せ ください 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   先ほど から 委員 も 御 指摘 に なら れ まし た よう に 、 ＩＲ の 区域 を ここ で だっ と つくっ て 集中 的 に 施設 を 整備 する と 、 地域 から 物 を 吸い込ん だり し て 周り が 疲弊 する ん じゃ ない か と か 、 そう いっ た 声 も あろう か と 思い ます 。 まさに 地元 と の コンセンサス と いう の が 大事 に なる 中 で 、 まさに その 主役 は 我々 は 地方 議会 で 、 地方 議員 の 皆 さん が その 声 を 受け取っ て 、 様々 に 地方 公共 団体 で 公式 に 議論 を する こと が 大事 で あろう と いう こと を 考え て おり ます 。   そう いっ た 意味 で 、 地方 公共 団体 の 議会 の 同意 を 要件 に する と いう よう な こと も 附帯 決議 に 盛り込ま れ まし た の で 、 こう し た こと を 踏まえ て 実施 法 が 制定 さ れ て いく 流れ に なっ て いる もの だ と 我々 も 承知 を いたし て いる ところ で あり ます 。 <br/> ○ 清水 貴之 君   先ほど も 、 運営 会社 の 選定 方法 に つい て も 非常 に 厳格 に 行っ て いく と いう 話 が あり まし た 。 真 に 適格 な 者 を しっかり と 選定 し て いく と いう こと な ん です が 、 その 上 で 、 適合 性 に つい て の 調査 、 これ に つい て 法制 上 の 措置 を とり ましょう と 、 事業 主体 と し て の 一体 性 及び 事業 活動 の 廉潔 性 が 確保 さ れる ため の 法制 上 の 措置 を と いう よう な 話 が あり ます 。 この 辺り 、 法制 上 で 、 法 を 整備 し て つくっ て いく と いう 話 に なっ て いる ん です が 、 この 辺り の 意図 で あっ たり 狙い で あっ たり 、 どう いっ た 思い で ある か 、 これ を お 聞か せ いただけ ます でしょう か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   ここ は 、 厳格 な 参入 基準 を 満たし て いる か 否 か の 調査 、 例えば カジノ 運営 者 の 関係 者 に つい て 厳格 な 背面 調査 を 行う など が 想定 さ れる と 、 こう いう ふう に 思い ます 。 具体 的 な 内容 に つい て は 、 本 法案 の 施行 後 一 年 以内 を 目途 と し て 政府 に おい て 策定 さ れる 実施 法案 の 中 に おい て 定め られる もの と 、 こう 思っ て おり ます けれど も 、 世界 で 最も 厳格 な 対策 を 取っ て 、 背面 調査 も しっかり やっ て 決め て いく と いう こと だ と 思っ て い ます 。 <br/> ○ 清水 貴之 君   入場 の 規制 、 ルール 作り に つい て も お 聞か せ を いただき たい と いう ふう に 思い ます 。 ＩＲ 整備 さ れ た 場合 、 カジノ の 入場 に つい て です 。   これ は もう 各国 いろんな 例 が ある と 思い ます 。 お 金 で ある 程度 入場 規制 を する 、 入場 料 を 取る と いう よう な 仕組み を 取っ て いる ところ も あり ます し 、 国籍 、 自国 の 方々 は 入れ ない と いう ところ 、 もう 皆 さん 御 存じ の とおり 様々 な ルール を 行っ て いる 中 で 、 やはり 日本 なら で は の ところ を つくっ て いか なけれ ば いけ ない と いう ふう に 思っ て い ます 。   細かく は また 実施 法 で と いう こと に なる の か も しれ ませ ん が 、 この 辺り 、 これ まで の 議論 の 中身 と か 経過 と か あり まし たら 教え て いただけ ます でしょう か 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   これ も 再三 、 同じ 議論 に なる か も しれ ませ ん が 、 先ほど から も 答弁 あり まし た よう に 、 我々 は 、 後発 だ から こそ 世界 で 一番 厳格 な 仕組み を 入れ て いける ん じゃ ない か と いう ふう に 考え て 、 それ を 目指し て この 法律 を 今 皆 様 に お 願い し て いる ところ で あり まし て 、 当然 、 青少年 の 健全 育成 、 暴力 団 員 等 の 関与 の 排除 、 ギャンブル 依存 症 へ の 対策 等 の ため に 入場 規制 を 考え て いる ところ で あり まし て 、 そして 加え て 言え ば 、 未 成年 者 に つい て は 入場 する こと も 禁止 す べき で あろう と か 、 写真 付き の 身分 証明 書 など で の 年齢 確認 等 も 考え て おり まし て 、 委員 お 触れ に なり まし た よう な 自己 排除 、 家族 排除 プログラム の 導入 、 これ ら は シンガポール で も 行わ れ て おり ます 。   各国 の そう いう いい ところ を しっかり と 集め た 形 で の 今後 の 実施 法 を イメージ し て いる ところ で あり ます 。 <br/> ○ 清水 貴之 君   この 附帯 決議 の 中 に は 、 その 入場 規制 の 際 に 、 それ に 加え て マイ ナンバー カード を 活用 し て は どう だ と いう よう な 話 も あり ます 。 そこ まで 、 もちろん マイ ナンバー が しっかり 普及 を し て い て 皆 さん もう 使っ て いる 状態 の 中 で だっ たら これ は まあ 有効 か な と も 思う ん です が 、 正直 そこ まで 現 時点 で マイ ナンバー カード が しっかり と 普及 さ れ て いる もの で も ない 、 様々 な 問題 も 起き て いる よう な ところ も ある と 。 この マイ ナンバー カード 自体 に つい て も 意見 の 分かれ て いる ところ も ある か な と いう ふう に も 理解 を し て おり ます けれど も 、 この マイ ナンバー カード の 活用 に つい て は どの よう な 話 が 出 て いる ん でしょう か 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   我々 、 当然 マイ ナンバー カード に つい て も 、 これ を 機 に 取得 を する 人 が 増えれ ば 、 それ も 社会 の ため に 資し た こと に なる の か な と いう ふう に は 考え て おり ます 。   いずれ に し て も 、 身分 証明 書 は それ だけ に 限る の か どう か と いう こと も 、 免許 証 等 も ある わけ です から 、 どの よう に 入れ て いく の か と いう こと も これ から 実施 法 の 中 で 、 もう 本当 に 細か な 技術 も どんどん どんどん と 進展 し て おり ます 、 海外 など で は 顔 を センサー で ピッ と やっ たら すぐ 誰 だ か 分かる と いう よう な 仕組み も ある と いう よう な こと です の で 、 どの 辺り に する か 、 その 技術 等 も 考え て 、 我々 やっ て いき たい と 思っ て おり ます 。 <br/> ○ 清水 貴之 君   あと もう 少し に なり まし た けれど も 、 ギャンブル 依存 症 対策 の 話 も 、 私 も お 聞き し たい と 思い ます 。   様々 な 対策 が ある 中 で 、 どっち か と いえ ば 私 は 施設 の 周辺 で の 対策 を どう する べき か と いう の を 考え て いく べき か な と いう ふう に 思っ て おり まし て 、 その 施設 周辺 で 、 もちろん 日本 全体 で 様々 な ギャンブル 依存 症 が あっ て 、 その 対策 も 必要 な ん です けれど も 、 そこ に カジノ 依存 症 の 方 が 、 まだ でき て い ない から まだ 現 時点 で は ない と いう 話 だっ た ん です けれど も 、 ほか の ギャンブル が ある わけ です から 、 そう いっ た 方々 が 、 気楽 に と 言う と ちょっと 言い 方 が 悪い か も しれ ませ ん けれど も 、 相談 が し やすい 体制 で あっ たり と か 、 それ に 対し て 対応 を 取り やすい 医療 体制 で あっ たり と いう の を 、 カジノ が できる の を きっかけ に その 周辺 で も 一緒 に なっ て 整備 を し て いく べき で は ない か と いう ふう に 思い ます が 、 いかが でしょう か 。 <br/> ○ 衆議 院 議員 （ 松浪 健太 君 ）   これ も 当然 の 話 だ と 思い ます けれど も 、 国 だけ で は なく て 、 国 も 当然 これ に は しっかり と し た 方針 を 持っ て 取り組む わけ で あり ます けれど も 、 この 附帯 決議 等 で 、 地方 公共 団体 に つい て も 、 それぞれ の 地域 に よっ て 様々 な 特性 が あろう か と 思い ます の で 、 その 地方 公共 団体 で も 積極 的 に 細やか な 体制 を 取っ て いただける よう に 、 今後 実施 法 を 作っ て いただき たい と いう ふう に 考え て おり ます 。 <br/> ○ 清水 貴之 君   次 は カジノ の 営業 規制 の 話 だっ た ん で 、 ちょっと 一 つ 、 先ほど お 答え いただき まし た 、 飛ばさ せ て いただい て 、 これ も ちょっと 近い ところ を 先ほど 私 個人 的 に 聞か せ て いただい た ん です が 、 附帯 決議 に も 入っ て おり ます の で 繰り返し に なる か も しれ ませ ん が 、 カジノ の 運営 会社 が 民間 事業 者 に なり ます 。 こう なる と 、 適用 さ れる 税制 ・ 会計 規則 等 、 この 辺り も 検討 し て いく べき で は ない か と いう の が 附帯 決議 に 入っ て おり ます 。 この 辺り の 考え を お 聞か せ いただけ ます でしょう か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   カジノ 施設 は 、 民設 民営 で あり ます けれど も 、 カジノ 管理 委員 会 に よる 厳格 な 規制 、 監督 の 下 で 運営 の 透明 性 の 確保 を 図る 必要 が ござい ます 。 衆議 院 の 内閣 委員 会 に おけ る 附帯 決議 の とおり 、 カジノ 運営 事業 者 に 適用 さ れる 税制 ・ 会計 規則 等 に つい て は 、 諸 外国 の 制度 を 十分 に 勘案 の 上 、 政府 に おい て 実施 法案 の 立案 過程 で 十分 な 検討 が 加え られ 、 適切 に 判断 さ れる もの と 思っ て おり ます 。 <br/> ○ 清水 貴之 君   今 、 カジノ 管理 委員 会 の 話 が 出 まし た 。 これ まで に も 出 て い ます けれど も 、 どう いっ た 組織 に し て いく の か と いう の も 大事 です し 、 その 中 の 、 入る 方々 が しっかり と 監督 を する 仕組み と いう の も つくっ て いか なけれ ば なり ませ ん 。 そう いっ た 人材 の 育成 と いう の も 非常 に 大切 な こと で は ない か な と いう ふう に 思っ て い ます 。 この 辺り は どの よう に 進め て いく と 思っ て いらっしゃる ん でしょう か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   この 法 案 全体 に おい て 、 カジノ 管理 委員 会 の 役割 と いう の は 極めて 重要 に なる わけ で あり ます 。 附帯 決議 で も あり ます と おり 、 独立 し た 強い 権限 を 持つ いわゆる 三 条 委員 会 と し て 設置 し て 、 カジノ 管理 委員 会 が カジノ 営業 規制 等 を 厳格 に 執行 できる 体制 の 構築 が 不可欠 で あり ます 。 特に 、 カジノ 導入 時 から 厳格 な 規制 を 執行 できる よう 、 十分 な 機構 ・ 定員 を 措置 する と とも に 、 適切 な 人材 を 配置 する ほか 、 厳格 な カジノ 営業 規制 等 や 関係 事業 者 に 対する 行政 処分 等 の 監督 を 有効 に 執行 できる 人材 育成 を す べき こと は 御 指摘 の とおり で ござい ます 。   具体 的 な 人材 育成 の 方法 に つい て は 、 諸 外国 の 経験 を しっかり と 勉強 し て 、 政府 に おい て 立案 過程 で 十分 に 検討 し て いく もの と 思っ て おり ます 。 <br/> ○ 清水 貴之 君   そして 、 その カジノ 管理 委員 会 と やはり 地元 の 警察 組織 と の 連携 と いう の も 大切 に なる と いう ふう に 思い ます 。 都 道 府 県 警察 その 他 関係 機関 と の 連携 、 これ は もう ＩＲ の 施設 と し て の 連携 も そう です 、 民間 事業 者 と の 連携 も そう です し 、 カジノ 管理 委員 会 と の 連携 も 、 これ も 必要 に なっ て くる 大切 な こと だ と 思い ます が 、 これ は どの よう な イメージ でしょう か 。 <br/> ○ 衆議 院 議員 （ 小沢 鋭 仁君 ）   これ も 附帯 決議 で 書い て いただき まし た よう に 、 この 特定 複合 観光 施設 の 設置 の 前提 と し て 犯罪 防止 ・ 治安 維持 、 青少年 の 健全 育成 、 依存 症 防止 等 の 観点 から 問題 を 生じ させ ない よう に する ため に 、 都 道 府 県 の 警察 その 他 の 関係 機関 の 必要 な 体制 を 確保 する よう 附帯 決議 と し て 決定 を し て いただい た ところ で ござい ます 。   特に 、 カジノ 管理 委員 会 と 都 道 府 県 警察 と の 関係 等 は 緊密 な 関係 を 持っ て いか なけれ ば いけ ない と 、 こう 思っ て おり ます し 、 そう いっ た 連携 の 具体 的 な 内容 は 、 実施 法 の 中 で よく 各 省庁 の 中 で 詰め て いただき たい と 我々 は 要望 し て いる ところ で ござい ます 。 <br/> ○ 清水 貴之 君   質問 は 以上 で 終わら せ て いただき ます けれど も 、 私 も 是非 ＩＲ 進め て いく べき だ と いう よう な 立場 です 。 ただ 、 その 一方 で 、 今日 も たくさん いろいろ 質問 に 上がっ た よう に 、 私 も 聞か せ て いただい た よう に 、 問題 点 、 解決 し て いか なけれ ば いけ ない 部分 と いう の も 多々 ある こと だ と いう ふう に 思っ て い ます 。   これ は あくまで **推進** **法** ( `特定複合観光施設区域の整備の推進に関する法律` ) です から 、 あと は 細かい 法律 で と いう 話 な ん です が 、 ここ も 、 **推進** **法** ( `特定複合観光施設区域の整備の推進に関する法律` ) が 通っ た から と いう 話 で は なく て 、 やはり 省庁 とも しっかり 連携 を 取り ながら 、 いい 施設 を 、 そして 皆 さん が 喜ん で もらえる よう な 施設 を 、 そして 長く 続く よう な 施設 を 、 問題 点 も しっかり と クリア を し ながら 、 で も 造っ て 良かっ た ね と 言わ れる よう な 施設 を 目指し て 、 もう 皆 さん に も 頑張っ て いただき たい です し 、 私 なんか も 一緒 に なっ て 取り組ん で いけ たら な と いう ふう に 思っ て おり ます 。   以上 で 質問 を 終わり ます 。 ありがとう ござい まし た 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   暫時 休憩 いたし ます 。       午後 四 時 五十 九 分 休憩           ─ ─ ─ ─ ─ ・ ─ ─ ─ ─ ─       午後 七 時 三十 分 開会 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   ただいま から 内閣 委員 会 を 再開 いたし ます 。   休憩 前 に 引き続き 、 **特定** **複合** **観光** **施設** **区域** **の** **整備** **の** **推進** **に** **関する** **法律** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) を 議題 と し て 質疑 を 行い ます 。   他 に 御 発言 も ない よう です から 、 質疑 は 終局 し た もの と 認め ます 。   本案 の 修正 に つい て 上月 良祐 君 から 発言 を 求め られ て おり ます の で 、 この 際 、 これ を 許し ます 。 上月 良祐 君 。 <br/> ○ 上月 良祐 君   私 は 、 **特定** **複合** **観光** **施設** **区域** **の** **整備** **の** **推進** **に** **関する** **法律** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に 対し 、 自由 民主 党 を 代表 し て 、 修正 の 動議 を 提出 いたし ます 。 その 内容 は 、 お 手元 に 配付 さ れ て おり ます 案文 の とおり です 。   これ より 、 その 趣旨 に つい て 御 説明 いたし ます 。   修正 の 要旨 は 、 政府 が カジノ 施設 の 設置 及び 運営 に 関し 講 ず べき カジノ 施設 の 入場 者 が 悪 影響 を 受ける こと を 防止 する ため に 必要 な 措置 と し て 、 ギャンブル 依存 症 等 の 防止 に つい て 明示 する と とも に 、 この 法律 の 規定 及び 第 五 条 の 規定 に 基づく 措置 に つい て は 、 この 法律 の 施行 後 五 年 以内 を 目途 と し て 必要 な 見直し が 行わ れる べき もの と する 旨 の 規定 を 置く もの で あり ます 。   何とぞ 委員 各位 の 御 賛同 を 賜り ます よう お 願い 申し上げ ます 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   これ より 原案 及び 修正 案 に つい て 討論 に 入り ます 。   御 意見 の ある 方 は 賛否 を 明らか に し て お 述べ 願い ます 。 <br/> ○ 相原 久美子 君   私 は 、 民進 党 ・ 新緑 風 会 を 代表 し て 、 **特定** **複合** **観光** **施設** **区域** **の** **整備** **の** **推進** **に** **関する** **法律** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に つい て 、 反対 の 立場 から 討論 を 行い ます 。   まず 初め に 、 今回 の この いわゆる **ＩＲ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) の 拙速 な 審議 に 対し 、 立法 府 と し て の 重大 な 懸念 を 表明 いたし ます 。   本 法案 が 衆議 院 の 内閣 委員 会 で の 審議 の 俎上 に 上っ た の は 、 僅か 二 週間 前 で あり ます 。 この 間 の 衆議 院 で の 審議 時間 は 六 時間 に 足ら ず 、 与党 公明 党 は 意見 集約 が でき ず 、 採決 に 当たっ て 党議 拘束 を 外す 異例 の 対応 を 行い まし た 。 参議 院 に おい て は 約 十 六 時間 の 審議 を 行い 、 参考 人 の 御 意見 も 伺う こと が でき た と は いえ 、 カジノ に 対する 国民 の 不安 を 払拭 する に は 全く 不 十分 と 言わ ざる を 得 ませ ん 。   以前 より 、 **ＩＲ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) は 、 国内 に おけ る 賭博 禁止 と の 整合 性 、 ギャンブル 依存 症 へ の 懸念 など 様々 な 問題 を はらん で おり 、 国民 の 間 で も 慎重 に 審議 す べし と の 声 の 大きい もの で あり まし た 。 日頃 政権 に 肯定 的 な 論調 で 知ら れ て いる 新聞 社 も 含め 、 全国 紙 が そろっ て 反対 の 社説 を 掲載 し 、 半数 以上 の 人々 が 懸念 を 示し 続け て いる 法案 で あり ます 。 果たして 、 私 たち は 委員 会 審議 に おい て この 声 に 十分 に 応え られ た と 胸 を 張れる でしょう か 。   また 、 議員 立法 で ある 本 法案 の 取扱い に つい て も 懸念 が あり ます 。   本来 、 議員 立法 は 、 事前 に 各党 間 で 丁寧 に 合意 形成 を 図る から こそ 委員 会 審議 が 短 時間 で 済む もの で あり ます が 、 それ を 今回 の よう に 与野党 で の 合意 も 十分 に 形成 し ない まま 審議 入り し 、 委員 会 で の 審議 も そこそこ に 一気 に 突破 する よう な やり 方 は 、 合意 形成 と 手続 を 重んじ て き た この 立法 府 の 在り 方 を 軽んじる もの に ほか なり ませ ん 。 まして や 、 ここ 参議 院 は 熟考 の 府 で あり 、 衆議 院 の 拙速 を 戒める べき 役割 を 自ら の 存在 意義 と し て き た の で あり ます 。 数 の 力 を 背景 に 合意 形成 と 手続 を 軽んじ て 採決 に ひた走る 姿 は 、 到底 容認 できる もの で は あり ませ ん 。   以上 の 理由 から 、 衆議 院 で の 新た な 審議 を 求め 、 法案 に 断固 反対 いたし ます 。 <br/> ○ 田村 智子 君   私 は 、 日本 共産 党 を 代表 し て 、 **ＩＲ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) 、 すなわち **カジノ** **・** **賭博** **解禁** **法** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) 及び 修正 案 に 反対 の 討論 を 行い ます 。   まず 、 この よう に 突然 修正 案 が 提案 さ れ 、 今朝 の 理事 会 で も 合意 に なら なかっ た 採決 が 突如 と し て 行わ れる こと に 強く 抗議 する もの です 。   これ まで の 本 会議 と 委員 会 質疑 で も 、 法務 委員 会 等 と の 連合 審査 、 地方 公聴 会 、 ギャンブル 依存 症 に 関わる 施設 の 視察 、 民間 カジノ 解禁 に つい て の 法務 省 の 見解 を まとめ た 当事 者 を 招致 し て の 参考 人 質疑 など が 要求 さ れ て おり 、 これ ら は 理事 会 協議 事項 と なっ て い た はず です 。 参議 院 の 審議 が 尽くさ れ た どころ か 、 乱暴 に 審議 を 断ち切ら れ た と 言わ なけれ ば なり ませ ん 。 これ で は 、 大 多数 の 国民 の 皆 さん が 反対 、 不安 の 声 を 上げる 中 、 衆議 院 に 続い て 参議 院 で も その 国民 の 声 を 乱暴 に 踏みにじっ た と 言わ なけれ ば なり ませ ん 。   本 法案 へ の 反対 の 理由 の 第 一 は 、 刑法 が 禁ずる 賭博 行為 で ある カジノ を ＩＲ と いう 隠れみの に よっ て 違法 で は ない と 、 日本 の 歴史 上 初めて 犯罪 で は ない 違法 性 の 阻却 を 行う こと です 。   発議 者 や カジノ 推進 者 は 、 本 法案 が カジノ を 含む 統合 型 の 観光 施設 を 整備 する もの で 、 カジノ を 解禁 する 法案 で は ない と いう ごまかし を 続け て い ます 。   しかし 、 これ まで の 審議 で 、 カジノ 抜き の ＩＲ なら ば 法案 は 必要 ない と 明確 に 答弁 が あり 、 さらに 、 本日 の 委員 会 質疑 で は 、 民間 カジノ に つい て 違法 性 の 阻却 の 法 整備 を 政府 が 行う こと に なる こと も 明言 さ れ まし た 。 違法 性 の 阻却 を どの よう に 行う か は 政府 が 判断 する が 、 本 法案 に よっ て ＩＲ の カジノ は 刑法 違反 に は なら ない と いう 結論 を 先 に 出し て しまう 、 この よう な 乱暴 な やり 方 で 刑法 に 大穴 を 空ける こと は 断じ て 許さ れ ませ ん 。   昨日 の 参考 人 質疑 で は 、 これ まで 賭博 が 公設 公営 に 限定 さ れ て き た 、 つまり は 民間 賭博 が 認め られ て こ なかっ た 理由 と し て 法務 省 が 示し た 八 項目 に つい て 、 弁護 士 の 中 で も 百 八十 度 異なる 解釈 論 が 展開 さ れ まし た 。 刑法 と いう 我が 国 の 基本 的 な 法律 に つい て この よう に 解釈 が 分かれ た まま で 、 どう し て ＩＲ の カジノ が 違法 性 が 阻却 さ れる と いう 結論 を 出す こと が できる の でしょう か 。   これ まで の 特別 法 に よっ て 公設 公営 の 賭博 のみ が 認め られ て き た その 理由 の 一 つ で ある 目的 の 公益 性 は 、 収益 の 使途 を 公益 性 の ある もの に 限る こと に よっ て 担保 さ れ て き た 、 これ が 法務 省 の 見解 です 。 これ は 、 民間 企業 が その 収益 を 上げる こと が なけれ ば 成り立た ない カジノ で は 決して クリア でき ない こと は 明白 で は あり ませ ん か 。 刑法 に 大穴 を 空ける 、 こんな 審議 で 採決 を 行う こと を 改めて 強く 抗議 を する ところ です 。   本 法案 へ の 反対 の 理由 の 第 二 は 、 ギャンブル 依存 症 対策 と どれ ほど 繰り返し 、 それ が 修正 案 に 盛り込ま れ た ところ で 、 一方 で カジノ を 解禁 する なら ば 、 新た な ギャンブル 依存 症 が 生み出さ れる 、 事後 の 処理 しか でき ない 、 大変 な 欠陥 の 法案 だ と いう こと です 。   これ まで も 、 パチンコ 、 スロット 、 あるいは 競馬 、 競艇 など で の ギャンブル 依存 症 が いかに 深刻 か と いう こと が この 委員 会 の 中 で も 審議 が さ れ て き まし た 。 マイナス が ある から と いっ て プラス 面 に 目 を つぶる の か 、 こんな 乱暴 な 議論 でし た 。 マイナス は 、 たった 一 つ の 家族 で あっ て も 家庭 の 崩壊 を もたらし ます 。 その 人 の 人生 を ぼろぼろ に し ます 。 その マイナス を 当たり前 の 前提 と し て 、 プラス が ある から いい ん だ と 、 ギャンブル 依存 症 に 目 を つぶり 、 新た な ギャンブル 依存 症 を 生み出す こと を 是 と する 、 この よう な **カジノ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) を 認める こと は 絶対 に でき ませ ん 。   本 法案 に 反対 の 理由 の 第 三 は 、 経済 成長 戦略 を カジノ に 頼る と いう こと が 余り に 情けなく 、 余り に 恥ずかしい から で あり ます 。   元々 賭博 は 人 の お 金 を 奪い取る 所業 で あり 、 経済 政策 と いう よう な 代物 で は あり ませ ん 。 その 奪っ た お 金 は カジノ の 営業 を 行う 者 に 吸い取ら れ て いく 、 その 一部 が 納付 金 に さ れ て 、 それ に よっ て 公益 性 が ある など と いう 、 こんな 議論 は 余り に も 情けない もの で は あり ませ ん か 。 推進 し て き た 大阪 商業 大学 の 谷岡 学長 が 言う よう に 、 これ から は 預金 で は なく 、 預金 を カジノ に 差し出せ と 言わ ん ばかり の 推進 派 の 議論 を 認める こと など 到底 できる もの で は あり ませ ん 。 だ から こそ 、 多く の 国民 が 今 も 反対 の 議論 、 声 を 上げ て いる の で は ない でしょう か 。   修正 案 が 急遽 出さ れ まし た が 、 これ ら の 修正 案 は 何 ら この 法 案 の 欠陥 を 修正 する もの で は なく 、 認める こと は でき ませ ん 。 引き続き の 審議 が 必要 で あり 、 衆議 院 に もう 一 度 審議 を 求める と いう の なら ば 、 きっぱり と 廃案 に し て 、 発議 者 に 対し て も 猛省 を 促し て 衆議 院 で の 審議 を 求める べき で あっ た 、 この こと を 強く 申し上げ 、 反対 の 討論 を 終わり ます 。 <br/> ○ 山本 太郎 君   私 は 、 **ＩＲ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) 、 いわゆる **カジノ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に 反対 の 立場 で 討論 を いたし ます 。 自由 ・ 社民 の 会派 、 希望 の 会 を 代表 し 、 反対 討論 を 行い ます 。   本当 に 悲しい 、 これ が 国会 な の って 思っ ちゃい ます よ 。 もう 慣れ た か も しれ ない 、 皆 さん は 、 こんな もの だ って 、 政治 は 。 で も 、 私 は まだ 日 が 浅い 、 三 年 しか たっ て い ない 。 だ から 、 まだ 町 の 感覚 、 人々 の 声 は 聞こえる 、 こんな 形 で こんな こと を 決め て いい ん です か って 。   私 たち を 食べ させ て くれ て いる の は 、 そして この 国 を 回せ て いる の は 税金 です よ って 。 誰 の 声 を 聞い て やる ん です か 。 国民 の 中 で 、 ギャンブル できる よう に し て くれ 、 カジノ 、 賭場 を 開い て くれ と いう 声 、 どれ ぐらい あり まし た 。 慎重 に やっ て くれ 、 ちょっと 行き 過ぎ て いる ん じゃ ない か 、 そう いう 声 の 方 が 多い じゃ ない です か 。 誰 の 声 を 聞い て 政治 を やる ん です か って 、 誰 の 国 の 政治 を 今 行っ て いる ん です か って 。   この カジノ と いう 考え 方 に 関し まし て は 、 特区 で 元々 あり まし た よ ね 。 特区 で でき なかっ た 理由 、 平たく 言う と 、 ばくち 場 を 開く こと を 特区 だけ で 合法 化 する に は 無理 が ある と いう 話 でし た 。 なら ば 違う ルート で 、 プログラム 法 と 実施 法 と いう 二 段階 で 分かり にくく 、 ばくち 場 を 開ける 場所 や 主体 を 合法 化 し て しまう 準備 法案 だ と も 言え ます 。   正面 から 来れ ば いい じゃ ない です か 。 ＩＲ 、 劇場 が どう し た 、 会議 場 が どう し た 、 もちろん それ も ある だろう けど 、 本丸 は カジノ じゃ ない か って 、 ばくち じゃ ない か 、 本丸 は と いう 話 な ん です よ 。 そして 、 この 先 の 法案 、 今回 の 法案 が 通っ た 先 の 法案 、 これ を 通せ ば 、 法務 省 と 警察 庁 の 懸念 、 刑法 百 八十 五 条 、 百 八十 六 条 に 係る 懸念 、 今回 と 先 の 立法 で 払拭 なさっ た 後 は 、 この 後 どう なっ て いく か と いう 話 です よ ね 。   一 度 その よう な 形 で 迂回 を さ せ て 、 ばくち 場 を 開ける よう に し た 上 で 、 先々 危険 性 と し て 考え られる の は 、 国家 戦略 特区 に 認定 すれ ば 、 カジノ を 運営 する 企業 に 税 控除 と いう インセンティブ まで 与える と いう 話 に なる 。 カジノ で もうけ て 、 その 税金 や 納付 金 を 使っ て 公益 に 資する こと を やり ます と いう 話 が 、 もし も 特区 に これ が 指定 さ れ て しまう よう な 先々 出来事 が あっ た と する なら ば 、 法人 税 は 控除 、 所得 税 も 控除 、 設備 投資 も 控除 で 、 もらえる 税金 も もらえ ない じゃ ない か って 。 また 企業 の ため に やっ て いる の か って 、 また 海外 の 企業 の ため に も 動く の か と いう 話 に なっ ちゃい ます よ 。 それ が 海外 企業 で あろう と 国内 の パチンコ 、 スロット 関係 企業 で あろう と 、 こんな やり 方 、 こんな 税金 の 使わ れ 方 に 納得 する 人 は い ませ ん 。   税金 を 掛け ない 、 皆 さん の 税 負担 なし で 民間 が やり ます から と うたっ て おい て 、 もし 先々 戦略 特区 内 で やる こと に よっ て その よう な こと に なれ ば 、 これ 一体 どう なる ん です か 。 もう 一 回 やり直し できる ん です か 。 でき ませ ん よ ね 。 じゃ 、 待っ て ください 、 この 戦略 特区 で 先々 やり ませ ん と いう 担保 、 この 法 案 の 中 に ある ん です か 。 取りあえず 突破 し て しまえ ば 関係 ない 話 な ん です か ね 。   しかも 、 これ が 外資 で あれ ば 、 税 控除 を 受け て 安く 事業 が やれ た 上 に 、 再 投資 も ほとんど する こと なく 、 日本 人 の 富裕 層 から は 大きく 、 そう で も ない 方々 から は それ なり に 資産 を 巻き上げる 、 国富 の 流出 を 税金 まで 使っ て お 手伝い を する こと に なる 。 これ で 納得 する の は 誰 でしょう 。 予定 地 を 地上げ する 不 動産 屋 です か 。 投資 する 人 、 ゼネコン 、 あと カジノ の 運営 企業 だけ 。 その よう な こと の 歯止め に なる 条文 も 入っ て い ない 。 プログラム 法 だ から と いう いつ も の せりふ です か ね 。 で も 、 それ ぐらい 気 を 付け て やら なきゃ いけ ない と いう 話 だ と 思う ん です 。   外資 事業 者 は 短期 な 利益 を 追求 する 傾向 が ある 。 その 結果 、 公共 性 、 周辺 産業 へ の 波及 効果 は 抑制 さ れる 。 超過 利益 の 海外 流出 、 つまり 日本 国内 に 再 投資 さ れ ない こと は 容易 に 想像 できる 。 外資 の カジノ の 運営 関連 会社 など に 対し て 日本 を 草刈り 場 と し て 差し上げる こと に なら ない か 、 その よう な 問題 点 。   二 〇 一 四 年 、 都内 で 開催 さ れ た カジノ の 国際 会議 、 ジャパン ・ ゲーミング ・ コングレス 。 主な 顧客 は 外国 人 観光 客 で は ない 、 日本 人 の 富裕 層 と いう 内容 の やり取り が 行わ れ て い た 。 日本 人 富裕 層 の 個人 金融 資産 量 を 日本 に できる 推定 カジノ 数 三 から 十 で 割る 、 海外 に 比べ て 、 日本 の 一 つ の カジノ 当たり の 個人 金融 資産 量 は 突出 し て いる から 日本 の カジノ は 莫大 な 利益 は 確実 、 その よう な 宣伝 が 日本 国内 で 行わ れ て いる 、 余り に も おかしい じゃ ない です か 。   ＩＲ で バラ色 経済 論 を 振りまく カジノ 幻想 の 現実 を 見 て みる 、 周辺 は どう なっ て いる 。 海外 で は 、 ＩＲ 、 カジノ で は コンプ と いう サービス 、 お 客 さん 、 カジノ が 囲い込ん で い ます よ ね 。 中 に 入っ たら 出 ない ん です よ 。 観光 地 の 近く に と か 日本 の すばらしい ところ を 見 て もらおう と いっ たって 、 この コンプ と いう 制度 も ある 上 に 、 その 中 から 出 ない ん です よ 。   コンプ と いう の は 英語 の コンプリメンタリー の 略 、 日本 語 に する と ポイント 優待 サービス 、 ＩＲ の カジノ や ホテル 、 レジャー 施設 の 中 だけ で 使える ポイント 制度 。 カジノ で の 賭け 額 の 一定 比率 が ポイント と し て 還元 、 その ポイント を 使っ て 宿泊 、 飲食 、 娯楽 等 が 無料 に なっ たり 、 あるいは 格安 で 利用 できる よう に なる 。   先日 参考 人 と し て 来 て いただい た 鳥畑 先生 の 論文 に よる と 、 ニュージャージー 州 カジノ 管理 委員 会 の レポート 、 二 〇 一 二 年 、 アトランティック シティー 内 に ある 十 二 の カジノ ホテル の 収入 五千 百 三十 九 万 ドル 、 約 六十 億 円 、 この 収益 の ため に 使わ れ た ホテル の コンプ 、 実 に 二千 七百 五十 八 万 ドル 、 約 三十 二 億 円 、 カジノ ホテル の 収入 の 半分 に 匹敵 する 額 に 上っ て いる 。 また 、 ラスベガス の ストリップ 地区 、 四十 三 の カジノ で 売上げ の 三 〇 ％ を コンプ に 費やし て おり 、 最終 的 に は 大 赤字 に なっ て いる 。   結局 、 コンプ の ポイント サービス で 多く の 客 が ＩＲ 施設 の ホテル や レストラン に 囲い込ま れ て 、 周辺 の 商店 、 レストラン は 寂れ て いっ て いる と いう 現実 が もう はっきり と 分かっ て いる の に 、 そして 外 に 観光 に 出 て いく なん と いう こと が ほとんど なくなっ ちゃう よ 、 中 が 楽しい ん だ もの 、 だ って 。 時計 も ない 、 窓 も ない 、 そんな ところ で 一 日 中 やり 続ける と いう よう な 状況 に 人々 が 陥っ た とき に 、 観光 も 何 も ない じゃ ない です か って 。 しかも 、 周辺 も そこ まで おいしい 思い でき ない よ 、 最初 だけ です よ 。 この よう な こと を 推し進め られ て いく なんて 余り に も おかし 過ぎる 。   で 、 これ 海外 企業 じゃ なく 国内 企業 だっ たら どう な ん だ って 。 セガサミーホールディングス 、 韓国 カジノ 最 大手 の パラダイス グループ と の 合弁 で 立ち上げる 統合 型 リゾート 、 ＩＲ 、 二 〇 一 七 年 五 月 開業 を 目指し て 既に 建設 中 。 日本 の 企業 だっ たら まだ いい か 、 そう 思う か も しれ ない 。 けれど も 、 この セガ サミー の 会長 さん 、 里見 治 さん 、 安倍 政権 、 深い つながり が ある と 。 セガ サミー 会長 の 娘 さん の 結婚 式 に 総理 や ほか の 閣僚 が わざわざ 出席 さ れる ほど の 仲 、 こう いっ た きずな で お 友達 人事 的 に カジノ の 運営 決定 さ れる 可能 性 否め ませ ん よ ね 。   そして 、 これ は セガ サミー に 限っ た 話 だけ で は あり ませ ん 。 パチンコ 、 スロット に 関する 企業 が カジノ の 運営 権 を 手 に 入れ た 場合 、 この 委員 会 で も 衆議 院 で も 一番 問題 に なっ た 依存 症 の 問題 に 対し て ちゃんと し た 規制 が でき ます か って 。 その 企業 の 母体 の 収益 、 すなわち パチンコ 、 スロット に 関連 する 収益 に 影響 する よう な 規制 が 、 ギャンブル 依存 症 対策 が 、 パチンコ 、 スロット 規制 が 本当 に できる の か って 。 依存 症 対策 、 進み にくく なる 原因 が 生まれ て くる ん じゃ ない でしょう か 。   ギャンブル 症 者 、 ギャンブル 依存 症 の 方 は 、 はまる ギャンブル 、 五 割 から 六 割 が パチンコ 、 スロット 。 パチンコ 、 スロット 絡み で ない の は 五 ％ 以下 で しか あり ませ ん 。 しかも 、 女性 で は ほぼ 全例 が パチンコ 、 スロット 。 パチンコ 、 スロット の ホール 、 全国 の ローソン より も 多い 一 万 二千 館 、 世界 中 に ある パチンコ 、 スロット の 機器 、 七百 二十 万 台 中 三 分 の 二 が 日本 に ある ん だ って 、 これ 誰 が つくり出し た ん です か って 、 国 です よ 、 政治 です よ って 。 それ に 対する 依存 症 者 が たくさん いる と 思わ れる 。 もう 既に 重症 化 し て いる 人 たち たくさん い ます よ 。 一 度 なっ たら なかなか 抜け られ ない 。 大根 が たくあん に なる よう な もの だ って 、 たくあん から 大根 に は 戻れ ない よう に 、 なかなか そこ から また 元 に 戻る なんて 大変 な 作業 な ん だ 。 それ を 国 と し て 野放し に する よう な 状況 で ずっと それ を エスカレート さ せ て いっ た 現実 が ある じゃ ない か って 。   カジノ 解禁 じゃ ない ん だ よ 、 ＩＲ が どう し た と いう 話 じゃ ない ん だ よ 。 まず 目 の 前 の ここ に 対策 しろ と いう 話 だ と 思う ん です よ 。 それ が 政治 な ん じゃ ない の 。 もちろん 話 の 中 に あり ます よ 、 そこ に なかなか 対策 費 が つぎ込ま れ ない から 、 だ から こそ 、 この ＩＲ を やり ながら 、 そこ から 得 た 収益 で やっ て いこう じゃ ない か 、 その 考え 方 も ある と 思う 。 で も 、 まず ＩＲ 、 カジノ と 言う 前 に 、 立ち止まる と いう 考え 方 を し て いただき たい ん です よ 。 胴元 が 勝っ て 商売 が 成り立つ 、 要 は 負け た 人 たち 、 この 人 たち を 踏み付け ながら 経済 成長 し て いく 企業 や そして 世界 が ある ん だ と し たら 、 それ 地獄 じゃ ない です か 。 世界 中 の カジノ が 寂れ て いる 原因 、 それ な ん じゃ ない です か 。   ギャンブル 、 自殺 と の 関係 で は 、 一 年 以内 の 自殺 は 一般 の 約 十 倍 、 生涯 の 自殺 は 一般 の 約 四十 倍 に も なる 。 これ ら は ほとんど マスコミ が 取り上げ ない 。 統計 データ が ある の に 知ら ない 人 の 方 が 多い だろう と ギャンブル 依存 問題 に 詳しい 医師 は 語り ます 。 そして 、 この よう に おっしゃい ます 。 最後 に これ を 読ま せ て ください 。   もし 日本 で カジノ が 合法 化 さ れ たら 、 もし **カジノ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) が 国会 で 可決 、 成立 し 、 カジノ が 合法 化 さ れれ ば 、 新た な ギャンブル 依存 症 が 大量 に 発生 する こと は 確実 で ある 。 カジノ 推進 派 から は 、 カジノ の 収益 から ギャンブル 依存 症 の 治療 施設 を 造り 、 その 治療 に 当たれ ば よい と いう 意見 が 聞か れる 。 依存 症 に なっ て しまっ た 後 、 ギャンブル から 離脱 する ため に 当事 者 と その 家族 など が どれ ほど の 努力 と 労力 と 年月 を 必要 と する か を 全く 理解 し て い ない と しか 考え られ ない 。   しかも 、 一旦 依存 症 に なれ ば 、 もう ギャンブル に 関する コントロール 、 取り戻す こと は でき ない 。 推進 派 は 、 ギャンブル 依存 症 が 本人 の 人生 ばかり か 家庭 を も 破壊 し て しまう ほど 深刻 な 病 で ある こと を 理解 し て い ない 。 そもそも 、 医療 で 現在 最も 重要 な 課題 の 一 つ と なっ て いる の が 病気 を 予防 する こと で ある 。 人々 が 病気 に なら ない ため の 施策 を 考え 実行 する の が 国 を 始め と する 行政 機関 の す べき こと で あり 、 我々 専門 家 の 仕事 で ある 。   この 法 案 に 賛成 する 皆 さん に お 聞き し たい 。 依存 症 を つくり出さ ない 最大 の 予防 策 と は 何 でしょう か 。 答え は もう 簡単 です 。 **カジノ** **法案** ( `特定複合観光施設区域の整備の推進に関する法律` ) の 廃案 と 、 現在 一番 依存 症 を 生み出し て いる パチンコ 、 スロット の 規制 です 。   反対 討論 を 終わり ます 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   他 に 御 意見 も ない よう です から 、 討論 は 終局 し た もの と 認め ます 。   それ で は 、 これ より **特定** **複合** **観光** **施設** **区域** **の** **整備** **の** **推進** **に** **関する** **法律** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に つい て 採決 に 入り ます 。   まず 、 上月 良祐 君 提出 の 修正 案 の 採決 を 行い ます 。   本 修正 案 に 賛成 の 方 の 挙手 を 願い ます 。       〔 賛成 者 挙手 〕 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   多数 と 認め ます 。 よっ て 、 上月 良祐 君 提出 の 修正 案 は 可決 さ れ まし た 。   次 に 、 ただいま 可決 さ れ まし た 修正 部分 を 除い た 原案 全部 の 採決 を 行い ます 。   修正 部分 を 除い た 原案 に 賛成 の 方 の 挙手 を 願い ます 。       〔 賛成 者 挙手 〕 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   多数 と 認め ます 。 よっ て 、 修正 部分 を 除い た 原案 は 可決 さ れ まし た 。   以上 の 結果 、 本案 は 多数 を もっ て 修正 議決 す べき もの と 決定 いたし まし た 。   この 際 、 上月 良祐 君 から 発言 を 求め られ て おり ます の で 、 これ を 許し ます 。 上月 良祐 君 。 <br/> ○ 上月 良祐 君   私 は 、 ただいま 可決 さ れ まし た **特定** **複合** **観光** **施設** **区域** **の** **整備** **の** **推進** **に** **関する** **法律** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に 対し 、 自由 民主 党 を 代表 し て 附帯 決議 案 を 提出 いたし ます 。   案文 を 朗読 いたし ます 。         **特定** **複合** **観光** **施設** **区域** **の** **整備** **の** **推進** **に** **関する** **法律** **案** ( `特定複合観光施設区域の整備の推進に関する法律` ) に 対する 附帯 決議 （ 案 ）     政府 は 、 本法 の 施行 に 当たっ て は 、 次 の 諸点 に 留意 し 、 その 運用 等 に つい て 遺憾 な き を 期す べき で ある 。   一   特定 複合 観光 施設 区域 の 整備 を 推進 する に 当たっ て は 、 特に 、 カジノ 施設 の 設置 及び 運営 に 伴う 有害 な 影響 を 排除 する 観点 、 我が 国 の 伝統 ・ 文化 ・ 芸術 を 活かし た 日本 らしい 国際 競争 力 の 高い 魅力 ある 観光 資源 を 整備 する 観点 、 並び に それ ら を 通じ た 観光 及び 地域 経済 の 振興 に 寄与 する 観点 に 特に 留意 する こと 。   二   政府 は 、 法 第 五 条 に 基づき 必要 と なる 法制 上 の 措置 を 講じる に 当たり 、 特定 複合 観光 施設 区域 の 整備 の 推進 に 係る 目的 の 公益 性 、 運営 主体 等 の 性格 、 収益 の 扱い 、 射幸 性 の 程度 、 運営 主体 の 廉潔 性 、 運営 主体 の 公的 管理 監督 、 運営 主体 の 財政 的 健全 性 、 副次 的 弊害 の 防止 等 の 観点 から 、 刑法 の 賭博 に 関する 法制 と の 整合 性 が 図ら れる よう 十分 な 検討 を 行う こと 。   三   特定 複合 観光 施設 に つい て は 、 国際 的 ・ 全国 的 な 視点 から 、 真 に 観光 及び 地域 経済 の 振興 の 効果 を 十分 に 発揮 できる 規模 の もの と し 、 その 際 、 特定 複合 観光 施設 全体 に 占める カジノ 施設 の 規模 に 上限 等 を 設ける と とも に 、 あくまで 一体 と し て の 特定 複合 観光 施設 区域 の 整備 が 主眼 で ある こと を 明確 に する こと 。   四   特定 複合 観光 施設 区域 の 数 に つい て は 、 我が 国 の 特定 複合 観光 施設 と し て の 国際 的 競争 力 の 観点 及び ギャンブル 等 依存 症 予防 等 の 観点 から 、 厳格 に 少数 に 限る こと と し 、 区域 認定 数 の 上限 を 法定 する こと 。   五   地方 公共 団体 が 特定 複合 観光 施設 区域 の 認定 申請 を 行う に 当たっ て は 、 公営 競技 の 法制 に 倣い 、 地方 議会 の 同意 を 要件 と する こと 。 また 、 地方 公共 団体 に よる 公聴 会 の 開催 など 、 地域 の 合意 形成 に 向け た 具体 的 な アクション や 依存 症 や 治安 維持 など の 地域 対策 を 、 国 の 認定 に 当たっ て は 十分 に 踏まえる こと 。   六   特定 複合 観光 施設 区域 の 整備 が 真 に 観光 及び 地域 経済 の 振興 に 寄与 する ため 、 また 、 特定 複合 観光 施設 の 設置 の 前提 と し て 犯罪 防止 ・ 治安 維持 、 青少年 の 健全 育成 、 依存 症 防止 等 の 観点 から 問題 を 生じ させ ない よう に する ため 、 特定 複合 観光 施設 区域 の 整備 の 推進 に おけ る 地方 公共 団体 の 役割 を 明確 化 する よう 検討 する こと 。   七   カジノ 施設 の 設置 及び 運営 を しよう と する 者 その 他 カジノ 施設 関係 者 に つい て は 、 真 に 適格 な 者 のみ が 選定 さ れる よう 厳格 な 要件 を 設ける と とも に 、 その 適合 性 に つい て 徹底 し た 調査 を 行う こと が できる よう 法制 上 の 措置 を 講ずる こと 。 また 、 カジノ 施設 を 含む 特定 複合 観光 施設 全体 の 健全 な 運営 等 を 確保 する ため 、 事業 主体 と し て の 一体 性 及び 事業 活動 の 廉潔 性 が 確保 さ れる よう 、 法制 上 の 措置 を 講ずる こと 。   八   依存 症 予防 等 の 観点 から 、 カジノ に は 厳格 な 入場 規制 を 導入 する こと 。 その 際 、 自己 排除 、 家族 排除 プログラム の 導入 、 入場 料 の 徴収 等 、 諸 外国 に おけ る カジノ 入場 規制 の 在り 方 や その 実効 性 等 を 十分 考慮 し 、 我が 国 に ふさわしい 、 清廉 な カジノ 運営 に 資する 法制 上 の 措置 を 講ずる こと 。   九   入場 規制 の 制度 設計 に 当たっ て は 、 個人 情報 の 保護 と の 調整 を 図り つつ 、 個人 番号 カード （ **行政** **手続** **に** **おけ** **る** **特定** **の** **個人** **を** **識別** **する** **ため** **の** **番号** **の** **利用** **等** **に** **関する** **法律** ( `行政手続における特定の個人を識別するための番号の利用等に関する法律` ) 第 二 条 第 七 項 に 定める 「 個人 番号 カード 」 を いう 。 ） の 活用 を 検討 する こと 。   十   ギャンブル 等 依存 症 患者 へ の 対策 を 抜本 的 に 強化 する こと 。 我が 国 に おけ る ギャンブル 等 依存 症 の 実態 把握 の ため の 体制 を 整備 し 、 その 原因 を 把握 ・ 分析 する と とも に 、 ギャンブル 等 依存 症 患者 の 相談 体制 や 臨床 医療 体制 を 強化 する こと 。 加え て 、 ギャンブル 等 依存 症 に 関する 教育 上 の 取組 を 整備 する こと 。 また 、 カジノ に とどまら ず 、 他 の ギャンブル ・ 遊技 等 に 起因 する 依存 症 を 含め 、 ギャンブル 等 依存 症 対策 に 関する 国 の 取組 を 抜本 的 に 強化 する ため 、 ギャンブル 等 依存 症 に 総合 的 に 対処 する ため の 仕組 ・ 体制 を 設ける と とも に 、 関係 省庁 が 十分 連携 し て 包括 的 な 取組 を 構築 し 、 強化 する こと 。 また 、 この ため に 十分 な 予算 を 確保 する こと 。   十 一   法 第 九 条 及び 第 十 条 に 定める 各種 規制 等 の 検討 に 当たっ て は 、 諸 外国 に おけ る カジノ 規制 の 現状 等 を 十分 踏まえる と とも に 、 犯罪 防止 ・ 治安 維持 、 青少年 の 健全 育成 、 依存 症 防止 等 の 観点 から 問題 を 生じ させ ない よう 、 世界 最高 水準 の 厳格 な カジノ 営業 規制 を 構築 する こと 。 なお 、 諸 外国 に おけ る いわゆる 「 ジャンケット 」 の 取扱 に つい て は きわめて 慎重 に 検討 を 行う こと 。   十 二   カジノ に おけ る マネー ・ ローンダリング の 防止 を 徹底 する 観点 から 、 第 七 項 の 事業 主体 の 廉潔 性 を 確保 する ため の 措置 、 第 八 項 及び 第 九 項 の カジノ へ の 厳格 な 入場 規制 を 導入 する ため の 措置 、 第 十 一 項 の 世界 最高 水準 の 厳格 な カジノ 営業 規制 を 構築 する ため の 措置 に 加え 、 マネー ・ ローンダリング 対策 に 関する 国際 基準 で ある ＦＡ ＴＦ 勧告 に 適切 に 対応 する ため 、 諸 外国 の 規制 の 現状 等 を 踏まえ つつ 、 カジノ の 顧客 の 取引 時 確認 、 確認 記録 の 作成 ・ 保存 、 疑わしい 取引 の 届出 等 に つい て 、 罰則 を 含む 必要 かつ 厳格 な 措置 を 講ずる こと 。 また 、 カジノ に おけ る マネー ・ ローンダリング の 防止 を 徹底 する 観点 から 、 厳格 な 税 の 執行 を 確保 する こと 。   十 三   カジノ 管理 委員 会 は 、 独立 し た 強い 権限 を 持つ いわゆる 三 条 委員 会 と し て 設置 し 、 カジノ 管理 委員 会 が カジノ 営業 規制 等 を 厳格 に 執行 できる 体制 の 構築 が 不可欠 で あり 、 特に 、 カジノ 導入 時 から 厳格 な 規制 を 執行 できる よう 、 十分 な 機構 ・ 定員 を 措置 する と とも に 、 適切 な 人材 を 配置 する ほか 、 厳格 な カジノ 営業 規制 等 や 関係 事業 者 に 対する 行政 処分 等 の 監督 を 有効 に 執行 できる 人材 育成 の 在り 方 も 検討 する こと 。 また 、 特定 複合 観光 施設 の 設置 の 前提 と し て 犯罪 防止 ・ 治安 維持 、 青少年 の 健全 育成 、 依存 症 防止 等 の 観点 から 問題 を 生じ させ ない よう に する ため 、 都 道 府 県 警察 その 他 の 関係 機関 の 必要 な 体制 を 確保 する と とも に 、 カジノ 管理 委員 会 と これ ら の 関係 機関 の 連携 体制 を 確保 する こと 。   十 四   カジノ の 運営 主体 が 民間 事業 者 に なる こと に 鑑み 、 カジノ 事業 者 に 適用 さ れる 税制 ・ 会計 規則 等 に つき 、 諸 外国 の 制度 を 十分 に 勘案 の 上 、 検討 を 行う こと 。   十 五   法 第 十 二 条 に 定める 納付 金 を 徴収 する こと と する 場合 は 、 その 使途 は 、 法 第 一 条 に 定める 特定 複合 観光 施設 区域 の 整備 の 推進 の 目的 と 整合 する もの と する と とも に 、 社会 福祉 、 文化 芸術 の 振興 等 の 公益 の ため に も 充てる こと を 検討 する こと 。 また 、 その 制度 設計 に 当たっ て は 、 依存 症 対策 の 実施 を はじめ 法 第 十 条 に 定める 必要 な 措置 の 実施 や 周辺 地方 公共 団体 等 に 十分 配慮 し た 検討 を 行う こと 。   十 六   以上 を 含め 、 法 第 五 条 に 定める 必要 と なる 法制 上 の 措置 の 検討 に 当たっ て は 、 十分 に 国民 的 な 議論 を 尽くす こと 。       右 決議 する 。   以上 で ござい ます 。   何とぞ 委員 各位 の 御 賛同 を お 願い 申し上げ ます 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   ただいま 上月 良祐 君 から 提出 さ れ まし た 附帯 決議 案 を 議題 と し 、 採決 を 行い ます 。   本 附帯 決議 案 に 賛成 の 方 の 挙手 を 願い ます 。       〔 賛成 者 挙手 〕 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   多数 と 認め ます 。 よっ て 、 上月 良祐 君 提出 の 附帯 決議 案 は 多数 を もっ て 本 委員 会 の 決議 と する こと に 決定 いたし まし た 。   ただいま の 決議 に 対し 、 菅 内閣 官房 長官 から 発言 を 求め られ て おり ます の で 、 この 際 、 これ を 許し ます 。 菅 内閣 官房 長官 。 <br/> ○ 国務 大臣 （ 菅 義偉 君 ）   ただいま 御 決議 の あり まし た 事項 に つき まし て は 、 その 御 趣旨 を 十分 に 尊重 し て まいり たい と 思い ます 。 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   なお 、 審査 報告 書 の 作成 に つき まし て は 、 これ を 委員 長 に 御 一任 願い たい と 存じ ます が 、 御 異議 ござい ませ ん か 。       〔 「 異議 なし 」 と 呼ぶ 者 あり 〕 <br/> ○ 委員 長 （ 難波 奨 二 君 ）   御 異議 ない と 認め 、 さよう 決定 いたし ます 。   本日 は これ にて 散会 いたし ます 。       午後 八 時 一 分 散会 nan 平成 二十 八 年 一 月 二十 日 （ 水曜 日 ）       午前 十 時 一 分 開議   　　　━━━━━━━━━━━━━ <br/> ○ 議事 日程   第 四 号     平成 二十 八 年 一 月 二十 日       午前 十 時 開議   第 一   平成 二十 七 年度 一般 会計 補正 予算 （ 第 1 号 ）   第 二   平成 二十 七 年度 特別 会計 補正 予算 （ 特 第 1 号 ）   第 三   **一般** **職** **の** **職員** **の** **給与** **に** **関する** **法律** **等** **の** **一** **部** **を** **改正** **する** **法律** **案** ( `NIL` ) （ 内閣 提出 、 衆議 院 送付 ）   第 四   **特別** **職** **の** **職員** **の** **給与** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) （ 内閣 提出 、 衆議 院 送付 ）   第 五   **裁判** **官** **の** **報酬** **等** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) （ 内閣 提出 、 衆議 院 送付 ）   第 六   **検察** **官** **の** **俸給** **等** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) （ 内閣 提出 、 衆議 院 送付 ）   第 七   **防衛** **省** **の** **職員** **の** **給与** **等** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) （ 内閣 提出 、 衆議 院 送付 ）   第 八   **地方** **交付** **税** **法** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) （ 内閣 提出 、 衆議 院 送付 ）   第 九   国務 大臣 の 報告 に 関する 件 （ 平成 二十 六 年度 決算 の 概要 に つい て ）   　　　━━━━━━━━━━━━━ <br/> ○ 本日 の 会議 に 付し た 案件   一 、 日程 第 一 より 第 八 まで   一 、 **国会** **議員** **の** **秘書** **の** **給与** **等** **に** **関する** **法律** **の** **一** **部** **を** **改正** **する** **法律** **案** ( `NIL` ) （ 衆議 院 提出 ）   一 、 日程 第 九           ─ ─ ─ ─ ─ ・ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   これ より 会議 を 開き ます 。   日程 第 一   平成 二十 七 年度 一般 会計 補正 予算 （ 第 1 号 ）   日程 第 二   平成 二十 七 年度 特別 会計 補正 予算 （ 特 第 1 号 ）   以上 両案 を 一括 し て 議題 と いたし ます 。   まず 、 委員 長 の 報告 を 求め ます 。 予算 委員 長 岸 宏一 君 。         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 審査 報告 書 は 本号 末尾 に 掲載 〕         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 岸 宏一 君 登壇 、 拍手 〕 <br/> ○ 岸 宏一 君   ただいま 議題 と なり まし た 平成 二十 七 年度 補正 予算 二 案 の 審査 の 経過 と 結果 を 御 報告 申し上げ ます 。   補正 予算 二 案 は 、 去る 一 月 四 日 に 国会 に 提出 さ れ 、 衆議 院 から 送付 の 後 、 一 月 十 五 日 に 財務 大臣 から 趣旨 説明 を 聴取 し 、 同日 から 昨日 まで 、 安倍 内閣 総理 大臣 及び 関係 大臣 に 対し 、 質疑 を 行っ て まいり まし た 。   質疑 は 、 臨時 会 召集 要求 へ の 政府 対応 、 一 億 総 活躍 社会 実現 へ の 取組 、 補正 予算 編成 の 要件 、 軽減 税率 導入 の 課題 と 問題 点 、 年金 生活 者 福祉 給付 金 の 妥当 性 、 Ｔ ＰＰ 協定 へ の 対応 、 名目 ＧＤＰ 六百 兆 円 の 実現 可能 性 、 伊勢 志摩 サミット に 向け た 取組 、 北朝鮮 の 核 問題 、 日韓 関係 の 改善 に 向け た 取組 、 若年 層 の 雇用 促進 策 、 子供 の 貧困 対策 、 給付 型 奨学 金 制度 の 必要 性 など 、 多岐 に わたり まし た が 、 その 詳細 は 会議 録 に よっ て 御 承知 願い たい と 存じ ます 。   質疑 を 終局 し 、 討論 、 採決 の 結果 、 平成 二十 七 年度 補正 予算 二 案 は 賛成 多数 を もっ て いずれ も 原案 どおり 可決 す べき もの と 決定 いたし まし た 。   以上 、 御 報告 を 申し上げ ます 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   両案 に 対し 、 討論 の 通告 が ござい ます 。 順次 発言 を 許し ます 。 西村 まさみ 君 。       〔 西村 まさみ 君 登壇 、 拍手 〕 <br/> ○ 西村 まさみ 君   民主 党 ・ 新緑 風 会 の 西村 まさみ で ござい ます 。   私 は 、 ただいま 議題 と なり まし た 平成 二十 七 年度 補正 予算 二 案 に 対し 、 会派 を 代表 し て 、 反対 の 立場 から 討論 を 行い ます 。   安倍 総理 は 、 年頭 、 六十 年 前 の 経済 白書 の 表現 を 用い 、 もはや デフレ で は ない と 宣言 さ れ まし た 。 しかし 、 個人 消費 や 実質 賃金 の 回復 が 見 られ ない 現在 、 果たして 本当 に デフレ から 脱却 し た の か 、 国民 の 生活 は 本当 に 良く なっ た の か 甚だ 疑問 で あり ます 。   平成 二十 六 年度 の 実質 ＧＤＰ は 前 年度 比 一 ・ 〇 ％ 減 と 五 年 ぶり の マイナス 成長 に 転落 し 、 二十 七 年 四 — 六 月 期 も 〇 ・ 五 ％ 減 と マイナス 成長 が 続き まし た 。 また 、 第 二 次 安倍 政権 の 三 年間 で 実質 賃金 は 減少 し 続け 、 最新 公表 分 の 平成 二十 七 年 十 一 月 に おい て も 前年 比 マイナス と いう 状況 に あり ます 。   私 は 歯科 の 開業 医 で あり ます 。 診療 所 や 病院 の ほとんど が 公的 価格 で ある 保険 診療 で なりわい を 立て て い ます 。 二十 四 年 、 二十 二 年 と 、 診療 報酬 改定 で 二 回 連続 で 民主 党 政権 時 は 全体 の プラス 改定 を 実現 し まし た 。 ところ が 、 平成 二十 八 年 診療 報酬 改定 で は 、 全体 で マイナス 〇 ・ 八 四 ％ 、 消費 増税 分 の 補填 が あっ た 平成 二十 六 年 に 続い て 実質 は 二 回 連続 の マイナス 改定 で あり 、 昨年 十 一 月 に 公表 さ れ た 医療 経済 実態 調査 に おい て も 、 歯科 の 診療 所 を 始め 医療 機関 の 経営 状態 は 改善 し て おら ず 、 厳しい 経営 状況 の 中 で 、 日々 、 国民 に 必要 な 医療 を 提供 する ため に 大変 な 思い で 診療 さ れ て いる 全国 の 医療 関係 者 の 悲痛 の 声 を 多く 聞き ます 。   国民 に とっ て この よう に 厳しい 経済 情勢 が 続く 状況 に おい て は 、 本質 的 な 景気 回復 に 向け た 補正 予算 を 編成 す べき で あり 、 それ に も かかわら ず 、 盛り込ま れ た 施策 の 多く は 平成 二十 八 年度 当初 予算 の 概算 要求 ・ 要望 に 含ま れ て い た もの の 前倒し で あり ます 。 当初 予算 を 見かけ 上 圧縮 する 一方 で 、 時間 的 制約 から 査定 が 甘く なり がち な 補正 予算 で の 安易 な 予算 計上 を 助長 する もの で あり 、 財政 規律 を 大きく 損なう 手法 と の 批判 は 免れ ませ ん 。   国民 生活 の 厳しい 現状 を 直視 せ ず 、 有効 で 具体 的 な 経済 政策 を 打ち出せ ない 安倍 内閣 に 猛省 を 促し 、 以下 、 本 補正 予算 案 の 反対 する 主な 理由 を 申し述べ ます 。   反対 の 第 一 の 理由 は 、 平成 二十 七 年度 補正 予算 が 緊急 性 の 低い 予算 で ある と いう 点 に あり ます 。   そもそも 補正 予算 と は 、 **財政** **法** ( `財政法` ) 第 二十 九 条 に より 、 予算 作成 後 に 生じ た 事由 に 基づき 特に 緊要 と なっ た 経費 の 支出 又 は 債務 の 負担 を 行う ため に 編成 できる もの と さ れ て い ます 。 一 億 総 活躍 社会 の 実現 に 向け た 対策 や Ｔ ＰＰ 関連 の 施策 が 盛り込ま れ て おり ます が 、 これ ら は 果たして 補正 予算 で 対応 する べき もの な の でしょう か 。   そして 、 待機 児童 の 解消 や 介護 施設 の 整備 と いっ た 社会 保障 に 関わる 重要 な 施策 は 、 制度 全体 と 財政 規模 を 俯瞰 し て 、 その 優先 順位 を 吟味 し た 上 で 恒久 的 な 施策 と し て 当初 予算 に 計上 する べき もの と 考え ます 。   また 、 保育 、 介護 の 現場 で は 人手 不足 が 大きな 問題 と なっ て い ます 。 依然 と し て 保育 、 介護 に 従事 する 人 たち の 処遇 改善 は 不 十分 です 。 そこ で 働く 人 が い なけれ ば 全く 意味 は あり ませ ん 。 人 へ の 投資 が 何 より 重要 で あり 、 一人一人 が 働き たい 分野 で 安心 し て 活躍 できる 社会 こそ 実現 す べき 社会 な の で は ない でしょう か 。   ＴＰＰ 協定 に つい て も 、 参加 国 が 大筋 合意 し た 段階 に すぎ ず 、 発効 の 時期 は まだ 決まっ て おり ませ ん 。 政府 は 、 平成 二十 七 年 秋 に 総合 的 な Ｔ ＰＰ 関連 政策 大綱 を 決定 し た もの の 、 より 具体 的 な 内容 は 二十 八 年 秋 を めど に 詰める と し て おり 、 本 補正 予算 に 関連 対策 を 計上 する 必要 性 を 見出す こと は 到底 でき ませ ん 。 さらに 、 過去 の ウルグアイ ・ ラウンド 関連 対策 に おい て 、 毎 年度 の 補正 予算 で 措置 する 手法 が 国会 で も 度々 批判 さ れ て き た に も かかわら ず 、 今回 も その 轍 を 踏もう と し て いる こと を 見過ごす こと も でき ませ ん 。 そもそも 、 秋 の 臨時 国会 を 開か ず 、 ＴＰＰ の 議論 を 避け て おい て 、 今 に なっ て 予算 だけ 通せ と は 、 誠 に 勝手 な 話 で は あり ませ ん か 。   反対 の 第 二 の 理由 は 、 過去 の 経済 政策 の 失敗 を 直視 せ ず 、 ばらまき を 繰り返し て いる 点 に あり ます 。   安倍 総理 は 、 昨年 九 月 に 突如 、 新 三 本 の 矢 を 掲げ まし た が 、 元々 の 三 本 の 矢 で ある 金融 緩和 、 財政 政策 、 成長 戦略 の 成果 を しっかり と 見極め た の でしょう か 。 物価 上昇 率 は 年率 二 ％ に 達せ ず 、 成長 戦略 も 芳しい 効果 が 見 られ ない 現況 を 直視 す べき で ある と 考え ます 。 新た な スローガン を 次々 と 打ち出し て 拙速 に 予算 を 付ける 前 に 、 これ まで の 三 本 の 矢 が 失敗 し た こと を 率直 に 認める こと が 必要 です 。   また 、 一 億 総 活躍 予算 の うち 三 割 以上 を 占める こと と なる 三 万 円 の 低 所得 年金 受給 者 向け の 臨時 福祉 給付 金 の 目的 を 、 政府 は 、 賃上げ の 恩恵 が 及ば ない 年金 受給 者 へ の 給付 に よっ て 消費 を 喚起 する こと と し て い ます 。 しかし 、 臨時 給付 金 の 消費 へ の 効果 は 極めて 疑わしい 。   かつて 麻生 内閣 が 平成 二十 一 年 に 定額 給付 金 を 支給 し た 際 の 消費 拡大 効果 は 、 受給 額 の 僅か 二 五 ％ に しか すぎ ませ ん でし た 。 政府 が 過去 の 失敗 を 反省 し て いれ ば 、 同様 の 政策 を 繰り返そう と する はず が あり ませ ん 。 仮 に この 補正 予算 が 成立 し て も 、 この 給付 金 の 執行 の ほとんど は 来 年度 に 繰り越さ れ 、 年金 生活 者 の 方々 が 実際 に 受け取る の は 五 月 、 六 月 と なり ます 。 つまり 、 これ は 合理 的 な 政策 と は 程遠く 、 ひとえに 夏 の 参議 院 選挙 対策 で ある こと は 誰 の 目 に も 明らか で あり ます 。   安倍 内閣 は 、 昨年 六 月 に 経済 ・ 財政 再生 計画 を 策定 し 、 年金 、 医療 等 の いわゆる 自然 増 を 毎年 〇 ・ 五 兆 円 程度 に 抑制 する と の 目安 を 設定 いたし まし た 。 これ を 踏まえ 、 平成 二十 八 年度 予算 に おけ る 社会 保障 関係 費 の 伸び は 、 概算 要求 時点 から 約 二千 億 円 が 削減 さ れ 、 目安 の 範囲 まで 抑制 さ れ た の で あり ます 。 安倍 総理 は 削減 額 が あり き で は ない と 説明 を し て い ます が 、 これ は 、 小泉 内閣 に おい て 国民 から 厳しい 批判 を 受け た 年間 二千 二百 億 円 の 社会 保障 の 機械 的 な 切捨て と 実質 に おい て 何 ら 変わり は あり ませ ん 。   かつて 社会 保障 費 の 一律 カット が 、 産科 、 小児 科 、 救急 医療 の 医師 を 不足 さ せ たり 、 救急 搬送 の たらい回し を 生じ させ 、 地域 医療 の 崩壊 に つながり 、 国民 が 必要 な 医療 を 受ける こと が でき なく なっ た こと を 、 昨年 の 五 月 の 本 会議 質疑 に おい て 総理 に も 財務 大臣 に も 厚生 労働 大臣 に も 私 は 申し上げ まし た が 、 その 認識 が いまだ に ない の でしょう か 。   当初 予算 で 社会 保障 関係 費 の 切り詰め を する 一方 、 補正 予算 で 選挙 対策 の ばらまき を しよう と する 財政 政策 は 、 断じ て 容認 でき ませ ん 。   第 三 の 理由 は 、 税収 の 上振れ 分 や 前 年度 の 剰余 金 を 借金 の 圧縮 で は なく 歳出 拡大 に 回し 、 財政 健全 化 から 目 を 背け て いる 点 で あり ます 。   この 補正 予算 に おい て は 、 約 四 兆 円 に 上る 税収 増 や 前 年度 剰余 金 の 多く が 新た な 歳出 に 充て られ 、 公債 発行 の 減額 は 僅か 四千 億 円 余り に とどまっ て い ます 。 日本 の 国 と 地方 の 債務 残高 は 名目 ＧＤＰ の 二 倍 を 超え て おり 、 財政 健全 化 を 進める と いう の で あれ ば 、 増収 分 は 歳出 増 で は なく 新た な 借金 の 抑制 や 返済 に 充てる の が 筋 で あり ます 。   政府 は 、 国 と 地方 の 基礎 的 財政 収支 対 ＧＤＰ 比 を 平成 三十 二 年度 に 黒字 化 する 目標 を 立てる 一方 、 内閣 府 の 中長期 試算 に おい て 、 その 目標 が 達成 でき ない 見込み で ある と いう こと を 示し て い ます 。 歳入 増 の 多く を 歳出 増 に 充てる 放漫 な 財政 運営 を 続け て いけ ば 、 財政 再建 目標 の 達成 が 更に 困難 に なる こと は 明白 で あり ます 。   以上 、 補正 予算 に 反対 する 主な 理由 を 述べ させ て いただき まし た 。   政策 の 緊要 性 や 実効 性 を 吟味 せ ず 、 安易 な 歳出 拡大 に 終始 し て いる 本 補正 予算 は 、 私 は 断じ て 容認 でき ませ ん 。 一 億 総 活躍 社会 の 実現 の 看板 の 下 に 十分 な 検討 の ない まま 思い付き の よう に 並べ られ た 政策 、 見切り 発車 の Ｔ Ｐ Ｐ 国内 農業 対策 、 若年 者 、 子育て 世帯 を ないがしろ に し た 選挙 対策 の ばらまき 、 果たして 持続 可能 な 経済 成長 が 可能 な の でしょう か 。   私 たち は 、 未来 へ の 責任 を 果たす べく 政治 に 向け て 全力 で 取り組む と いう 決意 を 申し上げ 、 私 から の 反対 討論 を 終わら せ て いただき ます 。   ありがとう ござい まし た 。 （ 拍手 ） <br/> ○ 議長 （ 山崎 正昭 君 ）   宇都 隆史 君 。       〔 宇都 隆史 君 登壇 、 拍手 〕 <br/> ○ 宇都 隆史 君   自由 民主 党 の 宇都 隆史 です 。   自由 民主 党 、 公明 党 を 代表 いたし まし て 、 平成 二十 七 年度 補正 予算 二 案 に 対し まし て 、 賛成 の 立場 から 討論 いたし ます 。   第 二 次 安倍 政権 が 誕生 し て から 三 年 が 経過 いたし まし た 。 安倍 政権 が デフレ 脱却 ・ 経済 再生 を 目標 に 掲げ 、 アベノミクス を 推し進め た 結果 、 企業 の 収益 は 過去 最高 の 水準 、 有効 求人 倍率 は 二十 三 年 ぶり の 高 水準 と なる 等 、 多く の 経済 指標 で 確か な 改善 の 傾向 を 見る こと が でき ます 。 アベノミクス の 成果 に より 、 我が 国 は 、 もはや デフレ で は ない と いう 状況 に まで 経済 を 取り戻す こと が でき まし た 。   しかし ながら 、 現状 に おい て は 、 地方 や 中 小 企業 を 中心 に 景気 回復 の 実感 は まだ かすか で 、 個人 消費 や 企業 の 設備 投資 の 伸び悩み など 、 完全 に デフレ から 脱却 し た と 言い 切れ ない 状況 で ある の も 事実 です 。 経済 再生 の 妨げ に は 、 少子 高齢 化 の 進行 に よる 人口 減少 と いう 我が 国 が 抱える 構造 的 な 課題 が あり ます 。 この まま 人口 減少 が 続き 、 国内 の 労働 力 や 国内 需要 が 縮小 し て いく の を 放置 すれ ば 、 日本 経済 及び 日本 の 国家 財政 は 成り立た ず 、 それ が 社会 保障 も 含め た 各種 政策 の 将来 に 対する 不安 へ と つながっ て い ます 。 アベノミクス に よる 成長 の 果実 が 得 られ つつ ある 今 こそ 、 更なる 追加 政策 を 行い 、 確実 な デフレ 脱却 を 実現 さ せ なけれ ば なり ませ ん 。   そこ で 、 昨年 、 安倍 総理 は 一 億 総 活躍 社会 を 打ち出し 、 その 実現 の ため に アベノミクス の 第 二 ステージ と なる 新 三 本 の 矢 を 放ち まし た 。 第 一 の 矢 で ある 強い 経済 で は 、 名目 ＧＤＰ 六百 兆 円 達成 を 目標 に 掲げ 、 その 実現 に 向け た 取組 から 得 られる 果実 を 第 二 、 第 三 の 矢 の 目標 で ある 希望 出生 率 一 ・ 八 、 介護 離職 ゼロ 実現 の ため に 分配 し 、 子育て や 介護 の 基盤 を 強化 し ます 。 子育て や 介護 に 対する 不安 が 解消 さ れ 、 将来 に 対する 見通し が 明るく なれ ば 、 消費 や 投資 が 促さ れ 、 経済 の 好 循環 が 一層 強化 さ れ ます 。 この よう に 成長 と 分配 の 好 循環 を 強化 し 、 持続 さ せる こと に よっ て 確実 な デフレ 脱却 を 実現 する こと が できる の です 。   また 、 昨年 十 月 に Ｔ Ｐ Ｐ が 大筋 合意 さ れ まし た 。 政府 が 、 Ｔ Ｐ Ｐ を 我が 国 の 経済 再生 を 進める 上 で の 有効 な カード と す べく 、 国会 決議 を 重く 受け止め 、 国民 生活 と 国益 を 守り 抜く ため の 粘り強い 交渉 を 乗り切っ た こと を 高く 評価 いたし ます 。 Ｔ Ｐ Ｐ が 最終 締結 さ れれ ば 、 我が 国 の 八 倍 の 人口 、 六 倍 の ＧＤＰ 規模 を 有する 世界 最大 の 市場 を 手 に する こと に より 、 国内 の 経済 、 産業 を 活性 化 さ せ 、 ＧＤＰ を 押し上げる 効果 が 期待 でき ます 。 アベノミクス の 成果 が 現れ 始め た 今 こそ 、 Ｔ Ｐ Ｐ を 最大 限 に 活用 し た 経済 の 押し上げ を 早期 に 現実 の もの と し 、 アベノミクス に よる 経済 再生 の 実体 を 国民 の 実感 に つなげ て いく よう に し て いか なけれ ば なり ませ ん 。 実体 を 実感 に つなげる の です 。   一方 、 Ｔ Ｐ Ｐ は 日本 経済 が 飛躍 する ため の 大きな チャンス で あり ます が 、 他国 に とっ て も それ は 同様 で 、 厳しい 競争 に さらさ れる 業種 、 特に 農林漁業 者 の 方々 に は 大きな 不安 が ある の も 事実 です 。 しかし 、 販路 や 輸出 を 拡大 し 、 各種 業 界 が 主導 する 体質 強化 、 攻め の 農林 水産 業 へ と 転換 を 図る こと は 、 日本 の 山野 、 里山 、 海 を 守る 上 で も 非常 に 重要 な こと です 。 攻める べき ところ は 攻め 、 守る べき ところ は 守る 、 農林漁業 者 や これ から の 担い手 が 将来 に 夢 を 持っ て 経営 に 取り組める よう な 措置 を 講じ て いか なけれ ば なり ませ ん 。   本 補正 予算 案 は 、 一 億 総 活躍 社会 を 推し進め 、 Ｔ Ｐ Ｐ を 活用 し 、 日本 が 大きく 飛躍 する ため の 第 一 歩 で あり ます 。 この 第 一 歩 を 力強く 踏み出す ため 、 一 日 も 早い 成立 が 求め られ ます 。   以下 、 本 補正 予算 に 賛成 する 主な 理由 を 三 点 申し述べ て まいり ます 。   第 一 に 、 一 億 総 活躍 社会 の 実現 に 向け て 、 緊急 に 実施 す べき 対策 に 的確 に 対応 し て いる 点 で あり ます 。   国民 一人一人 が 活躍 できる 社会 づくり を 進める 上 で 、 結婚 、 出産 へ の 希望 が 実現 し にくい 環境 や 介護 と 仕事 が 両立 し にくい 環境 を 改善 する こと は 喫緊 の 課題 です 。 本 補正 予算 案 で は 、 地域 に おけ る 結婚 に 向け た 活動 の 支援 や 保育 所 等 の 整備 、 保育 士 の 確保 など 、 結婚 、 子育て の 対象 と なる 若者 世代 の 支援 の ため の 経費 を 計上 する と とも に 、 介護 基盤 整備 の 加速 化 、 介護 人材 の 育成 確保 を する など 、 介護 と 仕事 を 両立 する ため の 、 お 年寄り や 介護 従事 者 の 支援 の ため の 経費 が 計上 さ れ て おり 、 希望 出生 率 一 ・ 八 と 介護 離職 ゼロ の 実現 に 直結 する 対策 が なさ れ て おり ます 。   また 、 本 補正 予算 案 に は 、 アベノミクス の 成果 の 再 配分 の 観点 から 、 景気 回復 に よる 賃金 引上げ 等 の 恩恵 が 及び にくい 低 所得 高齢 者 世帯 に 支援 を 行う ため に 、 年金 生活 者 等 支援 臨時 福祉 給付 金 が 計上 さ れ て おり ます 。 昨年 の 春闘 で の 賃金 引上げ 率 は 一 九 九 八 年 以来 十 七 年 ぶり の 高い 水準 で あり まし た 。 安倍 政権 発足 後 の 最低 賃金 の 引上げ 率 も 春闘 の 賃上げ 率 と とも に 上昇 傾向 に あり ます 。   その 他 、 中 小 企業 ・ 小 規模 事業 者 へ の 投資 促進 、 生産 性 を 高める ため の 経費 、 地方 創生 の 本格 展開 の ため の 経費 が 計上 さ れ て おり 、 我が 国 の 経済 の 根幹 を 支える 中 小 企業 の 強化 は 地方 創生 に 直接 寄与 する もの で あり 、 大いに 評価 いたし ます 。   賛成 する 第 二 の 理由 は 、 Ｔ ＰＰ 関連 政策 大綱 実現 に 向け た 施策 、 特に 攻め の 農林 水産 業 へ の 体質 強化 対策 に 予算 が 計上 さ れ て いる 点 で あり ます 。   攻め の 農林 水産 業 へ の 体質 転換 を 図る ため に は 、 その 足腰 を 強化 する 必要 が あり ます 。 農林 水産 業 に は 高齢 化 や 担い手 不足 等 様々 な 問題 が あり 、 一刻 も 早く 対策 を 打た なけれ ば なり ませ ん 。 政府 は 、 農林 水産 ・ 食品 の 輸出 目標 額 一 兆 円 を 前倒し し て 実現 する こと を 目標 に 掲げ 、 我が 国 農林 水産 物 の 一層 の 輸出 拡大 、 六 次 産業 化 、 地産 地消 に よる 地域 の 収益 力 強化 等 に より 、 攻め の 農林 水産 業 を 推進 し て おり ます 。   本 補正 予算 案 で は 、 今後 の 農業 界 を 牽引 し て いく 優れ た 経営 感覚 を 持っ た 担い手 を 支援 、 育成 し 、 人材 力 の 強化 を 進め 、 力強く 持続 可能 な 農業 構造 の 実現 に 必要 な 経費 を 計上 し て おり ます 。 また 、 畜産 、 酪農 の 収益 力 強化 の ため の 経費 、 林業 、 漁業 の 体質 強化 の ため の 経費 等 が 計上 さ れ て おり ます 。 魅力 ある 農林 水産 業 を つくり出し て いく こと は 、 農林 水産 業 の 抱える 諸 課題 の 解決 に も つながっ て まいり ます 。   賛成 する 第 三 の 理由 は 、 災害 復旧 、 防災 ・ 減災 、 東 日本 大 震災 から の 復興 加速 化 の ため の 的確 な 経費 が 計上 さ れ て いる 点 で あり ます 。   昨年 九 月 の 関東 ・ 東北 豪雨 は 各地 に 大きな 被害 を もたらし まし た 。 特に 、 鬼怒 川 の 氾濫 に よっ て 茨城 県 常総 市 で は 大きな 被害 を 受け て おり 、 一 日 も 早い 復旧 復興 が 望ま れ ます 。 また 、 東 日本 大 震災 から 間 も なく 五 年 が 経過 いたし ます 。 本年 三 月 で 集中 復興 期間 も 終了 いたし ます 。 いまだ に 多く の 避難 を 続け られ て おり 、 復興 を 一層 加速 さ せ なけれ ば なり ませ ん 。 国民 の 安心 、 安全 は 政府 が 取り組む べき 最 優先 事項 で あり 、 速やか な 成立 が 望ま れ ます 。   以上 、 賛成 する 主な 理由 を 三 点 述べ て まいり まし た が 、 付け加え て 、 本 補正 予算 案 の 財源 は 税収 増 、 前 年度 剰余 金 で 確保 さ れ て おり 、 平成 二十 七 年度 の プライマリー バランス 赤字 半減 目標 は 堅持 さ れ て いる こと も 評価 いたし ます 。   最後 に 、 本年 平成 二十 八 年 の えと は さる 年 です 。 「 申 」 と いう 漢字 は 、 果実 が 成熟 し て いく 状態 を 表し て いる と いい ます 。 その 果実 を 一 億 総 活躍 社会 へ と つなげる ため 、 今回 の 予算 は 時宜 を 得 た 内容 と なっ て おり ます 。   多く の 皆 様 から の 御 賛同 を 賜り ます よう お 願い を 申し上げ 、 私 の 賛成 討論 と いたし ます 。   御 清聴 ありがとう ござい まし た 。 （ 拍手 ） <br/> ○ 議長 （ 山崎 正昭 君 ）   辰巳 孝太郎 君 。       〔 辰巳 孝太郎 君 登壇 、 拍手 〕 <br/> ○ 辰巳 孝太郎 君   日本 共産 党 の 辰巳 孝太郎 で ござい ます 。   会派 を 代表 し て 、 政府 提出 の 二 〇一五年度補正予算二案 に 反対 の 立場 で 討論 を 行い ます 。   まず 冒頭 、 十 五 日 未明 に 起き た 長野 県 で の バス 事故 に おい て 十 五 名 の 尊い 命 が 失わ れ た こと に 心 から の お 悔やみ を 申し上げ ます と とも に 、 負傷 さ れ た 方々 の 一 日 も 早い 回復 を お 祈り いたし ます 。   運行 管理 や 安全 対策 に 関する 問題 、 バス 会社 が 法定 運賃 未満 で 受注 し て い た こと など が 明らか に なっ て い ます 。 更なる 事故 原因 の 真相 究明 と とも に 、 大本 に ある バス 事業 参入 の 規制 緩和 と バス 運転 手 の 長 時間 労働 を 抜本 的 に 見直す こと が 必要 です 。 この よう な 事故 が 二 度 と 起こら ない よう 、 国会 が その 責任 を 果たす こと を 強く 求める もの で あり ます 。   日本 共産 党 は 、 昨年 、 **戦争** **法** ( `平和安全法制` ) 、 ＴＰＰ 、 テロ 対策 、 沖縄 新 基地 建設 、 原発 再 稼働 問題 など 、 国政 、 外交 の 山積 する 問題 を ただす ため 、 野党 議員 と 共同 で 憲法 五十 三 条 の 規定 に 基づき 臨時 国会 開会 を 求め まし た 。 ところ が 、 安倍 政権 は 明確 な 理由 なく これ を 拒否 し まし た 。 集団 的 自衛 権 を 認める 違憲 立法 の ため に は 国会 を 延長 し 、 都合 が 悪く なれ ば 臨時 国会 召集 を し ない 。 まさに 憲法 無視 の 体質 を むき出し に し た の が 安倍 政権 です 。   我が 党 議員 が 委員 会 質疑 で 指摘 し た オスプレイ の 佐賀 空港 配備 や 佐世保 に おけ る 水陸 機動 団 の 陸自 、 海自 一体 の 新た な 基地 計画 など 、 **戦争** **法** ( `平和安全法制` ) 具体 化 の 一切 を 安倍 政権 は やめる べき で あり ます 。   日本 共産 党 は 、 米国 の 無法 な 戦争 に 加担 する ため の **戦争** **法** ( `平和安全法制` ) を 廃止 する ため 、 党派 を 超え た 共闘 を 進め 、 学生 や ママ たち 、 学者 、 文化 人 、 宗教 者 など 、 広範 な 市民 の 皆 さん と 結び付い て 最後 まで 闘い 抜い て まいり ます 。   この 間 の 予算 委員 会 の 審議 を 通じ て 、 安倍 政権 の 国民 生活 軽視 の 政治 姿勢 は より 一層 明らか に なり まし た 。 総理 は 、 アベノミクス で 経済 の 好 循環 が 生まれ 始め た 、 第 二 ステージ に 進む と し 、 一 億 総 活躍 社会 なる もの を 実現 し 、 今回 の 補正 予算 は その 皮切り と なる と 表明 し まし た が 、 暮らし や 経済 の 実態 は 好 循環 と は 程遠い の が 実態 です 。   国民 生活 基礎 調査 で は 、 生活 が 苦しい と 答え た 人 は 六 二 ・ 四 ％ で 、 年々 悪化 し て い ます 。 現在 の 税率 から 一 ％ も 下げ ない の に 、 軽減 など と 国民 を ごまかし ながら 消費 税 増税 を 強行 すれ ば 、 家計 に 深刻 な 打撃 を 与え 、 内需 を 更に 冷え込ま せ 、 日本 経済 を 悪化 さ せる の は 明らか です 。 政府 も 、 消費 税率 を 上げれ ば 上げる ほど 逆進 性 が 強まる こと を 認め まし た 。 逆進 性 の 緩和 を 言う の なら 、 一番 の 有効 策 は 消費 税 増税 を 中止 する こと で あり ます 。   しかも 、 審議 を 通じ て 、 これ まで 一 世帯 当たり 三 万 五千 円 と 説明 し て い た 増税 時 の 負担 増 は 、 実際 は 倍 近く の 六 万 二千 円 に なる こと を 政府 は 認め た の です 。 国民 を 欺い て 増税 する など 、 これ ほど の 背信 は あり ませ ん 。 消費 税 増税 は 中止 しか あり ませ ん 。   一方 、 大 企業 に は 相 変わら ず の 大盤 振る舞い です 。   安倍 政権 の 下 で 実施 し て き た 企業 減税 は 、 復興 特別 法人 税 の 一 年 前倒し 廃止 や 法人 税率 引下げ など で 年間 三 兆 円 に も 上り 、 来 年度 以降 は これ に 一 兆 円 が 加わり ます 。 しかし 、 減税 分 は 賃金 に は 回ら ず 、 その 大 部分 が 海外 投資 家 へ の 配当 へ と 流れ 、 大 企業 の 内部 留保 は とうとう 三百 兆 円 を 超え まし た 。 トリクルダウン の 破綻 は 明らか で あり ます 。   また 、 経団連 が 政治 献金 の 呼びかけ を 再開 し 、 二 〇 一 四 年 、 自民 党 が 受け取っ た 企業 献金 は 二十 二 億 円 を 超え まし た 。 自動 車 産業 など 法人 減税 の 一番 の 恩恵 者 から の 多額 の 政治 献金 は 、 まさに 税金 の 還流 で あり 、 経済 の 好 循環 は 自民 党 に もたらさ れ た と いう こと で は あり ませ ん か 。 政治 を 金 で ゆがめる 企業 献金 を なくす と の 口実 で 導入 さ れ た 政党 助成 金 と の 二 重 取り も 許せ ませ ん 。   そして 、 極め付け は 、 低 所得 者 の 高齢 者 一千 百 万 人 に 対する 三 万 円 の 臨時 福祉 給付 金 です 。 安倍 政権 は 六 月 まで に 配り 終える と し て おり ます 。 税金 を 使っ た 露骨 な 選挙 対策 で は あり ませ ん か 。 本 会議 で 安倍 首相 は 、 一 回 だけ な の で ばらまき で は ない と 答弁 し まし た けれど も 、 一 回 だけ だ から ばらまき な の です 。 苦しい 年金 生活 者 の 暮らし を 気遣う の なら 、 マクロ 経済 スライド を 撤回 し 、 最低 保障 年金 制度 の 創設 に 踏み出す べき で は あり ませ ん か 。   次 に 、 軍事 費 の 問題 です 。   補正 予算 案 と 本 予算 案 を 合わせる と 、 過去 最高 の 五 兆 一千 七百 十 八 億 円 と なり まし た 。 軽装 甲 機動 車 三十 八 両 、 九 六 式 装輪 装甲 車 八 両 、 ＮＢＣ 偵察 車 一両 の 購入 など 、 補正 予算 と 一体 に 二 〇 一 六 年度 本 予算 が 増強 さ れ 、 極めて 異常 な 軍拡 補正 予算 と なっ て おり ます 。 国民 の 多数 の 反対 を 押し切っ て 強行 採決 し た **戦争** **法** ( `平和安全法制` ) を 財政 面 から 支える もの で あり 、 到底 認める こと は でき ませ ん 。   さらに 、 辺野古 新 基地 建設 問題 です 。   普天間 基地 を より 一層 危険 に し て いる の は 、 県民 の 暮らし や 安全 より も 米軍 の 運用 を 最 優先 さ せる 日本 政府 の 態度 で あり 、 解決 する ため に は 、 移設 条件 なし の 撤去 以外 に あり ませ ん 。 政府 は 、 沖縄 県 民 の 総意 で ある 建白 書 を 受け入れ 、 基地 政策 の 転換 を 図る べき で あり ます 。   次 に 、 Ｔ Ｐ Ｐ です 。   本 会議 代表 質問 で 、 我が 党 の 井上 議員 が 、 政府 が 大幅 譲歩 し 、 国会 決議 を ほご に し 、 協定 案 全文 の 日本 語 版 すら 公表 し て い ない と 、 その 姿勢 を 追及 いたし まし た 。 政府 は 直後 に 暫定 仮訳 版 を 公表 し まし た けれど も 、 ＴＰＰ の 全容 を 知る 上 で 欠かせ ない 附属 書 など が 訳さ れ て い ない こと に 加え 、 交渉 経過 の 詳細 も 国民 に 明らか に さ れ て おり ませ ん 。   日本 共産 党 は 、 農業 関係 者 の 反対 の 声 を 押し切り 、 自民 党 自身 の 公約 も ほご に し て 大筋 合意 し た Ｔ Ｐ Ｐ の 批准 、 署名 に 断固 反対 する と とも に 、 Ｔ ＰＰ から の 撤退 を 求める もの で あり ます 。   最後 に 、 社会 保障 費 の 削減 と 格差 と 貧困 の 問題 です 。   小泉 政権 で は 、 消費 税 は 増税 し ない の で 痛み に 耐えよ と 、 毎年 二千 二百 億 円 の 社会 保障 費 自然 増 抑制 を 強行 いたし まし た 。 ところ が 、 安倍 政権 に おい て は 、 消費 税 増税 に 加え 、 社会 保障 費 自然 増 抑制 は 小泉 政権 時 を はるか に 上回る もの に なっ て おり 、 その 結果 、 医療 崩壊 、 介護 難民 を より 深刻 に し まし た 。 最後 の 命綱 で ある 生活 保護 費 も 無 慈悲 に 切り下げ 、 来 年度 は 、 年金 給付 の 引下げ 、 福祉 給付 金 の 半減 、 診療 報酬 の 減額 など 、 更なる 改悪 の オン パレード です 。   そんな 中 、 格差 と 貧困 が 広がり 、 とりわけ 女性 と 子供 の 貧困 は 深刻 です 。 一人親 家庭 の 子供 の 貧困 率 は 五 四 ・ 六 ％ で 、 ＯＥＣＤ 加盟 三十 四 か国 で 最悪 です 。 就労 世帯 ほど 貧困 率 が 進む と いう 、 世界 で も 類 の ない 異常 な 状態 です 。   ところ が 、 安倍 政権 は 、 こう し た 貧困 の 解決 を 口 で は 掲げ ながら 、 現実 から 目 を 背け 、 貧困 率 改善 の 数値 目標 さえ 掲げ て おり ませ ん 。 貧困 問題 を 自助 努力 で 解決 する こと は でき ませ ん 。 格差 と 貧困 は 政治 が つくり出し た もの で あり 、 解決 する 責任 は 政治 に ある 、 この 立場 に 立ち 切る こと が できる の か が 問わ れ て いる の です 。 <br/> ○ 議長 （ 山崎 正昭 君 ）   辰巳 君 、 時間 が 参り まし た 。 簡単 に 願い ます 。 <br/> ○ 辰巳 孝太郎 君 （ 続 ）   税制 、 社会 保障 制度 、 教育 制度 、 雇用 政策 など 、 あらゆる 政策 を 総 動員 し て 関連 予算 を 抜本 的 に 増額 し て 取り組む こと 、 これ こそ が 緊急 の 課題 で あり ます 。   以上 、 日本 共産 党 は 、 大 企業 と 富裕 層 優遇 を 改めて 能力 に 応じ た 税制 へ の 転換 を 図り 、 消費 税 の 増税 は 中止 し 、 国民 生活 を 応援 し て 経済 の 好 循環 を つくり 、 そして 日本 国 憲法 に 違反 する **戦争** **法** ( `平和安全法制` ) は 廃止 に する ため に 全力 を 尽くす 決意 を 述べ て 、 反対 討論 と いたし ます 。 （ 拍手 ） <br/> ○ 議長 （ 山崎 正昭 君 ）   これ にて 討論 は 終局 いたし まし た 。         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   これ より 両案 を 一括 し て 採決 いたし ます 。   両案 の 賛否 に つい て 、 投票 ボタン を お 押し 願い ます 。       〔 投票 開始 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   間 も なく 投票 を 終了 いたし ます 。 ─ ─ これ にて 投票 を 終了 いたし ます 。       〔 投票 終了 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   投票 の 結果 を 報告 いたし ます 。     投票 総数                   二百 三十 三         賛成                         百 三十 七         反対                           九十 六       よっ て 、 両案 は 可決 さ れ まし た 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 投票 者 氏名 は 本号 末尾 に 掲載 〕           ─ ─ ─ ─ ─ ・ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   日程 第 三   **一般** **職** **の** **職員** **の** **給与** **に** **関する** **法律** **等** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` )   日程 第 四   **特別** **職** **の** **職員** **の** **給与** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` )     （ いずれ も 内閣 提出 、 衆議 院 送付 ）   以上 両案 を 一括 し て 議題 と いたし ます 。   まず 、 委員 長 の 報告 を 求め ます 。 内閣 委員 長 神本 美恵子 君 。         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 審査 報告 書 及び 議案 は 本号 末尾 に 掲載 〕         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 神本 美恵子 君 登壇 、 拍手 〕 <br/> ○ 神本 美恵子 君   ただいま 議題 と なり まし た 両 法律 案 に つき まし て 、 内閣 委員 会 に おけ る 審査 の 経過 と 結果 を 御 報告 申し上げ ます 。   まず 、 **一般** **職** **の** **職員** **の** **給与** **に** **関する** **法律** **等** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) は 、 人事 院 の 国会 及び 内閣 に 対する 平成 二十 七 年 八 月 六 日 付け の 職員 の 給与 の 改定 に 関する 勧告 及び 職員 の 勤務 時間 の 改定 に 関する 勧告 に 鑑み 、 一般 職 の 国家 公務 員 に つい て 、 俸給 月額 、 初任 給 調整 手当 及び 勤勉 手当 の 額 の 改定 を 行う と とも に 、 職員 の 申告 を 考慮 し て 勤務 時間 を 割り振る 制度 の 対象 を 拡大 する 等 の 措置 を 講じよう と する もの で あり ます 。   次 に 、 **特別** **職** **の** **職員** **の** **給与** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) は 、 一般 職 の 国家 公務 員 の 給与 改定 に 伴い 、 特別 職 の 職員 の 給与 の 額 を 改定 しよう と する もの で あり ます 。   委員 会 に おき まし て は 、 両 法律 案 を 一括 し て 議題 と し 、 現行 の 官民 給与 の 比較 方法 の 妥当 性 、 **給与** **法** ( `NIL` ) 改正 に 伴う 国 の 非 常勤 職員 給与 の 取扱い 、 国 の 給与 改定 の 遅れ が 地方 に 与える 影響 等 に つい て 質疑 が 行わ れ まし た が 、 その 詳細 は 会議 録 に よっ て 御 承知 願い ます 。   質疑 を 終了 し た 後 、 生活 の 党 と 山本 太郎 と なかま たち の 山本 委員 より 、 **一般** **職** **の** **職員** **の** **給与** **に** **関する** **法律** **等** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) に 対し 、 **一般** **職** **の** **職員** **の** **給与** **に** **関する** **法律** ( `一般職の職員の給与に関する法律` ) に 定める 指定 職 俸給 表 の 俸給 月額 及び 同 俸給 表 の 適用 を 受ける 職員 に 係る 勤勉 手当 の 支給 割合 の 改定 は 行わ ない こと を 内容 と する 修正 案 が 提出 さ れ まし た 。   次い で 、 討論 に 入り まし た ところ 、 日本 共産 党 の 田村 委員 より 、 **一般** **職** **の** **職員** **の** **給与** **に** **関する** **法律** **等** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) の 原案 及び 修正 案 に 賛成 、 **特別** **職** **の** **職員** **の** **給与** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) に 反対 の 旨 の 意見 が 述べ られ まし た 。   次い で 、 順次 採決 の 結果 、 修正 案 は 賛成 少数 を もっ て 否決 さ れ 、 両 法律 案 は いずれ も 多数 を もっ て 原案 どおり 可決 す べき もの と 決定 いたし まし た 。   以上 、 御 報告 申し上げ ます 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   これ より 採決 を いたし ます 。   まず 、 **一般** **職** **の** **職員** **の** **給与** **に** **関する** **法律** **等** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) の 採決 を いたし ます 。   本案 の 賛否 に つい て 、 投票 ボタン を お 押し 願い ます 。       〔 投票 開始 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   間 も なく 投票 を 終了 いたし ます 。 ─ ─ これ にて 投票 を 終了 いたし ます 。       〔 投票 終了 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   投票 の 結果 を 報告 いたし ます 。     投票 総数                   二百 三十 五         賛成                       二百 二十 二         反対                             十 三       よっ て 、 本案 は 可決 さ れ まし た 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 投票 者 氏名 は 本号 末尾 に 掲載 〕         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   次 に 、 **特別** **職** **の** **職員** **の** **給与** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) の 採決 を いたし ます 。   本案 の 賛否 に つい て 、 投票 ボタン を お 押し 願い ます 。       〔 投票 開始 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   間 も なく 投票 を 終了 いたし ます 。 ─ ─ これ にて 投票 を 終了 いたし ます 。       〔 投票 終了 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   投票 の 結果 を 報告 いたし ます 。     投票 総数                   二百 三十 四         賛成                           二百 九         反対                           二十 五       よっ て 、 本案 は 可決 さ れ まし た 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 投票 者 氏名 は 本号 末尾 に 掲載 〕         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   申し上げ ます 。   賛成 、 反対 の こと に つき まし て は 、 簡潔 に 願い ます 。           ─ ─ ─ ─ ─ ・ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   日程 第 五   **裁判** **官** **の** **報酬** **等** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` )   日程 第 六   **検察** **官** **の** **俸給** **等** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` )     （ いずれ も 内閣 提出 、 衆議 院 送付 ）   以上 両案 を 一括 し て 議題 と いたし ます 。   まず 、 委員 長 の 報告 を 求め ます 。 法務 委員 長 魚住 裕一郎 君 。         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 審査 報告 書 及び 議案 は 本号 末尾 に 掲載 〕         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 魚住 裕一郎 君 登壇 、 拍手 〕 <br/> ○ 魚住 裕一郎 君   ただいま 議題 と なり まし た 両 法律 案 に つき まし て 、 法務 委員 会 に おけ る 審査 の 経過 と 結果 を 御 報告 申し上げ ます 。   両 法律 案 は 、 一般 の 政府 職員 の 給与 改定 に 伴い 、 裁判 官 の 報酬 月額 及び 検察 官 の 俸給 月額 の 改定 を 行おう と する もの で あり ます 。   委員 会 に おき まし て は 、 両 法律 案 を 一括 し て 議題 と し 、 裁判 官 及び 検察 官 の 職責 と 処遇 、 法曹 志願 者 減少 の 原因 と 法科 大学 院 が 抱える 課題 、 裁判 官 の 報酬 及び 検察 官 の 俸給 の 決定 の 在り 方 、 **少年** **法** ( `少年法` ) に よる 少年 の 更生 の 成果 等 に つい て 質疑 が 行わ れ まし た が 、 その 詳細 は 会議 録 に よっ て 御 承知 願い ます 。   質疑 を 終局 し 、 順次 採決 の 結果 、 両 法律 案 は いずれ も 全会 一致 を もっ て 原案 どおり 可決 す べき もの と 決定 いたし まし た 。   以上 、 御 報告 申し上げ ます 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   これ より 両案 を 一括 し て 採決 いたし ます 。   両案 の 賛否 に つい て 、 投票 ボタン を お 押し 願い ます 。       〔 投票 開始 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   間 も なく 投票 を 終了 いたし ます 。 ─ ─ これ にて 投票 を 終了 いたし ます 。       〔 投票 終了 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   投票 の 結果 を 報告 いたし ます 。     投票 総数                   二百 三十 四         賛成                       二百 二十 二         反対                             十 二       よっ て 、 両案 は 可決 さ れ まし た 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 投票 者 氏名 は 本号 末尾 に 掲載 〕           ─ ─ ─ ─ ─ ・ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   日程 第 七   **防衛** **省** **の** **職員** **の** **給与** **等** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) （ 内閣 提出 、 衆議 院 送付 ） を 議題 と いたし ます 。   まず 、 委員 長 の 報告 を 求め ます 。 外交 防衛 委員 長 佐藤 正久 君 。         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 審査 報告 書 及び 議案 は 本号 末尾 に 掲載 〕         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 佐藤 正久 君 登壇 、 拍手 〕 <br/> ○ 佐藤 正久 君   ただいま 議題 と なり まし た **防衛** **省** **の** **職員** **の** **給与** **等** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) に つき まし て 、 外交 防衛 委員 会 に おけ る 審査 の 経過 と 結果 を 御 報告 申し上げ ます 。   本 法律 案 は 、 一般 職 の 国家 公務 員 の 例 に 準じ て 、 防衛 省 職員 の 俸給 月額 等 を 改定 する 措置 を 講じよう と する もの で あり ます 。   委員 会 に おき まし て は 、 自衛 官 の 給与 改定 が 一般 職 国家 公務 員 の 給与 改定 に 準拠 する 理由 、 医官 及び 歯科 医官 に 対する 給与 の 在り 方 、 自衛 官 を 含む 防衛 省 職員 へ の フレックス タイム 制 の 拡充 、 運用 の 見通し 、 自衛 官 募集 の 現状 と その 強化 策 、 女性 自衛 官 活用 の ため の 対応 策 、 自衛 官 の 勤務 環境 の 改善 等 に つい て 質疑 が 行わ れ まし た が 、 詳細 は 会議 録 に よっ て 御 承知 願い ます 。   質疑 を 終え 、 採決 の 結果 、 本 法律 案 は 全会 一致 を もっ て 原案 どおり 可決 す べき もの と 決定 いたし まし た 。   以上 、 御 報告 申し上げ ます 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   これ より 採決 を いたし ます 。   本案 の 賛否 に つい て 、 投票 ボタン を お 押し 願い ます 。       〔 投票 開始 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   間 も なく 投票 を 終了 いたし ます 。 ─ ─ これ にて 投票 を 終了 いたし ます 。       〔 投票 終了 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   投票 の 結果 を 報告 いたし ます 。     投票 総数                   二百 三十 六         賛成                       二百 二十 四         反対                             十 二       よっ て 、 本案 は 可決 さ れ まし た 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 投票 者 氏名 は 本号 末尾 に 掲載 〕           ─ ─ ─ ─ ─ ・ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   日程 第 八   **地方** **交付** **税** **法** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) （ 内閣 提出 、 衆議 院 送付 ） を 議題 と いたし ます 。   まず 、 委員 長 の 報告 を 求め ます 。 総務 委員 長 山本 博司 君 。         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 審査 報告 書 及び 議案 は 本号 末尾 に 掲載 〕         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 山本 博司 君 登壇 、 拍手 〕 <br/> ○ 山本 博司 君   ただいま 議題 と なり まし た 法律 案 に つき まし て 、 総務 委員 会 に おけ る 審査 の 経過 と 結果 を 御 報告 申し上げ ます 。   本 法律 案 は 、 地方 財政 の 状況 等 に 鑑み 、 震災 復興 特別 交付 税 の うち 、 東 日本 大 震災 に 係る 復興 事業 等 の 実施 状況 に より 平成 二十 六 年度 の 決算 に おい て 不用 と なっ た 金額 を 減額 する ほか 、 補正 予算 に より 増加 し た 平成 二十 七 年度 分 の 地方 交付 税 の 額 の 一部 を 平成 二十 八 年度 分 の 地方 交付 税 の 総額 に 加算 し て 交付 する こと が できる こと と する もの で あり ます 。   委員 会 に おき まし て は 、 地方 交付 税 に よる 年度 間 調整 の 在り 方 、 震災 復興 特別 交付 税 の 不用 額 の 減額 に よる 復興 へ の 影響 の 有無 、 復興 ・ 創生 期間 に おけ る 人的 支援 等 に つい て 質疑 が 行わ れ まし た 。   質疑 を 終局 し 、 討論 に 入り まし た ところ 、 日本 共産 党 を 代表 し て 倉林 明子 委員 より 反対 する 旨 の 意見 が 述べ られ まし た 。   討論 を 終局 し 、 採決 の 結果 、 本 法律 案 は 多数 を もっ て 原案 どおり 可決 す べき もの と 決定 いたし まし た 。   以上 、 御 報告 申し上げ ます 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   これ より 採決 を いたし ます 。   本案 の 賛否 に つい て 、 投票 ボタン を お 押し 願い ます 。       〔 投票 開始 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   間 も なく 投票 を 終了 いたし ます 。 ─ ─ これ にて 投票 を 終了 いたし ます 。       〔 投票 終了 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   投票 の 結果 を 報告 いたし ます 。     投票 総数                   二百 三十 六         賛成                       二百 二十 六         反対                               十       よっ て 、 本案 は 可決 さ れ まし た 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 投票 者 氏名 は 本号 末尾 に 掲載 〕           ─ ─ ─ ─ ─ ・ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   この 際 、 日程 に 追加 し て 、   **国会** **議員** **の** **秘書** **の** **給与** **等** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) （ 衆議 院 提出 ） を 議題 と する こと に 御 異議 ござい ませ ん か 。       〔 「 異議 なし 」 と 呼ぶ 者 あり 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   御 異議 ない と 認め ます 。   まず 、 委員 長 の 報告 を 求め ます 。 議院 運営 委員 長 松山 政司 君 。         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 審査 報告 書 及び 議案 は 本号 末尾 に 掲載 〕         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 松山 政司 君 登壇 、 拍手 〕 <br/> ○ 松山 政司 君   ただいま 議題 と なり まし た 法律 案 に つき まし て 、 委員 会 に おけ る 審査 の 経過 と 結果 を 御 報告 を 申し上げ ます 。   **国会** **議員** **の** **秘書** **の** **給与** **等** **に** **関する** **法律** **の** **一部** **を** **改正** **する** **法律** **案** ( `NIL` ) は 、 政府 職員 の 給与 改定 に 伴い 、 議員 秘書 の 給料 月額 及び 勤勉 手当 の 支給 割合 を それぞれ 改定 しよう と する もの で ござい ます 。   委員 会 に おき まし て は 、 採決 の 結果 、 本 法律 案 は 全会 一致 を もっ て 原案 どおり 可決 す べき もの と 決定 を いたし まし た 。   以上 、 御 報告 を 申し上げ ます 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   これ より 採決 を いたし ます 。   本案 の 賛否 に つい て 、 投票 ボタン を お 押し 願い ます 。       〔 投票 開始 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   間 も なく 投票 を 終了 いたし ます 。 ─ ─ これ にて 投票 を 終了 いたし ます 。       〔 投票 終了 〕 <br/> ○ 議長 （ 山崎 正昭 君 ）   投票 の 結果 を 報告 いたし ます 。     投票 総数                   二百 三十 六         賛成                       二百 二十 四         反対                             十 二       よっ て 、 本案 は 可決 さ れ まし た 。 （ 拍手 ）         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─       〔 投票 者 氏名 は 本号 末尾 に 掲載 〕         ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ <br/> ○ 議長 （ 山崎 正昭 君 ）   これ にて 午後 一時 まで 休憩 いたし ます 。       午前 十 時 五十 三 分 休憩           ─ ─ ─ ─ ─ ・ ─ ─ ─ ─ ─       午後 一時 一 分 開議 <br/> ○ 副 議長 （ 輿石 東 君 ）   休憩 前 に 引き続き 、 会議 を 開き ます 。   日程 第 九   国務 大臣 の 報告 に 関する 件 （ 平成 二十 六 年度 決算 の 概要 に つい て ）   財務 大臣 から 発言 を 求め られ て おり ます 。 発言 を 許し ます 。 財務 大臣 麻生 太郎 君 。       〔 国務 大臣 麻生 太郎 君 登壇 、 拍手 〕 <br/> ○ 国務 大臣 （ 麻生 太郎 君 ）   平成 二十 六 年度 の 一般 会計 歳入 歳出 決算 、 特別 会計 歳入 歳出 決算 、 国税 収納 金 整理 資金 受払 計算 書 及び 政府 関係 機関 決算 書 を 会計 検査 院 の 検査 報告 と とも に 国会 に 提出 し 、 また 、 平成 二十 六 年度 の 国 の 債権 の 現在 額 並び に 物品 の 増減 及び 現在 額 に つき まし て も 国会 に 報告 を いたし て おり ます の で 、 その 概要 を 御 説明 さ せ て いただき ます 。   まず 、 平成 二十 六 年度 の 一般 会計 の 決算 に つき まし て は